In [58]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import lightgbm as lgbm
import catboost
from catboost import CatBoostClassifier
from ast import literal_eval

import sklearn
from sklearn.metrics import mean_absolute_error, mean_squared_error, roc_auc_score
from sklearn.model_selection import KFold, TimeSeriesSplit, StratifiedKFold, GroupKFold

import itertools

import datetime
import time
from tqdm import tqdm

import gc

In [6]:
print(lgbm.__version__)
print(catboost.__version__)
print(pd.__version__)
print(sklearn.__version__)
print(np.__version__)

3.1.0
0.24.3
1.0.5
0.23.1
1.18.5


In [59]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
sample_submission = pd.read_csv('./data/sample_submission.csv')

In [66]:
train

,game_id,winner,time,player,species,event,event_contents
0,0,1,0.00,0,T,Camera,"at (145.25, 21.5078125)"
1,0,1,0.00,1,T,Camera,"at (22.75, 147.0078125)"
2,0,1,0.02,0,T,Selection,['OrbitalCommand [3080001]']
3,0,1,0.02,0,T,Ability,(1360) - TrainSCV
4,0,1,0.14,0,T,Camera,"at (142.99609375, 24.50390625)"
...,...,...,...,...,...,...,...
33578027,38871,0,8.51,0,Z,Camera,"at (139.578125, 62.58203125)"
33578028,38871,0,8.52,1,T,GetControlGroup,NaN
33578029,38871,0,8.52,0,Z,Camera,"at (122.42578125, 45.4296875)"
33578030,38871,0,8.52,0,Z,Camera,"at (122.42578125, 43.25390625)"


In [67]:
def value (data):
    nRow, nCol = data.shape
    for i in tqdm(range(nRow)):#len(data)
        for j in range(nCol):
            if(data.columns[j] == "winner"):
            else if(data.columns[j] == "winner"):
            else if(data.columns[j] == "player0_species"):
                if(data.loc[i,j] == 'P'):
                    data.loc[i,j] = 0
                else if(data.loc[i,j] == 'T'):
                    data.loc[i,j] = 0
                else:
                    data.loc[i,j] = 0
            else if(data.columns[j] == "player1_species"):
                if(data.loc[i,j] == 'P'):
                    data.loc[i,j] = 0
                else if(data.loc[i,j] == 'T'):
                    data.loc[i,j] = 0
                else:
                    data.loc[i,j] = 0
            else if(data.columns[j] == "player0_apm"):
            else if(data.columns[j] == "player1_apm"):
            else if(data.columns[j] == "player0_X_apm"):
            else if(data.columns[j] == "player1_X_apm"):
            else:
                data.loc[i,j] = len(data.loc[i,j])
    
    x_data = data.iloc[:,1:]
    y_data = data.iloc[:,:1]
    return x_data, y_data

IndentationError: expected an indented block (<ipython-input-67-7e61d6f83305>, line 6)

In [7]:
def calc_unit_dict(x, dict_data, all_row_count):
    """
    아래 형태로 Data를 만듬
    key: tuple (game_id, player)
    value: dict (key: unit이름, value: unit_id set)
    
    예) 
    game_id: 0
    player: 0
    event_contents:
    ['CreepOnlyBlocker4x4 [1B00001]']
    "['SCV [30C0001]', 'SCV [3780001]']"
    
    변화 결과
    dict_data = {(0,0):{'SCV':set(30C0001, 3780001),
                        'CreepOnlyBlocker4x4':set(1B00001)}}
    """
    global progress
    g = x['game_id']
    p = x['player']
    k = (g,p)
    e = x['event_contents']
    split_item = e.split(',')
    if progress%100000==0:
        print(progress/all_row_count*100)
    progress+=1
    for c in split_item:
        t = c.strip('[\'] ').split(' ')
        if len(t)<2: # 이상한 Data, unit id가 없음..
            continue
        t0 = t[0] # unit 이름
        t1 = t[1].strip('[]') # unit id
        if k in dict_data: # dict_data에 game_id, player가 있다면
            d = dict_data[k]
            if t0 not in d:
                d[t0] = set()
            d[t0].add(t1)
        else: # 없을 때
            dict_data[k] = dict()
            d = dict_data[k]
            d[t0] = set()
            d[t0].add(t1)

train_unit_dict = dict()
progress = 0
slelection_row_count = train.loc[train['event']=='Selection'].shape[0]            
train.loc[train['event']=='Selection',['game_id','event_contents','player']].apply(lambda x: calc_unit_dict(x,train_unit_dict,slelection_row_count),axis=1)

test_unit_dict = dict()
progress = 0
slelection_row_count = test.loc[test['event']=='Selection'].shape[0]
test.loc[test['event']=='Selection',['game_id','event_contents','player']].apply(lambda x: calc_unit_dict(x,test_unit_dict,slelection_row_count),axis=1)

0.0
2.1006858739378407
4.201371747875681
6.302057621813523
8.402743495751363
10.503429369689202
12.604115243627046
14.704801117564886
16.805486991502725
18.906172865440567
21.006858739378405
23.107544613316247
25.20823048725409
27.308916361191933
29.40960223512977
31.51028810906761
33.61097398300545
35.71165985694329
37.812345730881134
39.91303160481897
42.01371747875681
44.11440335269465
46.21508922663249
48.315775100570335
50.41646097450818
52.51714684844602
54.617832722383866
56.7185185963217
58.81920447025954
60.919890344197384
63.02057621813522
65.12126209207307
67.2219479660109
69.32263383994875
71.42331971388658
73.52400558782442
75.62469146176227
77.7253773357001
79.82606320963794
81.92674908357579
84.02743495751362
86.12812083145147
88.2288067053893
90.32949257932715
92.43017845326499
94.53086432720283
96.63155020114067
98.73223607507852
0.0
2.452246785288383
4.904493570576766
7.356740355865149
9.808987141153532
12.261233926441916
14.713480711730298
17.165727497018683
19.61797

2           None
31          None
65          None
86          None
97          None
            ... 
28714812    None
28714813    None
28714819    None
28714821    None
28714838    None
Length: 4077893, dtype: object

In [10]:
train_unit_df = pd.DataFrame(train_unit_dict).T.reset_index()
train_unit_df.rename(columns={'level_0':'game_id','level_1':'player'}, inplace=True)

test_unit_df = pd.DataFrame(test_unit_dict).T.reset_index()
test_unit_df.rename(columns={'level_0':'game_id','level_1':'player'}, inplace=True)

In [55]:
train_unit_df

,game_id,player,OrbitalCommand,SCV,CreepOnlyBlocker4x4,Barracks,Reaper,SupplyDepotLowered,Factory,None,...,DestructibleIce2x6Vertical,BroodLordWeapon,BanelingBurrowed,BroodLordAWeapon,ExtendingBridgeNWWide8,DestructibleIce2x6Horizontal,XelNagaDestructibleBlocker8W,CreepTumor,CreepTumorQueen,HydraliskBurrowed
0,0,0,"{3B80001, 3080001}","{3380001, 3180001, 3880001, 3140001, 3580003, ...",{1B00001},{38C0001},{3DC0002},{3740001},{3E40001},{1},...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,1,"{33C0001, 4340002}","{3D00001, 3480001, 3640001, 3840001, 3440001, ...",NaN,"{3A40002, 39C0001}",NaN,NaN,{3D40001},NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{44C0002, 5200003, 5C0001}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,14,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38810,38867,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{4A00001, 40C0006, 4140004}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38811,38868,0,{3000001},"{3F80001, 3180001, 3880001, 3740001, 3A40001, ...",NaN,"{38C0001, 3D80001}","{4200001, 4600001}",NaN,{4800001},NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38812,38868,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{4840003},...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38813,38871,1,"{3FC0003, 3E00001, 3540001}","{5580003, 4A40005, 5040004, 3800001, 38C0001, ...",NaN,"{3AC0001, 5680002, 54C0003}",{3E40004},"{4A80003, 3140002}",{3D00004},{2880001},...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
test_unit_df

,game_id,player,Nexus,Probe,Pylon,WarpGate,PhotonCannon,Stargate,Sentry,Zealot,...,ChangelingZergling,ChangelingMarineShield,CreepTumorQueen,DestructibleIce2x6Vertical,ExtendingBridgeNEWide8Out,Corruptor,BroodLord,BroodLordAWeapon,InfestedTerran,AiurLightBridgeNE10Out
0,38872,1,"{3100001, 3C00008}","{3380001, 3400001, 3180001, 3880001, 3140001, ...",{3C40001},"{3C40003, 3780001}",{3C00002},{3E80001},{40C0001},"{41C0001, 3D40001}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,38872,0,"{3AC0001, 2DC0001}","{3080001, 3A40001, 3800001, 3480001, 38C0001, ...",NaN,"{3A00002, 3680001}",{3C00001},{32C0002},NaN,"{3D00004, 3E40001, 4040001}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,38873,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,38873,1,{3C40001},"{3640001, 3B80001, 3780001, 3680001, 35C0001, ...",NaN,{3980004},NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,38874,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33520,55656,1,{3440001},"{3A40001, 3480001, 38C0001, 3E40018, 3E80001, ...","{3C80001, 3C40001}",NaN,{3D80001},NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33521,55657,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33522,55657,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33523,55658,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
bad_feature_fold1 = ['Ability_player_0_isnull', 'Ability_player_1_isnull', 'AddToControlGroup_player_0', 'AddToControlGroup_player_0_AddToControlGroup_player_1_diff', 'AddToControlGroup_player_0_isnull', 'AddToControlGroup_player_1', 'AdeptPiercingWeapon_len_player_0', 'Adept_right_click_target_count_diff', 'Adept_right_click_target_count_player_0', 'AiurLightBridgeAbandonedNE10Out_len_player_0', 'AiurLightBridgeAbandonedNE10Out_len_player_1', 'AiurLightBridgeAbandonedNE8Out_len_player_0', 'AiurLightBridgeAbandonedNE8Out_len_player_1', 'AiurLightBridgeAbandonedNE8_len_player_0', 'AiurLightBridgeAbandonedNE8_len_player_1', 'AiurLightBridgeNE10Out_len_player_0', 'AiurLightBridgeNE10Out_len_player_1', 'AiurLightBridgeNE8_len_player_0', 'AiurLightBridgeNE8_len_player_1', 'Archon_len_player_0', 'Archon_len_player_1', 'Armory_len_player_0', 'Assimilator_len_player_0', 'Assimilator_len_player_1', 'AutoTurret_len_player_0', 'AutoTurret_len_player_1', 'BanelingBurrowed_len_player_0', 'BanelingBurrowed_len_player_1', 'BanelingCocoon_len_player_0_Factory_len_player_0_diff', 'BanelingCocoon_len_player_0_Hatchery_len_player_0_diff', 'BanelingCocoon_len_player_0_MULE_len_player_0_diff', 'BanelingCocoon_len_player_0_Reaper_len_player_0_diff', 'BanelingCocoon_len_player_0_Starport_len_player_0_diff', 'BanelingCocoon_len_player_1_Hatchery_len_player_1_diff', 'BanelingCocoon_len_player_1_Hatchery_len_player_1_diff_BanelingCocoon_len_player_0_Hatchery_len_player_0_diff_diff', 'BanelingCocoon_len_player_1_Hellion_len_player_1_diff', 'BanelingCocoon_len_player_1_Overlord_len_player_1_diff', 'BanelingCocoon_len_player_1_Overlord_len_player_1_diff_BanelingCocoon_len_player_0_Overlord_len_player_0_diff_diff', 'BanelingCocoon_len_player_1_Queen_len_player_1_diff', 'BanelingCocoon_len_player_1_Reaper_len_player_1_diff', 'BanelingCocoon_len_player_1_Reaper_len_player_1_diff_BanelingCocoon_len_player_0_Reaper_len_player_0_diff_diff', 'BanelingCocoon_len_player_1_Starport_len_player_1_diff', 'BanelingCocoon_len_player_1_Starport_len_player_1_diff_BanelingCocoon_len_player_0_Starport_len_player_0_diff_diff', 'BanelingCocoon_right_click_target_count_player_1', 'BanelingNest_len_player_1', 'Baneling_len_player_0', 'Baneling_right_click_target_count_diff', 'Baneling_right_click_target_count_player_0', 'Baneling_right_click_target_count_player_1', 'Banshee_len_player_0', 'Banshee_len_player_1', 'BarracksReactor_len_player_1', 'Barracks_len_player_0_BanelingCocoon_len_player_0_diff', 'Barracks_len_player_0_Factory_len_player_0_diff', 'Barracks_len_player_0_Hatchery_len_player_0_diff', 'Barracks_len_player_0_OrbitalCommand_len_player_0_diff', 'Barracks_len_player_0_Overlord_len_player_0_diff', 'Barracks_len_player_0_Queen_len_player_0_diff', 'Barracks_len_player_0_Reaper_len_player_0_diff', 'Barracks_len_player_0_Roach_len_player_0_diff', 'Barracks_len_player_0_Starport_len_player_0_diff', 'Barracks_len_player_0_SupplyDepotLowered_len_player_0_diff', 'Barracks_len_player_1_BanelingCocoon_len_player_1_diff', 'Barracks_len_player_1_Factory_len_player_1_diff_Barracks_len_player_0_Factory_len_player_0_diff_diff', 'Barracks_len_player_1_Hellion_len_player_1_diff_Barracks_len_player_0_Hellion_len_player_0_diff_diff', 'Barracks_len_player_1_OrbitalCommand_len_player_1_diff', 'Barracks_len_player_1_OrbitalCommand_len_player_1_diff_Barracks_len_player_0_OrbitalCommand_len_player_0_diff_diff', 'Barracks_len_player_1_Overlord_len_player_1_diff', 'Barracks_len_player_1_Overlord_len_player_1_diff_Barracks_len_player_0_Overlord_len_player_0_diff_diff', 'Barracks_len_player_1_Queen_len_player_1_diff_Barracks_len_player_0_Queen_len_player_0_diff_diff', 'Barracks_len_player_1_Reaper_len_player_1_diff', 'Barracks_len_player_1_Roach_len_player_1_diff', 'Barracks_len_player_1_Starport_len_player_1_diff_Barracks_len_player_0_Starport_len_player_0_diff_diff', 'Barracks_right_click_target_count_diff', 'Battlecruiser_len_player_0', 'Battlecruiser_len_player_1', 'Beacon_TerranSmall_len_player_0', 'Beacon_TerranSmall_len_player_1', 'Beacon_Terran_len_player_0', 'Beacon_Terran_len_player_1', 'BroodLordAWeapon_len_player_0', 'BroodLordAWeapon_len_player_1', 'BroodLordWeapon_len_player_0', 'BroodLordWeapon_len_player_1', 'BroodLord_len_player_0', 'BroodLord_len_player_1', 'Broodling_len_player_1', 'Bunker_len_player_1', 'Bunker_right_click_target_count_diff', 'Bunker_right_click_target_count_player_1', 'BypassArmorDrone_len_player_0', 'BypassArmorDrone_len_player_1', 'Camera_player_0', 'Camera_player_0_Camera_player_1_diff', 'Camera_player_0_isnull', 'Camera_player_1_isnull', 'Carrier_len_player_0', 'Carrier_len_player_1', 'ChangelingMarineShield_len_player_1', 'ChangelingMarine_len_player_0', 'ChangelingMarine_len_player_1', 'ChangelingZealot_len_player_0', 'ChangelingZealot_len_player_1', 'ChangelingZerglingWings_len_player_0', 'ChangelingZerglingWings_len_player_1', 'ChangelingZergling_len_player_0', 'ChangelingZergling_len_player_1', 'Changeling_len_player_0', 'Changeling_len_player_1', 'Colossus_len_player_0', 'ControlGroup_player_0_ControlGroup_player_1_diff', 'ControlGroup_player_0_isnull', 'ControlGroup_player_1', 'ControlGroup_player_1_isnull', 'Corruptor_len_player_0', 'Corruptor_len_player_1', 'CreepOnlyBlocker4x4_len_player_0', 'CreepOnlyBlocker4x4_len_player_1', 'CreepTumorBurrowed_len_player_1', 'CreepTumorBurrowed_right_click_target_count_player_0', 'CreepTumorBurrowed_right_click_target_count_player_1', 'CreepTumorQueen_len_player_0', 'CreepTumorQueen_len_player_1', 'CreepTumor_len_player_0', 'CreepTumor_len_player_1', 'CyberneticsCore_right_click_target_count_diff', 'CyberneticsCore_right_click_target_count_player_0', 'CyberneticsCore_right_click_target_count_player_1', 'CycloneMissileLargeAir_len_player_0', 'CycloneMissileLargeAir_len_player_1', 'Cyclone_len_player_0', 'Cyclone_len_player_1', 'DarkShrine_len_player_0', 'DarkShrine_len_player_1', 'DarkTemplar_len_player_0', 'DarkTemplar_len_player_1', 'DestructibleIce2x4Horizontal_len_player_0', 'DestructibleIce2x4Horizontal_len_player_1', 'DestructibleIce2x4Vertical_len_player_0', 'DestructibleIce2x4Vertical_len_player_1', 'DestructibleIce2x4Vertical_right_click_target_count_player_0', 'DestructibleIce2x4Vertical_right_click_target_count_player_1', 'DestructibleIce2x6Horizontal_len_player_0', 'DestructibleIce2x6Horizontal_len_player_1', 'DestructibleIce2x6Vertical_len_player_0', 'DestructibleIce2x6Vertical_len_player_1', 'DestructibleSignsConstruction_len_player_1', 'DestructibleSignsConstruction_right_click_target_count_diff', 'DestructibleSignsFunny_len_player_0', 'DestructibleSignsIcons_len_player_1', 'DestructibleSignsIcons_right_click_target_count_player_0', 'DestructibleSignsIcons_right_click_target_count_player_1', 'Disruptor_len_player_0', 'Disruptor_len_player_1', 'DroneBurrowed_len_player_0', 'Drone_len_player_0', 'Drone_len_player_0_BanelingCocoon_len_player_0_diff', 'Drone_len_player_0_Factory_len_player_0_diff', 'Drone_len_player_0_Hatchery_len_player_0_diff', 'Drone_len_player_0_OrbitalCommand_len_player_0_diff', 'Drone_len_player_0_Probe_len_player_0_diff', 'Drone_len_player_0_Queen_len_player_0_diff', 'Drone_len_player_0_Reaper_len_player_0_diff', 'Drone_len_player_0_Roach_len_player_0_diff', 'Drone_len_player_0_Starport_len_player_0_diff', 'Drone_len_player_0_Zergling_len_player_0_diff', 'Drone_len_player_1_BanelingCocoon_len_player_1_diff', 'Drone_len_player_1_BanelingCocoon_len_player_1_diff_Drone_len_player_0_BanelingCocoon_len_player_0_diff_diff', 'Drone_len_player_1_Barracks_len_player_1_diff', 'Drone_len_player_1_Factory_len_player_1_diff', 'Drone_len_player_1_Factory_len_player_1_diff_Drone_len_player_0_Factory_len_player_0_diff_diff', 'Drone_len_player_1_Hellion_len_player_1_diff', 'Drone_len_player_1_MULE_len_player_1_diff_Drone_len_player_0_MULE_len_player_0_diff_diff', 'Drone_len_player_1_Marine_len_player_1_diff', 'Drone_len_player_1_Overlord_len_player_1_diff_Drone_len_player_0_Overlord_len_player_0_diff_diff', 'Drone_len_player_1_Probe_len_player_1_diff_Drone_len_player_0_Probe_len_player_0_diff_diff', 'Drone_len_player_1_Queen_len_player_1_diff', 'Drone_len_player_1_Queen_len_player_1_diff_Drone_len_player_0_Queen_len_player_0_diff_diff', 'Drone_len_player_1_Roach_len_player_1_diff_Drone_len_player_0_Roach_len_player_0_diff_diff', 'Drone_len_player_1_Starport_len_player_1_diff', 'Drone_len_player_1_Starport_len_player_1_diff_Drone_len_player_0_Starport_len_player_0_diff_diff', 'Drone_len_player_1_SupplyDepotLowered_len_player_1_diff', 'Drone_len_player_1_SupplyDepotLowered_len_player_1_diff_Drone_len_player_0_SupplyDepotLowered_len_player_0_diff_diff', 'Drone_len_player_1_Zergling_len_player_1_diff_Drone_len_player_0_Zergling_len_player_0_diff_diff', 'Egg_len_player_0_Barracks_len_player_0_diff', 'Egg_len_player_0_MULE_len_player_0_diff', 'Egg_len_player_0_Marine_len_player_0_diff', 'Egg_len_player_0_Reaper_len_player_0_diff', 'Egg_len_player_0_SCV_len_player_0_diff', 'Egg_len_player_1', 'Egg_len_player_1_Barracks_len_player_1_diff', 'Egg_len_player_1_Factory_len_player_1_diff', 'Egg_len_player_1_Hatchery_len_player_1_diff', 'Egg_len_player_1_Hellion_len_player_1_diff', 'Egg_len_player_1_MULE_len_player_1_diff', 'Egg_len_player_1_MULE_len_player_1_diff_Egg_len_player_0_MULE_len_player_0_diff_diff', 'Egg_len_player_1_OrbitalCommand_len_player_1_diff', 'Egg_len_player_1_OrbitalCommand_len_player_1_diff_Egg_len_player_0_OrbitalCommand_len_player_0_diff_diff', 'Egg_len_player_1_Overlord_len_player_1_diff', 'Egg_len_player_1_Probe_len_player_1_diff', 'Egg_len_player_1_Queen_len_player_1_diff', 'Egg_len_player_1_Reaper_len_player_1_diff', 'Egg_len_player_1_SCV_len_player_1_diff', 'Egg_len_player_1_Starport_len_player_1_diff_Egg_len_player_0_Starport_len_player_0_diff_diff', 'EngineeringBay_right_click_target_count_diff', 'EngineeringBay_right_click_target_count_player_0', 'EngineeringBay_right_click_target_count_player_1', 'ExtendingBridgeNEWide10Out_len_player_1', 'ExtendingBridgeNEWide10_len_player_0', 'ExtendingBridgeNEWide10_len_player_1', 'ExtendingBridgeNEWide8Out_len_player_0', 'ExtendingBridgeNEWide8Out_len_player_1', 'ExtendingBridgeNWWide8Out_len_player_0', 'ExtendingBridgeNWWide8Out_len_player_1', 'ExtendingBridgeNWWide8_len_player_0', 'ExtendingBridgeNWWide8_len_player_1', 'Extractor_len_player_0', 'Extractor_right_click_target_count_player_0', 'FactoryReactor_len_player_0', 'FactoryReactor_len_player_1', 'FactoryTechLab_len_player_1', 'Factory_len_player_0_Hatchery_len_player_0_diff', 'Factory_len_player_1', 'Factory_len_player_1_Hatchery_len_player_1_diff', 'Factory_len_player_1_Hatchery_len_player_1_diff_Factory_len_player_0_Hatchery_len_player_0_diff_diff', 'Factory_len_player_1_Hellion_len_player_1_diff', 'Factory_len_player_1_MULE_len_player_1_diff', 'Factory_len_player_1_Reaper_len_player_1_diff', 'Factory_len_player_1_Reaper_len_player_1_diff_Factory_len_player_0_Reaper_len_player_0_diff_diff', 'Factory_len_player_1_Starport_len_player_1_diff', 'Factory_right_click_target_count_player_0', 'Factory_right_click_target_count_player_1', 'FleetBeacon_len_player_0', 'Forge_len_player_0', 'Forge_right_click_target_count_diff', 'Forge_right_click_target_count_player_0', 'Forge_right_click_target_count_player_1', 'FungalGrowthMissile_len_player_1', 'FungalGrowthMissile_right_click_target_count_player_0', 'FusionCore_len_player_0', 'FusionCore_len_player_1', 'GetControlGroup_player_0_GetControlGroup_player_1_diff', 'GetControlGroup_player_0_isnull', 'GetControlGroup_player_1', 'GetControlGroup_player_1_isnull', 'GhostAcademy_len_player_0', 'GhostAcademy_len_player_1', 'Ghost_len_player_0', 'Ghost_len_player_1', 'GlaiveWurmBounceWeapon_len_player_0', 'GlaiveWurmBounceWeapon_len_player_1', 'GreaterSpire_len_player_0', 'GreaterSpire_len_player_1', 'Hatchery_len_player_0', 'Hatchery_len_player_0_Reaper_len_player_0_diff', 'Hatchery_len_player_0_Roach_len_player_0_diff', 'Hatchery_len_player_0_Starport_len_player_0_diff', 'Hatchery_len_player_1', 'Hatchery_len_player_1_Reaper_len_player_1_diff', 'Hatchery_len_player_1_Roach_len_player_1_diff', 'Hatchery_len_player_1_Starport_len_player_1_diff', 'Hatchery_right_click_target_count_diff', 'Hellion_len_player_0_Hatchery_len_player_0_diff', 'Hellion_len_player_0_MULE_len_player_0_diff', 'Hellion_len_player_0_Roach_len_player_0_diff', 'Hellion_len_player_0_Starport_len_player_0_diff', 'Hellion_len_player_1', 'Hellion_len_player_1_Hatchery_len_player_1_diff', 'Hellion_len_player_1_Hatchery_len_player_1_diff_Hellion_len_player_0_Hatchery_len_player_0_diff_diff', 'Hellion_len_player_1_MULE_len_player_1_diff_Hellion_len_player_0_MULE_len_player_0_diff_diff', 'Hellion_len_player_1_Reaper_len_player_1_diff', 'Hellion_len_player_1_Reaper_len_player_1_diff_Hellion_len_player_0_Reaper_len_player_0_diff_diff', 'Hellion_len_player_1_Roach_len_player_1_diff', 'Hellion_len_player_1_Roach_len_player_1_diff_Hellion_len_player_0_Roach_len_player_0_diff_diff', 'Hellion_right_click_target_count_diff', 'Hellion_right_click_target_count_player_0', 'HighTemplar_len_player_0', 'HighTemplar_len_player_1', 'Hive_len_player_0', 'Hive_right_click_target_count_player_0', 'HydraliskDen_len_player_1', 'Hydralisk_len_player_1', 'Immortal_len_player_1', 'InfestationPit_len_player_1', 'InfestorBurrowed_len_player_1', 'Infestor_len_player_1', 'Lair_right_click_target_count_player_0', 'Lair_right_click_target_count_player_1', 'LocustMPFlying_len_player_0', 'LocustMPFlying_len_player_1', 'LurkerEgg_len_player_0', 'LurkerEgg_len_player_1', 'Lurker_len_player_0', 'Lurker_len_player_1', 'MULE_len_player_0', 'MULE_len_player_0_Reaper_len_player_0_diff', 'MULE_len_player_1', 'MULE_len_player_1_Hatchery_len_player_1_diff', 'MULE_len_player_1_Reaper_len_player_1_diff_MULE_len_player_0_Reaper_len_player_0_diff_diff', 'MULE_len_player_1_Starport_len_player_1_diff', 'MULE_len_player_1_Starport_len_player_1_diff_MULE_len_player_0_Starport_len_player_0_diff_diff', 'MULE_right_click_target_count_diff', 'MULE_right_click_target_count_player_0', 'MULE_right_click_target_count_player_1', 'Marine_len_player_0_BanelingCocoon_len_player_0_diff', 'Marine_len_player_0_Hatchery_len_player_0_diff', 'Marine_len_player_0_Hellion_len_player_0_diff', 'Marine_len_player_0_OrbitalCommand_len_player_0_diff', 'Marine_len_player_0_Probe_len_player_0_diff', 'Marine_len_player_0_Reaper_len_player_0_diff', 'Marine_len_player_0_Roach_len_player_0_diff', 'Marine_len_player_0_Zergling_len_player_0_diff', 'Marine_len_player_1', 'Marine_len_player_1_BanelingCocoon_len_player_1_diff', 'Marine_len_player_1_Barracks_len_player_1_diff_Marine_len_player_0_Barracks_len_player_0_diff_diff', 'Marine_len_player_1_Hatchery_len_player_1_diff', 'Marine_len_player_1_Hatchery_len_player_1_diff_Marine_len_player_0_Hatchery_len_player_0_diff_diff', 'Marine_len_player_1_Overlord_len_player_1_diff', 'Marine_len_player_1_Overlord_len_player_1_diff_Marine_len_player_0_Overlord_len_player_0_diff_diff', 'Marine_len_player_1_Queen_len_player_1_diff', 'Marine_len_player_1_Reaper_len_player_1_diff', 'Marine_len_player_1_Reaper_len_player_1_diff_Marine_len_player_0_Reaper_len_player_0_diff_diff', 'Marine_len_player_1_Roach_len_player_1_diff', 'Marine_len_player_1_Roach_len_player_1_diff_Marine_len_player_0_Roach_len_player_0_diff_diff', 'Marine_len_player_1_SupplyDepotLowered_len_player_1_diff', 'Marine_len_player_1_SupplyDepotLowered_len_player_1_diff_Marine_len_player_0_SupplyDepotLowered_len_player_0_diff_diff', 'Marine_right_click_target_count_diff', 'Marine_right_click_target_count_player_0', 'Marine_right_click_target_count_player_1', 'Medivac_len_player_0', 'Medivac_len_player_1', 'MissileTurret_len_player_0', 'MissileTurret_len_player_1', 'Mutalisk_len_player_1', 'None_len_player_0', 'None_right_click_target_count_diff', 'NydusNetwork_len_player_1', 'NydusWorm_len_player_0', 'NydusWorm_len_player_1', 'Observer_len_player_0', 'OrbitalCommand_len_player_0', 'OrbitalCommand_len_player_0_BanelingCocoon_len_player_0_diff', 'OrbitalCommand_len_player_0_Factory_len_player_0_diff', 'OrbitalCommand_len_player_0_Hatchery_len_player_0_diff', 'OrbitalCommand_len_player_0_Hellion_len_player_0_diff', 'OrbitalCommand_len_player_0_MULE_len_player_0_diff', 'OrbitalCommand_len_player_0_Overlord_len_player_0_diff', 'OrbitalCommand_len_player_0_Queen_len_player_0_diff', 'OrbitalCommand_len_player_0_Reaper_len_player_0_diff', 'OrbitalCommand_len_player_0_Roach_len_player_0_diff', 'OrbitalCommand_len_player_1', 'OrbitalCommand_len_player_1_BanelingCocoon_len_player_1_diff', 'OrbitalCommand_len_player_1_Hellion_len_player_1_diff_OrbitalCommand_len_player_0_Hellion_len_player_0_diff_diff', 'OrbitalCommand_len_player_1_MULE_len_player_1_diff', 'OrbitalCommand_len_player_1_Overlord_len_player_1_diff_OrbitalCommand_len_player_0_Overlord_len_player_0_diff_diff', 'OrbitalCommand_len_player_1_Queen_len_player_1_diff', 'OrbitalCommand_len_player_1_Queen_len_player_1_diff_OrbitalCommand_len_player_0_Queen_len_player_0_diff_diff', 'OrbitalCommand_len_player_1_Reaper_len_player_1_diff', 'OrbitalCommand_len_player_1_Reaper_len_player_1_diff_OrbitalCommand_len_player_0_Reaper_len_player_0_diff_diff', 'OrbitalCommand_len_player_1_Roach_len_player_1_diff', 'OrbitalCommand_len_player_1_Roach_len_player_1_diff_OrbitalCommand_len_player_0_Roach_len_player_0_diff_diff', 'OrbitalCommand_len_player_1_Starport_len_player_1_diff_OrbitalCommand_len_player_0_Starport_len_player_0_diff_diff', 'OrbitalCommand_len_player_1_SupplyDepotLowered_len_player_1_diff_OrbitalCommand_len_player_0_SupplyDepotLowered_len_player_0_diff_diff', 'OverlordGenerateCreepKeybind_len_player_0', 'OverlordGenerateCreepKeybind_len_player_1', 'Overlord_len_player_0', 'Overlord_len_player_0_Hatchery_len_player_0_diff', 'Overlord_len_player_0_Hellion_len_player_0_diff', 'Overlord_len_player_0_MULE_len_player_0_diff', 'Overlord_len_player_0_Reaper_len_player_0_diff', 'Overlord_len_player_1_Hatchery_len_player_1_diff', 'Overlord_len_player_1_Hellion_len_player_1_diff', 'Overlord_len_player_1_Hellion_len_player_1_diff_Overlord_len_player_0_Hellion_len_player_0_diff_diff', 'Overlord_len_player_1_MULE_len_player_1_diff', 'Overlord_len_player_1_Queen_len_player_1_diff', 'Overlord_len_player_1_Reaper_len_player_1_diff_Overlord_len_player_0_Reaper_len_player_0_diff_diff', 'Overlord_len_player_1_Starport_len_player_1_diff_Overlord_len_player_0_Starport_len_player_0_diff_diff', 'Overlord_right_click_target_count_diff', 'Overlord_right_click_target_count_player_1', 'OverseerCocoon_len_player_0', 'OverseerCocoon_len_player_1', 'Overseer_len_player_0', 'Phoenix_len_player_0', 'Phoenix_len_player_1', 'PhotonCannon_len_player_0', 'PhotonCannon_len_player_1', 'PhotonCannon_right_click_target_count_player_0', 'PhotonCannon_right_click_target_count_player_1', 'PhysicsCapsule_len_player_0', 'PhysicsCapsule_len_player_1', 'PhysicsCapsule_right_click_target_count_diff', 'PhysicsCapsule_right_click_target_count_player_0', 'PhysicsCapsule_right_click_target_count_player_1', 'Probe_len_player_0', 'Probe_len_player_0_BanelingCocoon_len_player_0_diff', 'Probe_len_player_0_Factory_len_player_0_diff', 'Probe_len_player_0_Hellion_len_player_0_diff', 'Probe_len_player_0_OrbitalCommand_len_player_0_diff', 'Probe_len_player_0_Reaper_len_player_0_diff', 'Probe_len_player_0_Roach_len_player_0_diff', 'Probe_len_player_1_BanelingCocoon_len_player_1_diff', 'Probe_len_player_1_Barracks_len_player_1_diff_Probe_len_player_0_Barracks_len_player_0_diff_diff', 'Probe_len_player_1_Factory_len_player_1_diff_Probe_len_player_0_Factory_len_player_0_diff_diff', 'Probe_len_player_1_Hatchery_len_player_1_diff', 'Probe_len_player_1_MULE_len_player_1_diff', 'Probe_len_player_1_OrbitalCommand_len_player_1_diff', 'Probe_len_player_1_OrbitalCommand_len_player_1_diff_Probe_len_player_0_OrbitalCommand_len_player_0_diff_diff', 'Probe_len_player_1_Overlord_len_player_1_diff', 'Probe_len_player_1_Queen_len_player_1_diff', 'Probe_len_player_1_Roach_len_player_1_diff_Probe_len_player_0_Roach_len_player_0_diff_diff', 'Probe_len_player_1_Starport_len_player_1_diff_Probe_len_player_0_Starport_len_player_0_diff_diff', 'Probe_len_player_1_SupplyDepotLowered_len_player_1_diff', 'Probe_len_player_1_SupplyDepotLowered_len_player_1_diff_Probe_len_player_0_SupplyDepotLowered_len_player_0_diff_diff', 'Pylon_len_player_0', 'Pylon_right_click_target_count_diff', 'Pylon_right_click_target_count_player_1', 'QueenBurrowed_len_player_0', 'QueenBurrowed_len_player_1', 'Queen_len_player_0', 'Queen_len_player_0_Hatchery_len_player_0_diff', 'Queen_len_player_0_Hellion_len_player_0_diff', 'Queen_len_player_0_MULE_len_player_0_diff', 'Queen_len_player_1', 'Queen_len_player_1_Factory_len_player_1_diff', 'Queen_len_player_1_Hatchery_len_player_1_diff', 'Queen_len_player_1_Hellion_len_player_1_diff', 'Queen_len_player_1_Hellion_len_player_1_diff_Queen_len_player_0_Hellion_len_player_0_diff_diff', 'Queen_len_player_1_MULE_len_player_1_diff', 'Queen_len_player_1_Reaper_len_player_1_diff', 'Queen_len_player_1_Roach_len_player_1_diff', 'Queen_len_player_1_Starport_len_player_1_diff', 'Queen_right_click_target_count_player_0', 'Queen_right_click_target_count_player_1', 'Ravager_len_player_0', 'Raven_len_player_0', 'Raven_len_player_1', 'Reactor_len_player_1', 'Reaper_len_player_0', 'Reaper_len_player_0_Starport_len_player_0_diff', 'Reaper_len_player_1', 'Reaper_len_player_1_Starport_len_player_1_diff', 'Reaper_right_click_target_count_player_0', 'Refinery_len_player_1', 'RichMineralField750_len_player_0', 'RichMineralField750_len_player_1', 'RichMineralField_len_player_0', 'RichMineralField_len_player_1', 'Right Click_player_0', 'Right Click_player_0_isnull', 'Right Click_player_1', 'Right Click_player_1_isnull', 'RoachBurrowed_len_player_0', 'RoachBurrowed_len_player_1', 'RoachWarren_len_player_0', 'RoachWarren_len_player_1', 'Roach_len_player_0', 'Roach_len_player_1', 'Roach_len_player_1_Starport_len_player_1_diff', 'Roach_len_player_1_Starport_len_player_1_diff_Roach_len_player_0_Starport_len_player_0_diff_diff', 'Roach_right_click_target_count_diff', 'Roach_right_click_target_count_player_1', 'RoboticsBay_len_player_0', 'Rocks2x2NonConjoined_len_player_1', 'SCV_len_player_0', 'SCV_len_player_0_BanelingCocoon_len_player_0_diff', 'SCV_len_player_0_Barracks_len_player_0_diff', 'SCV_len_player_0_Drone_len_player_0_diff', 'SCV_len_player_0_Factory_len_player_0_diff', 'SCV_len_player_0_OrbitalCommand_len_player_0_diff', 'SCV_len_player_0_Queen_len_player_0_diff', 'SCV_len_player_0_Roach_len_player_0_diff', 'SCV_len_player_0_Starport_len_player_0_diff', 'SCV_len_player_0_SupplyDepotLowered_len_player_0_diff', 'SCV_len_player_1', 'SCV_len_player_1_BanelingCocoon_len_player_1_diff', 'SCV_len_player_1_BanelingCocoon_len_player_1_diff_SCV_len_player_0_BanelingCocoon_len_player_0_diff_diff', 'SCV_len_player_1_Barracks_len_player_1_diff', 'SCV_len_player_1_Hatchery_len_player_1_diff', 'SCV_len_player_1_Hatchery_len_player_1_diff_SCV_len_player_0_Hatchery_len_player_0_diff_diff', 'SCV_len_player_1_MULE_len_player_1_diff', 'SCV_len_player_1_Marine_len_player_1_diff', 'SCV_len_player_1_Marine_len_player_1_diff_SCV_len_player_0_Marine_len_player_0_diff_diff', 'SCV_len_player_1_OrbitalCommand_len_player_1_diff', 'SCV_len_player_1_Queen_len_player_1_diff_SCV_len_player_0_Queen_len_player_0_diff_diff', 'SCV_len_player_1_Reaper_len_player_1_diff', 'SCV_len_player_1_Reaper_len_player_1_diff_SCV_len_player_0_Reaper_len_player_0_diff_diff', 'SCV_len_player_1_Roach_len_player_1_diff', 'SCV_len_player_1_Roach_len_player_1_diff_SCV_len_player_0_Roach_len_player_0_diff_diff', 'SCV_len_player_1_Starport_len_player_1_diff', 'SCV_len_player_1_SupplyDepotLowered_len_player_1_diff', 'SCV_len_player_1_Zergling_len_player_1_diff_SCV_len_player_0_Zergling_len_player_0_diff_diff', 'Selection_player_0', 'Selection_player_0_isnull', 'Selection_player_1_isnull', 'SensorTower_len_player_0', 'SensorTower_len_player_1', 'Sentry_len_player_0', 'SetControlGroup_player_0_isnull', 'SiegeTankSieged_len_player_1', 'SiegeTankSieged_right_click_target_count_diff', 'SiegeTankSieged_right_click_target_count_player_0', 'SiegeTank_len_player_0', 'SiegeTank_right_click_target_count_player_0', 'SiegeTank_right_click_target_count_player_1', 'SpawningPool_len_player_0', 'SpawningPool_right_click_target_count_player_0', 'SpineCrawlerUprooted_len_player_0', 'SpineCrawlerUprooted_len_player_1', 'SpineCrawler_len_player_0', 'SpineCrawler_len_player_1', 'Spire_len_player_0', 'Spire_len_player_1', 'SporeCrawlerUprooted_len_player_0', 'SporeCrawlerUprooted_len_player_1', 'SporeCrawler_len_player_0', 'SporeCrawler_len_player_1', 'Stalker_right_click_target_count_diff', 'Stalker_right_click_target_count_player_0', 'StarportReactor_len_player_0', 'StarportReactor_len_player_1', 'StarportTechLab_len_player_0', 'StarportTechLab_len_player_1', 'Starport_len_player_0', 'Starport_len_player_1', 'Starport_right_click_target_count_player_0', 'SupplyDepotLowered_len_player_0_BanelingCocoon_len_player_0_diff', 'SupplyDepotLowered_len_player_0_Factory_len_player_0_diff', 'SupplyDepotLowered_len_player_0_Hellion_len_player_0_diff', 'SupplyDepotLowered_len_player_0_MULE_len_player_0_diff', 'SupplyDepotLowered_len_player_0_Queen_len_player_0_diff', 'SupplyDepotLowered_len_player_0_Starport_len_player_0_diff', 'SupplyDepotLowered_len_player_1', 'SupplyDepotLowered_len_player_1_BanelingCocoon_len_player_1_diff', 'SupplyDepotLowered_len_player_1_BanelingCocoon_len_player_1_diff_SupplyDepotLowered_len_player_0_BanelingCocoon_len_player_0_diff_diff', 'SupplyDepotLowered_len_player_1_Factory_len_player_1_diff', 'SupplyDepotLowered_len_player_1_Factory_len_player_1_diff_SupplyDepotLowered_len_player_0_Factory_len_player_0_diff_diff', 'SupplyDepotLowered_len_player_1_Hatchery_len_player_1_diff', 'SupplyDepotLowered_len_player_1_Hellion_len_player_1_diff', 'SupplyDepotLowered_len_player_1_Hellion_len_player_1_diff_SupplyDepotLowered_len_player_0_Hellion_len_player_0_diff_diff', 'SupplyDepotLowered_len_player_1_Overlord_len_player_1_diff_SupplyDepotLowered_len_player_0_Overlord_len_player_0_diff_diff', 'SupplyDepotLowered_len_player_1_Queen_len_player_1_diff', 'SupplyDepotLowered_len_player_1_Reaper_len_player_1_diff_SupplyDepotLowered_len_player_0_Reaper_len_player_0_diff_diff', 'SupplyDepotLowered_len_player_1_Roach_len_player_1_diff', 'SupplyDepotLowered_len_player_1_Roach_len_player_1_diff_SupplyDepotLowered_len_player_0_Roach_len_player_0_diff_diff', 'SupplyDepotLowered_len_player_1_Starport_len_player_1_diff', 'SupplyDepot_right_click_target_count_diff', 'SupplyDepot_right_click_target_count_player_1', 'SwarmHost_len_player_1', 'TechLab_len_player_1', 'TemplarArchive_len_player_0', 'Thor_len_player_0', 'Thor_len_player_1', 'TwilightCouncil_len_player_0', 'TwilightCouncil_len_player_1', 'UltraliskCavern_len_player_1', 'VikingAssault_len_player_0', 'VikingAssault_len_player_1', 'Viking_len_player_0', 'Viking_len_player_1', 'VoidRay_len_player_0', 'WarpGate_right_click_target_count_diff', 'WarpPrismPhasing_len_player_0', 'WarpPrismPhasing_len_player_1', 'WarpPrism_len_player_0', 'WarpPrism_len_player_1', 'Weapon_len_player_0', 'Weapon_len_player_1', 'XelNagaDestructibleBlocker8W_len_player_0', 'XelNagaDestructibleBlocker8W_len_player_1', 'XelNagaTower_len_player_0', 'XelNagaTower_len_player_1', 'Zealot_len_player_0', 'Zealot_len_player_1', 'Zealot_right_click_target_count_player_0', 'ZerglingBurrowed_len_player_0', 'ZerglingBurrowed_len_player_1', 'Zergling_len_player_0_BanelingCocoon_len_player_0_diff', 'Zergling_len_player_0_Barracks_len_player_0_diff', 'Zergling_len_player_0_MULE_len_player_0_diff', 'Zergling_len_player_0_OrbitalCommand_len_player_0_diff', 'Zergling_len_player_0_Overlord_len_player_0_diff', 'Zergling_len_player_0_Queen_len_player_0_diff', 'Zergling_len_player_0_Starport_len_player_0_diff', 'Zergling_len_player_0_SupplyDepotLowered_len_player_0_diff', 'Zergling_len_player_1', 'Zergling_len_player_1_Barracks_len_player_1_diff_Zergling_len_player_0_Barracks_len_player_0_diff_diff', 'Zergling_len_player_1_Factory_len_player_1_diff', 'Zergling_len_player_1_Factory_len_player_1_diff_Zergling_len_player_0_Factory_len_player_0_diff_diff', 'Zergling_len_player_1_Hatchery_len_player_1_diff_Zergling_len_player_0_Hatchery_len_player_0_diff_diff', 'Zergling_len_player_1_Hellion_len_player_1_diff', 'Zergling_len_player_1_OrbitalCommand_len_player_1_diff', 'Zergling_len_player_1_Probe_len_player_1_diff', 'Zergling_len_player_1_Queen_len_player_1_diff_Zergling_len_player_0_Queen_len_player_0_diff_diff', 'Zergling_len_player_1_Reaper_len_player_1_diff', 'Zergling_len_player_1_Reaper_len_player_1_diff_Zergling_len_player_0_Reaper_len_player_0_diff_diff', 'Zergling_len_player_1_Roach_len_player_1_diff_Zergling_len_player_0_Roach_len_player_0_diff_diff', 'Zergling_len_player_1_Starport_len_player_1_diff', 'Zergling_len_player_1_Starport_len_player_1_diff_Zergling_len_player_0_Starport_len_player_0_diff_diff', 'Zergling_right_click_target_count_diff', 'Zergling_right_click_target_count_player_0', 'ability_code_1000_player_1', 'ability_code_1020_first_time_diff', 'ability_code_1020_player_0_ability_code_16E2_player_0_diff', 'ability_code_1020_player_0_ability_code_480_player_0_diff', 'ability_code_1020_player_1', 'ability_code_1020_player_1_ability_code_102A_player_1_diff', 'ability_code_1020_player_1_ability_code_1261_player_1_diff', 'ability_code_1020_player_1_ability_code_16E2_player_1_diff', 'ability_code_1020_player_1_ability_code_1E60_player_1_diff', 'ability_code_1020_player_1_ability_code_1E60_player_1_diff_ability_code_1020_player_0_ability_code_1E60_player_0_diff_diff', 'ability_code_1020_player_1_ability_code_480_player_1_diff', 'ability_code_1020_player_1_ability_code_480_player_1_diff_ability_code_1020_player_0_ability_code_480_player_0_diff_diff', 'ability_code_1021_diff', 'ability_code_1021_first_time_player_1', 'ability_code_1021_player_0_ability_code_1261_player_0_diff', 'ability_code_1021_player_0_ability_code_15E0_player_0_diff', 'ability_code_1021_player_0_ability_code_16E2_player_0_diff', 'ability_code_1021_player_0_ability_code_1821_player_0_diff', 'ability_code_1021_player_0_ability_code_1C60_player_0_diff', 'ability_code_1021_player_0_ability_code_1E60_player_0_diff', 'ability_code_1021_player_0_ability_code_480_player_0_diff', 'ability_code_1021_player_0_ability_code_B40_player_0_diff', 'ability_code_1021_player_0_ability_code_DE0_player_0_diff', 'ability_code_1021_player_1', 'ability_code_1021_player_1_ability_code_1020_player_1_diff', 'ability_code_1021_player_1_ability_code_1020_player_1_diff_ability_code_1021_player_0_ability_code_1020_player_0_diff_diff', 'ability_code_1021_player_1_ability_code_1022_player_1_diff', 'ability_code_1021_player_1_ability_code_1022_player_1_diff_ability_code_1021_player_0_ability_code_1022_player_0_diff_diff', 'ability_code_1021_player_1_ability_code_1023_player_1_diff_ability_code_1021_player_0_ability_code_1023_player_0_diff_diff', 'ability_code_1021_player_1_ability_code_102A_player_1_diff', 'ability_code_1021_player_1_ability_code_1261_player_1_diff', 'ability_code_1021_player_1_ability_code_1261_player_1_diff_ability_code_1021_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_1021_player_1_ability_code_13A0_player_1_diff', 'ability_code_1021_player_1_ability_code_13A0_player_1_diff_ability_code_1021_player_0_ability_code_13A0_player_0_diff_diff', 'ability_code_1021_player_1_ability_code_16E2_player_1_diff_ability_code_1021_player_0_ability_code_16E2_player_0_diff_diff', 'ability_code_1021_player_1_ability_code_1821_player_1_diff_ability_code_1021_player_0_ability_code_1821_player_0_diff_diff', 'ability_code_1021_player_1_ability_code_1822_player_1_diff_ability_code_1021_player_0_ability_code_1822_player_0_diff_diff', 'ability_code_1021_player_1_ability_code_1C60_player_1_diff_ability_code_1021_player_0_ability_code_1C60_player_0_diff_diff', 'ability_code_1021_player_1_ability_code_480_player_1_diff_ability_code_1021_player_0_ability_code_480_player_0_diff_diff', 'ability_code_1021_player_1_ability_code_B40_player_1_diff', 'ability_code_1021_player_1_ability_code_DE0_player_1_diff', 'ability_code_1022_first_time_diff', 'ability_code_1022_first_time_player_1', 'ability_code_1022_player_0_ability_code_102A_player_0_diff', 'ability_code_1022_player_0_ability_code_13A0_player_0_diff', 'ability_code_1022_player_0_ability_code_16E2_player_0_diff', 'ability_code_1022_player_0_ability_code_1821_player_0_diff', 'ability_code_1022_player_0_ability_code_1E60_player_0_diff', 'ability_code_1022_player_0_ability_code_480_player_0_diff', 'ability_code_1022_player_1', 'ability_code_1022_player_1_ability_code_1023_player_1_diff', 'ability_code_1022_player_1_ability_code_1023_player_1_diff_ability_code_1022_player_0_ability_code_1023_player_0_diff_diff', 'ability_code_1022_player_1_ability_code_102A_player_1_diff', 'ability_code_1022_player_1_ability_code_102A_player_1_diff_ability_code_1022_player_0_ability_code_102A_player_0_diff_diff', 'ability_code_1022_player_1_ability_code_1261_player_1_diff', 'ability_code_1022_player_1_ability_code_1261_player_1_diff_ability_code_1022_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_1022_player_1_ability_code_16E2_player_1_diff', 'ability_code_1022_player_1_ability_code_1822_player_1_diff', 'ability_code_1022_player_1_ability_code_1822_player_1_diff_ability_code_1022_player_0_ability_code_1822_player_0_diff_diff', 'ability_code_1022_player_1_ability_code_1E60_player_1_diff', 'ability_code_1022_player_1_ability_code_1E60_player_1_diff_ability_code_1022_player_0_ability_code_1E60_player_0_diff_diff', 'ability_code_1023_last_first_diff', 'ability_code_1023_last_first_player_1', 'ability_code_1023_player_0', 'ability_code_1023_player_0_ability_code_1020_player_0_diff', 'ability_code_1023_player_0_ability_code_102A_player_0_diff', 'ability_code_1023_player_0_ability_code_1261_player_0_diff', 'ability_code_1023_player_0_ability_code_13A0_player_0_diff', 'ability_code_1023_player_0_ability_code_15E0_player_0_diff', 'ability_code_1023_player_0_ability_code_16E2_player_0_diff', 'ability_code_1023_player_0_ability_code_1821_player_0_diff', 'ability_code_1023_player_0_ability_code_1822_player_0_diff', 'ability_code_1023_player_0_ability_code_1E60_player_0_diff', 'ability_code_1023_player_0_ability_code_480_player_0_diff', 'ability_code_1023_player_1', 'ability_code_1023_player_1_ability_code_1020_player_1_diff', 'ability_code_1023_player_1_ability_code_1020_player_1_diff_ability_code_1023_player_0_ability_code_1020_player_0_diff_diff', 'ability_code_1023_player_1_ability_code_102A_player_1_diff_ability_code_1023_player_0_ability_code_102A_player_0_diff_diff', 'ability_code_1023_player_1_ability_code_1261_player_1_diff', 'ability_code_1023_player_1_ability_code_16E2_player_1_diff', 'ability_code_1023_player_1_ability_code_16E2_player_1_diff_ability_code_1023_player_0_ability_code_16E2_player_0_diff_diff', 'ability_code_1023_player_1_ability_code_1821_player_1_diff', 'ability_code_1023_player_1_ability_code_1821_player_1_diff_ability_code_1023_player_0_ability_code_1821_player_0_diff_diff', 'ability_code_1023_player_1_ability_code_1822_player_1_diff', 'ability_code_1023_player_1_ability_code_1C60_player_1_diff_ability_code_1023_player_0_ability_code_1C60_player_0_diff_diff', 'ability_code_1023_player_1_ability_code_1E60_player_1_diff', 'ability_code_1023_player_1_ability_code_1E60_player_1_diff_ability_code_1023_player_0_ability_code_1E60_player_0_diff_diff', 'ability_code_1023_player_1_ability_code_480_player_1_diff_ability_code_1023_player_0_ability_code_480_player_0_diff_diff', 'ability_code_1024_diff', 'ability_code_1024_first_time_player_0', 'ability_code_1024_player_0', 'ability_code_1024_player_1', 'ability_code_1025_diff', 'ability_code_1025_first_time_player_0', 'ability_code_1025_first_time_player_1', 'ability_code_1025_player_0', 'ability_code_1025_player_1', 'ability_code_1026_diff', 'ability_code_1026_first_time_diff', 'ability_code_1026_player_1', 'ability_code_1028_diff', 'ability_code_1028_player_0', 'ability_code_1028_player_1', 'ability_code_1029_diff', 'ability_code_1029_player_0', 'ability_code_1029_player_1', 'ability_code_102A_diff', 'ability_code_102A_first_time_player_1', 'ability_code_102A_player_0_ability_code_1261_player_0_diff', 'ability_code_102A_player_1_ability_code_1261_player_1_diff_ability_code_102A_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_102A_player_1_ability_code_480_player_1_diff', 'ability_code_102B_first_time_diff', 'ability_code_102B_player_0', 'ability_code_102B_player_1', 'ability_code_102D_first_time_diff', 'ability_code_102D_first_time_player_0', 'ability_code_102D_first_time_player_1', 'ability_code_102D_player_0', 'ability_code_102D_player_1', 'ability_code_102F_diff', 'ability_code_102F_player_0', 'ability_code_102F_player_1', 'ability_code_103E_player_0', 'ability_code_103E_player_1', 'ability_code_1060_player_0', 'ability_code_1060_player_1', 'ability_code_1080_diff', 'ability_code_1080_player_0', 'ability_code_1080_player_1', 'ability_code_1081_diff', 'ability_code_1081_player_0', 'ability_code_1081_player_1', 'ability_code_10E0_diff', 'ability_code_10E0_first_time_diff', 'ability_code_10E0_first_time_player_0', 'ability_code_10E0_first_time_player_1', 'ability_code_10E0_player_0', 'ability_code_1100_player_0', 'ability_code_1100_player_1', 'ability_code_1120_diff', 'ability_code_1120_player_0', 'ability_code_1120_player_1', 'ability_code_1121_diff', 'ability_code_1121_player_0', 'ability_code_1121_player_1', 'ability_code_1140_diff', 'ability_code_1140_player_0', 'ability_code_1140_player_1', 'ability_code_1142_diff', 'ability_code_1142_first_time_diff', 'ability_code_1142_first_time_player_1', 'ability_code_1142_player_1', 'ability_code_1143_player_1', 'ability_code_1160_diff', 'ability_code_1160_first_time_diff', 'ability_code_1160_first_time_player_1', 'ability_code_1160_player_0', 'ability_code_1160_player_1', 'ability_code_1180_diff', 'ability_code_1180_player_0', 'ability_code_1180_player_1', 'ability_code_11A0_diff', 'ability_code_11A0_player_0', 'ability_code_11A0_player_1', 'ability_code_11C0_diff', 'ability_code_11C0_player_0', 'ability_code_11C0_player_1', 'ability_code_11E1_diff', 'ability_code_11E1_player_1', 'ability_code_11E3_player_0', 'ability_code_11E3_player_1', 'ability_code_1201_diff', 'ability_code_1201_player_0', 'ability_code_1201_player_1', 'ability_code_1203_diff', 'ability_code_1203_player_0', 'ability_code_1203_player_1', 'ability_code_1204_diff', 'ability_code_1204_player_0', 'ability_code_1204_player_1', 'ability_code_1220_diff', 'ability_code_1240_player_0', 'ability_code_1240_player_1', 'ability_code_1260_diff', 'ability_code_1260_player_0', 'ability_code_1260_player_1', 'ability_code_1261_first_time_player_1', 'ability_code_1261_player_0', 'ability_code_1261_player_0_ability_code_480_player_0_diff', 'ability_code_1261_player_1', 'ability_code_1261_player_1_ability_code_480_player_1_diff', 'ability_code_1261_player_1_ability_code_480_player_1_diff_ability_code_1261_player_0_ability_code_480_player_0_diff_diff', 'ability_code_127E_diff', 'ability_code_127E_player_0', 'ability_code_127E_player_1', 'ability_code_1280_diff', 'ability_code_1280_first_time_diff', 'ability_code_1280_player_1', 'ability_code_12A0_first_time_diff', 'ability_code_12A0_player_1', 'ability_code_12A1_first_time_diff', 'ability_code_12A1_first_time_player_1', 'ability_code_12A1_player_0', 'ability_code_12A1_player_1', 'ability_code_12BE_diff', 'ability_code_12BE_player_0', 'ability_code_12BE_player_1', 'ability_code_12C0_diff', 'ability_code_12C0_first_time_diff', 'ability_code_12C0_first_time_player_1', 'ability_code_12C0_player_0', 'ability_code_12C0_player_1', 'ability_code_12E0_player_1', 'ability_code_12E1_diff', 'ability_code_12E1_first_time_diff', 'ability_code_12E1_first_time_player_0', 'ability_code_12E1_first_time_player_1', 'ability_code_12E1_player_0', 'ability_code_12E1_player_1', 'ability_code_12FE_diff', 'ability_code_12FE_player_0', 'ability_code_12FE_player_1', 'ability_code_1300_diff', 'ability_code_1300_first_time_diff', 'ability_code_1300_first_time_player_0', 'ability_code_1300_player_1', 'ability_code_1320_diff', 'ability_code_1320_first_time_diff', 'ability_code_1320_player_0', 'ability_code_1320_player_1', 'ability_code_1340_first_time_player_0', 'ability_code_1340_player_1', 'ability_code_1360_first_time_player_1', 'ability_code_1360_last_first_player_1', 'ability_code_1360_player_0_ability_code_1020_player_0_diff', 'ability_code_1360_player_0_ability_code_1022_player_0_diff', 'ability_code_1360_player_0_ability_code_1261_player_0_diff', 'ability_code_1360_player_0_ability_code_13A0_player_0_diff', 'ability_code_1360_player_0_ability_code_13E0_player_0_diff', 'ability_code_1360_player_0_ability_code_1820_player_0_diff', 'ability_code_1360_player_0_ability_code_1822_player_0_diff', 'ability_code_1360_player_0_ability_code_1C60_player_0_diff', 'ability_code_1360_player_0_ability_code_1E60_player_0_diff', 'ability_code_1360_player_0_ability_code_480_player_0_diff', 'ability_code_1360_player_0_ability_code_DE0_player_0_diff', 'ability_code_1360_player_1_ability_code_1020_player_1_diff_ability_code_1360_player_0_ability_code_1020_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_1021_player_1_diff_ability_code_1360_player_0_ability_code_1021_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_1022_player_1_diff', 'ability_code_1360_player_1_ability_code_1022_player_1_diff_ability_code_1360_player_0_ability_code_1022_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_1023_player_1_diff_ability_code_1360_player_0_ability_code_1023_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_1261_player_1_diff_ability_code_1360_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_13A0_player_1_diff_ability_code_1360_player_0_ability_code_13A0_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_13E0_player_1_diff', 'ability_code_1360_player_1_ability_code_13E0_player_1_diff_ability_code_1360_player_0_ability_code_13E0_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_15E0_player_1_diff_ability_code_1360_player_0_ability_code_15E0_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_16E2_player_1_diff', 'ability_code_1360_player_1_ability_code_1820_player_1_diff', 'ability_code_1360_player_1_ability_code_1820_player_1_diff_ability_code_1360_player_0_ability_code_1820_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_1821_player_1_diff', 'ability_code_1360_player_1_ability_code_1822_player_1_diff', 'ability_code_1360_player_1_ability_code_1C60_player_1_diff_ability_code_1360_player_0_ability_code_1C60_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_1E60_player_1_diff', 'ability_code_1360_player_1_ability_code_1E60_player_1_diff_ability_code_1360_player_0_ability_code_1E60_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_480_player_1_diff', 'ability_code_1360_player_1_ability_code_480_player_1_diff_ability_code_1360_player_0_ability_code_480_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_B40_player_1_diff_ability_code_1360_player_0_ability_code_B40_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_DE0_player_1_diff', 'ability_code_1360_player_1_ability_code_DE0_player_1_diff_ability_code_1360_player_0_ability_code_DE0_player_0_diff_diff', 'ability_code_1380_diff', 'ability_code_1380_first_time_diff', 'ability_code_1380_player_0', 'ability_code_13A0_diff', 'ability_code_13A0_first_time_player_1', 'ability_code_13A0_player_0_ability_code_1020_player_0_diff', 'ability_code_13A0_player_0_ability_code_102A_player_0_diff', 'ability_code_13A0_player_0_ability_code_1261_player_0_diff', 'ability_code_13A0_player_0_ability_code_1821_player_0_diff', 'ability_code_13A0_player_0_ability_code_1E60_player_0_diff', 'ability_code_13A0_player_0_ability_code_480_player_0_diff', 'ability_code_13A0_player_1', 'ability_code_13A0_player_1_ability_code_1020_player_1_diff', 'ability_code_13A0_player_1_ability_code_102A_player_1_diff', 'ability_code_13A0_player_1_ability_code_102A_player_1_diff_ability_code_13A0_player_0_ability_code_102A_player_0_diff_diff', 'ability_code_13A0_player_1_ability_code_1261_player_1_diff', 'ability_code_13A0_player_1_ability_code_1261_player_1_diff_ability_code_13A0_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_13A0_player_1_ability_code_15E0_player_1_diff', 'ability_code_13A0_player_1_ability_code_16E2_player_1_diff', 'ability_code_13A0_player_1_ability_code_1821_player_1_diff', 'ability_code_13A0_player_1_ability_code_1821_player_1_diff_ability_code_13A0_player_0_ability_code_1821_player_0_diff_diff', 'ability_code_13A0_player_1_ability_code_1C60_player_1_diff', 'ability_code_13A0_player_1_ability_code_1E60_player_1_diff', 'ability_code_13A0_player_1_ability_code_480_player_1_diff', 'ability_code_13A0_player_1_ability_code_480_player_1_diff_ability_code_13A0_player_0_ability_code_480_player_0_diff_diff', 'ability_code_13C0_diff', 'ability_code_13C0_player_1', 'ability_code_13E0_first_time_player_0', 'ability_code_13E0_player_0', 'ability_code_13E0_player_0_ability_code_1022_player_0_diff', 'ability_code_13E0_player_0_ability_code_102A_player_0_diff', 'ability_code_13E0_player_0_ability_code_1261_player_0_diff', 'ability_code_13E0_player_0_ability_code_15E0_player_0_diff', 'ability_code_13E0_player_0_ability_code_1821_player_0_diff', 'ability_code_13E0_player_0_ability_code_1822_player_0_diff', 'ability_code_13E0_player_0_ability_code_1E60_player_0_diff', 'ability_code_13E0_player_0_ability_code_480_player_0_diff', 'ability_code_13E0_player_0_ability_code_B40_player_0_diff', 'ability_code_13E0_player_1_ability_code_102A_player_1_diff', 'ability_code_13E0_player_1_ability_code_1261_player_1_diff_ability_code_13E0_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_13E0_player_1_ability_code_13A0_player_1_diff', 'ability_code_13E0_player_1_ability_code_13A0_player_1_diff_ability_code_13E0_player_0_ability_code_13A0_player_0_diff_diff', 'ability_code_13E0_player_1_ability_code_15E0_player_1_diff', 'ability_code_13E0_player_1_ability_code_16E2_player_1_diff', 'ability_code_13E0_player_1_ability_code_16E2_player_1_diff_ability_code_13E0_player_0_ability_code_16E2_player_0_diff_diff', 'ability_code_13E0_player_1_ability_code_1821_player_1_diff', 'ability_code_13E0_player_1_ability_code_1821_player_1_diff_ability_code_13E0_player_0_ability_code_1821_player_0_diff_diff', 'ability_code_13E0_player_1_ability_code_1822_player_1_diff', 'ability_code_13E0_player_1_ability_code_1822_player_1_diff_ability_code_13E0_player_0_ability_code_1822_player_0_diff_diff', 'ability_code_13E0_player_1_ability_code_1C60_player_1_diff', 'ability_code_13E0_player_1_ability_code_1E60_player_1_diff', 'ability_code_13E0_player_1_ability_code_1E60_player_1_diff_ability_code_13E0_player_0_ability_code_1E60_player_0_diff_diff', 'ability_code_13E0_player_1_ability_code_480_player_1_diff', 'ability_code_13E0_player_1_ability_code_480_player_1_diff_ability_code_13E0_player_0_ability_code_480_player_0_diff_diff', 'ability_code_13E0_player_1_ability_code_DE0_player_1_diff', 'ability_code_13E1_diff', 'ability_code_13E1_first_time_diff', 'ability_code_13E1_first_time_player_1', 'ability_code_13E1_player_1', 'ability_code_13E2_diff', 'ability_code_13E2_player_0', 'ability_code_13E2_player_1', 'ability_code_13E3_first_time_diff', 'ability_code_13E3_player_0', 'ability_code_1401_player_0', 'ability_code_1401_player_1', 'ability_code_1404_player_0', 'ability_code_1404_player_1', 'ability_code_1405_diff', 'ability_code_1405_first_time_player_1', 'ability_code_1405_player_0', 'ability_code_1405_player_1', 'ability_code_1406_diff', 'ability_code_1406_player_0', 'ability_code_1406_player_1', 'ability_code_1407_first_time_player_1', 'ability_code_1407_player_1', 'ability_code_1418_first_time_diff', 'ability_code_1418_player_1', 'ability_code_1420_diff', 'ability_code_1420_first_time_diff', 'ability_code_1420_first_time_player_0', 'ability_code_1420_player_0', 'ability_code_1420_player_1', 'ability_code_1421_player_0', 'ability_code_1421_player_1', 'ability_code_1422_player_1', 'ability_code_1423_diff', 'ability_code_1423_player_1', 'ability_code_1424_diff', 'ability_code_1424_player_0', 'ability_code_1424_player_1', 'ability_code_1426_diff', 'ability_code_1426_player_1', 'ability_code_1440_diff', 'ability_code_1440_player_0', 'ability_code_1440_player_1', 'ability_code_1441_diff', 'ability_code_1441_player_0', 'ability_code_1441_player_1', 'ability_code_1442_diff', 'ability_code_1442_first_time_diff', 'ability_code_1442_first_time_player_0', 'ability_code_1442_first_time_player_1', 'ability_code_1442_player_0', 'ability_code_1442_player_1', 'ability_code_1443_diff', 'ability_code_1443_player_0', 'ability_code_1443_player_1', 'ability_code_1445_diff', 'ability_code_1445_player_0', 'ability_code_1445_player_1', 'ability_code_1446_diff', 'ability_code_1446_first_time_diff', 'ability_code_1446_first_time_player_0', 'ability_code_1446_first_time_player_1', 'ability_code_1446_player_0', 'ability_code_1447_diff', 'ability_code_1447_player_0', 'ability_code_1447_player_1', 'ability_code_1480_diff', 'ability_code_1480_player_0', 'ability_code_1480_player_1', 'ability_code_14A0_diff', 'ability_code_14A0_first_time_diff', 'ability_code_14A0_first_time_player_1', 'ability_code_14A0_player_0', 'ability_code_14A0_player_1', 'ability_code_14A1_first_time_diff', 'ability_code_14A1_first_time_player_0', 'ability_code_14A1_player_0', 'ability_code_14A1_player_1', 'ability_code_14A2_diff', 'ability_code_14A2_player_0', 'ability_code_14A2_player_1', 'ability_code_14C1_diff', 'ability_code_14C1_player_0', 'ability_code_14C1_player_1', 'ability_code_14C4_diff', 'ability_code_14C4_player_0', 'ability_code_14C4_player_1', 'ability_code_14E0_diff', 'ability_code_14E0_player_0', 'ability_code_14E0_player_1', 'ability_code_14E3_diff', 'ability_code_14E3_player_0', 'ability_code_14E3_player_1', 'ability_code_14E9_diff', 'ability_code_14E9_player_0', 'ability_code_14E9_player_1', 'ability_code_14ED_diff', 'ability_code_14ED_player_0', 'ability_code_14ED_player_1', 'ability_code_14EE_diff', 'ability_code_14EE_player_0', 'ability_code_14EE_player_1', 'ability_code_14EF_diff', 'ability_code_14EF_player_0', 'ability_code_14EF_player_1', 'ability_code_1500_diff', 'ability_code_1500_player_0', 'ability_code_1500_player_1', 'ability_code_1525_diff', 'ability_code_1525_player_0', 'ability_code_1525_player_1', 'ability_code_1526_diff', 'ability_code_1526_player_0', 'ability_code_1526_player_1', 'ability_code_152B_diff', 'ability_code_152B_player_0', 'ability_code_152B_player_1', 'ability_code_152E_player_1', 'ability_code_152F_diff', 'ability_code_152F_player_0', 'ability_code_152F_player_1', 'ability_code_1541_first_time_diff', 'ability_code_1541_player_0', 'ability_code_1542_first_time_diff', 'ability_code_1542_first_time_player_0', 'ability_code_1542_first_time_player_1', 'ability_code_1543_player_0', 'ability_code_1544_first_time_diff', 'ability_code_1544_first_time_player_0', 'ability_code_1544_first_time_player_1', 'ability_code_1544_player_0', 'ability_code_1545_player_0', 'ability_code_1546_first_time_player_0', 'ability_code_1547_diff', 'ability_code_1547_first_time_diff', 'ability_code_1547_first_time_player_1', 'ability_code_1547_player_1', 'ability_code_1549_first_time_player_0', 'ability_code_1549_player_0', 'ability_code_1549_player_1', 'ability_code_154A_player_0', 'ability_code_154A_player_1', 'ability_code_154B_player_0', 'ability_code_154C_player_0', 'ability_code_154D_player_0', 'ability_code_154D_player_1', 'ability_code_154E_diff', 'ability_code_154E_player_0', 'ability_code_154E_player_1', 'ability_code_1562_diff', 'ability_code_1562_player_0', 'ability_code_1563_diff', 'ability_code_1563_player_0', 'ability_code_1563_player_1', 'ability_code_1580_player_0', 'ability_code_1581_first_time_diff', 'ability_code_1581_player_0', 'ability_code_1583_diff', 'ability_code_1583_player_1', 'ability_code_1584_diff', 'ability_code_1584_player_0', 'ability_code_1584_player_1', 'ability_code_1585_diff', 'ability_code_1585_player_0', 'ability_code_1585_player_1', 'ability_code_1586_first_time_diff', 'ability_code_1586_player_1', 'ability_code_15A0_player_0', 'ability_code_15A0_player_1', 'ability_code_15A2_player_0', 'ability_code_15A4_diff', 'ability_code_15A4_player_0', 'ability_code_15A4_player_1', 'ability_code_15A8_diff', 'ability_code_15A8_first_time_diff', 'ability_code_15A8_first_time_player_0', 'ability_code_15A8_player_0', 'ability_code_15A8_player_1', 'ability_code_15A9_diff', 'ability_code_15A9_player_0', 'ability_code_15A9_player_1', 'ability_code_15C0_diff', 'ability_code_15C0_player_0', 'ability_code_15C1_diff', 'ability_code_15C1_first_time_player_0', 'ability_code_15C1_player_0', 'ability_code_15C2_player_0', 'ability_code_15C2_player_1', 'ability_code_15C3_diff', 'ability_code_15C3_first_time_diff', 'ability_code_15C3_first_time_player_0', 'ability_code_15D2_diff', 'ability_code_15D2_player_0', 'ability_code_15D2_player_1', 'ability_code_15E0_first_time_player_0', 'ability_code_15E0_player_0', 'ability_code_15E0_player_0_ability_code_1020_player_0_diff', 'ability_code_15E0_player_0_ability_code_102A_player_0_diff', 'ability_code_15E0_player_0_ability_code_1261_player_0_diff', 'ability_code_15E0_player_0_ability_code_16E2_player_0_diff', 'ability_code_15E0_player_0_ability_code_1E60_player_0_diff', 'ability_code_15E0_player_0_ability_code_480_player_0_diff', 'ability_code_15E0_player_1', 'ability_code_15E0_player_1_ability_code_1020_player_1_diff', 'ability_code_15E0_player_1_ability_code_102A_player_1_diff', 'ability_code_15E0_player_1_ability_code_1261_player_1_diff', 'ability_code_15E0_player_1_ability_code_16E2_player_1_diff_ability_code_15E0_player_0_ability_code_16E2_player_0_diff_diff', 'ability_code_15E0_player_1_ability_code_1C60_player_1_diff', 'ability_code_15E0_player_1_ability_code_1E60_player_1_diff', 'ability_code_1600_diff', 'ability_code_1600_player_1', 'ability_code_1680_diff', 'ability_code_1680_player_1', 'ability_code_1681_diff', 'ability_code_1681_player_0', 'ability_code_1681_player_1', 'ability_code_1683_player_0', 'ability_code_1684_diff', 'ability_code_1684_player_0', 'ability_code_1684_player_1', 'ability_code_1686_diff', 'ability_code_1686_player_0', 'ability_code_1686_player_1', 'ability_code_1687_diff', 'ability_code_1687_player_0', 'ability_code_1687_player_1', 'ability_code_16A1_diff', 'ability_code_16A1_player_0', 'ability_code_16A1_player_1', 'ability_code_16A2_diff', 'ability_code_16A2_player_0', 'ability_code_16A2_player_1', 'ability_code_16A5_diff', 'ability_code_16A5_player_0', 'ability_code_16A5_player_1', 'ability_code_16C4_diff', 'ability_code_16C4_player_0', 'ability_code_16C4_player_1', 'ability_code_16E0_player_1', 'ability_code_16E2_player_0', 'ability_code_16E2_player_0_ability_code_1261_player_0_diff', 'ability_code_16E2_player_1', 'ability_code_16E2_player_1_ability_code_102A_player_1_diff', 'ability_code_16E2_player_1_ability_code_102A_player_1_diff_ability_code_16E2_player_0_ability_code_102A_player_0_diff_diff', 'ability_code_16E2_player_1_ability_code_1261_player_1_diff', 'ability_code_16E2_player_1_ability_code_1261_player_1_diff_ability_code_16E2_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_16E2_player_1_ability_code_480_player_1_diff', 'ability_code_16E2_player_1_ability_code_480_player_1_diff_ability_code_16E2_player_0_ability_code_480_player_0_diff_diff', 'ability_code_16E3_diff', 'ability_code_16E3_first_time_diff', 'ability_code_16E3_player_0', 'ability_code_16E3_player_1', 'ability_code_16E4_diff', 'ability_code_16E4_first_time_diff', 'ability_code_16E4_first_time_player_0', 'ability_code_16E4_first_time_player_1', 'ability_code_16E4_player_1', 'ability_code_16E5_diff', 'ability_code_16E5_player_0', 'ability_code_16E5_player_1', 'ability_code_16E6_player_1', 'ability_code_16E7_diff', 'ability_code_16E7_player_0', 'ability_code_16E7_player_1', 'ability_code_16E8_player_0', 'ability_code_16E9_diff', 'ability_code_16E9_player_0', 'ability_code_16E9_player_1', 'ability_code_16EA_diff', 'ability_code_16EA_first_time_diff', 'ability_code_16EA_player_0', 'ability_code_16ED_diff', 'ability_code_16ED_player_1', 'ability_code_16EE_first_time_diff', 'ability_code_16EE_first_time_player_0', 'ability_code_16EE_first_time_player_1', 'ability_code_16EE_player_0', 'ability_code_16EE_player_1', 'ability_code_16EF_first_time_diff', 'ability_code_16EF_first_time_player_0', 'ability_code_16EF_player_0', 'ability_code_16EF_player_1', 'ability_code_1700_diff', 'ability_code_1700_player_0', 'ability_code_1700_player_1', 'ability_code_1720_player_0', 'ability_code_1721_diff', 'ability_code_1721_player_0', 'ability_code_1721_player_1', 'ability_code_1723_first_time_diff', 'ability_code_1723_first_time_player_0', 'ability_code_1723_player_0', 'ability_code_1724_player_0', 'ability_code_1726_player_1', 'ability_code_1727_diff', 'ability_code_1727_player_0', 'ability_code_1727_player_1', 'ability_code_1740_first_time_diff', 'ability_code_1740_player_1', 'ability_code_1741_diff', 'ability_code_1741_player_0', 'ability_code_1741_player_1', 'ability_code_1760_player_0', 'ability_code_1760_player_1', 'ability_code_17A1_diff', 'ability_code_17A1_player_0', 'ability_code_17A1_player_1', 'ability_code_17A3_player_1', 'ability_code_17C1_player_0', 'ability_code_17C1_player_1', 'ability_code_17E1_player_0', 'ability_code_1800_diff', 'ability_code_1800_player_0', 'ability_code_1800_player_1', 'ability_code_1803_diff', 'ability_code_1803_player_0', 'ability_code_1803_player_1', 'ability_code_1820_first_time_diff', 'ability_code_1820_last_first_player_1', 'ability_code_1820_player_0', 'ability_code_1820_player_0_ability_code_1020_player_0_diff', 'ability_code_1820_player_0_ability_code_1021_player_0_diff', 'ability_code_1820_player_0_ability_code_1022_player_0_diff', 'ability_code_1820_player_0_ability_code_1023_player_0_diff', 'ability_code_1820_player_0_ability_code_102A_player_0_diff', 'ability_code_1820_player_0_ability_code_1261_player_0_diff', 'ability_code_1820_player_0_ability_code_13A0_player_0_diff', 'ability_code_1820_player_0_ability_code_13E0_player_0_diff', 'ability_code_1820_player_0_ability_code_15E0_player_0_diff', 'ability_code_1820_player_0_ability_code_1822_player_0_diff', 'ability_code_1820_player_0_ability_code_1E60_player_0_diff', 'ability_code_1820_player_0_ability_code_480_player_0_diff', 'ability_code_1820_player_0_ability_code_B40_player_0_diff', 'ability_code_1820_player_1_ability_code_1021_player_1_diff', 'ability_code_1820_player_1_ability_code_1021_player_1_diff_ability_code_1820_player_0_ability_code_1021_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_1022_player_1_diff', 'ability_code_1820_player_1_ability_code_1022_player_1_diff_ability_code_1820_player_0_ability_code_1022_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_1023_player_1_diff', 'ability_code_1820_player_1_ability_code_102A_player_1_diff', 'ability_code_1820_player_1_ability_code_102A_player_1_diff_ability_code_1820_player_0_ability_code_102A_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_1261_player_1_diff', 'ability_code_1820_player_1_ability_code_1261_player_1_diff_ability_code_1820_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_13A0_player_1_diff', 'ability_code_1820_player_1_ability_code_13E0_player_1_diff', 'ability_code_1820_player_1_ability_code_15E0_player_1_diff', 'ability_code_1820_player_1_ability_code_15E0_player_1_diff_ability_code_1820_player_0_ability_code_15E0_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_16E2_player_1_diff', 'ability_code_1820_player_1_ability_code_16E2_player_1_diff_ability_code_1820_player_0_ability_code_16E2_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_1822_player_1_diff_ability_code_1820_player_0_ability_code_1822_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_1C60_player_1_diff', 'ability_code_1820_player_1_ability_code_1E60_player_1_diff', 'ability_code_1820_player_1_ability_code_480_player_1_diff', 'ability_code_1820_player_1_ability_code_480_player_1_diff_ability_code_1820_player_0_ability_code_480_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_B40_player_1_diff', 'ability_code_1820_player_1_ability_code_B40_player_1_diff_ability_code_1820_player_0_ability_code_B40_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_DE0_player_1_diff', 'ability_code_1820_player_1_ability_code_DE0_player_1_diff_ability_code_1820_player_0_ability_code_DE0_player_0_diff_diff', 'ability_code_1821_diff', 'ability_code_1821_first_time_player_0', 'ability_code_1821_first_time_player_1', 'ability_code_1821_player_0', 'ability_code_1821_player_0_ability_code_102A_player_0_diff', 'ability_code_1821_player_0_ability_code_1261_player_0_diff', 'ability_code_1821_player_0_ability_code_15E0_player_0_diff', 'ability_code_1821_player_0_ability_code_16E2_player_0_diff', 'ability_code_1821_player_0_ability_code_1E60_player_0_diff', 'ability_code_1821_player_1', 'ability_code_1821_player_1_ability_code_1020_player_1_diff', 'ability_code_1821_player_1_ability_code_102A_player_1_diff', 'ability_code_1821_player_1_ability_code_102A_player_1_diff_ability_code_1821_player_0_ability_code_102A_player_0_diff_diff', 'ability_code_1821_player_1_ability_code_1261_player_1_diff', 'ability_code_1821_player_1_ability_code_1261_player_1_diff_ability_code_1821_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_1821_player_1_ability_code_15E0_player_1_diff', 'ability_code_1821_player_1_ability_code_16E2_player_1_diff', 'ability_code_1821_player_1_ability_code_16E2_player_1_diff_ability_code_1821_player_0_ability_code_16E2_player_0_diff_diff', 'ability_code_1821_player_1_ability_code_1C60_player_1_diff', 'ability_code_1821_player_1_ability_code_1C60_player_1_diff_ability_code_1821_player_0_ability_code_1C60_player_0_diff_diff', 'ability_code_1821_player_1_ability_code_1E60_player_1_diff', 'ability_code_1821_player_1_ability_code_1E60_player_1_diff_ability_code_1821_player_0_ability_code_1E60_player_0_diff_diff', 'ability_code_1821_player_1_ability_code_480_player_1_diff', 'ability_code_1821_player_1_ability_code_480_player_1_diff_ability_code_1821_player_0_ability_code_480_player_0_diff_diff', 'ability_code_1822_first_time_player_0', 'ability_code_1822_first_time_player_1', 'ability_code_1822_last_first_player_0', 'ability_code_1822_player_0', 'ability_code_1822_player_0_ability_code_1020_player_0_diff', 'ability_code_1822_player_0_ability_code_102A_player_0_diff', 'ability_code_1822_player_0_ability_code_1261_player_0_diff', 'ability_code_1822_player_0_ability_code_13A0_player_0_diff', 'ability_code_1822_player_0_ability_code_15E0_player_0_diff', 'ability_code_1822_player_0_ability_code_16E2_player_0_diff', 'ability_code_1822_player_0_ability_code_1C60_player_0_diff', 'ability_code_1822_player_0_ability_code_480_player_0_diff', 'ability_code_1822_player_1', 'ability_code_1822_player_1_ability_code_1020_player_1_diff', 'ability_code_1822_player_1_ability_code_102A_player_1_diff', 'ability_code_1822_player_1_ability_code_102A_player_1_diff_ability_code_1822_player_0_ability_code_102A_player_0_diff_diff', 'ability_code_1822_player_1_ability_code_1261_player_1_diff', 'ability_code_1822_player_1_ability_code_1261_player_1_diff_ability_code_1822_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_1822_player_1_ability_code_13A0_player_1_diff', 'ability_code_1822_player_1_ability_code_15E0_player_1_diff', 'ability_code_1822_player_1_ability_code_16E2_player_1_diff', 'ability_code_1822_player_1_ability_code_16E2_player_1_diff_ability_code_1822_player_0_ability_code_16E2_player_0_diff_diff', 'ability_code_1822_player_1_ability_code_1821_player_1_diff_ability_code_1822_player_0_ability_code_1821_player_0_diff_diff', 'ability_code_1822_player_1_ability_code_1C60_player_1_diff', 'ability_code_1822_player_1_ability_code_480_player_1_diff_ability_code_1822_player_0_ability_code_480_player_0_diff_diff', 'ability_code_1823_diff', 'ability_code_1823_player_0', 'ability_code_1823_player_1', 'ability_code_1824_diff', 'ability_code_1824_player_1', 'ability_code_1829_first_time_diff', 'ability_code_1829_player_0', 'ability_code_182A_diff', 'ability_code_182A_player_0', 'ability_code_182A_player_1', 'ability_code_182B_diff', 'ability_code_182B_player_0', 'ability_code_182B_player_1', 'ability_code_182E_diff', 'ability_code_182E_player_0', 'ability_code_182E_player_1', 'ability_code_1860_diff', 'ability_code_1860_player_0', 'ability_code_1860_player_1', 'ability_code_1880_diff', 'ability_code_1880_player_0', 'ability_code_1880_player_1', 'ability_code_18A0_diff', 'ability_code_18A0_player_0', 'ability_code_18A0_player_1', 'ability_code_18C0_diff', 'ability_code_18C0_player_0', 'ability_code_18C0_player_1', 'ability_code_1920_diff', 'ability_code_1920_player_0', 'ability_code_1920_player_1', 'ability_code_1940_diff', 'ability_code_1940_player_0', 'ability_code_1940_player_1', 'ability_code_1960_diff', 'ability_code_1960_player_0', 'ability_code_1960_player_1', 'ability_code_1980_diff', 'ability_code_1980_player_0', 'ability_code_1980_player_1', 'ability_code_1A62_diff', 'ability_code_1A62_player_0', 'ability_code_1A62_player_1', 'ability_code_1A63_diff', 'ability_code_1A63_player_0', 'ability_code_1A63_player_1', 'ability_code_1AC3_diff', 'ability_code_1AC3_player_0', 'ability_code_1AC3_player_1', 'ability_code_1AC4_player_0', 'ability_code_1AC4_player_1', 'ability_code_1AC5_diff', 'ability_code_1AC5_player_0', 'ability_code_1AC6_diff', 'ability_code_1AC6_first_time_diff', 'ability_code_1AC6_first_time_player_0', 'ability_code_1AC6_first_time_player_1', 'ability_code_1AC6_player_0', 'ability_code_1AC6_player_1', 'ability_code_1AE0_diff', 'ability_code_1AE0_player_0', 'ability_code_1AE0_player_1', 'ability_code_1B00_diff', 'ability_code_1B00_player_0', 'ability_code_1B00_player_1', 'ability_code_1B21_diff', 'ability_code_1B21_player_0', 'ability_code_1B21_player_1', 'ability_code_1B40_diff', 'ability_code_1B40_player_0', 'ability_code_1B40_player_1', 'ability_code_1B60_diff', 'ability_code_1B60_player_1', 'ability_code_1BA0_diff', 'ability_code_1BA0_player_1', 'ability_code_1BA1_diff', 'ability_code_1BA1_player_0', 'ability_code_1BA1_player_1', 'ability_code_1BC0_diff', 'ability_code_1BC0_player_0', 'ability_code_1BC0_player_1', 'ability_code_1BC1_diff', 'ability_code_1BC1_player_0', 'ability_code_1BC1_player_1', 'ability_code_1BE2_diff', 'ability_code_1BE2_player_0', 'ability_code_1BE2_player_1', 'ability_code_1C00_diff', 'ability_code_1C00_player_0', 'ability_code_1C00_player_1', 'ability_code_1C60_first_time_player_0', 'ability_code_1C60_first_time_player_1', 'ability_code_1C60_last_first_diff', 'ability_code_1C60_last_first_player_0', 'ability_code_1C60_last_first_player_1', 'ability_code_1C60_player_0', 'ability_code_1C60_player_0_ability_code_16E2_player_0_diff', 'ability_code_1C60_player_0_ability_code_1E60_player_0_diff', 'ability_code_1C60_player_0_ability_code_480_player_0_diff', 'ability_code_1C60_player_1', 'ability_code_1C60_player_1_ability_code_1020_player_1_diff', 'ability_code_1C60_player_1_ability_code_1261_player_1_diff_ability_code_1C60_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_1C60_player_1_ability_code_1E60_player_1_diff', 'ability_code_1C60_player_1_ability_code_1E60_player_1_diff_ability_code_1C60_player_0_ability_code_1E60_player_0_diff_diff', 'ability_code_1C60_player_1_ability_code_480_player_1_diff_ability_code_1C60_player_0_ability_code_480_player_0_diff_diff', 'ability_code_1C61_diff', 'ability_code_1C61_player_0', 'ability_code_1C61_player_1', 'ability_code_1C80_diff', 'ability_code_1C80_first_time_diff', 'ability_code_1C80_first_time_player_0', 'ability_code_1C80_first_time_player_1', 'ability_code_1C80_player_0', 'ability_code_1CA0_diff', 'ability_code_1CA0_player_0', 'ability_code_1CA0_player_1', 'ability_code_1CC0_diff', 'ability_code_1CC0_player_0', 'ability_code_1CC0_player_1', 'ability_code_1CE0_diff', 'ability_code_1CE0_player_0', 'ability_code_1CE0_player_1', 'ability_code_1D00_diff', 'ability_code_1D00_player_0', 'ability_code_1D00_player_1', 'ability_code_1D20_player_0', 'ability_code_1D40_diff', 'ability_code_1D40_player_0', 'ability_code_1D40_player_1', 'ability_code_1D60_diff', 'ability_code_1D60_player_0', 'ability_code_1D60_player_1', 'ability_code_1D80_player_0', 'ability_code_1D80_player_1', 'ability_code_1D81_diff', 'ability_code_1D81_player_0', 'ability_code_1D81_player_1', 'ability_code_1D83_diff', 'ability_code_1D83_player_0', 'ability_code_1D83_player_1', 'ability_code_1D86_diff', 'ability_code_1D86_first_time_player_0', 'ability_code_1D86_first_time_player_1', 'ability_code_1D86_player_1', 'ability_code_1DA1_diff', 'ability_code_1DA1_player_1', 'ability_code_1DA2_player_0', 'ability_code_1DA2_player_1', 'ability_code_1DC0_diff', 'ability_code_1DC0_player_0', 'ability_code_1DC0_player_1', 'ability_code_1E20_diff', 'ability_code_1E20_player_0', 'ability_code_1E60_first_time_diff', 'ability_code_1E60_first_time_player_0', 'ability_code_1E60_first_time_player_1', 'ability_code_1E60_player_0', 'ability_code_1E60_player_0_ability_code_102A_player_0_diff', 'ability_code_1E60_player_0_ability_code_1261_player_0_diff', 'ability_code_1E60_player_0_ability_code_16E2_player_0_diff', 'ability_code_1E60_player_0_ability_code_480_player_0_diff', 'ability_code_1E60_player_1', 'ability_code_1E60_player_1_ability_code_102A_player_1_diff', 'ability_code_1E60_player_1_ability_code_102A_player_1_diff_ability_code_1E60_player_0_ability_code_102A_player_0_diff_diff', 'ability_code_1E60_player_1_ability_code_1261_player_1_diff', 'ability_code_1E60_player_1_ability_code_480_player_1_diff', 'ability_code_1EA0_diff', 'ability_code_1EA0_player_0', 'ability_code_1EA0_player_1', 'ability_code_1FC0_diff', 'ability_code_1FC0_player_0', 'ability_code_1FC0_player_1', 'ability_code_1FE0_diff', 'ability_code_1FE0_player_0', 'ability_code_1FE0_player_1', 'ability_code_2040_diff', 'ability_code_2040_player_0', 'ability_code_2040_player_1', 'ability_code_2060_diff', 'ability_code_2060_player_0', 'ability_code_2060_player_1', 'ability_code_2061_diff', 'ability_code_2061_player_0', 'ability_code_2061_player_1', 'ability_code_2080_first_time_diff', 'ability_code_2080_first_time_player_1', 'ability_code_2080_player_1', 'ability_code_20A0_player_0', 'ability_code_20C0_diff', 'ability_code_20C0_player_0', 'ability_code_20C0_player_1', 'ability_code_20E0_diff', 'ability_code_20E0_player_0', 'ability_code_20E1_diff', 'ability_code_20E1_player_0', 'ability_code_20E1_player_1', 'ability_code_2100_diff', 'ability_code_2100_player_0', 'ability_code_2100_player_1', 'ability_code_2101_diff', 'ability_code_2101_player_0', 'ability_code_2101_player_1', 'ability_code_2120_diff', 'ability_code_2120_first_time_diff', 'ability_code_2120_player_0', 'ability_code_213E_diff', 'ability_code_213E_player_0', 'ability_code_213E_player_1', 'ability_code_2140_diff', 'ability_code_2140_player_0', 'ability_code_2140_player_1', 'ability_code_2160_diff', 'ability_code_2160_player_0', 'ability_code_2160_player_1', 'ability_code_2180_diff', 'ability_code_2180_player_0', 'ability_code_2180_player_1', 'ability_code_22A0_diff', 'ability_code_22A0_player_0', 'ability_code_22A1_diff', 'ability_code_22A1_player_0', 'ability_code_22A1_player_1', 'ability_code_26C0_diff', 'ability_code_26C0_player_0', 'ability_code_26C0_player_1', 'ability_code_2720_first_time_diff', 'ability_code_2720_first_time_player_0', 'ability_code_2720_player_0', 'ability_code_2720_player_1', 'ability_code_2B00_player_0', 'ability_code_2B40_diff', 'ability_code_2B40_player_0', 'ability_code_2B40_player_1', 'ability_code_2C80_diff', 'ability_code_2C80_player_0', 'ability_code_2C80_player_1', 'ability_code_2DE0_diff', 'ability_code_2DE0_player_0', 'ability_code_2DE0_player_1', 'ability_code_2F60_diff', 'ability_code_2F60_player_0', 'ability_code_2F60_player_1', 'ability_code_3040_diff', 'ability_code_3040_first_time_diff', 'ability_code_3040_first_time_player_0', 'ability_code_3040_first_time_player_1', 'ability_code_3040_player_0', 'ability_code_3040_player_1', 'ability_code_3060_diff', 'ability_code_3060_player_0', 'ability_code_3060_player_1', 'ability_code_3120_diff', 'ability_code_3120_player_0', 'ability_code_3120_player_1', 'ability_code_3140_diff', 'ability_code_3140_player_0', 'ability_code_3140_player_1', 'ability_code_3160_diff', 'ability_code_3160_player_0', 'ability_code_3160_player_1', 'ability_code_31A0_diff', 'ability_code_31A0_first_time_player_0', 'ability_code_31A0_first_time_player_1', 'ability_code_31A0_player_0', 'ability_code_31A0_player_1', 'ability_code_3380_diff', 'ability_code_3380_player_0', 'ability_code_3380_player_1', 'ability_code_3480_first_time_diff', 'ability_code_3480_first_time_player_0', 'ability_code_3480_player_0', 'ability_code_3480_player_1', 'ability_code_39A0_diff', 'ability_code_39A0_player_0', 'ability_code_39A0_player_1', 'ability_code_4020_player_0', 'ability_code_4021_diff', 'ability_code_4021_player_0', 'ability_code_4021_player_1', 'ability_code_4040_diff', 'ability_code_4040_player_0', 'ability_code_4040_player_1', 'ability_code_40A0_player_0', 'ability_code_4120_diff', 'ability_code_4120_player_0', 'ability_code_4120_player_1', 'ability_code_4160_diff', 'ability_code_4160_player_1', 'ability_code_41A0_diff', 'ability_code_41A0_player_0', 'ability_code_41A0_player_1', 'ability_code_41E0_diff', 'ability_code_41E0_player_0', 'ability_code_41E0_player_1', 'ability_code_4220_diff', 'ability_code_4220_player_0', 'ability_code_4220_player_1', 'ability_code_42A1_diff', 'ability_code_42A1_player_0', 'ability_code_42A1_player_1', 'ability_code_42C0_diff', 'ability_code_42C0_player_0', 'ability_code_42C1_diff', 'ability_code_42C1_player_0', 'ability_code_42C1_player_1', 'ability_code_43E0_diff', 'ability_code_43E0_player_0', 'ability_code_43E0_player_1', 'ability_code_480_diff', 'ability_code_480_first_time_player_0', 'ability_code_480_first_time_player_1', 'ability_code_480_player_0', 'ability_code_480_player_1', 'ability_code_483_diff', 'ability_code_483_player_0', 'ability_code_483_player_1', 'ability_code_4AE0_diff', 'ability_code_4AE0_player_0', 'ability_code_4AE0_player_1', 'ability_code_4AFE_diff', 'ability_code_4AFE_player_0', 'ability_code_4AFE_player_1', 'ability_code_4B80_diff', 'ability_code_4B80_first_time_diff', 'ability_code_4B80_first_time_player_0', 'ability_code_4B80_player_0', 'ability_code_4B80_player_1', 'ability_code_4C0_diff', 'ability_code_4C0_player_0', 'ability_code_4C0_player_1', 'ability_code_4C1_diff', 'ability_code_4C1_first_time_player_0', 'ability_code_4C1_player_0', 'ability_code_4C2_first_time_player_1', 'ability_code_4C2_player_1', 'ability_code_4C3_diff', 'ability_code_4C3_player_1', 'ability_code_4C60_player_1', 'ability_code_4C80_player_0', 'ability_code_4C80_player_1', 'ability_code_4E40_diff', 'ability_code_4E40_first_time_player_1', 'ability_code_4E40_player_1', 'ability_code_4EA0_diff', 'ability_code_4EA0_player_0', 'ability_code_4EA0_player_1', 'ability_code_4EC0_diff', 'ability_code_4EC0_player_0', 'ability_code_4EC0_player_1', 'ability_code_55E0_diff', 'ability_code_55E0_player_0', 'ability_code_55E0_player_1', 'ability_code_5600_diff', 'ability_code_5600_player_0', 'ability_code_5600_player_1', 'ability_code_5640_diff', 'ability_code_5640_player_0', 'ability_code_5640_player_1', 'ability_code_5641_diff', 'ability_code_5641_player_0', 'ability_code_5641_player_1', 'ability_code_56A0_diff', 'ability_code_56A0_player_0', 'ability_code_56A0_player_1', 'ability_code_5700_diff', 'ability_code_5700_player_0', 'ability_code_5700_player_1', 'ability_code_5A0_first_time_diff', 'ability_code_5A0_player_0', 'ability_code_5A0_player_0_ability_code_1021_player_0_diff', 'ability_code_5A0_player_0_ability_code_1022_player_0_diff', 'ability_code_5A0_player_0_ability_code_102A_player_0_diff', 'ability_code_5A0_player_0_ability_code_13A0_player_0_diff', 'ability_code_5A0_player_0_ability_code_13E0_player_0_diff', 'ability_code_5A0_player_0_ability_code_15E0_player_0_diff', 'ability_code_5A0_player_0_ability_code_1820_player_0_diff', 'ability_code_5A0_player_0_ability_code_1821_player_0_diff', 'ability_code_5A0_player_1', 'ability_code_5A0_player_1_ability_code_1020_player_1_diff', 'ability_code_5A0_player_1_ability_code_1020_player_1_diff_ability_code_5A0_player_0_ability_code_1020_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_1021_player_1_diff', 'ability_code_5A0_player_1_ability_code_1021_player_1_diff_ability_code_5A0_player_0_ability_code_1021_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_1022_player_1_diff', 'ability_code_5A0_player_1_ability_code_1023_player_1_diff', 'ability_code_5A0_player_1_ability_code_1023_player_1_diff_ability_code_5A0_player_0_ability_code_1023_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_1261_player_1_diff', 'ability_code_5A0_player_1_ability_code_1261_player_1_diff_ability_code_5A0_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_1360_player_1_diff', 'ability_code_5A0_player_1_ability_code_1360_player_1_diff_ability_code_5A0_player_0_ability_code_1360_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_16E2_player_1_diff', 'ability_code_5A0_player_1_ability_code_16E2_player_1_diff_ability_code_5A0_player_0_ability_code_16E2_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_1820_player_1_diff_ability_code_5A0_player_0_ability_code_1820_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_1821_player_1_diff', 'ability_code_5A0_player_1_ability_code_1821_player_1_diff_ability_code_5A0_player_0_ability_code_1821_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_1822_player_1_diff', 'ability_code_5A0_player_1_ability_code_1822_player_1_diff_ability_code_5A0_player_0_ability_code_1822_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_1E60_player_1_diff_ability_code_5A0_player_0_ability_code_1E60_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_480_player_1_diff_ability_code_5A0_player_0_ability_code_480_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_B40_player_1_diff_ability_code_5A0_player_0_ability_code_B40_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_DE0_player_1_diff_ability_code_5A0_player_0_ability_code_DE0_player_0_diff_diff', 'ability_code_780_diff', 'ability_code_780_player_0', 'ability_code_780_player_1', 'ability_code_7A0_diff', 'ability_code_7A0_player_0', 'ability_code_7A0_player_1', 'ability_code_7C0_diff', 'ability_code_7C0_player_0', 'ability_code_7C0_player_1', 'ability_code_7E0_diff', 'ability_code_7E0_player_0', 'ability_code_7E0_player_1', 'ability_code_820_diff', 'ability_code_820_player_0', 'ability_code_820_player_1', 'ability_code_880_diff', 'ability_code_880_player_0', 'ability_code_880_player_1', 'ability_code_8A0_player_0', 'ability_code_8A2_diff', 'ability_code_8A2_player_0', 'ability_code_8A2_player_1', 'ability_code_8E0_diff', 'ability_code_8E0_player_0', 'ability_code_8E0_player_1', 'ability_code_900_diff', 'ability_code_900_player_0', 'ability_code_900_player_1', 'ability_code_920_first_time_diff', 'ability_code_920_first_time_player_0', 'ability_code_920_player_0', 'ability_code_960_diff', 'ability_code_960_player_1', 'ability_code_9E0_diff', 'ability_code_9E0_player_0', 'ability_code_9E0_player_1', 'ability_code_A40_diff', 'ability_code_A40_player_0', 'ability_code_A40_player_1', 'ability_code_B00_diff', 'ability_code_B00_player_0', 'ability_code_B01_diff', 'ability_code_B01_player_0', 'ability_code_B01_player_1', 'ability_code_B40_diff', 'ability_code_B40_first_time_player_0', 'ability_code_B40_last_first_player_0', 'ability_code_B40_player_0_ability_code_1022_player_0_diff', 'ability_code_B40_player_0_ability_code_102A_player_0_diff', 'ability_code_B40_player_0_ability_code_1261_player_0_diff', 'ability_code_B40_player_0_ability_code_13A0_player_0_diff', 'ability_code_B40_player_0_ability_code_15E0_player_0_diff', 'ability_code_B40_player_0_ability_code_16E2_player_0_diff', 'ability_code_B40_player_0_ability_code_1821_player_0_diff', 'ability_code_B40_player_0_ability_code_1822_player_0_diff', 'ability_code_B40_player_0_ability_code_1E60_player_0_diff', 'ability_code_B40_player_0_ability_code_480_player_0_diff', 'ability_code_B40_player_0_ability_code_DE0_player_0_diff', 'ability_code_B40_player_1', 'ability_code_B40_player_1_ability_code_1023_player_1_diff', 'ability_code_B40_player_1_ability_code_1023_player_1_diff_ability_code_B40_player_0_ability_code_1023_player_0_diff_diff', 'ability_code_B40_player_1_ability_code_102A_player_1_diff', 'ability_code_B40_player_1_ability_code_102A_player_1_diff_ability_code_B40_player_0_ability_code_102A_player_0_diff_diff', 'ability_code_B40_player_1_ability_code_1261_player_1_diff', 'ability_code_B40_player_1_ability_code_1261_player_1_diff_ability_code_B40_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_B40_player_1_ability_code_13A0_player_1_diff', 'ability_code_B40_player_1_ability_code_13A0_player_1_diff_ability_code_B40_player_0_ability_code_13A0_player_0_diff_diff', 'ability_code_B40_player_1_ability_code_15E0_player_1_diff', 'ability_code_B40_player_1_ability_code_15E0_player_1_diff_ability_code_B40_player_0_ability_code_15E0_player_0_diff_diff', 'ability_code_B40_player_1_ability_code_16E2_player_1_diff', 'ability_code_B40_player_1_ability_code_16E2_player_1_diff_ability_code_B40_player_0_ability_code_16E2_player_0_diff_diff', 'ability_code_B40_player_1_ability_code_1821_player_1_diff', 'ability_code_B40_player_1_ability_code_1821_player_1_diff_ability_code_B40_player_0_ability_code_1821_player_0_diff_diff', 'ability_code_B40_player_1_ability_code_1822_player_1_diff', 'ability_code_B40_player_1_ability_code_1C60_player_1_diff', 'ability_code_B40_player_1_ability_code_1E60_player_1_diff', 'ability_code_B40_player_1_ability_code_480_player_1_diff_ability_code_B40_player_0_ability_code_480_player_0_diff_diff', 'ability_code_B40_player_1_ability_code_DE0_player_1_diff', 'ability_code_B40_player_1_ability_code_DE0_player_1_diff_ability_code_B40_player_0_ability_code_DE0_player_0_diff_diff', 'ability_code_B60_player_0', 'ability_code_B61_diff', 'ability_code_B61_player_0', 'ability_code_B61_player_1', 'ability_code_BE0_diff', 'ability_code_BE0_player_0', 'ability_code_BE0_player_1', 'ability_code_CC0_player_0', 'ability_code_D00_diff', 'ability_code_D00_player_0', 'ability_code_D00_player_1', 'ability_code_D20_diff', 'ability_code_D20_player_0', 'ability_code_D20_player_1', 'ability_code_D40_player_0', 'ability_code_D40_player_1', 'ability_code_D41_player_1', 'ability_code_D61_player_1', 'ability_code_D62_diff', 'ability_code_D62_player_0', 'ability_code_D62_player_1', 'ability_code_DE0_first_time_player_0', 'ability_code_DE0_player_0', 'ability_code_DE0_player_0_ability_code_1020_player_0_diff', 'ability_code_DE0_player_0_ability_code_1022_player_0_diff', 'ability_code_DE0_player_0_ability_code_1023_player_0_diff', 'ability_code_DE0_player_0_ability_code_102A_player_0_diff', 'ability_code_DE0_player_0_ability_code_1261_player_0_diff', 'ability_code_DE0_player_0_ability_code_13A0_player_0_diff', 'ability_code_DE0_player_0_ability_code_15E0_player_0_diff', 'ability_code_DE0_player_0_ability_code_16E2_player_0_diff', 'ability_code_DE0_player_0_ability_code_1822_player_0_diff', 'ability_code_DE0_player_0_ability_code_1C60_player_0_diff', 'ability_code_DE0_player_0_ability_code_1E60_player_0_diff', 'ability_code_DE0_player_0_ability_code_480_player_0_diff', 'ability_code_DE0_player_1', 'ability_code_DE0_player_1_ability_code_1022_player_1_diff', 'ability_code_DE0_player_1_ability_code_1023_player_1_diff', 'ability_code_DE0_player_1_ability_code_102A_player_1_diff', 'ability_code_DE0_player_1_ability_code_1261_player_1_diff', 'ability_code_DE0_player_1_ability_code_13A0_player_1_diff', 'ability_code_DE0_player_1_ability_code_13A0_player_1_diff_ability_code_DE0_player_0_ability_code_13A0_player_0_diff_diff', 'ability_code_DE0_player_1_ability_code_15E0_player_1_diff', 'ability_code_DE0_player_1_ability_code_1822_player_1_diff_ability_code_DE0_player_0_ability_code_1822_player_0_diff_diff', 'ability_code_DE0_player_1_ability_code_1C60_player_1_diff', 'ability_code_DE0_player_1_ability_code_480_player_1_diff', 'ability_code_DE0_player_1_ability_code_480_player_1_diff_ability_code_DE0_player_0_ability_code_480_player_0_diff_diff', 'ability_code_E00_diff', 'ability_code_E00_player_0', 'ability_code_E00_player_1', 'ability_code_E20_player_0', 'ability_code_E20_player_1', 'ability_code_E80_diff', 'ability_code_E80_first_time_diff', 'ability_code_E80_first_time_player_0', 'ability_code_E80_player_0', 'ability_code_E80_player_1', 'ability_code_EE0_diff', 'ability_code_EE0_player_0', 'ability_code_EE0_player_1', 'ability_code_EE1_player_1', 'ability_code_F00_diff', 'ability_code_F00_player_0', 'ability_code_F00_player_1', 'ability_code_F01_diff', 'ability_code_F01_player_0', 'ability_code_F40_diff', 'ability_code_F40_player_0', 'ability_code_F40_player_1', 'ability_code_F41_diff', 'ability_code_F41_player_0', 'ability_code_F41_player_1', 'ability_code_F60_player_0', 'ability_code_F61_first_time_diff', 'ability_code_F61_player_1', 'ability_code_F80_diff', 'ability_code_F80_first_time_diff', 'ability_code_F80_player_1', 'ability_code_F81_first_time_diff', 'ability_code_F81_player_1', 'ability_code_FC0_diff', 'ability_code_FC0_player_0', 'ability_code_FC0_player_1', 'ability_code_FC1_diff', 'ability_code_FC1_player_0', 'ability_code_FC1_player_1', 'ability_code_FE0_first_time_diff', 'ability_code_FE0_first_time_player_0', 'ability_code_FE0_player_0', 'ability_code_FE1_diff', 'ability_code_FE1_player_0', 'ability_code_FE1_player_1', 'all_unit_diff', 'all_unit_len_player_0_Barracks_len_player_0_diff', 'all_unit_len_player_0_Egg_len_player_0_diff', 'all_unit_len_player_0_Factory_len_player_0_diff', 'all_unit_len_player_0_Hellion_len_player_0_diff', 'all_unit_len_player_0_MULE_len_player_0_diff', 'all_unit_len_player_0_OrbitalCommand_len_player_0_diff', 'all_unit_len_player_0_Overlord_len_player_0_diff', 'all_unit_len_player_0_Roach_len_player_0_diff', 'all_unit_len_player_1_BanelingCocoon_len_player_1_diff', 'all_unit_len_player_1_Barracks_len_player_1_diff_all_unit_len_player_0_Barracks_len_player_0_diff_diff', 'all_unit_len_player_1_Drone_len_player_1_diff', 'all_unit_len_player_1_Drone_len_player_1_diff_all_unit_len_player_0_Drone_len_player_0_diff_diff', 'all_unit_len_player_1_Egg_len_player_1_diff', 'all_unit_len_player_1_Factory_len_player_1_diff', 'all_unit_len_player_1_Factory_len_player_1_diff_all_unit_len_player_0_Factory_len_player_0_diff_diff', 'all_unit_len_player_1_Hatchery_len_player_1_diff', 'all_unit_len_player_1_Hatchery_len_player_1_diff_all_unit_len_player_0_Hatchery_len_player_0_diff_diff', 'all_unit_len_player_1_Marine_len_player_1_diff', 'all_unit_len_player_1_Marine_len_player_1_diff_all_unit_len_player_0_Marine_len_player_0_diff_diff', 'all_unit_len_player_1_Overlord_len_player_1_diff', 'all_unit_len_player_1_Overlord_len_player_1_diff_all_unit_len_player_0_Overlord_len_player_0_diff_diff', 'all_unit_len_player_1_Probe_len_player_1_diff_all_unit_len_player_0_Probe_len_player_0_diff_diff', 'all_unit_len_player_1_Queen_len_player_1_diff', 'all_unit_len_player_1_Queen_len_player_1_diff_all_unit_len_player_0_Queen_len_player_0_diff_diff', 'all_unit_len_player_1_Reaper_len_player_1_diff', 'all_unit_len_player_1_Roach_len_player_1_diff_all_unit_len_player_0_Roach_len_player_0_diff_diff', 'all_unit_len_player_1_SCV_len_player_1_diff', 'all_unit_len_player_1_Starport_len_player_1_diff', 'all_unit_len_player_1_Starport_len_player_1_diff_all_unit_len_player_0_Starport_len_player_0_diff_diff', 'all_unit_len_player_1_SupplyDepotLowered_len_player_1_diff', 'all_unit_len_player_1_SupplyDepotLowered_len_player_1_diff_all_unit_len_player_0_SupplyDepotLowered_len_player_0_diff_diff', 'build_count_player_0', 'camera_pos_x_max_player_1', 'camera_pos_x_mean_player_0', 'camera_pos_x_mean_player_1', 'camera_pos_x_min_player_0', 'camera_pos_x_min_player_1', 'camera_pos_y_max_player_1', 'camera_pos_y_min_player_0', 'game_id_time_max_div_nunique', 'species_player_0', 'species_player_1', 'species_war_kind', 'time_count_player_diff', 'under_1_Ability_player_0', 'under_1_AddToControlGroup_diff', 'under_1_AddToControlGroup_player_0', 'under_1_AddToControlGroup_player_1', 'under_1_Camera_player_0', 'under_1_Camera_player_1', 'under_1_ControlGroup_diff', 'under_1_ControlGroup_player_0', 'under_1_ControlGroup_player_1', 'under_1_GetControlGroup_player_0', 'under_1_Right Click_player_1', 'under_1_Selection_diff', 'under_1_Selection_player_1', 'under_1_SetControlGroup_player_0', 'under_1_SetControlGroup_player_1', 'under_3_AddToControlGroup_diff', 'under_3_Camera_diff', 'under_3_Camera_player_0', 'under_3_ControlGroup_diff', 'under_3_ControlGroup_player_0', 'under_3_ControlGroup_player_1', 'under_3_GetControlGroup_player_1', 'under_3_SetControlGroup_diff', 'under_3_SetControlGroup_player_0', 'under_3_SetControlGroup_player_1', 'under_5_Ability_diff', 'under_5_Ability_player_0', 'under_5_AddToControlGroup_diff', 'under_5_AddToControlGroup_player_1', 'under_5_Camera_player_0', 'under_5_ControlGroup_diff', 'under_5_ControlGroup_player_0', 'under_5_GetControlGroup_diff', 'under_5_GetControlGroup_player_1', 'under_5_Selection_diff', 'under_5_SetControlGroup_diff', 'under_5_SetControlGroup_player_0', 'under_5_SetControlGroup_player_1', 'worker_count_player_0']
bad_feature_fold2 = ['Ability_player_0_isnull', 'Ability_player_1_isnull', 'AddToControlGroup_player_0', 'AddToControlGroup_player_0_AddToControlGroup_player_1_diff', 'AddToControlGroup_player_1_isnull', 'AdeptPiercingWeapon_len_player_0', 'Adept_len_player_0', 'Adept_right_click_target_count_diff', 'Adept_right_click_target_count_player_0', 'Adept_right_click_target_count_player_1', 'AiurLightBridgeAbandonedNE10Out_len_player_1', 'AiurLightBridgeAbandonedNE8Out_len_player_0', 'AiurLightBridgeAbandonedNE8Out_len_player_1', 'AiurLightBridgeAbandonedNE8_len_player_0', 'AiurLightBridgeAbandonedNE8_len_player_1', 'AiurLightBridgeNE10Out_len_player_0', 'AiurLightBridgeNE10Out_len_player_1', 'AiurLightBridgeNE8_len_player_0', 'AiurLightBridgeNE8_len_player_1', 'Archon_len_player_0', 'Archon_len_player_1', 'Armory_len_player_1', 'Assimilator_len_player_0', 'Assimilator_right_click_target_count_diff', 'Assimilator_right_click_target_count_player_1', 'AutoTurret_len_player_0', 'AutoTurret_len_player_1', 'BanelingBurrowed_len_player_0', 'BanelingBurrowed_len_player_1', 'BanelingCocoon_len_player_0', 'BanelingCocoon_len_player_0_Hellion_len_player_0_diff', 'BanelingCocoon_len_player_0_MULE_len_player_0_diff', 'BanelingCocoon_len_player_0_Overlord_len_player_0_diff', 'BanelingCocoon_len_player_1', 'BanelingCocoon_len_player_1_Factory_len_player_1_diff', 'BanelingCocoon_len_player_1_Hatchery_len_player_1_diff', 'BanelingCocoon_len_player_1_Hatchery_len_player_1_diff_BanelingCocoon_len_player_0_Hatchery_len_player_0_diff_diff', 'BanelingCocoon_len_player_1_Hellion_len_player_1_diff', 'BanelingCocoon_len_player_1_MULE_len_player_1_diff', 'BanelingCocoon_len_player_1_Overlord_len_player_1_diff_BanelingCocoon_len_player_0_Overlord_len_player_0_diff_diff', 'BanelingCocoon_len_player_1_Roach_len_player_1_diff', 'BanelingCocoon_len_player_1_Starport_len_player_1_diff', 'BanelingCocoon_right_click_target_count_player_0', 'Baneling_len_player_0', 'Baneling_right_click_target_count_diff', 'Baneling_right_click_target_count_player_0', 'Baneling_right_click_target_count_player_1', 'Banshee_len_player_1', 'BarracksReactor_len_player_0', 'BarracksReactor_len_player_1', 'BarracksTechLab_len_player_0', 'Barracks_len_player_0_BanelingCocoon_len_player_0_diff', 'Barracks_len_player_0_MULE_len_player_0_diff', 'Barracks_len_player_0_Queen_len_player_0_diff', 'Barracks_len_player_0_Reaper_len_player_0_diff', 'Barracks_len_player_1', 'Barracks_len_player_1_BanelingCocoon_len_player_1_diff', 'Barracks_len_player_1_Hatchery_len_player_1_diff', 'Barracks_len_player_1_Hellion_len_player_1_diff', 'Barracks_len_player_1_MULE_len_player_1_diff', 'Barracks_len_player_1_OrbitalCommand_len_player_1_diff', 'Barracks_len_player_1_Overlord_len_player_1_diff', 'Barracks_len_player_1_Overlord_len_player_1_diff_Barracks_len_player_0_Overlord_len_player_0_diff_diff', 'Barracks_len_player_1_Reaper_len_player_1_diff_Barracks_len_player_0_Reaper_len_player_0_diff_diff', 'Barracks_len_player_1_Roach_len_player_1_diff', 'Barracks_len_player_1_Starport_len_player_1_diff_Barracks_len_player_0_Starport_len_player_0_diff_diff', 'Barracks_len_player_1_SupplyDepotLowered_len_player_1_diff', 'Barracks_len_player_1_SupplyDepotLowered_len_player_1_diff_Barracks_len_player_0_SupplyDepotLowered_len_player_0_diff_diff', 'Barracks_right_click_target_count_diff', 'Barracks_right_click_target_count_player_0', 'Battlecruiser_len_player_0', 'Battlecruiser_len_player_1', 'Beacon_TerranSmall_len_player_0', 'Beacon_TerranSmall_len_player_1', 'Beacon_Terran_len_player_0', 'Beacon_Terran_len_player_1', 'BroodLordAWeapon_len_player_0', 'BroodLordAWeapon_len_player_1', 'BroodLordWeapon_len_player_0', 'BroodLordWeapon_len_player_1', 'BroodLord_len_player_0', 'BroodLord_len_player_1', 'Broodling_len_player_0', 'Broodling_len_player_1', 'Bunker_len_player_1', 'Bunker_right_click_target_count_player_1', 'BypassArmorDrone_len_player_0', 'BypassArmorDrone_len_player_1', 'Camera_player_0_isnull', 'Camera_player_1_isnull', 'Carrier_len_player_0', 'Carrier_len_player_1', 'ChangelingMarineShield_len_player_1', 'ChangelingMarine_len_player_0', 'ChangelingMarine_len_player_1', 'ChangelingZealot_len_player_0', 'ChangelingZealot_len_player_1', 'ChangelingZerglingWings_len_player_0', 'ChangelingZerglingWings_len_player_1', 'ChangelingZergling_len_player_0', 'ChangelingZergling_len_player_1', 'Changeling_len_player_0', 'Changeling_len_player_1', 'Colossus_len_player_0', 'Colossus_len_player_1', 'CommandCenterFlying_len_player_1', 'CommandCenter_len_player_1', 'ControlGroup_player_0', 'ControlGroup_player_0_isnull', 'ControlGroup_player_1', 'ControlGroup_player_1_isnull', 'Corruptor_len_player_0', 'Corruptor_len_player_1', 'CreepOnlyBlocker4x4_len_player_0', 'CreepOnlyBlocker4x4_len_player_1', 'CreepOnlyBlocker4x4_right_click_target_count_diff', 'CreepTumorBurrowed_right_click_target_count_player_0', 'CreepTumorQueen_len_player_0', 'CreepTumorQueen_len_player_1', 'CreepTumor_len_player_0', 'CreepTumor_len_player_1', 'CyberneticsCore_len_player_0', 'CyberneticsCore_len_player_1', 'CyberneticsCore_right_click_target_count_player_0', 'CycloneMissileLargeAir_len_player_0', 'Cyclone_len_player_0', 'Cyclone_len_player_1', 'DarkShrine_len_player_0', 'DarkShrine_len_player_1', 'DarkTemplar_len_player_0', 'DarkTemplar_len_player_1', 'DestructibleIce2x4Horizontal_len_player_0', 'DestructibleIce2x4Horizontal_len_player_1', 'DestructibleIce2x4Vertical_len_player_0', 'DestructibleIce2x4Vertical_len_player_1', 'DestructibleIce2x4Vertical_right_click_target_count_diff', 'DestructibleIce2x4Vertical_right_click_target_count_player_0', 'DestructibleIce2x4Vertical_right_click_target_count_player_1', 'DestructibleIce2x6Horizontal_len_player_0', 'DestructibleIce2x6Horizontal_len_player_1', 'DestructibleIce2x6Vertical_len_player_0', 'DestructibleIce2x6Vertical_len_player_1', 'DestructibleSignsConstruction_len_player_0', 'DestructibleSignsConstruction_right_click_target_count_player_1', 'DestructibleSignsFunny_len_player_0', 'DestructibleSignsIcons_len_player_1', 'DestructibleSignsIcons_right_click_target_count_player_1', 'Disruptor_len_player_0', 'Disruptor_len_player_1', 'DroneBurrowed_len_player_0', 'Drone_len_player_0_Barracks_len_player_0_diff', 'Drone_len_player_0_Factory_len_player_0_diff', 'Drone_len_player_0_Hatchery_len_player_0_diff', 'Drone_len_player_0_Hellion_len_player_0_diff', 'Drone_len_player_0_Reaper_len_player_0_diff', 'Drone_len_player_0_Roach_len_player_0_diff', 'Drone_len_player_0_SupplyDepotLowered_len_player_0_diff', 'Drone_len_player_1_Barracks_len_player_1_diff', 'Drone_len_player_1_Barracks_len_player_1_diff_Drone_len_player_0_Barracks_len_player_0_diff_diff', 'Drone_len_player_1_Hatchery_len_player_1_diff', 'Drone_len_player_1_Hellion_len_player_1_diff', 'Drone_len_player_1_MULE_len_player_1_diff', 'Drone_len_player_1_Marine_len_player_1_diff', 'Drone_len_player_1_OrbitalCommand_len_player_1_diff_Drone_len_player_0_OrbitalCommand_len_player_0_diff_diff', 'Drone_len_player_1_Probe_len_player_1_diff', 'Drone_len_player_1_Probe_len_player_1_diff_Drone_len_player_0_Probe_len_player_0_diff_diff', 'Drone_len_player_1_Queen_len_player_1_diff_Drone_len_player_0_Queen_len_player_0_diff_diff', 'Drone_len_player_1_Reaper_len_player_1_diff', 'Drone_len_player_1_Reaper_len_player_1_diff_Drone_len_player_0_Reaper_len_player_0_diff_diff', 'Drone_len_player_1_Starport_len_player_1_diff', 'Drone_len_player_1_SupplyDepotLowered_len_player_1_diff', 'Drone_len_player_1_Zergling_len_player_1_diff_Drone_len_player_0_Zergling_len_player_0_diff_diff', 'Egg_len_player_0_Barracks_len_player_0_diff', 'Egg_len_player_0_Hellion_len_player_0_diff', 'Egg_len_player_0_MULE_len_player_0_diff', 'Egg_len_player_0_Marine_len_player_0_diff', 'Egg_len_player_0_OrbitalCommand_len_player_0_diff', 'Egg_len_player_0_Overlord_len_player_0_diff', 'Egg_len_player_0_Queen_len_player_0_diff', 'Egg_len_player_0_Reaper_len_player_0_diff', 'Egg_len_player_0_Starport_len_player_0_diff', 'Egg_len_player_1_BanelingCocoon_len_player_1_diff_Egg_len_player_0_BanelingCocoon_len_player_0_diff_diff', 'Egg_len_player_1_Hatchery_len_player_1_diff', 'Egg_len_player_1_Hellion_len_player_1_diff', 'Egg_len_player_1_MULE_len_player_1_diff', 'Egg_len_player_1_Marine_len_player_1_diff', 'Egg_len_player_1_OrbitalCommand_len_player_1_diff', 'Egg_len_player_1_Probe_len_player_1_diff', 'Egg_len_player_1_SCV_len_player_1_diff', 'Egg_len_player_1_SupplyDepotLowered_len_player_1_diff', 'EngineeringBay_right_click_target_count_diff', 'EngineeringBay_right_click_target_count_player_0', 'EngineeringBay_right_click_target_count_player_1', 'ExtendingBridgeNEWide10Out_len_player_1', 'ExtendingBridgeNEWide10_len_player_0', 'ExtendingBridgeNEWide10_len_player_1', 'ExtendingBridgeNEWide8Out_len_player_0', 'ExtendingBridgeNEWide8Out_len_player_1', 'ExtendingBridgeNWWide8Out_len_player_0', 'ExtendingBridgeNWWide8Out_len_player_1', 'ExtendingBridgeNWWide8_len_player_0', 'ExtendingBridgeNWWide8_len_player_1', 'Extractor_len_player_0', 'Extractor_len_player_1', 'Extractor_right_click_target_count_player_1', 'FactoryReactor_len_player_0', 'FactoryReactor_len_player_1', 'FactoryTechLab_len_player_0', 'FactoryTechLab_len_player_1', 'Factory_len_player_1', 'Factory_len_player_1_Hatchery_len_player_1_diff_Factory_len_player_0_Hatchery_len_player_0_diff_diff', 'Factory_len_player_1_MULE_len_player_1_diff', 'Factory_len_player_1_Reaper_len_player_1_diff', 'Factory_len_player_1_Roach_len_player_1_diff', 'Factory_len_player_1_Starport_len_player_1_diff', 'Factory_right_click_target_count_player_1', 'FleetBeacon_len_player_0', 'Forge_len_player_0', 'Forge_right_click_target_count_player_1', 'FungalGrowthMissile_len_player_0', 'FungalGrowthMissile_right_click_target_count_diff', 'FungalGrowthMissile_right_click_target_count_player_0', 'FungalGrowthMissile_right_click_target_count_player_1', 'FusionCore_len_player_0', 'FusionCore_len_player_1', 'GetControlGroup_player_0_isnull', 'GetControlGroup_player_1_isnull', 'GhostAcademy_len_player_0', 'GhostAcademy_len_player_1', 'Ghost_len_player_0', 'Ghost_len_player_1', 'GlaiveWurmBounceWeapon_len_player_0', 'GlaiveWurmBounceWeapon_len_player_1', 'GreaterSpire_len_player_0', 'GreaterSpire_len_player_1', 'Hatchery_len_player_0', 'Hatchery_len_player_0_Reaper_len_player_0_diff', 'Hatchery_len_player_1', 'Hatchery_len_player_1_Starport_len_player_1_diff', 'Hatchery_len_player_1_Starport_len_player_1_diff_Hatchery_len_player_0_Starport_len_player_0_diff_diff', 'Hatchery_right_click_target_count_player_0', 'Hellion_len_player_0', 'Hellion_len_player_0_Hatchery_len_player_0_diff', 'Hellion_len_player_0_Reaper_len_player_0_diff', 'Hellion_len_player_0_Roach_len_player_0_diff', 'Hellion_len_player_1', 'Hellion_len_player_1_Hatchery_len_player_1_diff', 'Hellion_len_player_1_MULE_len_player_1_diff_Hellion_len_player_0_MULE_len_player_0_diff_diff', 'Hellion_len_player_1_Reaper_len_player_1_diff', 'Hellion_len_player_1_Reaper_len_player_1_diff_Hellion_len_player_0_Reaper_len_player_0_diff_diff', 'Hellion_len_player_1_Roach_len_player_1_diff', 'Hellion_len_player_1_Roach_len_player_1_diff_Hellion_len_player_0_Roach_len_player_0_diff_diff', 'Hellion_right_click_target_count_diff', 'Hellion_right_click_target_count_player_0', 'Hellion_right_click_target_count_player_1', 'HighTemplar_len_player_0', 'Hive_right_click_target_count_player_1', 'HydraliskDen_len_player_0', 'HydraliskDen_len_player_1', 'Hydralisk_len_player_1', 'Immortal_len_player_0', 'InfestationPit_len_player_0', 'InfestationPit_len_player_1', 'InfestorBurrowed_len_player_1', 'Infestor_len_player_1', 'Lair_right_click_target_count_diff', 'Lair_right_click_target_count_player_0', 'Lair_right_click_target_count_player_1', 'Larva_len_player_1', 'LocustMPFlying_len_player_0', 'LocustMPFlying_len_player_1', 'LurkerEgg_len_player_0', 'LurkerEgg_len_player_1', 'Lurker_len_player_0', 'Lurker_len_player_1', 'MULE_len_player_0', 'MULE_len_player_0_Hatchery_len_player_0_diff', 'MULE_len_player_0_Roach_len_player_0_diff', 'MULE_len_player_1', 'MULE_len_player_1_Hatchery_len_player_1_diff', 'MULE_len_player_1_Hatchery_len_player_1_diff_MULE_len_player_0_Hatchery_len_player_0_diff_diff', 'MULE_len_player_1_Reaper_len_player_1_diff', 'MULE_len_player_1_Reaper_len_player_1_diff_MULE_len_player_0_Reaper_len_player_0_diff_diff', 'MULE_len_player_1_Roach_len_player_1_diff', 'MULE_len_player_1_Roach_len_player_1_diff_MULE_len_player_0_Roach_len_player_0_diff_diff', 'MULE_len_player_1_Starport_len_player_1_diff', 'MULE_len_player_1_Starport_len_player_1_diff_MULE_len_player_0_Starport_len_player_0_diff_diff', 'MULE_right_click_target_count_diff', 'MULE_right_click_target_count_player_0', 'MULE_right_click_target_count_player_1', 'Marine_len_player_0', 'Marine_len_player_0_BanelingCocoon_len_player_0_diff', 'Marine_len_player_0_Factory_len_player_0_diff', 'Marine_len_player_0_Hatchery_len_player_0_diff', 'Marine_len_player_0_Hellion_len_player_0_diff', 'Marine_len_player_0_MULE_len_player_0_diff', 'Marine_len_player_0_Probe_len_player_0_diff', 'Marine_len_player_0_Queen_len_player_0_diff', 'Marine_len_player_0_Roach_len_player_0_diff', 'Marine_len_player_0_Starport_len_player_0_diff', 'Marine_len_player_0_SupplyDepotLowered_len_player_0_diff', 'Marine_len_player_1', 'Marine_len_player_1_BanelingCocoon_len_player_1_diff', 'Marine_len_player_1_Hatchery_len_player_1_diff', 'Marine_len_player_1_Hatchery_len_player_1_diff_Marine_len_player_0_Hatchery_len_player_0_diff_diff', 'Marine_len_player_1_OrbitalCommand_len_player_1_diff', 'Marine_len_player_1_Overlord_len_player_1_diff', 'Marine_len_player_1_Probe_len_player_1_diff', 'Marine_len_player_1_Queen_len_player_1_diff', 'Marine_len_player_1_Reaper_len_player_1_diff', 'Marine_len_player_1_Reaper_len_player_1_diff_Marine_len_player_0_Reaper_len_player_0_diff_diff', 'Marine_len_player_1_Roach_len_player_1_diff', 'Marine_len_player_1_SupplyDepotLowered_len_player_1_diff_Marine_len_player_0_SupplyDepotLowered_len_player_0_diff_diff', 'Marine_len_player_1_Zergling_len_player_1_diff', 'Marine_right_click_target_count_diff', 'Marine_right_click_target_count_player_1', 'Medivac_len_player_0', 'Medivac_right_click_target_count_diff', 'Medivac_right_click_target_count_player_0', 'Medivac_right_click_target_count_player_1', 'MissileTurret_len_player_0', 'MissileTurret_len_player_1', 'Mutalisk_len_player_0', 'Mutalisk_len_player_1', 'None_len_player_0', 'NydusNetwork_len_player_1', 'NydusWorm_len_player_0', 'NydusWorm_len_player_1', 'Observer_len_player_0', 'Observer_len_player_1', 'OrbitalCommand_len_player_0_BanelingCocoon_len_player_0_diff', 'OrbitalCommand_len_player_0_Hellion_len_player_0_diff', 'OrbitalCommand_len_player_0_MULE_len_player_0_diff', 'OrbitalCommand_len_player_0_Overlord_len_player_0_diff', 'OrbitalCommand_len_player_0_Reaper_len_player_0_diff', 'OrbitalCommand_len_player_0_Roach_len_player_0_diff', 'OrbitalCommand_len_player_1', 'OrbitalCommand_len_player_1_BanelingCocoon_len_player_1_diff_OrbitalCommand_len_player_0_BanelingCocoon_len_player_0_diff_diff', 'OrbitalCommand_len_player_1_Factory_len_player_1_diff', 'OrbitalCommand_len_player_1_Factory_len_player_1_diff_OrbitalCommand_len_player_0_Factory_len_player_0_diff_diff', 'OrbitalCommand_len_player_1_Hatchery_len_player_1_diff_OrbitalCommand_len_player_0_Hatchery_len_player_0_diff_diff', 'OrbitalCommand_len_player_1_Hellion_len_player_1_diff', 'OrbitalCommand_len_player_1_MULE_len_player_1_diff', 'OrbitalCommand_len_player_1_Overlord_len_player_1_diff', 'OrbitalCommand_len_player_1_Roach_len_player_1_diff', 'OrbitalCommand_len_player_1_Roach_len_player_1_diff_OrbitalCommand_len_player_0_Roach_len_player_0_diff_diff', 'OrbitalCommand_len_player_1_Starport_len_player_1_diff', 'OrbitalCommand_len_player_1_SupplyDepotLowered_len_player_1_diff', 'OverlordGenerateCreepKeybind_len_player_0', 'OverlordGenerateCreepKeybind_len_player_1', 'Overlord_len_player_0', 'Overlord_len_player_0_Factory_len_player_0_diff', 'Overlord_len_player_0_Hellion_len_player_0_diff', 'Overlord_len_player_0_MULE_len_player_0_diff', 'Overlord_len_player_0_Reaper_len_player_0_diff', 'Overlord_len_player_0_Roach_len_player_0_diff', 'Overlord_len_player_0_Starport_len_player_0_diff', 'Overlord_len_player_1', 'Overlord_len_player_1_Factory_len_player_1_diff', 'Overlord_len_player_1_Hatchery_len_player_1_diff', 'Overlord_len_player_1_Hatchery_len_player_1_diff_Overlord_len_player_0_Hatchery_len_player_0_diff_diff', 'Overlord_len_player_1_Hellion_len_player_1_diff', 'Overlord_len_player_1_MULE_len_player_1_diff', 'Overlord_len_player_1_Reaper_len_player_1_diff', 'Overlord_len_player_1_Reaper_len_player_1_diff_Overlord_len_player_0_Reaper_len_player_0_diff_diff', 'Overlord_len_player_1_Starport_len_player_1_diff', 'Overlord_len_player_1_Starport_len_player_1_diff_Overlord_len_player_0_Starport_len_player_0_diff_diff', 'Overlord_right_click_target_count_player_1', 'OverseerCocoon_len_player_0', 'OverseerCocoon_len_player_1', 'Phoenix_len_player_0', 'Phoenix_len_player_1', 'PhotonCannon_len_player_0', 'PhotonCannon_len_player_1', 'PhotonCannon_right_click_target_count_diff', 'PhotonCannon_right_click_target_count_player_0', 'PhotonCannon_right_click_target_count_player_1', 'PhysicsCapsule_len_player_0', 'PhysicsCapsule_len_player_1', 'PhysicsCapsule_right_click_target_count_diff', 'PhysicsCapsule_right_click_target_count_player_0', 'PhysicsCapsule_right_click_target_count_player_1', 'PlanetaryFortress_len_player_1', 'Probe_len_player_0', 'Probe_len_player_0_BanelingCocoon_len_player_0_diff', 'Probe_len_player_0_Hellion_len_player_0_diff', 'Probe_len_player_0_OrbitalCommand_len_player_0_diff', 'Probe_len_player_0_Overlord_len_player_0_diff', 'Probe_len_player_0_Reaper_len_player_0_diff', 'Probe_len_player_0_Starport_len_player_0_diff', 'Probe_len_player_1', 'Probe_len_player_1_Barracks_len_player_1_diff', 'Probe_len_player_1_Factory_len_player_1_diff', 'Probe_len_player_1_Hellion_len_player_1_diff', 'Probe_len_player_1_Hellion_len_player_1_diff_Probe_len_player_0_Hellion_len_player_0_diff_diff', 'Probe_len_player_1_MULE_len_player_1_diff', 'Probe_len_player_1_OrbitalCommand_len_player_1_diff', 'Probe_len_player_1_Overlord_len_player_1_diff', 'Probe_len_player_1_Queen_len_player_1_diff', 'Probe_len_player_1_Reaper_len_player_1_diff', 'Probe_len_player_1_Reaper_len_player_1_diff_Probe_len_player_0_Reaper_len_player_0_diff_diff', 'Probe_len_player_1_Roach_len_player_1_diff', 'Probe_len_player_1_Starport_len_player_1_diff', 'Probe_right_click_target_count_player_1', 'Pylon_len_player_1', 'QueenBurrowed_len_player_0', 'QueenBurrowed_len_player_1', 'Queen_len_player_0', 'Queen_len_player_0_Hatchery_len_player_0_diff', 'Queen_len_player_0_Hellion_len_player_0_diff', 'Queen_len_player_0_MULE_len_player_0_diff', 'Queen_len_player_0_Roach_len_player_0_diff', 'Queen_len_player_1', 'Queen_len_player_1_Factory_len_player_1_diff_Queen_len_player_0_Factory_len_player_0_diff_diff', 'Queen_len_player_1_Hellion_len_player_1_diff', 'Queen_len_player_1_Hellion_len_player_1_diff_Queen_len_player_0_Hellion_len_player_0_diff_diff', 'Queen_len_player_1_Reaper_len_player_1_diff_Queen_len_player_0_Reaper_len_player_0_diff_diff', 'Queen_len_player_1_Starport_len_player_1_diff', 'Queen_right_click_target_count_diff', 'Queen_right_click_target_count_player_0', 'Queen_right_click_target_count_player_1', 'Raven_len_player_1', 'Reactor_len_player_0', 'Reactor_len_player_1', 'Reaper_len_player_0', 'Reaper_len_player_1', 'Reaper_len_player_1_Roach_len_player_1_diff', 'Reaper_len_player_1_Roach_len_player_1_diff_Reaper_len_player_0_Roach_len_player_0_diff_diff', 'Reaper_len_player_1_Starport_len_player_1_diff', 'Reaper_right_click_target_count_player_1', 'Refinery_len_player_0', 'Refinery_len_player_1', 'RichMineralField750_len_player_0', 'RichMineralField750_len_player_1', 'RichMineralField_len_player_0', 'RichMineralField_len_player_1', 'Right Click_player_0_isnull', 'Right Click_player_1_isnull', 'RoachBurrowed_len_player_0', 'RoachBurrowed_len_player_1', 'Roach_len_player_0', 'Roach_len_player_0_Starport_len_player_0_diff', 'Roach_len_player_1', 'Roach_len_player_1_Starport_len_player_1_diff', 'Roach_right_click_target_count_diff', 'Roach_right_click_target_count_player_0', 'Roach_right_click_target_count_player_1', 'RoboticsBay_len_player_0', 'Rocks2x2NonConjoined_len_player_1', 'SCV_len_player_0', 'SCV_len_player_0_BanelingCocoon_len_player_0_diff', 'SCV_len_player_0_Drone_len_player_0_diff', 'SCV_len_player_0_Factory_len_player_0_diff', 'SCV_len_player_0_Hatchery_len_player_0_diff', 'SCV_len_player_0_MULE_len_player_0_diff', 'SCV_len_player_0_Marine_len_player_0_diff', 'SCV_len_player_0_OrbitalCommand_len_player_0_diff', 'SCV_len_player_0_Overlord_len_player_0_diff', 'SCV_len_player_0_Reaper_len_player_0_diff', 'SCV_len_player_0_Roach_len_player_0_diff', 'SCV_len_player_0_Starport_len_player_0_diff', 'SCV_len_player_0_SupplyDepotLowered_len_player_0_diff', 'SCV_len_player_0_Zergling_len_player_0_diff', 'SCV_len_player_1', 'SCV_len_player_1_BanelingCocoon_len_player_1_diff', 'SCV_len_player_1_BanelingCocoon_len_player_1_diff_SCV_len_player_0_BanelingCocoon_len_player_0_diff_diff', 'SCV_len_player_1_Barracks_len_player_1_diff', 'SCV_len_player_1_Drone_len_player_1_diff_SCV_len_player_0_Drone_len_player_0_diff_diff', 'SCV_len_player_1_Factory_len_player_1_diff_SCV_len_player_0_Factory_len_player_0_diff_diff', 'SCV_len_player_1_Hatchery_len_player_1_diff_SCV_len_player_0_Hatchery_len_player_0_diff_diff', 'SCV_len_player_1_Hellion_len_player_1_diff', 'SCV_len_player_1_Hellion_len_player_1_diff_SCV_len_player_0_Hellion_len_player_0_diff_diff', 'SCV_len_player_1_Marine_len_player_1_diff', 'SCV_len_player_1_Marine_len_player_1_diff_SCV_len_player_0_Marine_len_player_0_diff_diff', 'SCV_len_player_1_OrbitalCommand_len_player_1_diff', 'SCV_len_player_1_Overlord_len_player_1_diff', 'SCV_len_player_1_Overlord_len_player_1_diff_SCV_len_player_0_Overlord_len_player_0_diff_diff', 'SCV_len_player_1_Probe_len_player_1_diff', 'SCV_len_player_1_Probe_len_player_1_diff_SCV_len_player_0_Probe_len_player_0_diff_diff', 'SCV_len_player_1_Reaper_len_player_1_diff', 'SCV_len_player_1_Reaper_len_player_1_diff_SCV_len_player_0_Reaper_len_player_0_diff_diff', 'SCV_len_player_1_Roach_len_player_1_diff', 'SCV_len_player_1_SupplyDepotLowered_len_player_1_diff', 'SCV_len_player_1_SupplyDepotLowered_len_player_1_diff_SCV_len_player_0_SupplyDepotLowered_len_player_0_diff_diff', 'SCV_len_player_1_Zergling_len_player_1_diff', 'SCV_len_player_1_Zergling_len_player_1_diff_SCV_len_player_0_Zergling_len_player_0_diff_diff', 'SCV_right_click_target_count_player_0', 'SCV_right_click_target_count_player_1', 'Selection_player_0_isnull', 'Selection_player_1_isnull', 'SensorTower_len_player_0', 'SensorTower_len_player_1', 'Sentry_len_player_0', 'SetControlGroup_player_0_SetControlGroup_player_1_diff', 'SetControlGroup_player_0_isnull', 'SetControlGroup_player_1', 'SetControlGroup_player_1_isnull', 'SiegeTankSieged_len_player_1', 'SiegeTankSieged_right_click_target_count_diff', 'SiegeTankSieged_right_click_target_count_player_0', 'SiegeTank_len_player_0', 'SiegeTank_right_click_target_count_player_0', 'SiegeTank_right_click_target_count_player_1', 'SpawningPool_len_player_1', 'SpawningPool_right_click_target_count_diff', 'SpawningPool_right_click_target_count_player_0', 'SpineCrawlerUprooted_len_player_0', 'SpineCrawlerUprooted_len_player_1', 'SpineCrawler_len_player_1', 'Spire_len_player_0', 'Spire_len_player_1', 'SporeCrawlerUprooted_len_player_0', 'SporeCrawlerUprooted_len_player_1', 'SporeCrawler_len_player_0', 'SporeCrawler_len_player_1', 'Stalker_len_player_0', 'Stalker_right_click_target_count_player_0', 'StarportReactor_len_player_0', 'StarportReactor_len_player_1', 'StarportTechLab_len_player_0', 'StarportTechLab_len_player_1', 'Starport_len_player_0', 'Starport_len_player_1', 'Starport_right_click_target_count_player_0', 'Starport_right_click_target_count_player_1', 'SupplyDepotLowered_len_player_0', 'SupplyDepotLowered_len_player_0_BanelingCocoon_len_player_0_diff', 'SupplyDepotLowered_len_player_0_Overlord_len_player_0_diff', 'SupplyDepotLowered_len_player_0_Queen_len_player_0_diff', 'SupplyDepotLowered_len_player_0_Roach_len_player_0_diff', 'SupplyDepotLowered_len_player_1', 'SupplyDepotLowered_len_player_1_BanelingCocoon_len_player_1_diff', 'SupplyDepotLowered_len_player_1_BanelingCocoon_len_player_1_diff_SupplyDepotLowered_len_player_0_BanelingCocoon_len_player_0_diff_diff', 'SupplyDepotLowered_len_player_1_Factory_len_player_1_diff', 'SupplyDepotLowered_len_player_1_Factory_len_player_1_diff_SupplyDepotLowered_len_player_0_Factory_len_player_0_diff_diff', 'SupplyDepotLowered_len_player_1_Hatchery_len_player_1_diff', 'SupplyDepotLowered_len_player_1_Hatchery_len_player_1_diff_SupplyDepotLowered_len_player_0_Hatchery_len_player_0_diff_diff', 'SupplyDepotLowered_len_player_1_Hellion_len_player_1_diff', 'SupplyDepotLowered_len_player_1_MULE_len_player_1_diff', 'SupplyDepotLowered_len_player_1_Overlord_len_player_1_diff', 'SupplyDepotLowered_len_player_1_Overlord_len_player_1_diff_SupplyDepotLowered_len_player_0_Overlord_len_player_0_diff_diff', 'SupplyDepotLowered_len_player_1_Queen_len_player_1_diff', 'SupplyDepotLowered_len_player_1_Roach_len_player_1_diff', 'SupplyDepotLowered_len_player_1_Starport_len_player_1_diff_SupplyDepotLowered_len_player_0_Starport_len_player_0_diff_diff', 'SupplyDepot_right_click_target_count_player_1', 'SwarmHost_len_player_1', 'TechLab_len_player_1', 'TemplarArchive_len_player_0', 'TemplarArchive_len_player_1', 'Thor_len_player_0', 'Thor_len_player_1', 'UltraliskCavern_len_player_1', 'VikingAssault_len_player_0', 'VikingAssault_len_player_1', 'Viking_len_player_0', 'Viking_len_player_1', 'VoidRay_len_player_1', 'WarpGate_right_click_target_count_player_0', 'WarpPrismPhasing_len_player_0', 'WarpPrismPhasing_len_player_1', 'WarpPrism_len_player_0', 'WarpPrism_len_player_1', 'Weapon_len_player_0', 'Weapon_len_player_1', 'XelNagaDestructibleBlocker8W_len_player_0', 'XelNagaDestructibleBlocker8W_len_player_1', 'XelNagaTower_len_player_0', 'XelNagaTower_len_player_1', 'XelNagaTower_right_click_target_count_player_0', 'Zealot_len_player_0', 'Zealot_len_player_1', 'Zealot_right_click_target_count_player_0', 'Zealot_right_click_target_count_player_1', 'ZerglingBurrowed_len_player_0', 'ZerglingBurrowed_len_player_1', 'Zergling_len_player_0', 'Zergling_len_player_0_BanelingCocoon_len_player_0_diff', 'Zergling_len_player_0_Hatchery_len_player_0_diff', 'Zergling_len_player_0_Hellion_len_player_0_diff', 'Zergling_len_player_0_MULE_len_player_0_diff', 'Zergling_len_player_0_OrbitalCommand_len_player_0_diff', 'Zergling_len_player_0_Overlord_len_player_0_diff', 'Zergling_len_player_0_Probe_len_player_0_diff', 'Zergling_len_player_0_Queen_len_player_0_diff', 'Zergling_len_player_0_Reaper_len_player_0_diff', 'Zergling_len_player_0_Roach_len_player_0_diff', 'Zergling_len_player_1', 'Zergling_len_player_1_Barracks_len_player_1_diff', 'Zergling_len_player_1_Barracks_len_player_1_diff_Zergling_len_player_0_Barracks_len_player_0_diff_diff', 'Zergling_len_player_1_Factory_len_player_1_diff', 'Zergling_len_player_1_Factory_len_player_1_diff_Zergling_len_player_0_Factory_len_player_0_diff_diff', 'Zergling_len_player_1_Hatchery_len_player_1_diff_Zergling_len_player_0_Hatchery_len_player_0_diff_diff', 'Zergling_len_player_1_Hellion_len_player_1_diff', 'Zergling_len_player_1_MULE_len_player_1_diff', 'Zergling_len_player_1_OrbitalCommand_len_player_1_diff', 'Zergling_len_player_1_Probe_len_player_1_diff', 'Zergling_len_player_1_Probe_len_player_1_diff_Zergling_len_player_0_Probe_len_player_0_diff_diff', 'Zergling_len_player_1_Queen_len_player_1_diff', 'Zergling_len_player_1_Queen_len_player_1_diff_Zergling_len_player_0_Queen_len_player_0_diff_diff', 'Zergling_len_player_1_Reaper_len_player_1_diff', 'Zergling_len_player_1_Roach_len_player_1_diff', 'Zergling_len_player_1_Roach_len_player_1_diff_Zergling_len_player_0_Roach_len_player_0_diff_diff', 'Zergling_len_player_1_Starport_len_player_1_diff', 'Zergling_len_player_1_Starport_len_player_1_diff_Zergling_len_player_0_Starport_len_player_0_diff_diff', 'Zergling_len_player_1_SupplyDepotLowered_len_player_1_diff', 'ability_code_1000_diff', 'ability_code_1000_player_1', 'ability_code_1020_player_0', 'ability_code_1020_player_0_ability_code_102A_player_0_diff', 'ability_code_1020_player_0_ability_code_1E60_player_0_diff', 'ability_code_1020_player_1', 'ability_code_1020_player_1_ability_code_102A_player_1_diff', 'ability_code_1020_player_1_ability_code_1261_player_1_diff', 'ability_code_1020_player_1_ability_code_1E60_player_1_diff', 'ability_code_1020_player_1_ability_code_1E60_player_1_diff_ability_code_1020_player_0_ability_code_1E60_player_0_diff_diff', 'ability_code_1020_player_1_ability_code_480_player_1_diff', 'ability_code_1021_first_time_diff', 'ability_code_1021_first_time_player_1', 'ability_code_1021_last_first_player_1', 'ability_code_1021_player_0', 'ability_code_1021_player_0_ability_code_1022_player_0_diff', 'ability_code_1021_player_0_ability_code_1023_player_0_diff', 'ability_code_1021_player_0_ability_code_1261_player_0_diff', 'ability_code_1021_player_0_ability_code_15E0_player_0_diff', 'ability_code_1021_player_0_ability_code_16E2_player_0_diff', 'ability_code_1021_player_0_ability_code_1822_player_0_diff', 'ability_code_1021_player_0_ability_code_1E60_player_0_diff', 'ability_code_1021_player_0_ability_code_480_player_0_diff', 'ability_code_1021_player_0_ability_code_B40_player_0_diff', 'ability_code_1021_player_0_ability_code_DE0_player_0_diff', 'ability_code_1021_player_1_ability_code_1020_player_1_diff_ability_code_1021_player_0_ability_code_1020_player_0_diff_diff', 'ability_code_1021_player_1_ability_code_1022_player_1_diff', 'ability_code_1021_player_1_ability_code_1022_player_1_diff_ability_code_1021_player_0_ability_code_1022_player_0_diff_diff', 'ability_code_1021_player_1_ability_code_1023_player_1_diff_ability_code_1021_player_0_ability_code_1023_player_0_diff_diff', 'ability_code_1021_player_1_ability_code_102A_player_1_diff_ability_code_1021_player_0_ability_code_102A_player_0_diff_diff', 'ability_code_1021_player_1_ability_code_1261_player_1_diff', 'ability_code_1021_player_1_ability_code_1261_player_1_diff_ability_code_1021_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_1021_player_1_ability_code_13A0_player_1_diff', 'ability_code_1021_player_1_ability_code_13A0_player_1_diff_ability_code_1021_player_0_ability_code_13A0_player_0_diff_diff', 'ability_code_1021_player_1_ability_code_15E0_player_1_diff', 'ability_code_1021_player_1_ability_code_16E2_player_1_diff', 'ability_code_1021_player_1_ability_code_16E2_player_1_diff_ability_code_1021_player_0_ability_code_16E2_player_0_diff_diff', 'ability_code_1021_player_1_ability_code_1821_player_1_diff_ability_code_1021_player_0_ability_code_1821_player_0_diff_diff', 'ability_code_1021_player_1_ability_code_1822_player_1_diff_ability_code_1021_player_0_ability_code_1822_player_0_diff_diff', 'ability_code_1021_player_1_ability_code_1C60_player_1_diff', 'ability_code_1021_player_1_ability_code_1C60_player_1_diff_ability_code_1021_player_0_ability_code_1C60_player_0_diff_diff', 'ability_code_1021_player_1_ability_code_480_player_1_diff', 'ability_code_1021_player_1_ability_code_B40_player_1_diff', 'ability_code_1021_player_1_ability_code_B40_player_1_diff_ability_code_1021_player_0_ability_code_B40_player_0_diff_diff', 'ability_code_1021_player_1_ability_code_DE0_player_1_diff', 'ability_code_1022_diff', 'ability_code_1022_first_time_player_1', 'ability_code_1022_last_first_diff', 'ability_code_1022_player_0', 'ability_code_1022_player_0_ability_code_1020_player_0_diff', 'ability_code_1022_player_0_ability_code_15E0_player_0_diff', 'ability_code_1022_player_0_ability_code_16E2_player_0_diff', 'ability_code_1022_player_0_ability_code_1821_player_0_diff', 'ability_code_1022_player_0_ability_code_1822_player_0_diff', 'ability_code_1022_player_0_ability_code_1C60_player_0_diff', 'ability_code_1022_player_0_ability_code_1E60_player_0_diff', 'ability_code_1022_player_1', 'ability_code_1022_player_1_ability_code_1020_player_1_diff', 'ability_code_1022_player_1_ability_code_1020_player_1_diff_ability_code_1022_player_0_ability_code_1020_player_0_diff_diff', 'ability_code_1022_player_1_ability_code_1023_player_1_diff', 'ability_code_1022_player_1_ability_code_1023_player_1_diff_ability_code_1022_player_0_ability_code_1023_player_0_diff_diff', 'ability_code_1022_player_1_ability_code_102A_player_1_diff_ability_code_1022_player_0_ability_code_102A_player_0_diff_diff', 'ability_code_1022_player_1_ability_code_1261_player_1_diff', 'ability_code_1022_player_1_ability_code_1261_player_1_diff_ability_code_1022_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_1022_player_1_ability_code_13A0_player_1_diff_ability_code_1022_player_0_ability_code_13A0_player_0_diff_diff', 'ability_code_1022_player_1_ability_code_16E2_player_1_diff', 'ability_code_1022_player_1_ability_code_16E2_player_1_diff_ability_code_1022_player_0_ability_code_16E2_player_0_diff_diff', 'ability_code_1022_player_1_ability_code_1822_player_1_diff', 'ability_code_1022_player_1_ability_code_1822_player_1_diff_ability_code_1022_player_0_ability_code_1822_player_0_diff_diff', 'ability_code_1022_player_1_ability_code_1C60_player_1_diff', 'ability_code_1022_player_1_ability_code_1C60_player_1_diff_ability_code_1022_player_0_ability_code_1C60_player_0_diff_diff', 'ability_code_1022_player_1_ability_code_1E60_player_1_diff', 'ability_code_1022_player_1_ability_code_1E60_player_1_diff_ability_code_1022_player_0_ability_code_1E60_player_0_diff_diff', 'ability_code_1022_player_1_ability_code_480_player_1_diff_ability_code_1022_player_0_ability_code_480_player_0_diff_diff', 'ability_code_1023_first_time_player_0', 'ability_code_1023_first_time_player_1', 'ability_code_1023_last_first_diff', 'ability_code_1023_last_first_player_1', 'ability_code_1023_player_0', 'ability_code_1023_player_0_ability_code_102A_player_0_diff', 'ability_code_1023_player_0_ability_code_1261_player_0_diff', 'ability_code_1023_player_0_ability_code_15E0_player_0_diff', 'ability_code_1023_player_0_ability_code_16E2_player_0_diff', 'ability_code_1023_player_0_ability_code_1821_player_0_diff', 'ability_code_1023_player_0_ability_code_1822_player_0_diff', 'ability_code_1023_player_0_ability_code_1E60_player_0_diff', 'ability_code_1023_player_0_ability_code_480_player_0_diff', 'ability_code_1023_player_1', 'ability_code_1023_player_1_ability_code_1020_player_1_diff', 'ability_code_1023_player_1_ability_code_102A_player_1_diff', 'ability_code_1023_player_1_ability_code_102A_player_1_diff_ability_code_1023_player_0_ability_code_102A_player_0_diff_diff', 'ability_code_1023_player_1_ability_code_1261_player_1_diff', 'ability_code_1023_player_1_ability_code_1261_player_1_diff_ability_code_1023_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_1023_player_1_ability_code_13A0_player_1_diff', 'ability_code_1023_player_1_ability_code_15E0_player_1_diff_ability_code_1023_player_0_ability_code_15E0_player_0_diff_diff', 'ability_code_1023_player_1_ability_code_16E2_player_1_diff', 'ability_code_1023_player_1_ability_code_1821_player_1_diff', 'ability_code_1023_player_1_ability_code_1822_player_1_diff', 'ability_code_1023_player_1_ability_code_1C60_player_1_diff', 'ability_code_1023_player_1_ability_code_1C60_player_1_diff_ability_code_1023_player_0_ability_code_1C60_player_0_diff_diff', 'ability_code_1023_player_1_ability_code_1E60_player_1_diff', 'ability_code_1023_player_1_ability_code_1E60_player_1_diff_ability_code_1023_player_0_ability_code_1E60_player_0_diff_diff', 'ability_code_1023_player_1_ability_code_480_player_1_diff_ability_code_1023_player_0_ability_code_480_player_0_diff_diff', 'ability_code_1024_diff', 'ability_code_1024_player_0', 'ability_code_1024_player_1', 'ability_code_1025_diff', 'ability_code_1025_first_time_diff', 'ability_code_1026_diff', 'ability_code_1026_first_time_diff', 'ability_code_1026_first_time_player_0', 'ability_code_1026_player_0', 'ability_code_1026_player_1', 'ability_code_1028_diff', 'ability_code_1028_player_0', 'ability_code_1028_player_1', 'ability_code_1029_diff', 'ability_code_1029_player_0', 'ability_code_1029_player_1', 'ability_code_102A_diff', 'ability_code_102A_first_time_player_1', 'ability_code_102A_player_0', 'ability_code_102A_player_0_ability_code_1261_player_0_diff', 'ability_code_102A_player_0_ability_code_480_player_0_diff', 'ability_code_102A_player_1', 'ability_code_102A_player_1_ability_code_1261_player_1_diff', 'ability_code_102A_player_1_ability_code_1261_player_1_diff_ability_code_102A_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_102A_player_1_ability_code_480_player_1_diff', 'ability_code_102A_player_1_ability_code_480_player_1_diff_ability_code_102A_player_0_ability_code_480_player_0_diff_diff', 'ability_code_102B_first_time_diff', 'ability_code_102B_first_time_player_0', 'ability_code_102B_first_time_player_1', 'ability_code_102B_player_0', 'ability_code_102B_player_1', 'ability_code_102D_diff', 'ability_code_102D_first_time_diff', 'ability_code_102D_first_time_player_1', 'ability_code_102D_player_0', 'ability_code_102D_player_1', 'ability_code_102F_diff', 'ability_code_102F_player_0', 'ability_code_102F_player_1', 'ability_code_103E_diff', 'ability_code_103E_player_0', 'ability_code_103E_player_1', 'ability_code_1060_diff', 'ability_code_1060_player_1', 'ability_code_1080_diff', 'ability_code_1080_player_0', 'ability_code_1080_player_1', 'ability_code_1081_diff', 'ability_code_1081_player_0', 'ability_code_1081_player_1', 'ability_code_10E0_first_time_diff', 'ability_code_10E0_player_0', 'ability_code_10E0_player_1', 'ability_code_1100_diff', 'ability_code_1100_player_0', 'ability_code_1100_player_1', 'ability_code_1120_player_0', 'ability_code_1120_player_1', 'ability_code_1121_diff', 'ability_code_1121_player_0', 'ability_code_1121_player_1', 'ability_code_1140_diff', 'ability_code_1140_player_0', 'ability_code_1140_player_1', 'ability_code_1142_diff', 'ability_code_1142_first_time_diff', 'ability_code_1142_first_time_player_0', 'ability_code_1142_first_time_player_1', 'ability_code_1142_player_0', 'ability_code_1142_player_1', 'ability_code_1143_diff', 'ability_code_1143_player_0', 'ability_code_1143_player_1', 'ability_code_1160_diff', 'ability_code_1160_first_time_diff', 'ability_code_1160_first_time_player_1', 'ability_code_1160_player_0', 'ability_code_1160_player_1', 'ability_code_1180_diff', 'ability_code_1180_player_0', 'ability_code_1180_player_1', 'ability_code_11A0_diff', 'ability_code_11A0_player_0', 'ability_code_11A0_player_1', 'ability_code_11C0_diff', 'ability_code_11C0_player_0', 'ability_code_11C0_player_1', 'ability_code_11E1_diff', 'ability_code_11E1_player_0', 'ability_code_11E1_player_1', 'ability_code_11E3_diff', 'ability_code_11E3_player_0', 'ability_code_11E3_player_1', 'ability_code_1201_diff', 'ability_code_1201_player_0', 'ability_code_1201_player_1', 'ability_code_1203_diff', 'ability_code_1203_player_0', 'ability_code_1203_player_1', 'ability_code_1204_diff', 'ability_code_1204_player_0', 'ability_code_1204_player_1', 'ability_code_1220_diff', 'ability_code_1220_player_0', 'ability_code_1220_player_1', 'ability_code_1240_diff', 'ability_code_1240_player_0', 'ability_code_1240_player_1', 'ability_code_1260_diff', 'ability_code_1260_first_time_diff', 'ability_code_1260_player_0', 'ability_code_1260_player_1', 'ability_code_1261_first_time_diff', 'ability_code_1261_first_time_player_1', 'ability_code_1261_player_0', 'ability_code_1261_player_1', 'ability_code_1261_player_1_ability_code_480_player_1_diff', 'ability_code_127E_diff', 'ability_code_127E_player_0', 'ability_code_127E_player_1', 'ability_code_1280_diff', 'ability_code_1280_first_time_diff', 'ability_code_1280_first_time_player_0', 'ability_code_1280_first_time_player_1', 'ability_code_1280_player_0', 'ability_code_1280_player_1', 'ability_code_12A0_first_time_diff', 'ability_code_12A0_player_1', 'ability_code_12A1_diff', 'ability_code_12A1_first_time_diff', 'ability_code_12A1_first_time_player_0', 'ability_code_12A1_first_time_player_1', 'ability_code_12A1_player_0', 'ability_code_12A1_player_1', 'ability_code_12BE_diff', 'ability_code_12BE_player_0', 'ability_code_12BE_player_1', 'ability_code_12C0_diff', 'ability_code_12C0_first_time_player_1', 'ability_code_12C0_player_1', 'ability_code_12E0_diff', 'ability_code_12E0_player_0', 'ability_code_12E0_player_1', 'ability_code_12E1_diff', 'ability_code_12E1_first_time_diff', 'ability_code_12E1_first_time_player_0', 'ability_code_12E1_first_time_player_1', 'ability_code_12E1_player_0', 'ability_code_12E1_player_1', 'ability_code_12FE_player_1', 'ability_code_1300_diff', 'ability_code_1300_first_time_diff', 'ability_code_1300_first_time_player_1', 'ability_code_1300_player_0', 'ability_code_1300_player_1', 'ability_code_1320_diff', 'ability_code_1320_first_time_diff', 'ability_code_1320_first_time_player_1', 'ability_code_1320_player_1', 'ability_code_1340_diff', 'ability_code_1340_first_time_diff', 'ability_code_1340_first_time_player_0', 'ability_code_1340_first_time_player_1', 'ability_code_1340_player_1', 'ability_code_1360_diff', 'ability_code_1360_first_time_player_1', 'ability_code_1360_last_first_diff', 'ability_code_1360_player_0_ability_code_1020_player_0_diff', 'ability_code_1360_player_0_ability_code_1022_player_0_diff', 'ability_code_1360_player_0_ability_code_102A_player_0_diff', 'ability_code_1360_player_0_ability_code_15E0_player_0_diff', 'ability_code_1360_player_0_ability_code_16E2_player_0_diff', 'ability_code_1360_player_0_ability_code_1820_player_0_diff', 'ability_code_1360_player_0_ability_code_1821_player_0_diff', 'ability_code_1360_player_0_ability_code_1822_player_0_diff', 'ability_code_1360_player_0_ability_code_1C60_player_0_diff', 'ability_code_1360_player_0_ability_code_1E60_player_0_diff', 'ability_code_1360_player_0_ability_code_480_player_0_diff', 'ability_code_1360_player_0_ability_code_DE0_player_0_diff', 'ability_code_1360_player_1', 'ability_code_1360_player_1_ability_code_1020_player_1_diff', 'ability_code_1360_player_1_ability_code_1020_player_1_diff_ability_code_1360_player_0_ability_code_1020_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_1021_player_1_diff', 'ability_code_1360_player_1_ability_code_1022_player_1_diff', 'ability_code_1360_player_1_ability_code_102A_player_1_diff', 'ability_code_1360_player_1_ability_code_102A_player_1_diff_ability_code_1360_player_0_ability_code_102A_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_1261_player_1_diff', 'ability_code_1360_player_1_ability_code_1261_player_1_diff_ability_code_1360_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_13E0_player_1_diff', 'ability_code_1360_player_1_ability_code_13E0_player_1_diff_ability_code_1360_player_0_ability_code_13E0_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_15E0_player_1_diff', 'ability_code_1360_player_1_ability_code_15E0_player_1_diff_ability_code_1360_player_0_ability_code_15E0_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_16E2_player_1_diff', 'ability_code_1360_player_1_ability_code_16E2_player_1_diff_ability_code_1360_player_0_ability_code_16E2_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_1820_player_1_diff', 'ability_code_1360_player_1_ability_code_1820_player_1_diff_ability_code_1360_player_0_ability_code_1820_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_1821_player_1_diff', 'ability_code_1360_player_1_ability_code_1821_player_1_diff_ability_code_1360_player_0_ability_code_1821_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_1822_player_1_diff', 'ability_code_1360_player_1_ability_code_1822_player_1_diff_ability_code_1360_player_0_ability_code_1822_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_1C60_player_1_diff', 'ability_code_1360_player_1_ability_code_1E60_player_1_diff', 'ability_code_1360_player_1_ability_code_1E60_player_1_diff_ability_code_1360_player_0_ability_code_1E60_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_480_player_1_diff_ability_code_1360_player_0_ability_code_480_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_B40_player_1_diff_ability_code_1360_player_0_ability_code_B40_player_0_diff_diff', 'ability_code_1380_diff', 'ability_code_1380_first_time_diff', 'ability_code_1380_player_0', 'ability_code_1380_player_1', 'ability_code_13A0_diff', 'ability_code_13A0_first_time_diff', 'ability_code_13A0_first_time_player_1', 'ability_code_13A0_player_0_ability_code_102A_player_0_diff', 'ability_code_13A0_player_0_ability_code_1261_player_0_diff', 'ability_code_13A0_player_0_ability_code_16E2_player_0_diff', 'ability_code_13A0_player_0_ability_code_1821_player_0_diff', 'ability_code_13A0_player_0_ability_code_1E60_player_0_diff', 'ability_code_13A0_player_0_ability_code_480_player_0_diff', 'ability_code_13A0_player_1', 'ability_code_13A0_player_1_ability_code_1020_player_1_diff', 'ability_code_13A0_player_1_ability_code_1020_player_1_diff_ability_code_13A0_player_0_ability_code_1020_player_0_diff_diff', 'ability_code_13A0_player_1_ability_code_102A_player_1_diff', 'ability_code_13A0_player_1_ability_code_102A_player_1_diff_ability_code_13A0_player_0_ability_code_102A_player_0_diff_diff', 'ability_code_13A0_player_1_ability_code_1261_player_1_diff', 'ability_code_13A0_player_1_ability_code_1261_player_1_diff_ability_code_13A0_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_13A0_player_1_ability_code_15E0_player_1_diff', 'ability_code_13A0_player_1_ability_code_15E0_player_1_diff_ability_code_13A0_player_0_ability_code_15E0_player_0_diff_diff', 'ability_code_13A0_player_1_ability_code_16E2_player_1_diff', 'ability_code_13A0_player_1_ability_code_16E2_player_1_diff_ability_code_13A0_player_0_ability_code_16E2_player_0_diff_diff', 'ability_code_13A0_player_1_ability_code_1821_player_1_diff', 'ability_code_13A0_player_1_ability_code_1821_player_1_diff_ability_code_13A0_player_0_ability_code_1821_player_0_diff_diff', 'ability_code_13A0_player_1_ability_code_1C60_player_1_diff_ability_code_13A0_player_0_ability_code_1C60_player_0_diff_diff', 'ability_code_13A0_player_1_ability_code_1E60_player_1_diff', 'ability_code_13A0_player_1_ability_code_1E60_player_1_diff_ability_code_13A0_player_0_ability_code_1E60_player_0_diff_diff', 'ability_code_13A0_player_1_ability_code_480_player_1_diff', 'ability_code_13A0_player_1_ability_code_480_player_1_diff_ability_code_13A0_player_0_ability_code_480_player_0_diff_diff', 'ability_code_13C0_diff', 'ability_code_13C0_first_time_player_0', 'ability_code_13C0_player_0', 'ability_code_13C0_player_1', 'ability_code_13E0_first_time_player_0', 'ability_code_13E0_player_0_ability_code_1021_player_0_diff', 'ability_code_13E0_player_0_ability_code_1022_player_0_diff', 'ability_code_13E0_player_0_ability_code_1023_player_0_diff', 'ability_code_13E0_player_0_ability_code_16E2_player_0_diff', 'ability_code_13E0_player_0_ability_code_1821_player_0_diff', 'ability_code_13E0_player_0_ability_code_1822_player_0_diff', 'ability_code_13E0_player_0_ability_code_1E60_player_0_diff', 'ability_code_13E0_player_0_ability_code_480_player_0_diff', 'ability_code_13E0_player_0_ability_code_DE0_player_0_diff', 'ability_code_13E0_player_1', 'ability_code_13E0_player_1_ability_code_1020_player_1_diff', 'ability_code_13E0_player_1_ability_code_1021_player_1_diff', 'ability_code_13E0_player_1_ability_code_1022_player_1_diff_ability_code_13E0_player_0_ability_code_1022_player_0_diff_diff', 'ability_code_13E0_player_1_ability_code_1023_player_1_diff', 'ability_code_13E0_player_1_ability_code_102A_player_1_diff', 'ability_code_13E0_player_1_ability_code_1261_player_1_diff', 'ability_code_13E0_player_1_ability_code_1261_player_1_diff_ability_code_13E0_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_13E0_player_1_ability_code_15E0_player_1_diff', 'ability_code_13E0_player_1_ability_code_16E2_player_1_diff', 'ability_code_13E0_player_1_ability_code_16E2_player_1_diff_ability_code_13E0_player_0_ability_code_16E2_player_0_diff_diff', 'ability_code_13E0_player_1_ability_code_1821_player_1_diff', 'ability_code_13E0_player_1_ability_code_1821_player_1_diff_ability_code_13E0_player_0_ability_code_1821_player_0_diff_diff', 'ability_code_13E0_player_1_ability_code_1822_player_1_diff', 'ability_code_13E0_player_1_ability_code_1822_player_1_diff_ability_code_13E0_player_0_ability_code_1822_player_0_diff_diff', 'ability_code_13E0_player_1_ability_code_1C60_player_1_diff', 'ability_code_13E0_player_1_ability_code_1E60_player_1_diff', 'ability_code_13E0_player_1_ability_code_1E60_player_1_diff_ability_code_13E0_player_0_ability_code_1E60_player_0_diff_diff', 'ability_code_13E0_player_1_ability_code_480_player_1_diff', 'ability_code_13E0_player_1_ability_code_480_player_1_diff_ability_code_13E0_player_0_ability_code_480_player_0_diff_diff', 'ability_code_13E0_player_1_ability_code_B40_player_1_diff', 'ability_code_13E0_player_1_ability_code_B40_player_1_diff_ability_code_13E0_player_0_ability_code_B40_player_0_diff_diff', 'ability_code_13E0_player_1_ability_code_DE0_player_1_diff', 'ability_code_13E1_first_time_player_1', 'ability_code_13E1_player_0', 'ability_code_13E1_player_1', 'ability_code_13E2_diff', 'ability_code_13E2_player_0', 'ability_code_13E2_player_1', 'ability_code_13E3_first_time_diff', 'ability_code_13E3_first_time_player_0', 'ability_code_13E3_player_0', 'ability_code_13E3_player_1', 'ability_code_1401_player_0', 'ability_code_1401_player_1', 'ability_code_1404_diff', 'ability_code_1404_player_0', 'ability_code_1404_player_1', 'ability_code_1405_first_time_diff', 'ability_code_1405_first_time_player_1', 'ability_code_1405_player_0', 'ability_code_1405_player_1', 'ability_code_1406_diff', 'ability_code_1406_player_0', 'ability_code_1406_player_1', 'ability_code_1407_diff', 'ability_code_1407_first_time_diff', 'ability_code_1407_player_0', 'ability_code_1407_player_1', 'ability_code_1418_first_time_diff', 'ability_code_1418_first_time_player_1', 'ability_code_1418_player_1', 'ability_code_1420_diff', 'ability_code_1420_first_time_diff', 'ability_code_1420_player_0', 'ability_code_1421_player_0', 'ability_code_1421_player_1', 'ability_code_1422_player_1', 'ability_code_1423_diff', 'ability_code_1423_player_0', 'ability_code_1423_player_1', 'ability_code_1424_diff', 'ability_code_1424_first_time_diff', 'ability_code_1424_first_time_player_0', 'ability_code_1424_player_0', 'ability_code_1424_player_1', 'ability_code_1426_diff', 'ability_code_1426_player_1', 'ability_code_1440_diff', 'ability_code_1440_player_0', 'ability_code_1440_player_1', 'ability_code_1441_diff', 'ability_code_1441_player_0', 'ability_code_1441_player_1', 'ability_code_1442_diff', 'ability_code_1442_first_time_diff', 'ability_code_1442_first_time_player_1', 'ability_code_1442_player_0', 'ability_code_1442_player_1', 'ability_code_1443_diff', 'ability_code_1443_player_0', 'ability_code_1443_player_1', 'ability_code_1445_diff', 'ability_code_1445_player_0', 'ability_code_1445_player_1', 'ability_code_1446_first_time_diff', 'ability_code_1446_first_time_player_1', 'ability_code_1446_player_0', 'ability_code_1446_player_1', 'ability_code_1447_diff', 'ability_code_1447_player_0', 'ability_code_1447_player_1', 'ability_code_1480_diff', 'ability_code_1480_player_0', 'ability_code_1480_player_1', 'ability_code_14A0_diff', 'ability_code_14A0_player_0', 'ability_code_14A0_player_1', 'ability_code_14A1_player_0', 'ability_code_14A1_player_1', 'ability_code_14A2_diff', 'ability_code_14A2_player_0', 'ability_code_14A2_player_1', 'ability_code_14C1_diff', 'ability_code_14C1_player_0', 'ability_code_14C1_player_1', 'ability_code_14C4_diff', 'ability_code_14C4_player_0', 'ability_code_14C4_player_1', 'ability_code_14E0_diff', 'ability_code_14E0_player_0', 'ability_code_14E0_player_1', 'ability_code_14E3_diff', 'ability_code_14E3_player_0', 'ability_code_14E3_player_1', 'ability_code_14E9_diff', 'ability_code_14E9_player_0', 'ability_code_14E9_player_1', 'ability_code_14ED_diff', 'ability_code_14ED_player_0', 'ability_code_14ED_player_1', 'ability_code_14EE_diff', 'ability_code_14EE_player_0', 'ability_code_14EE_player_1', 'ability_code_14EF_diff', 'ability_code_14EF_player_0', 'ability_code_14EF_player_1', 'ability_code_1500_diff', 'ability_code_1500_player_0', 'ability_code_1500_player_1', 'ability_code_1525_diff', 'ability_code_1525_player_0', 'ability_code_1525_player_1', 'ability_code_1526_diff', 'ability_code_1526_player_0', 'ability_code_1526_player_1', 'ability_code_152B_diff', 'ability_code_152B_player_0', 'ability_code_152B_player_1', 'ability_code_152E_diff', 'ability_code_152E_player_0', 'ability_code_152E_player_1', 'ability_code_152F_diff', 'ability_code_152F_player_0', 'ability_code_152F_player_1', 'ability_code_1540_first_time_diff', 'ability_code_1540_player_0', 'ability_code_1541_first_time_player_1', 'ability_code_1541_player_0', 'ability_code_1541_player_1', 'ability_code_1542_first_time_diff', 'ability_code_1542_first_time_player_0', 'ability_code_1542_first_time_player_1', 'ability_code_1542_player_1', 'ability_code_1543_first_time_diff', 'ability_code_1544_first_time_diff', 'ability_code_1544_player_0', 'ability_code_1544_player_1', 'ability_code_1545_player_0', 'ability_code_1546_player_0', 'ability_code_1546_player_1', 'ability_code_1547_first_time_diff', 'ability_code_1547_first_time_player_0', 'ability_code_1547_first_time_player_1', 'ability_code_1547_player_0', 'ability_code_1547_player_1', 'ability_code_1549_first_time_diff', 'ability_code_1549_player_0', 'ability_code_1549_player_1', 'ability_code_154A_player_0', 'ability_code_154B_player_0', 'ability_code_154C_player_0', 'ability_code_154C_player_1', 'ability_code_154D_first_time_diff', 'ability_code_154D_first_time_player_1', 'ability_code_154D_player_0', 'ability_code_154E_diff', 'ability_code_154E_player_0', 'ability_code_154E_player_1', 'ability_code_1562_diff', 'ability_code_1562_player_0', 'ability_code_1563_diff', 'ability_code_1563_player_0', 'ability_code_1563_player_1', 'ability_code_1580_first_time_diff', 'ability_code_1580_first_time_player_0', 'ability_code_1580_player_0', 'ability_code_1581_diff', 'ability_code_1581_first_time_diff', 'ability_code_1581_first_time_player_1', 'ability_code_1581_player_0', 'ability_code_1581_player_1', 'ability_code_1583_diff', 'ability_code_1583_player_1', 'ability_code_1584_diff', 'ability_code_1584_player_0', 'ability_code_1584_player_1', 'ability_code_1585_diff', 'ability_code_1585_player_0', 'ability_code_1585_player_1', 'ability_code_1586_first_time_diff', 'ability_code_1586_first_time_player_0', 'ability_code_1586_player_0', 'ability_code_15A0_player_0', 'ability_code_15A0_player_1', 'ability_code_15A2_diff', 'ability_code_15A2_player_0', 'ability_code_15A2_player_1', 'ability_code_15A4_player_0', 'ability_code_15A8_diff', 'ability_code_15A8_first_time_diff', 'ability_code_15A8_first_time_player_1', 'ability_code_15A8_player_0', 'ability_code_15A8_player_1', 'ability_code_15A9_diff', 'ability_code_15A9_player_0', 'ability_code_15A9_player_1', 'ability_code_15C0_diff', 'ability_code_15C0_player_0', 'ability_code_15C0_player_1', 'ability_code_15C1_diff', 'ability_code_15C1_first_time_diff', 'ability_code_15C1_first_time_player_0', 'ability_code_15C1_player_0', 'ability_code_15C1_player_1', 'ability_code_15C2_player_0', 'ability_code_15C2_player_1', 'ability_code_15C3_diff', 'ability_code_15C3_first_time_diff', 'ability_code_15C3_player_1', 'ability_code_15D2_diff', 'ability_code_15D2_player_0', 'ability_code_15D2_player_1', 'ability_code_15E0_diff', 'ability_code_15E0_first_time_diff', 'ability_code_15E0_player_0', 'ability_code_15E0_player_0_ability_code_1020_player_0_diff', 'ability_code_15E0_player_0_ability_code_102A_player_0_diff', 'ability_code_15E0_player_0_ability_code_1261_player_0_diff', 'ability_code_15E0_player_0_ability_code_16E2_player_0_diff', 'ability_code_15E0_player_0_ability_code_1C60_player_0_diff', 'ability_code_15E0_player_0_ability_code_1E60_player_0_diff', 'ability_code_15E0_player_1', 'ability_code_15E0_player_1_ability_code_1020_player_1_diff', 'ability_code_15E0_player_1_ability_code_102A_player_1_diff', 'ability_code_15E0_player_1_ability_code_102A_player_1_diff_ability_code_15E0_player_0_ability_code_102A_player_0_diff_diff', 'ability_code_15E0_player_1_ability_code_1261_player_1_diff', 'ability_code_15E0_player_1_ability_code_16E2_player_1_diff', 'ability_code_15E0_player_1_ability_code_1C60_player_1_diff', 'ability_code_15E0_player_1_ability_code_1E60_player_1_diff', 'ability_code_15E0_player_1_ability_code_1E60_player_1_diff_ability_code_15E0_player_0_ability_code_1E60_player_0_diff_diff', 'ability_code_15E0_player_1_ability_code_480_player_1_diff', 'ability_code_15E0_player_1_ability_code_480_player_1_diff_ability_code_15E0_player_0_ability_code_480_player_0_diff_diff', 'ability_code_1600_diff', 'ability_code_1600_player_1', 'ability_code_1680_diff', 'ability_code_1680_player_0', 'ability_code_1680_player_1', 'ability_code_1681_diff', 'ability_code_1681_player_0', 'ability_code_1681_player_1', 'ability_code_1684_diff', 'ability_code_1684_player_0', 'ability_code_1684_player_1', 'ability_code_1686_diff', 'ability_code_1686_player_0', 'ability_code_1686_player_1', 'ability_code_1687_diff', 'ability_code_1687_player_0', 'ability_code_1687_player_1', 'ability_code_16A1_diff', 'ability_code_16A1_player_0', 'ability_code_16A1_player_1', 'ability_code_16A2_diff', 'ability_code_16A2_player_0', 'ability_code_16A2_player_1', 'ability_code_16A5_diff', 'ability_code_16A5_player_0', 'ability_code_16A5_player_1', 'ability_code_16C4_diff', 'ability_code_16C4_player_0', 'ability_code_16C4_player_1', 'ability_code_16E0_diff', 'ability_code_16E0_first_time_diff', 'ability_code_16E0_player_0', 'ability_code_16E0_player_1', 'ability_code_16E2_diff', 'ability_code_16E2_player_0', 'ability_code_16E2_player_0_ability_code_102A_player_0_diff', 'ability_code_16E2_player_0_ability_code_1261_player_0_diff', 'ability_code_16E2_player_0_ability_code_480_player_0_diff', 'ability_code_16E2_player_1', 'ability_code_16E2_player_1_ability_code_102A_player_1_diff', 'ability_code_16E2_player_1_ability_code_1261_player_1_diff', 'ability_code_16E2_player_1_ability_code_1261_player_1_diff_ability_code_16E2_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_16E2_player_1_ability_code_480_player_1_diff_ability_code_16E2_player_0_ability_code_480_player_0_diff_diff', 'ability_code_16E3_diff', 'ability_code_16E3_first_time_diff', 'ability_code_16E3_player_0', 'ability_code_16E3_player_1', 'ability_code_16E4_diff', 'ability_code_16E4_first_time_diff', 'ability_code_16E4_player_1', 'ability_code_16E5_diff', 'ability_code_16E5_player_1', 'ability_code_16E6_diff', 'ability_code_16E6_player_0', 'ability_code_16E6_player_1', 'ability_code_16E7_diff', 'ability_code_16E7_player_0', 'ability_code_16E7_player_1', 'ability_code_16E8_diff', 'ability_code_16E8_player_0', 'ability_code_16E8_player_1', 'ability_code_16E9_diff', 'ability_code_16E9_player_0', 'ability_code_16E9_player_1', 'ability_code_16EA_player_0', 'ability_code_16EA_player_1', 'ability_code_16ED_diff', 'ability_code_16ED_first_time_diff', 'ability_code_16ED_player_1', 'ability_code_16EE_first_time_diff', 'ability_code_16EE_player_1', 'ability_code_16EF_diff', 'ability_code_16EF_first_time_diff', 'ability_code_16EF_first_time_player_1', 'ability_code_16EF_player_0', 'ability_code_16EF_player_1', 'ability_code_1700_diff', 'ability_code_1700_player_0', 'ability_code_1700_player_1', 'ability_code_1701_player_0', 'ability_code_1720_player_0', 'ability_code_1721_diff', 'ability_code_1721_player_0', 'ability_code_1723_first_time_diff', 'ability_code_1723_player_0', 'ability_code_1723_player_1', 'ability_code_1724_diff', 'ability_code_1724_player_0', 'ability_code_1724_player_1', 'ability_code_1726_player_0', 'ability_code_1727_diff', 'ability_code_1727_player_0', 'ability_code_1727_player_1', 'ability_code_1740_first_time_diff', 'ability_code_1740_player_0', 'ability_code_1740_player_1', 'ability_code_1741_diff', 'ability_code_1741_player_0', 'ability_code_1741_player_1', 'ability_code_1760_player_1', 'ability_code_17A1_diff', 'ability_code_17A1_player_0', 'ability_code_17A1_player_1', 'ability_code_17A3_player_1', 'ability_code_17C1_player_0', 'ability_code_17C1_player_1', 'ability_code_17E1_player_0', 'ability_code_17E1_player_1', 'ability_code_1800_diff', 'ability_code_1800_player_0', 'ability_code_1800_player_1', 'ability_code_1803_diff', 'ability_code_1803_player_0', 'ability_code_1803_player_1', 'ability_code_1820_first_time_diff', 'ability_code_1820_first_time_player_0', 'ability_code_1820_first_time_player_1', 'ability_code_1820_last_first_player_1', 'ability_code_1820_player_0', 'ability_code_1820_player_0_ability_code_1020_player_0_diff', 'ability_code_1820_player_0_ability_code_1022_player_0_diff', 'ability_code_1820_player_0_ability_code_1023_player_0_diff', 'ability_code_1820_player_0_ability_code_102A_player_0_diff', 'ability_code_1820_player_0_ability_code_1261_player_0_diff', 'ability_code_1820_player_0_ability_code_13A0_player_0_diff', 'ability_code_1820_player_0_ability_code_15E0_player_0_diff', 'ability_code_1820_player_0_ability_code_16E2_player_0_diff', 'ability_code_1820_player_0_ability_code_1822_player_0_diff', 'ability_code_1820_player_0_ability_code_1C60_player_0_diff', 'ability_code_1820_player_0_ability_code_B40_player_0_diff', 'ability_code_1820_player_1', 'ability_code_1820_player_1_ability_code_1020_player_1_diff', 'ability_code_1820_player_1_ability_code_1020_player_1_diff_ability_code_1820_player_0_ability_code_1020_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_1021_player_1_diff', 'ability_code_1820_player_1_ability_code_1021_player_1_diff_ability_code_1820_player_0_ability_code_1021_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_1022_player_1_diff', 'ability_code_1820_player_1_ability_code_1023_player_1_diff', 'ability_code_1820_player_1_ability_code_102A_player_1_diff', 'ability_code_1820_player_1_ability_code_1261_player_1_diff', 'ability_code_1820_player_1_ability_code_13A0_player_1_diff', 'ability_code_1820_player_1_ability_code_13A0_player_1_diff_ability_code_1820_player_0_ability_code_13A0_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_13E0_player_1_diff', 'ability_code_1820_player_1_ability_code_15E0_player_1_diff', 'ability_code_1820_player_1_ability_code_15E0_player_1_diff_ability_code_1820_player_0_ability_code_15E0_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_16E2_player_1_diff_ability_code_1820_player_0_ability_code_16E2_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_1821_player_1_diff', 'ability_code_1820_player_1_ability_code_1822_player_1_diff_ability_code_1820_player_0_ability_code_1822_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_1C60_player_1_diff', 'ability_code_1820_player_1_ability_code_1C60_player_1_diff_ability_code_1820_player_0_ability_code_1C60_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_1E60_player_1_diff_ability_code_1820_player_0_ability_code_1E60_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_480_player_1_diff', 'ability_code_1820_player_1_ability_code_B40_player_1_diff', 'ability_code_1820_player_1_ability_code_DE0_player_1_diff_ability_code_1820_player_0_ability_code_DE0_player_0_diff_diff', 'ability_code_1821_player_0', 'ability_code_1821_player_0_ability_code_1020_player_0_diff', 'ability_code_1821_player_0_ability_code_102A_player_0_diff', 'ability_code_1821_player_0_ability_code_1261_player_0_diff', 'ability_code_1821_player_0_ability_code_15E0_player_0_diff', 'ability_code_1821_player_0_ability_code_1C60_player_0_diff', 'ability_code_1821_player_0_ability_code_1E60_player_0_diff', 'ability_code_1821_player_1', 'ability_code_1821_player_1_ability_code_1020_player_1_diff', 'ability_code_1821_player_1_ability_code_102A_player_1_diff', 'ability_code_1821_player_1_ability_code_1261_player_1_diff', 'ability_code_1821_player_1_ability_code_1261_player_1_diff_ability_code_1821_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_1821_player_1_ability_code_15E0_player_1_diff', 'ability_code_1821_player_1_ability_code_16E2_player_1_diff', 'ability_code_1821_player_1_ability_code_16E2_player_1_diff_ability_code_1821_player_0_ability_code_16E2_player_0_diff_diff', 'ability_code_1821_player_1_ability_code_1C60_player_1_diff', 'ability_code_1821_player_1_ability_code_1C60_player_1_diff_ability_code_1821_player_0_ability_code_1C60_player_0_diff_diff', 'ability_code_1821_player_1_ability_code_1E60_player_1_diff', 'ability_code_1821_player_1_ability_code_480_player_1_diff', 'ability_code_1822_first_time_player_1', 'ability_code_1822_last_first_player_0', 'ability_code_1822_player_0', 'ability_code_1822_player_0_ability_code_1020_player_0_diff', 'ability_code_1822_player_0_ability_code_102A_player_0_diff', 'ability_code_1822_player_0_ability_code_1261_player_0_diff', 'ability_code_1822_player_0_ability_code_13A0_player_0_diff', 'ability_code_1822_player_0_ability_code_15E0_player_0_diff', 'ability_code_1822_player_0_ability_code_1821_player_0_diff', 'ability_code_1822_player_0_ability_code_1C60_player_0_diff', 'ability_code_1822_player_0_ability_code_1E60_player_0_diff', 'ability_code_1822_player_0_ability_code_480_player_0_diff', 'ability_code_1822_player_1', 'ability_code_1822_player_1_ability_code_1020_player_1_diff_ability_code_1822_player_0_ability_code_1020_player_0_diff_diff', 'ability_code_1822_player_1_ability_code_102A_player_1_diff', 'ability_code_1822_player_1_ability_code_1261_player_1_diff', 'ability_code_1822_player_1_ability_code_13A0_player_1_diff', 'ability_code_1822_player_1_ability_code_13A0_player_1_diff_ability_code_1822_player_0_ability_code_13A0_player_0_diff_diff', 'ability_code_1822_player_1_ability_code_15E0_player_1_diff', 'ability_code_1822_player_1_ability_code_16E2_player_1_diff', 'ability_code_1822_player_1_ability_code_16E2_player_1_diff_ability_code_1822_player_0_ability_code_16E2_player_0_diff_diff', 'ability_code_1822_player_1_ability_code_1C60_player_1_diff', 'ability_code_1823_player_0', 'ability_code_1823_player_1', 'ability_code_1824_diff', 'ability_code_1824_player_0', 'ability_code_1824_player_1', 'ability_code_1829_player_0', 'ability_code_182A_diff', 'ability_code_182A_player_0', 'ability_code_182A_player_1', 'ability_code_182B_diff', 'ability_code_182B_player_0', 'ability_code_182B_player_1', 'ability_code_182E_diff', 'ability_code_182E_player_0', 'ability_code_182E_player_1', 'ability_code_1860_diff', 'ability_code_1860_player_0', 'ability_code_1860_player_1', 'ability_code_1880_diff', 'ability_code_1880_player_0', 'ability_code_1880_player_1', 'ability_code_18A0_diff', 'ability_code_18A0_player_0', 'ability_code_18A0_player_1', 'ability_code_18C0_diff', 'ability_code_18C0_player_0', 'ability_code_18C0_player_1', 'ability_code_1920_diff', 'ability_code_1920_player_0', 'ability_code_1920_player_1', 'ability_code_1940_diff', 'ability_code_1940_player_0', 'ability_code_1940_player_1', 'ability_code_1960_diff', 'ability_code_1960_player_0', 'ability_code_1960_player_1', 'ability_code_1980_diff', 'ability_code_1980_player_0', 'ability_code_1980_player_1', 'ability_code_1A62_diff', 'ability_code_1A62_player_0', 'ability_code_1A62_player_1', 'ability_code_1A63_diff', 'ability_code_1A63_player_0', 'ability_code_1A63_player_1', 'ability_code_1AC0_diff', 'ability_code_1AC0_player_0', 'ability_code_1AC0_player_1', 'ability_code_1AC1_diff', 'ability_code_1AC1_player_0', 'ability_code_1AC1_player_1', 'ability_code_1AC3_diff', 'ability_code_1AC3_player_0', 'ability_code_1AC3_player_1', 'ability_code_1AC4_diff', 'ability_code_1AC4_player_0', 'ability_code_1AC4_player_1', 'ability_code_1AC5_player_0', 'ability_code_1AC5_player_1', 'ability_code_1AC6_diff', 'ability_code_1AC6_first_time_diff', 'ability_code_1AC6_first_time_player_0', 'ability_code_1AC6_first_time_player_1', 'ability_code_1AC6_player_0', 'ability_code_1AE0_diff', 'ability_code_1AE0_player_0', 'ability_code_1AE0_player_1', 'ability_code_1B00_diff', 'ability_code_1B00_player_0', 'ability_code_1B00_player_1', 'ability_code_1B21_diff', 'ability_code_1B21_player_0', 'ability_code_1B21_player_1', 'ability_code_1B40_diff', 'ability_code_1B40_player_0', 'ability_code_1B40_player_1', 'ability_code_1B60_diff', 'ability_code_1B60_player_1', 'ability_code_1BA0_player_0', 'ability_code_1BA0_player_1', 'ability_code_1BA1_diff', 'ability_code_1BA1_player_0', 'ability_code_1BA1_player_1', 'ability_code_1BC0_player_0', 'ability_code_1BC0_player_1', 'ability_code_1BC1_diff', 'ability_code_1BC1_player_0', 'ability_code_1BC1_player_1', 'ability_code_1BE2_diff', 'ability_code_1BE2_player_0', 'ability_code_1BE2_player_1', 'ability_code_1C00_diff', 'ability_code_1C00_player_0', 'ability_code_1C60_last_first_player_0', 'ability_code_1C60_player_0_ability_code_1020_player_0_diff', 'ability_code_1C60_player_0_ability_code_102A_player_0_diff', 'ability_code_1C60_player_0_ability_code_16E2_player_0_diff', 'ability_code_1C60_player_0_ability_code_1E60_player_0_diff', 'ability_code_1C60_player_0_ability_code_480_player_0_diff', 'ability_code_1C60_player_1', 'ability_code_1C60_player_1_ability_code_1020_player_1_diff_ability_code_1C60_player_0_ability_code_1020_player_0_diff_diff', 'ability_code_1C60_player_1_ability_code_102A_player_1_diff', 'ability_code_1C60_player_1_ability_code_102A_player_1_diff_ability_code_1C60_player_0_ability_code_102A_player_0_diff_diff', 'ability_code_1C60_player_1_ability_code_1261_player_1_diff', 'ability_code_1C60_player_1_ability_code_16E2_player_1_diff', 'ability_code_1C60_player_1_ability_code_16E2_player_1_diff_ability_code_1C60_player_0_ability_code_16E2_player_0_diff_diff', 'ability_code_1C60_player_1_ability_code_1E60_player_1_diff', 'ability_code_1C60_player_1_ability_code_1E60_player_1_diff_ability_code_1C60_player_0_ability_code_1E60_player_0_diff_diff', 'ability_code_1C61_diff', 'ability_code_1C61_player_0', 'ability_code_1C61_player_1', 'ability_code_1C80_first_time_diff', 'ability_code_1C80_first_time_player_0', 'ability_code_1C80_player_0', 'ability_code_1CA0_diff', 'ability_code_1CA0_player_0', 'ability_code_1CA0_player_1', 'ability_code_1CC0_diff', 'ability_code_1CC0_player_0', 'ability_code_1CC0_player_1', 'ability_code_1CE0_diff', 'ability_code_1CE0_player_0', 'ability_code_1CE0_player_1', 'ability_code_1D00_diff', 'ability_code_1D00_player_0', 'ability_code_1D00_player_1', 'ability_code_1D20_diff', 'ability_code_1D20_player_0', 'ability_code_1D20_player_1', 'ability_code_1D40_diff', 'ability_code_1D40_player_0', 'ability_code_1D40_player_1', 'ability_code_1D60_diff', 'ability_code_1D60_player_0', 'ability_code_1D60_player_1', 'ability_code_1D80_diff', 'ability_code_1D80_player_0', 'ability_code_1D81_diff', 'ability_code_1D81_player_0', 'ability_code_1D81_player_1', 'ability_code_1D83_diff', 'ability_code_1D83_player_0', 'ability_code_1D83_player_1', 'ability_code_1D86_diff', 'ability_code_1D86_first_time_player_0', 'ability_code_1D86_player_0', 'ability_code_1D86_player_1', 'ability_code_1DA0_player_0', 'ability_code_1DA1_player_0', 'ability_code_1DA1_player_1', 'ability_code_1DA2_diff', 'ability_code_1DA2_player_0', 'ability_code_1DA2_player_1', 'ability_code_1DC0_diff', 'ability_code_1DC0_player_0', 'ability_code_1DC0_player_1', 'ability_code_1E20_diff', 'ability_code_1E20_player_0', 'ability_code_1E60_first_time_diff', 'ability_code_1E60_player_0', 'ability_code_1E60_player_0_ability_code_1261_player_0_diff', 'ability_code_1E60_player_0_ability_code_480_player_0_diff', 'ability_code_1E60_player_1', 'ability_code_1E60_player_1_ability_code_102A_player_1_diff', 'ability_code_1E60_player_1_ability_code_1261_player_1_diff', 'ability_code_1E60_player_1_ability_code_1261_player_1_diff_ability_code_1E60_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_1EA0_diff', 'ability_code_1EA0_player_0', 'ability_code_1EA0_player_1', 'ability_code_1FC0_diff', 'ability_code_1FC0_player_0', 'ability_code_1FC0_player_1', 'ability_code_1FE0_diff', 'ability_code_1FE0_player_0', 'ability_code_1FE0_player_1', 'ability_code_2040_diff', 'ability_code_2040_player_0', 'ability_code_2040_player_1', 'ability_code_2060_diff', 'ability_code_2060_player_0', 'ability_code_2060_player_1', 'ability_code_2061_diff', 'ability_code_2061_player_0', 'ability_code_2061_player_1', 'ability_code_2080_first_time_diff', 'ability_code_2080_first_time_player_1', 'ability_code_2080_player_0', 'ability_code_20A0_player_0', 'ability_code_20A0_player_1', 'ability_code_20C0_diff', 'ability_code_20C0_player_0', 'ability_code_20C0_player_1', 'ability_code_20E0_diff', 'ability_code_20E0_player_0', 'ability_code_20E0_player_1', 'ability_code_20E1_diff', 'ability_code_20E1_player_0', 'ability_code_20E1_player_1', 'ability_code_2100_diff', 'ability_code_2100_player_0', 'ability_code_2100_player_1', 'ability_code_2101_diff', 'ability_code_2101_player_0', 'ability_code_2101_player_1', 'ability_code_2120_first_time_diff', 'ability_code_2120_first_time_player_0', 'ability_code_2120_player_1', 'ability_code_213E_diff', 'ability_code_213E_player_0', 'ability_code_213E_player_1', 'ability_code_2140_diff', 'ability_code_2140_player_0', 'ability_code_2140_player_1', 'ability_code_2160_diff', 'ability_code_2160_player_0', 'ability_code_2160_player_1', 'ability_code_2180_diff', 'ability_code_2180_player_0', 'ability_code_2180_player_1', 'ability_code_22A0_diff', 'ability_code_22A0_first_time_diff', 'ability_code_22A0_first_time_player_0', 'ability_code_22A0_first_time_player_1', 'ability_code_22A0_player_0', 'ability_code_22A0_player_1', 'ability_code_22A1_player_0', 'ability_code_22A1_player_1', 'ability_code_26C0_diff', 'ability_code_26C0_player_0', 'ability_code_26C0_player_1', 'ability_code_2720_diff', 'ability_code_2720_player_0', 'ability_code_2720_player_1', 'ability_code_2B00_player_0', 'ability_code_2B40_diff', 'ability_code_2B40_player_0', 'ability_code_2B40_player_1', 'ability_code_2C80_diff', 'ability_code_2C80_player_0', 'ability_code_2C80_player_1', 'ability_code_2DE0_diff', 'ability_code_2DE0_player_0', 'ability_code_2DE0_player_1', 'ability_code_2F60_diff', 'ability_code_2F60_player_0', 'ability_code_2F60_player_1', 'ability_code_3040_first_time_diff', 'ability_code_3040_first_time_player_1', 'ability_code_3040_player_0', 'ability_code_3040_player_1', 'ability_code_3060_diff', 'ability_code_3060_player_0', 'ability_code_3060_player_1', 'ability_code_3120_diff', 'ability_code_3120_player_0', 'ability_code_3120_player_1', 'ability_code_3140_diff', 'ability_code_3140_player_0', 'ability_code_3140_player_1', 'ability_code_3160_diff', 'ability_code_3160_player_1', 'ability_code_31A0_first_time_diff', 'ability_code_31A0_player_0', 'ability_code_31A0_player_1', 'ability_code_3380_diff', 'ability_code_3380_player_0', 'ability_code_3380_player_1', 'ability_code_3480_diff', 'ability_code_3480_first_time_diff', 'ability_code_3480_first_time_player_0', 'ability_code_3480_first_time_player_1', 'ability_code_3480_player_0', 'ability_code_3480_player_1', 'ability_code_39A0_diff', 'ability_code_39A0_player_0', 'ability_code_39A0_player_1', 'ability_code_4020_diff', 'ability_code_4020_player_0', 'ability_code_4020_player_1', 'ability_code_4021_diff', 'ability_code_4021_player_0', 'ability_code_4021_player_1', 'ability_code_4040_diff', 'ability_code_4040_player_0', 'ability_code_4040_player_1', 'ability_code_40A0_player_0', 'ability_code_40A0_player_1', 'ability_code_4120_diff', 'ability_code_4120_player_0', 'ability_code_4120_player_1', 'ability_code_4160_diff', 'ability_code_4160_player_0', 'ability_code_4160_player_1', 'ability_code_41A0_diff', 'ability_code_41A0_player_0', 'ability_code_41A0_player_1', 'ability_code_41E0_diff', 'ability_code_41E0_player_0', 'ability_code_41E0_player_1', 'ability_code_4220_diff', 'ability_code_4220_player_0', 'ability_code_4220_player_1', 'ability_code_42A1_diff', 'ability_code_42A1_player_0', 'ability_code_42A1_player_1', 'ability_code_42C0_diff', 'ability_code_42C1_diff', 'ability_code_42C1_player_0', 'ability_code_42C1_player_1', 'ability_code_43E0_diff', 'ability_code_43E0_player_0', 'ability_code_43E0_player_1', 'ability_code_480_diff', 'ability_code_480_first_time_player_0', 'ability_code_480_player_0', 'ability_code_480_player_1', 'ability_code_483_diff', 'ability_code_483_player_0', 'ability_code_483_player_1', 'ability_code_4AE0_diff', 'ability_code_4AE0_player_0', 'ability_code_4AFE_diff', 'ability_code_4AFE_player_0', 'ability_code_4AFE_player_1', 'ability_code_4B80_diff', 'ability_code_4B80_first_time_diff', 'ability_code_4B80_first_time_player_0', 'ability_code_4B80_player_0', 'ability_code_4B80_player_1', 'ability_code_4C0_diff', 'ability_code_4C0_player_0', 'ability_code_4C0_player_1', 'ability_code_4C1_diff', 'ability_code_4C1_first_time_diff', 'ability_code_4C1_first_time_player_0', 'ability_code_4C1_first_time_player_1', 'ability_code_4C1_player_0', 'ability_code_4C2_first_time_diff', 'ability_code_4C2_first_time_player_0', 'ability_code_4C2_first_time_player_1', 'ability_code_4C2_player_0', 'ability_code_4C3_diff', 'ability_code_4C3_player_0', 'ability_code_4C3_player_1', 'ability_code_4C60_diff', 'ability_code_4C60_player_0', 'ability_code_4C60_player_1', 'ability_code_4C80_diff', 'ability_code_4C80_player_0', 'ability_code_4C80_player_1', 'ability_code_4E40_first_time_diff', 'ability_code_4E40_first_time_player_0', 'ability_code_4E40_player_0', 'ability_code_4E40_player_1', 'ability_code_4EA0_diff', 'ability_code_4EA0_player_0', 'ability_code_4EA0_player_1', 'ability_code_4EC0_diff', 'ability_code_4EC0_player_0', 'ability_code_4EC0_player_1', 'ability_code_55E0_diff', 'ability_code_55E0_player_0', 'ability_code_55E0_player_1', 'ability_code_5600_diff', 'ability_code_5600_player_0', 'ability_code_5600_player_1', 'ability_code_5640_diff', 'ability_code_5640_player_0', 'ability_code_5640_player_1', 'ability_code_5641_diff', 'ability_code_5641_player_0', 'ability_code_5641_player_1', 'ability_code_56A0_diff', 'ability_code_56A0_player_0', 'ability_code_56A0_player_1', 'ability_code_5700_diff', 'ability_code_5700_player_0', 'ability_code_5700_player_1', 'ability_code_5A0_diff', 'ability_code_5A0_first_time_player_0', 'ability_code_5A0_last_first_diff', 'ability_code_5A0_last_first_player_0', 'ability_code_5A0_player_0', 'ability_code_5A0_player_0_ability_code_1021_player_0_diff', 'ability_code_5A0_player_0_ability_code_1022_player_0_diff', 'ability_code_5A0_player_0_ability_code_1261_player_0_diff', 'ability_code_5A0_player_0_ability_code_13E0_player_0_diff', 'ability_code_5A0_player_0_ability_code_15E0_player_0_diff', 'ability_code_5A0_player_0_ability_code_1822_player_0_diff', 'ability_code_5A0_player_0_ability_code_480_player_0_diff', 'ability_code_5A0_player_0_ability_code_B40_player_0_diff', 'ability_code_5A0_player_1', 'ability_code_5A0_player_1_ability_code_1020_player_1_diff_ability_code_5A0_player_0_ability_code_1020_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_1022_player_1_diff', 'ability_code_5A0_player_1_ability_code_1022_player_1_diff_ability_code_5A0_player_0_ability_code_1022_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_102A_player_1_diff_ability_code_5A0_player_0_ability_code_102A_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_1261_player_1_diff', 'ability_code_5A0_player_1_ability_code_1261_player_1_diff_ability_code_5A0_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_1360_player_1_diff', 'ability_code_5A0_player_1_ability_code_1360_player_1_diff_ability_code_5A0_player_0_ability_code_1360_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_13A0_player_1_diff', 'ability_code_5A0_player_1_ability_code_13E0_player_1_diff', 'ability_code_5A0_player_1_ability_code_13E0_player_1_diff_ability_code_5A0_player_0_ability_code_13E0_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_15E0_player_1_diff_ability_code_5A0_player_0_ability_code_15E0_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_16E2_player_1_diff_ability_code_5A0_player_0_ability_code_16E2_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_1820_player_1_diff', 'ability_code_5A0_player_1_ability_code_1822_player_1_diff', 'ability_code_5A0_player_1_ability_code_1C60_player_1_diff_ability_code_5A0_player_0_ability_code_1C60_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_1E60_player_1_diff', 'ability_code_5A0_player_1_ability_code_1E60_player_1_diff_ability_code_5A0_player_0_ability_code_1E60_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_480_player_1_diff_ability_code_5A0_player_0_ability_code_480_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_B40_player_1_diff', 'ability_code_5A0_player_1_ability_code_DE0_player_1_diff_ability_code_5A0_player_0_ability_code_DE0_player_0_diff_diff', 'ability_code_780_diff', 'ability_code_780_player_0', 'ability_code_780_player_1', 'ability_code_7A0_diff', 'ability_code_7A0_player_0', 'ability_code_7A0_player_1', 'ability_code_7C0_diff', 'ability_code_7C0_player_0', 'ability_code_7C0_player_1', 'ability_code_7E0_diff', 'ability_code_7E0_player_0', 'ability_code_7E0_player_1', 'ability_code_820_diff', 'ability_code_820_player_0', 'ability_code_820_player_1', 'ability_code_880_diff', 'ability_code_880_player_0', 'ability_code_880_player_1', 'ability_code_8A0_player_0', 'ability_code_8A2_diff', 'ability_code_8A2_player_0', 'ability_code_8A2_player_1', 'ability_code_8E0_diff', 'ability_code_8E0_player_0', 'ability_code_8E0_player_1', 'ability_code_900_diff', 'ability_code_900_player_0', 'ability_code_900_player_1', 'ability_code_920_player_0', 'ability_code_960_diff', 'ability_code_960_player_1', 'ability_code_9E0_diff', 'ability_code_9E0_player_0', 'ability_code_9E0_player_1', 'ability_code_A40_diff', 'ability_code_A40_player_0', 'ability_code_A40_player_1', 'ability_code_B00_diff', 'ability_code_B00_player_0', 'ability_code_B01_diff', 'ability_code_B01_player_0', 'ability_code_B01_player_1', 'ability_code_B40_diff', 'ability_code_B40_first_time_diff', 'ability_code_B40_first_time_player_0', 'ability_code_B40_first_time_player_1', 'ability_code_B40_last_first_player_0', 'ability_code_B40_player_0', 'ability_code_B40_player_0_ability_code_1022_player_0_diff', 'ability_code_B40_player_0_ability_code_102A_player_0_diff', 'ability_code_B40_player_0_ability_code_1261_player_0_diff', 'ability_code_B40_player_0_ability_code_13A0_player_0_diff', 'ability_code_B40_player_0_ability_code_16E2_player_0_diff', 'ability_code_B40_player_0_ability_code_1821_player_0_diff', 'ability_code_B40_player_0_ability_code_1822_player_0_diff', 'ability_code_B40_player_0_ability_code_1E60_player_0_diff', 'ability_code_B40_player_0_ability_code_DE0_player_0_diff', 'ability_code_B40_player_1', 'ability_code_B40_player_1_ability_code_1020_player_1_diff', 'ability_code_B40_player_1_ability_code_1022_player_1_diff', 'ability_code_B40_player_1_ability_code_1023_player_1_diff', 'ability_code_B40_player_1_ability_code_102A_player_1_diff', 'ability_code_B40_player_1_ability_code_102A_player_1_diff_ability_code_B40_player_0_ability_code_102A_player_0_diff_diff', 'ability_code_B40_player_1_ability_code_1261_player_1_diff', 'ability_code_B40_player_1_ability_code_1261_player_1_diff_ability_code_B40_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_B40_player_1_ability_code_13A0_player_1_diff', 'ability_code_B40_player_1_ability_code_15E0_player_1_diff', 'ability_code_B40_player_1_ability_code_15E0_player_1_diff_ability_code_B40_player_0_ability_code_15E0_player_0_diff_diff', 'ability_code_B40_player_1_ability_code_16E2_player_1_diff', 'ability_code_B40_player_1_ability_code_16E2_player_1_diff_ability_code_B40_player_0_ability_code_16E2_player_0_diff_diff', 'ability_code_B40_player_1_ability_code_1821_player_1_diff', 'ability_code_B40_player_1_ability_code_1821_player_1_diff_ability_code_B40_player_0_ability_code_1821_player_0_diff_diff', 'ability_code_B40_player_1_ability_code_1822_player_1_diff', 'ability_code_B40_player_1_ability_code_1E60_player_1_diff', 'ability_code_B40_player_1_ability_code_1E60_player_1_diff_ability_code_B40_player_0_ability_code_1E60_player_0_diff_diff', 'ability_code_B40_player_1_ability_code_480_player_1_diff', 'ability_code_B40_player_1_ability_code_480_player_1_diff_ability_code_B40_player_0_ability_code_480_player_0_diff_diff', 'ability_code_B40_player_1_ability_code_DE0_player_1_diff', 'ability_code_B40_player_1_ability_code_DE0_player_1_diff_ability_code_B40_player_0_ability_code_DE0_player_0_diff_diff', 'ability_code_B60_diff', 'ability_code_B60_player_0', 'ability_code_B60_player_1', 'ability_code_B61_diff', 'ability_code_B61_player_0', 'ability_code_B61_player_1', 'ability_code_BE0_diff', 'ability_code_BE0_player_0', 'ability_code_BE0_player_1', 'ability_code_CC0_diff', 'ability_code_D00_diff', 'ability_code_D00_player_0', 'ability_code_D00_player_1', 'ability_code_D20_player_0', 'ability_code_D20_player_1', 'ability_code_D40_diff', 'ability_code_D40_player_0', 'ability_code_D40_player_1', 'ability_code_D41_player_1', 'ability_code_D61_player_1', 'ability_code_D62_diff', 'ability_code_D62_player_0', 'ability_code_D62_player_1', 'ability_code_DE0_player_0', 'ability_code_DE0_player_0_ability_code_1020_player_0_diff', 'ability_code_DE0_player_0_ability_code_1022_player_0_diff', 'ability_code_DE0_player_0_ability_code_1023_player_0_diff', 'ability_code_DE0_player_0_ability_code_102A_player_0_diff', 'ability_code_DE0_player_0_ability_code_1261_player_0_diff', 'ability_code_DE0_player_0_ability_code_13A0_player_0_diff', 'ability_code_DE0_player_0_ability_code_15E0_player_0_diff', 'ability_code_DE0_player_0_ability_code_16E2_player_0_diff', 'ability_code_DE0_player_0_ability_code_1822_player_0_diff', 'ability_code_DE0_player_0_ability_code_1C60_player_0_diff', 'ability_code_DE0_player_0_ability_code_480_player_0_diff', 'ability_code_DE0_player_1', 'ability_code_DE0_player_1_ability_code_1020_player_1_diff', 'ability_code_DE0_player_1_ability_code_1022_player_1_diff', 'ability_code_DE0_player_1_ability_code_1022_player_1_diff_ability_code_DE0_player_0_ability_code_1022_player_0_diff_diff', 'ability_code_DE0_player_1_ability_code_1023_player_1_diff', 'ability_code_DE0_player_1_ability_code_102A_player_1_diff', 'ability_code_DE0_player_1_ability_code_102A_player_1_diff_ability_code_DE0_player_0_ability_code_102A_player_0_diff_diff', 'ability_code_DE0_player_1_ability_code_1261_player_1_diff', 'ability_code_DE0_player_1_ability_code_13A0_player_1_diff', 'ability_code_DE0_player_1_ability_code_15E0_player_1_diff', 'ability_code_DE0_player_1_ability_code_16E2_player_1_diff_ability_code_DE0_player_0_ability_code_16E2_player_0_diff_diff', 'ability_code_DE0_player_1_ability_code_1821_player_1_diff', 'ability_code_DE0_player_1_ability_code_1822_player_1_diff_ability_code_DE0_player_0_ability_code_1822_player_0_diff_diff', 'ability_code_DE0_player_1_ability_code_1C60_player_1_diff', 'ability_code_DE0_player_1_ability_code_480_player_1_diff_ability_code_DE0_player_0_ability_code_480_player_0_diff_diff', 'ability_code_E00_diff', 'ability_code_E00_player_0', 'ability_code_E00_player_1', 'ability_code_E20_diff', 'ability_code_E20_player_0', 'ability_code_E20_player_1', 'ability_code_E80_first_time_diff', 'ability_code_E80_first_time_player_0', 'ability_code_E80_player_0', 'ability_code_E80_player_1', 'ability_code_EE0_diff', 'ability_code_EE0_player_0', 'ability_code_EE0_player_1', 'ability_code_EE1_player_0', 'ability_code_EE1_player_1', 'ability_code_F00_diff', 'ability_code_F00_player_0', 'ability_code_F00_player_1', 'ability_code_F01_player_0', 'ability_code_F40_player_0', 'ability_code_F40_player_1', 'ability_code_F41_diff', 'ability_code_F41_player_0', 'ability_code_F41_player_1', 'ability_code_F60_player_1', 'ability_code_F61_diff', 'ability_code_F61_first_time_diff', 'ability_code_F61_first_time_player_1', 'ability_code_F61_player_0', 'ability_code_F61_player_1', 'ability_code_F80_first_time_diff', 'ability_code_F80_player_0', 'ability_code_F80_player_1', 'ability_code_F81_diff', 'ability_code_F81_first_time_diff', 'ability_code_F81_first_time_player_1', 'ability_code_F81_player_0', 'ability_code_F81_player_1', 'ability_code_FC0_diff', 'ability_code_FC0_player_0', 'ability_code_FC0_player_1', 'ability_code_FC1_diff', 'ability_code_FC1_player_0', 'ability_code_FC1_player_1', 'ability_code_FE0_first_time_player_0', 'ability_code_FE0_first_time_player_1', 'ability_code_FE0_player_0', 'ability_code_FE0_player_1', 'ability_code_FE1_diff', 'ability_code_FE1_player_0', 'ability_code_FE1_player_1', 'all_unit_len_player_0_BanelingCocoon_len_player_0_diff', 'all_unit_len_player_0_Barracks_len_player_0_diff', 'all_unit_len_player_0_Drone_len_player_0_diff', 'all_unit_len_player_0_Factory_len_player_0_diff', 'all_unit_len_player_0_Hellion_len_player_0_diff', 'all_unit_len_player_0_MULE_len_player_0_diff', 'all_unit_len_player_0_OrbitalCommand_len_player_0_diff', 'all_unit_len_player_0_Overlord_len_player_0_diff', 'all_unit_len_player_0_Probe_len_player_0_diff', 'all_unit_len_player_0_SCV_len_player_0_diff', 'all_unit_len_player_0_Starport_len_player_0_diff', 'all_unit_len_player_0_SupplyDepotLowered_len_player_0_diff', 'all_unit_len_player_0_Zergling_len_player_0_diff', 'all_unit_len_player_1_Barracks_len_player_1_diff', 'all_unit_len_player_1_Egg_len_player_1_diff', 'all_unit_len_player_1_Egg_len_player_1_diff_all_unit_len_player_0_Egg_len_player_0_diff_diff', 'all_unit_len_player_1_Factory_len_player_1_diff', 'all_unit_len_player_1_Factory_len_player_1_diff_all_unit_len_player_0_Factory_len_player_0_diff_diff', 'all_unit_len_player_1_Hellion_len_player_1_diff', 'all_unit_len_player_1_MULE_len_player_1_diff', 'all_unit_len_player_1_MULE_len_player_1_diff_all_unit_len_player_0_MULE_len_player_0_diff_diff', 'all_unit_len_player_1_Overlord_len_player_1_diff', 'all_unit_len_player_1_Overlord_len_player_1_diff_all_unit_len_player_0_Overlord_len_player_0_diff_diff', 'all_unit_len_player_1_Probe_len_player_1_diff_all_unit_len_player_0_Probe_len_player_0_diff_diff', 'all_unit_len_player_1_Reaper_len_player_1_diff', 'all_unit_len_player_1_SCV_len_player_1_diff', 'all_unit_len_player_1_Starport_len_player_1_diff', 'all_unit_len_player_1_SupplyDepotLowered_len_player_1_diff', 'build_count_player_0', 'camera_pos_x_min_player_0', 'camera_pos_x_min_player_1', 'species_player_0', 'species_player_1', 'species_war_kind', 'time_sum_player_diff', 'under_1_Ability_player_0', 'under_1_AddToControlGroup_player_0', 'under_1_AddToControlGroup_player_1', 'under_1_Camera_diff', 'under_1_ControlGroup_player_0', 'under_1_ControlGroup_player_1', 'under_1_GetControlGroup_player_1', 'under_1_SetControlGroup_player_0', 'under_3_Ability_player_0', 'under_3_AddToControlGroup_diff', 'under_3_AddToControlGroup_player_0', 'under_3_Camera_player_1', 'under_3_ControlGroup_player_1', 'under_3_GetControlGroup_player_1', 'under_3_SetControlGroup_player_0', 'under_5_Ability_diff', 'under_5_AddToControlGroup_diff', 'under_5_AddToControlGroup_player_0', 'under_5_AddToControlGroup_player_1', 'under_5_Camera_diff', 'under_5_Camera_player_1', 'under_5_ControlGroup_diff', 'under_5_ControlGroup_player_1', 'under_5_GetControlGroup_diff', 'under_5_SetControlGroup_diff', 'work_diff']
bad_feature_fold3 = ['Ability_player_0_isnull', 'Ability_player_1_isnull', 'AddToControlGroup_player_0', 'AddToControlGroup_player_0_isnull', 'AdeptPiercingWeapon_len_player_0', 'AdeptPiercingWeapon_len_player_1', 'Adept_len_player_0', 'Adept_right_click_target_count_diff', 'Adept_right_click_target_count_player_0', 'AiurLightBridgeAbandonedNE10Out_len_player_0', 'AiurLightBridgeAbandonedNE10Out_len_player_1', 'AiurLightBridgeAbandonedNE8Out_len_player_0', 'AiurLightBridgeAbandonedNE8Out_len_player_1', 'AiurLightBridgeAbandonedNE8_len_player_0', 'AiurLightBridgeAbandonedNE8_len_player_1', 'AiurLightBridgeNE10Out_len_player_0', 'AiurLightBridgeNE10Out_len_player_1', 'AiurLightBridgeNE8_len_player_0', 'AiurLightBridgeNE8_len_player_1', 'Archon_len_player_0', 'Archon_len_player_1', 'Armory_len_player_1', 'AutoTurret_len_player_0', 'AutoTurret_len_player_1', 'BanelingBurrowed_len_player_0', 'BanelingBurrowed_len_player_1', 'BanelingCocoon_len_player_0', 'BanelingCocoon_len_player_0_Hellion_len_player_0_diff', 'BanelingCocoon_len_player_0_MULE_len_player_0_diff', 'BanelingCocoon_len_player_0_Overlord_len_player_0_diff', 'BanelingCocoon_len_player_0_Queen_len_player_0_diff', 'BanelingCocoon_len_player_0_Roach_len_player_0_diff', 'BanelingCocoon_len_player_1', 'BanelingCocoon_len_player_1_Factory_len_player_1_diff', 'BanelingCocoon_len_player_1_Hatchery_len_player_1_diff_BanelingCocoon_len_player_0_Hatchery_len_player_0_diff_diff', 'BanelingCocoon_len_player_1_MULE_len_player_1_diff', 'BanelingCocoon_len_player_1_MULE_len_player_1_diff_BanelingCocoon_len_player_0_MULE_len_player_0_diff_diff', 'BanelingCocoon_len_player_1_Overlord_len_player_1_diff', 'BanelingCocoon_len_player_1_Overlord_len_player_1_diff_BanelingCocoon_len_player_0_Overlord_len_player_0_diff_diff', 'BanelingCocoon_len_player_1_Queen_len_player_1_diff_BanelingCocoon_len_player_0_Queen_len_player_0_diff_diff', 'BanelingCocoon_len_player_1_Reaper_len_player_1_diff', 'BanelingCocoon_len_player_1_Roach_len_player_1_diff', 'BanelingCocoon_len_player_1_Starport_len_player_1_diff', 'BanelingNest_len_player_1', 'Baneling_len_player_0', 'Baneling_len_player_1', 'Baneling_right_click_target_count_diff', 'Baneling_right_click_target_count_player_0', 'Baneling_right_click_target_count_player_1', 'Banshee_len_player_1', 'BarracksReactor_len_player_0', 'BarracksReactor_len_player_1', 'BarracksTechLab_len_player_0', 'BarracksTechLab_len_player_1', 'Barracks_len_player_0', 'Barracks_len_player_0_BanelingCocoon_len_player_0_diff', 'Barracks_len_player_0_Queen_len_player_0_diff', 'Barracks_len_player_0_Reaper_len_player_0_diff', 'Barracks_len_player_0_Starport_len_player_0_diff', 'Barracks_len_player_0_SupplyDepotLowered_len_player_0_diff', 'Barracks_len_player_1_Factory_len_player_1_diff_Barracks_len_player_0_Factory_len_player_0_diff_diff', 'Barracks_len_player_1_Hatchery_len_player_1_diff_Barracks_len_player_0_Hatchery_len_player_0_diff_diff', 'Barracks_len_player_1_Hellion_len_player_1_diff', 'Barracks_len_player_1_Overlord_len_player_1_diff', 'Barracks_len_player_1_Overlord_len_player_1_diff_Barracks_len_player_0_Overlord_len_player_0_diff_diff', 'Barracks_len_player_1_Queen_len_player_1_diff_Barracks_len_player_0_Queen_len_player_0_diff_diff', 'Barracks_len_player_1_Reaper_len_player_1_diff', 'Barracks_len_player_1_Starport_len_player_1_diff', 'Barracks_len_player_1_Starport_len_player_1_diff_Barracks_len_player_0_Starport_len_player_0_diff_diff', 'Barracks_len_player_1_SupplyDepotLowered_len_player_1_diff', 'Barracks_right_click_target_count_diff', 'Barracks_right_click_target_count_player_0', 'Battlecruiser_len_player_0', 'Battlecruiser_len_player_1', 'Beacon_TerranSmall_len_player_0', 'Beacon_TerranSmall_len_player_1', 'Beacon_Terran_len_player_0', 'Beacon_Terran_len_player_1', 'BroodLordAWeapon_len_player_0', 'BroodLordAWeapon_len_player_1', 'BroodLordWeapon_len_player_0', 'BroodLordWeapon_len_player_1', 'BroodLord_len_player_0', 'BroodLord_len_player_1', 'Broodling_len_player_0', 'Broodling_len_player_1', 'BypassArmorDrone_len_player_0', 'Camera_player_0_Camera_player_1_diff', 'Camera_player_0_isnull', 'Camera_player_1', 'Camera_player_1_isnull', 'Carrier_len_player_0', 'Carrier_len_player_1', 'ChangelingMarineShield_len_player_1', 'ChangelingMarine_len_player_0', 'ChangelingMarine_len_player_1', 'ChangelingZealot_len_player_0', 'ChangelingZealot_len_player_1', 'ChangelingZerglingWings_len_player_0', 'ChangelingZerglingWings_len_player_1', 'ChangelingZergling_len_player_0', 'ChangelingZergling_len_player_1', 'Changeling_len_player_0', 'Changeling_len_player_1', 'Colossus_len_player_0', 'Colossus_len_player_1', 'CommandCenter_len_player_1', 'ControlGroup_player_0', 'ControlGroup_player_0_ControlGroup_player_1_diff', 'ControlGroup_player_1', 'ControlGroup_player_1_isnull', 'Corruptor_len_player_0', 'Corruptor_len_player_1', 'CreepOnlyBlocker4x4_len_player_0', 'CreepOnlyBlocker4x4_len_player_1', 'CreepOnlyBlocker4x4_right_click_target_count_diff', 'CreepOnlyBlocker4x4_right_click_target_count_player_0', 'CreepOnlyBlocker4x4_right_click_target_count_player_1', 'CreepTumorBurrowed_len_player_0', 'CreepTumorBurrowed_len_player_1', 'CreepTumorBurrowed_right_click_target_count_player_0', 'CreepTumorQueen_len_player_0', 'CreepTumorQueen_len_player_1', 'CreepTumor_len_player_0', 'CreepTumor_len_player_1', 'CyberneticsCore_right_click_target_count_diff', 'CycloneMissileLargeAir_len_player_0', 'Cyclone_len_player_0', 'Cyclone_len_player_1', 'DarkShrine_len_player_0', 'DarkShrine_len_player_1', 'DarkTemplar_len_player_0', 'DestructibleIce2x4Horizontal_len_player_0', 'DestructibleIce2x4Horizontal_len_player_1', 'DestructibleIce2x4Vertical_len_player_0', 'DestructibleIce2x4Vertical_len_player_1', 'DestructibleIce2x4Vertical_right_click_target_count_player_0', 'DestructibleIce2x4Vertical_right_click_target_count_player_1', 'DestructibleIce2x6Horizontal_len_player_0', 'DestructibleIce2x6Horizontal_len_player_1', 'DestructibleIce2x6Vertical_len_player_0', 'DestructibleIce2x6Vertical_len_player_1', 'DestructibleSignsIcons_len_player_1', 'DestructibleSignsIcons_right_click_target_count_diff', 'DestructibleSignsIcons_right_click_target_count_player_0', 'Disruptor_len_player_0', 'Disruptor_len_player_1', 'Drone_len_player_0_BanelingCocoon_len_player_0_diff', 'Drone_len_player_0_Factory_len_player_0_diff', 'Drone_len_player_0_Hellion_len_player_0_diff', 'Drone_len_player_0_Marine_len_player_0_diff', 'Drone_len_player_1_BanelingCocoon_len_player_1_diff_Drone_len_player_0_BanelingCocoon_len_player_0_diff_diff', 'Drone_len_player_1_Barracks_len_player_1_diff', 'Drone_len_player_1_Factory_len_player_1_diff', 'Drone_len_player_1_Hellion_len_player_1_diff', 'Drone_len_player_1_MULE_len_player_1_diff', 'Drone_len_player_1_Marine_len_player_1_diff_Drone_len_player_0_Marine_len_player_0_diff_diff', 'Drone_len_player_1_Overlord_len_player_1_diff', 'Drone_len_player_1_Overlord_len_player_1_diff_Drone_len_player_0_Overlord_len_player_0_diff_diff', 'Drone_len_player_1_Probe_len_player_1_diff', 'Drone_len_player_1_Probe_len_player_1_diff_Drone_len_player_0_Probe_len_player_0_diff_diff', 'Drone_len_player_1_Queen_len_player_1_diff', 'Drone_len_player_1_Queen_len_player_1_diff_Drone_len_player_0_Queen_len_player_0_diff_diff', 'Drone_len_player_1_Reaper_len_player_1_diff_Drone_len_player_0_Reaper_len_player_0_diff_diff', 'Drone_len_player_1_Roach_len_player_1_diff_Drone_len_player_0_Roach_len_player_0_diff_diff', 'Drone_len_player_1_Starport_len_player_1_diff', 'Drone_len_player_1_SupplyDepotLowered_len_player_1_diff', 'Drone_len_player_1_SupplyDepotLowered_len_player_1_diff_Drone_len_player_0_SupplyDepotLowered_len_player_0_diff_diff', 'Drone_len_player_1_Zergling_len_player_1_diff', 'Drone_len_player_1_Zergling_len_player_1_diff_Drone_len_player_0_Zergling_len_player_0_diff_diff', 'Drone_right_click_target_count_diff', 'Drone_right_click_target_count_player_1', 'Egg_len_player_0_Barracks_len_player_0_diff', 'Egg_len_player_0_Factory_len_player_0_diff', 'Egg_len_player_0_Hatchery_len_player_0_diff', 'Egg_len_player_0_MULE_len_player_0_diff', 'Egg_len_player_0_Marine_len_player_0_diff', 'Egg_len_player_0_Queen_len_player_0_diff', 'Egg_len_player_0_Reaper_len_player_0_diff', 'Egg_len_player_0_SCV_len_player_0_diff', 'Egg_len_player_0_Starport_len_player_0_diff', 'Egg_len_player_1_BanelingCocoon_len_player_1_diff', 'Egg_len_player_1_Factory_len_player_1_diff_Egg_len_player_0_Factory_len_player_0_diff_diff', 'Egg_len_player_1_Hatchery_len_player_1_diff_Egg_len_player_0_Hatchery_len_player_0_diff_diff', 'Egg_len_player_1_MULE_len_player_1_diff', 'Egg_len_player_1_OrbitalCommand_len_player_1_diff_Egg_len_player_0_OrbitalCommand_len_player_0_diff_diff', 'Egg_len_player_1_Overlord_len_player_1_diff', 'Egg_len_player_1_Probe_len_player_1_diff', 'Egg_len_player_1_Reaper_len_player_1_diff', 'Egg_len_player_1_SCV_len_player_1_diff_Egg_len_player_0_SCV_len_player_0_diff_diff', 'Egg_len_player_1_SupplyDepotLowered_len_player_1_diff', 'Egg_len_player_1_SupplyDepotLowered_len_player_1_diff_Egg_len_player_0_SupplyDepotLowered_len_player_0_diff_diff', 'EngineeringBay_len_player_1', 'EngineeringBay_right_click_target_count_diff', 'EngineeringBay_right_click_target_count_player_0', 'ExtendingBridgeNEWide10Out_len_player_0', 'ExtendingBridgeNEWide10Out_len_player_1', 'ExtendingBridgeNEWide10_len_player_0', 'ExtendingBridgeNEWide10_len_player_1', 'ExtendingBridgeNEWide8Out_len_player_0', 'ExtendingBridgeNEWide8Out_len_player_1', 'ExtendingBridgeNWWide8Out_len_player_0', 'ExtendingBridgeNWWide8Out_len_player_1', 'ExtendingBridgeNWWide8_len_player_0', 'ExtendingBridgeNWWide8_len_player_1', 'Extractor_len_player_0', 'Extractor_len_player_1', 'Extractor_right_click_target_count_diff', 'Extractor_right_click_target_count_player_0', 'FactoryReactor_len_player_0', 'FactoryReactor_len_player_1', 'FactoryTechLab_len_player_1', 'Factory_len_player_0_Hatchery_len_player_0_diff', 'Factory_len_player_0_Hellion_len_player_0_diff', 'Factory_len_player_0_Roach_len_player_0_diff', 'Factory_len_player_0_Starport_len_player_0_diff', 'Factory_len_player_1', 'Factory_len_player_1_Hatchery_len_player_1_diff_Factory_len_player_0_Hatchery_len_player_0_diff_diff', 'Factory_len_player_1_Hellion_len_player_1_diff', 'Factory_len_player_1_Hellion_len_player_1_diff_Factory_len_player_0_Hellion_len_player_0_diff_diff', 'Factory_len_player_1_MULE_len_player_1_diff', 'Factory_len_player_1_Reaper_len_player_1_diff', 'Factory_len_player_1_Roach_len_player_1_diff', 'Factory_len_player_1_Starport_len_player_1_diff', 'Factory_right_click_target_count_diff', 'FleetBeacon_len_player_0', 'Forge_len_player_0', 'Forge_right_click_target_count_diff', 'Forge_right_click_target_count_player_1', 'FungalGrowthMissile_len_player_1', 'FungalGrowthMissile_right_click_target_count_diff', 'FungalGrowthMissile_right_click_target_count_player_0', 'FusionCore_len_player_0', 'FusionCore_len_player_1', 'GhostAcademy_len_player_0', 'GhostAcademy_len_player_1', 'Ghost_len_player_0', 'Ghost_len_player_1', 'GlaiveWurmBounceWeapon_len_player_0', 'GlaiveWurmBounceWeapon_len_player_1', 'GreaterSpire_len_player_0', 'GreaterSpire_len_player_1', 'Hatchery_len_player_0', 'Hatchery_len_player_0_Reaper_len_player_0_diff', 'Hatchery_len_player_1', 'Hatchery_len_player_1_Reaper_len_player_1_diff_Hatchery_len_player_0_Reaper_len_player_0_diff_diff', 'Hatchery_len_player_1_Roach_len_player_1_diff', 'Hatchery_len_player_1_Starport_len_player_1_diff', 'Hatchery_right_click_target_count_player_0', 'Hellion_len_player_0', 'Hellion_len_player_0_Hatchery_len_player_0_diff', 'Hellion_len_player_0_MULE_len_player_0_diff', 'Hellion_len_player_0_Roach_len_player_0_diff', 'Hellion_len_player_1', 'Hellion_len_player_1_Hatchery_len_player_1_diff', 'Hellion_len_player_1_MULE_len_player_1_diff', 'Hellion_len_player_1_MULE_len_player_1_diff_Hellion_len_player_0_MULE_len_player_0_diff_diff', 'Hellion_len_player_1_Reaper_len_player_1_diff', 'Hellion_len_player_1_Roach_len_player_1_diff', 'Hellion_right_click_target_count_diff', 'Hellion_right_click_target_count_player_1', 'HighTemplar_len_player_0', 'HighTemplar_len_player_1', 'Hive_right_click_target_count_diff', 'Hive_right_click_target_count_player_0', 'Hive_right_click_target_count_player_1', 'HydraliskDen_len_player_1', 'Hydralisk_len_player_0', 'Hydralisk_len_player_1', 'Immortal_len_player_1', 'InfestationPit_len_player_0', 'InfestationPit_len_player_1', 'InfestorBurrowed_len_player_1', 'Infestor_len_player_1', 'Lair_right_click_target_count_player_0', 'Larva_len_player_0', 'Larva_len_player_1', 'LocustMPFlying_len_player_0', 'LocustMPFlying_len_player_1', 'LurkerEgg_len_player_0', 'LurkerEgg_len_player_1', 'Lurker_len_player_0', 'Lurker_len_player_1', 'MULE_len_player_0', 'MULE_len_player_0_Hatchery_len_player_0_diff', 'MULE_len_player_0_Reaper_len_player_0_diff', 'MULE_len_player_0_Roach_len_player_0_diff', 'MULE_len_player_1', 'MULE_len_player_1_Hatchery_len_player_1_diff', 'MULE_len_player_1_Reaper_len_player_1_diff', 'MULE_len_player_1_Reaper_len_player_1_diff_MULE_len_player_0_Reaper_len_player_0_diff_diff', 'MULE_len_player_1_Roach_len_player_1_diff', 'MULE_len_player_1_Starport_len_player_1_diff', 'MULE_right_click_target_count_diff', 'MULE_right_click_target_count_player_0', 'MULE_right_click_target_count_player_1', 'Marine_len_player_0', 'Marine_len_player_0_Barracks_len_player_0_diff', 'Marine_len_player_0_Hatchery_len_player_0_diff', 'Marine_len_player_0_Hellion_len_player_0_diff', 'Marine_len_player_0_MULE_len_player_0_diff', 'Marine_len_player_0_Queen_len_player_0_diff', 'Marine_len_player_0_Starport_len_player_0_diff', 'Marine_len_player_0_SupplyDepotLowered_len_player_0_diff', 'Marine_len_player_1', 'Marine_len_player_1_BanelingCocoon_len_player_1_diff', 'Marine_len_player_1_BanelingCocoon_len_player_1_diff_Marine_len_player_0_BanelingCocoon_len_player_0_diff_diff', 'Marine_len_player_1_Barracks_len_player_1_diff', 'Marine_len_player_1_Barracks_len_player_1_diff_Marine_len_player_0_Barracks_len_player_0_diff_diff', 'Marine_len_player_1_Factory_len_player_1_diff', 'Marine_len_player_1_Factory_len_player_1_diff_Marine_len_player_0_Factory_len_player_0_diff_diff', 'Marine_len_player_1_Hatchery_len_player_1_diff', 'Marine_len_player_1_Hatchery_len_player_1_diff_Marine_len_player_0_Hatchery_len_player_0_diff_diff', 'Marine_len_player_1_Hellion_len_player_1_diff', 'Marine_len_player_1_Hellion_len_player_1_diff_Marine_len_player_0_Hellion_len_player_0_diff_diff', 'Marine_len_player_1_MULE_len_player_1_diff', 'Marine_len_player_1_Overlord_len_player_1_diff', 'Marine_len_player_1_Probe_len_player_1_diff', 'Marine_len_player_1_Probe_len_player_1_diff_Marine_len_player_0_Probe_len_player_0_diff_diff', 'Marine_len_player_1_Queen_len_player_1_diff_Marine_len_player_0_Queen_len_player_0_diff_diff', 'Marine_len_player_1_Reaper_len_player_1_diff', 'Marine_len_player_1_Roach_len_player_1_diff', 'Marine_len_player_1_Starport_len_player_1_diff', 'Marine_len_player_1_Starport_len_player_1_diff_Marine_len_player_0_Starport_len_player_0_diff_diff', 'Marine_len_player_1_SupplyDepotLowered_len_player_1_diff', 'Marine_len_player_1_Zergling_len_player_1_diff', 'Marine_right_click_target_count_diff', 'Medivac_len_player_0', 'Medivac_len_player_1', 'Medivac_right_click_target_count_player_0', 'MissileTurret_len_player_0', 'MissileTurret_len_player_1', 'Mutalisk_len_player_1', 'NydusNetwork_len_player_1', 'NydusWorm_len_player_0', 'NydusWorm_len_player_1', 'Observer_len_player_0', 'OrbitalCommand_len_player_0_Factory_len_player_0_diff', 'OrbitalCommand_len_player_0_Starport_len_player_0_diff', 'OrbitalCommand_len_player_0_SupplyDepotLowered_len_player_0_diff', 'OrbitalCommand_len_player_1', 'OrbitalCommand_len_player_1_BanelingCocoon_len_player_1_diff', 'OrbitalCommand_len_player_1_BanelingCocoon_len_player_1_diff_OrbitalCommand_len_player_0_BanelingCocoon_len_player_0_diff_diff', 'OrbitalCommand_len_player_1_Hatchery_len_player_1_diff', 'OrbitalCommand_len_player_1_Hatchery_len_player_1_diff_OrbitalCommand_len_player_0_Hatchery_len_player_0_diff_diff', 'OrbitalCommand_len_player_1_MULE_len_player_1_diff', 'OrbitalCommand_len_player_1_MULE_len_player_1_diff_OrbitalCommand_len_player_0_MULE_len_player_0_diff_diff', 'OrbitalCommand_len_player_1_Overlord_len_player_1_diff', 'OrbitalCommand_len_player_1_Queen_len_player_1_diff', 'OrbitalCommand_len_player_1_Reaper_len_player_1_diff', 'OrbitalCommand_len_player_1_Roach_len_player_1_diff', 'OrbitalCommand_len_player_1_Starport_len_player_1_diff_OrbitalCommand_len_player_0_Starport_len_player_0_diff_diff', 'OrbitalCommand_len_player_1_SupplyDepotLowered_len_player_1_diff_OrbitalCommand_len_player_0_SupplyDepotLowered_len_player_0_diff_diff', 'OrbitalCommand_right_click_target_count_player_1', 'OverlordGenerateCreepKeybind_len_player_0', 'OverlordGenerateCreepKeybind_len_player_1', 'Overlord_len_player_0_Hellion_len_player_0_diff', 'Overlord_len_player_0_MULE_len_player_0_diff', 'Overlord_len_player_1', 'Overlord_len_player_1_Hatchery_len_player_1_diff_Overlord_len_player_0_Hatchery_len_player_0_diff_diff', 'Overlord_len_player_1_Hellion_len_player_1_diff', 'Overlord_len_player_1_Hellion_len_player_1_diff_Overlord_len_player_0_Hellion_len_player_0_diff_diff', 'Overlord_len_player_1_MULE_len_player_1_diff', 'Overlord_len_player_1_MULE_len_player_1_diff_Overlord_len_player_0_MULE_len_player_0_diff_diff', 'Overlord_len_player_1_Queen_len_player_1_diff', 'Overlord_len_player_1_Reaper_len_player_1_diff_Overlord_len_player_0_Reaper_len_player_0_diff_diff', 'Overlord_len_player_1_Starport_len_player_1_diff_Overlord_len_player_0_Starport_len_player_0_diff_diff', 'Overlord_right_click_target_count_player_1', 'OverseerCocoon_len_player_0', 'OverseerCocoon_len_player_1', 'Phoenix_len_player_0', 'Phoenix_len_player_1', 'PhotonCannon_len_player_1', 'PhotonCannon_right_click_target_count_player_1', 'PhysicsCapsule_len_player_0', 'PhysicsCapsule_len_player_1', 'PhysicsCapsule_right_click_target_count_diff', 'PhysicsCapsule_right_click_target_count_player_0', 'PhysicsCapsule_right_click_target_count_player_1', 'PlanetaryFortress_len_player_1', 'Probe_len_player_0', 'Probe_len_player_0_Barracks_len_player_0_diff', 'Probe_len_player_0_Hellion_len_player_0_diff', 'Probe_len_player_0_MULE_len_player_0_diff', 'Probe_len_player_0_OrbitalCommand_len_player_0_diff', 'Probe_len_player_0_Reaper_len_player_0_diff', 'Probe_len_player_0_Roach_len_player_0_diff', 'Probe_len_player_0_Starport_len_player_0_diff', 'Probe_len_player_1_BanelingCocoon_len_player_1_diff_Probe_len_player_0_BanelingCocoon_len_player_0_diff_diff', 'Probe_len_player_1_Barracks_len_player_1_diff', 'Probe_len_player_1_Factory_len_player_1_diff_Probe_len_player_0_Factory_len_player_0_diff_diff', 'Probe_len_player_1_Hellion_len_player_1_diff', 'Probe_len_player_1_MULE_len_player_1_diff', 'Probe_len_player_1_MULE_len_player_1_diff_Probe_len_player_0_MULE_len_player_0_diff_diff', 'Probe_len_player_1_OrbitalCommand_len_player_1_diff', 'Probe_len_player_1_Overlord_len_player_1_diff', 'Probe_len_player_1_Queen_len_player_1_diff', 'Probe_len_player_1_Reaper_len_player_1_diff', 'Probe_len_player_1_Starport_len_player_1_diff', 'Probe_len_player_1_SupplyDepotLowered_len_player_1_diff', 'Pylon_len_player_0', 'Pylon_len_player_1', 'Pylon_right_click_target_count_player_0', 'QueenBurrowed_len_player_0', 'QueenBurrowed_len_player_1', 'Queen_len_player_0', 'Queen_len_player_0_Factory_len_player_0_diff', 'Queen_len_player_0_Hatchery_len_player_0_diff', 'Queen_len_player_0_Hellion_len_player_0_diff', 'Queen_len_player_0_Reaper_len_player_0_diff', 'Queen_len_player_0_Roach_len_player_0_diff', 'Queen_len_player_0_Starport_len_player_0_diff', 'Queen_len_player_1', 'Queen_len_player_1_Factory_len_player_1_diff', 'Queen_len_player_1_Hatchery_len_player_1_diff', 'Queen_len_player_1_Hatchery_len_player_1_diff_Queen_len_player_0_Hatchery_len_player_0_diff_diff', 'Queen_len_player_1_Hellion_len_player_1_diff', 'Queen_len_player_1_Hellion_len_player_1_diff_Queen_len_player_0_Hellion_len_player_0_diff_diff', 'Queen_len_player_1_MULE_len_player_1_diff', 'Queen_len_player_1_Reaper_len_player_1_diff', 'Queen_len_player_1_Reaper_len_player_1_diff_Queen_len_player_0_Reaper_len_player_0_diff_diff', 'Queen_len_player_1_Starport_len_player_1_diff', 'Queen_right_click_target_count_player_1', 'Raven_len_player_0', 'Raven_len_player_1', 'Reactor_len_player_1', 'Reaper_len_player_0', 'Reaper_len_player_0_Roach_len_player_0_diff', 'Reaper_len_player_1', 'Reaper_len_player_1_Roach_len_player_1_diff', 'Reaper_len_player_1_Starport_len_player_1_diff', 'Refinery_len_player_0', 'Refinery_len_player_1', 'Refinery_right_click_target_count_diff', 'RichMineralField750_len_player_0', 'RichMineralField750_len_player_1', 'RichMineralField_len_player_0', 'RichMineralField_len_player_1', 'Right Click_player_0_isnull', 'Right Click_player_1', 'Right Click_player_1_isnull', 'RoachBurrowed_len_player_0', 'RoachBurrowed_len_player_1', 'Roach_len_player_0_Starport_len_player_0_diff', 'Roach_len_player_1', 'Roach_len_player_1_Starport_len_player_1_diff', 'Roach_right_click_target_count_diff', 'Roach_right_click_target_count_player_0', 'Roach_right_click_target_count_player_1', 'RoboticsBay_len_player_0', 'RoboticsFacility_len_player_0', 'Rocks2x2NonConjoined_len_player_1', 'SCV_len_player_0', 'SCV_len_player_0_Hatchery_len_player_0_diff', 'SCV_len_player_0_Overlord_len_player_0_diff', 'SCV_len_player_0_Queen_len_player_0_diff', 'SCV_len_player_0_Roach_len_player_0_diff', 'SCV_len_player_0_SupplyDepotLowered_len_player_0_diff', 'SCV_len_player_0_Zergling_len_player_0_diff', 'SCV_len_player_1', 'SCV_len_player_1_BanelingCocoon_len_player_1_diff', 'SCV_len_player_1_Barracks_len_player_1_diff', 'SCV_len_player_1_Drone_len_player_1_diff', 'SCV_len_player_1_Factory_len_player_1_diff_SCV_len_player_0_Factory_len_player_0_diff_diff', 'SCV_len_player_1_Hatchery_len_player_1_diff', 'SCV_len_player_1_Hatchery_len_player_1_diff_SCV_len_player_0_Hatchery_len_player_0_diff_diff', 'SCV_len_player_1_Hellion_len_player_1_diff_SCV_len_player_0_Hellion_len_player_0_diff_diff', 'SCV_len_player_1_MULE_len_player_1_diff', 'SCV_len_player_1_MULE_len_player_1_diff_SCV_len_player_0_MULE_len_player_0_diff_diff', 'SCV_len_player_1_Overlord_len_player_1_diff_SCV_len_player_0_Overlord_len_player_0_diff_diff', 'SCV_len_player_1_Probe_len_player_1_diff_SCV_len_player_0_Probe_len_player_0_diff_diff', 'SCV_len_player_1_Queen_len_player_1_diff', 'SCV_len_player_1_Queen_len_player_1_diff_SCV_len_player_0_Queen_len_player_0_diff_diff', 'SCV_len_player_1_Roach_len_player_1_diff', 'SCV_len_player_1_Starport_len_player_1_diff_SCV_len_player_0_Starport_len_player_0_diff_diff', 'SCV_len_player_1_SupplyDepotLowered_len_player_1_diff', 'SCV_right_click_target_count_diff', 'SCV_right_click_target_count_player_0', 'SCV_right_click_target_count_player_1', 'Selection_player_0_isnull', 'Selection_player_1_isnull', 'SensorTower_len_player_0', 'SensorTower_len_player_1', 'Sentry_len_player_0', 'Sentry_len_player_1', 'SetControlGroup_player_0_isnull', 'SetControlGroup_player_1', 'SiegeTankSieged_len_player_1', 'SiegeTankSieged_right_click_target_count_diff', 'SiegeTankSieged_right_click_target_count_player_0', 'SiegeTank_len_player_1', 'SiegeTank_right_click_target_count_diff', 'SiegeTank_right_click_target_count_player_0', 'SpawningPool_len_player_0', 'SpawningPool_right_click_target_count_diff', 'SpawningPool_right_click_target_count_player_0', 'SpawningPool_right_click_target_count_player_1', 'SpineCrawlerUprooted_len_player_0', 'SpineCrawlerUprooted_len_player_1', 'SpineCrawler_len_player_0', 'SpineCrawler_len_player_1', 'Spire_len_player_0', 'Spire_len_player_1', 'SporeCrawlerUprooted_len_player_0', 'SporeCrawlerUprooted_len_player_1', 'SporeCrawler_len_player_1', 'Stalker_len_player_0', 'Stalker_right_click_target_count_diff', 'StarportReactor_len_player_0', 'StarportReactor_len_player_1', 'StarportTechLab_len_player_0', 'StarportTechLab_len_player_1', 'Starport_len_player_0', 'Starport_len_player_1', 'Starport_right_click_target_count_diff', 'Starport_right_click_target_count_player_0', 'SupplyDepotLowered_len_player_0', 'SupplyDepotLowered_len_player_0_BanelingCocoon_len_player_0_diff', 'SupplyDepotLowered_len_player_0_Factory_len_player_0_diff', 'SupplyDepotLowered_len_player_0_MULE_len_player_0_diff', 'SupplyDepotLowered_len_player_0_Overlord_len_player_0_diff', 'SupplyDepotLowered_len_player_0_Queen_len_player_0_diff', 'SupplyDepotLowered_len_player_1', 'SupplyDepotLowered_len_player_1_BanelingCocoon_len_player_1_diff', 'SupplyDepotLowered_len_player_1_BanelingCocoon_len_player_1_diff_SupplyDepotLowered_len_player_0_BanelingCocoon_len_player_0_diff_diff', 'SupplyDepotLowered_len_player_1_Hatchery_len_player_1_diff', 'SupplyDepotLowered_len_player_1_Hatchery_len_player_1_diff_SupplyDepotLowered_len_player_0_Hatchery_len_player_0_diff_diff', 'SupplyDepotLowered_len_player_1_Hellion_len_player_1_diff', 'SupplyDepotLowered_len_player_1_Hellion_len_player_1_diff_SupplyDepotLowered_len_player_0_Hellion_len_player_0_diff_diff', 'SupplyDepotLowered_len_player_1_MULE_len_player_1_diff', 'SupplyDepotLowered_len_player_1_Queen_len_player_1_diff', 'SupplyDepotLowered_len_player_1_Roach_len_player_1_diff', 'SupplyDepotLowered_len_player_1_Starport_len_player_1_diff_SupplyDepotLowered_len_player_0_Starport_len_player_0_diff_diff', 'SupplyDepotLowered_right_click_target_count_diff', 'SupplyDepotLowered_right_click_target_count_player_1', 'SupplyDepot_right_click_target_count_diff', 'SupplyDepot_right_click_target_count_player_0', 'SwarmHost_len_player_1', 'TechLab_len_player_0', 'TechLab_len_player_1', 'TemplarArchive_len_player_0', 'Thor_len_player_0', 'Thor_len_player_1', 'UltraliskCavern_len_player_1', 'VikingAssault_len_player_0', 'VikingAssault_len_player_1', 'Viking_len_player_0', 'Viking_len_player_1', 'VoidRay_len_player_0', 'WarpGate_right_click_target_count_diff', 'WarpPrismPhasing_len_player_0', 'WarpPrismPhasing_len_player_1', 'WarpPrism_len_player_0', 'Weapon_len_player_0', 'Weapon_len_player_1', 'XelNagaDestructibleBlocker8W_len_player_0', 'XelNagaDestructibleBlocker8W_len_player_1', 'XelNagaTower_len_player_0', 'XelNagaTower_len_player_1', 'XelNagaTower_right_click_target_count_diff', 'XelNagaTower_right_click_target_count_player_1', 'Zealot_len_player_0', 'Zealot_len_player_1', 'Zealot_right_click_target_count_player_0', 'ZerglingBurrowed_len_player_0', 'ZerglingBurrowed_len_player_1', 'Zergling_len_player_0', 'Zergling_len_player_0_BanelingCocoon_len_player_0_diff', 'Zergling_len_player_0_Factory_len_player_0_diff', 'Zergling_len_player_0_Hatchery_len_player_0_diff', 'Zergling_len_player_0_Hellion_len_player_0_diff', 'Zergling_len_player_0_MULE_len_player_0_diff', 'Zergling_len_player_0_Queen_len_player_0_diff', 'Zergling_len_player_0_Reaper_len_player_0_diff', 'Zergling_len_player_0_Roach_len_player_0_diff', 'Zergling_len_player_1', 'Zergling_len_player_1_Barracks_len_player_1_diff', 'Zergling_len_player_1_Barracks_len_player_1_diff_Zergling_len_player_0_Barracks_len_player_0_diff_diff', 'Zergling_len_player_1_Factory_len_player_1_diff', 'Zergling_len_player_1_Factory_len_player_1_diff_Zergling_len_player_0_Factory_len_player_0_diff_diff', 'Zergling_len_player_1_MULE_len_player_1_diff', 'Zergling_len_player_1_MULE_len_player_1_diff_Zergling_len_player_0_MULE_len_player_0_diff_diff', 'Zergling_len_player_1_OrbitalCommand_len_player_1_diff', 'Zergling_len_player_1_Overlord_len_player_1_diff_Zergling_len_player_0_Overlord_len_player_0_diff_diff', 'Zergling_len_player_1_Reaper_len_player_1_diff', 'Zergling_len_player_1_Reaper_len_player_1_diff_Zergling_len_player_0_Reaper_len_player_0_diff_diff', 'Zergling_len_player_1_Roach_len_player_1_diff_Zergling_len_player_0_Roach_len_player_0_diff_diff', 'Zergling_len_player_1_Starport_len_player_1_diff', 'Zergling_len_player_1_SupplyDepotLowered_len_player_1_diff', 'Zergling_len_player_1_SupplyDepotLowered_len_player_1_diff_Zergling_len_player_0_SupplyDepotLowered_len_player_0_diff_diff', 'ability_code_1000_diff', 'ability_code_1000_player_0', 'ability_code_1000_player_1', 'ability_code_1020_diff', 'ability_code_1020_first_time_player_1', 'ability_code_1020_player_0', 'ability_code_1020_player_0_ability_code_102A_player_0_diff', 'ability_code_1020_player_0_ability_code_1261_player_0_diff', 'ability_code_1020_player_0_ability_code_1E60_player_0_diff', 'ability_code_1020_player_1', 'ability_code_1020_player_1_ability_code_102A_player_1_diff_ability_code_1020_player_0_ability_code_102A_player_0_diff_diff', 'ability_code_1020_player_1_ability_code_1261_player_1_diff', 'ability_code_1020_player_1_ability_code_16E2_player_1_diff', 'ability_code_1020_player_1_ability_code_1E60_player_1_diff', 'ability_code_1020_player_1_ability_code_480_player_1_diff', 'ability_code_1020_player_1_ability_code_480_player_1_diff_ability_code_1020_player_0_ability_code_480_player_0_diff_diff', 'ability_code_1021_first_time_player_1', 'ability_code_1021_last_first_diff', 'ability_code_1021_player_0_ability_code_1020_player_0_diff', 'ability_code_1021_player_0_ability_code_102A_player_0_diff', 'ability_code_1021_player_0_ability_code_1261_player_0_diff', 'ability_code_1021_player_0_ability_code_13A0_player_0_diff', 'ability_code_1021_player_0_ability_code_15E0_player_0_diff', 'ability_code_1021_player_0_ability_code_16E2_player_0_diff', 'ability_code_1021_player_0_ability_code_1821_player_0_diff', 'ability_code_1021_player_0_ability_code_1C60_player_0_diff', 'ability_code_1021_player_0_ability_code_1E60_player_0_diff', 'ability_code_1021_player_1_ability_code_1020_player_1_diff', 'ability_code_1021_player_1_ability_code_1022_player_1_diff', 'ability_code_1021_player_1_ability_code_1022_player_1_diff_ability_code_1021_player_0_ability_code_1022_player_0_diff_diff', 'ability_code_1021_player_1_ability_code_1023_player_1_diff', 'ability_code_1021_player_1_ability_code_102A_player_1_diff', 'ability_code_1021_player_1_ability_code_1261_player_1_diff', 'ability_code_1021_player_1_ability_code_15E0_player_1_diff_ability_code_1021_player_0_ability_code_15E0_player_0_diff_diff', 'ability_code_1021_player_1_ability_code_16E2_player_1_diff_ability_code_1021_player_0_ability_code_16E2_player_0_diff_diff', 'ability_code_1021_player_1_ability_code_1822_player_1_diff', 'ability_code_1021_player_1_ability_code_1822_player_1_diff_ability_code_1021_player_0_ability_code_1822_player_0_diff_diff', 'ability_code_1021_player_1_ability_code_1C60_player_1_diff', 'ability_code_1021_player_1_ability_code_1C60_player_1_diff_ability_code_1021_player_0_ability_code_1C60_player_0_diff_diff', 'ability_code_1021_player_1_ability_code_1E60_player_1_diff', 'ability_code_1021_player_1_ability_code_1E60_player_1_diff_ability_code_1021_player_0_ability_code_1E60_player_0_diff_diff', 'ability_code_1021_player_1_ability_code_480_player_1_diff', 'ability_code_1021_player_1_ability_code_B40_player_1_diff', 'ability_code_1021_player_1_ability_code_B40_player_1_diff_ability_code_1021_player_0_ability_code_B40_player_0_diff_diff', 'ability_code_1021_player_1_ability_code_DE0_player_1_diff', 'ability_code_1021_player_1_ability_code_DE0_player_1_diff_ability_code_1021_player_0_ability_code_DE0_player_0_diff_diff', 'ability_code_1022_diff', 'ability_code_1022_first_time_diff', 'ability_code_1022_first_time_player_0', 'ability_code_1022_first_time_player_1', 'ability_code_1022_player_0', 'ability_code_1022_player_0_ability_code_1261_player_0_diff', 'ability_code_1022_player_0_ability_code_13A0_player_0_diff', 'ability_code_1022_player_0_ability_code_16E2_player_0_diff', 'ability_code_1022_player_0_ability_code_1821_player_0_diff', 'ability_code_1022_player_0_ability_code_1822_player_0_diff', 'ability_code_1022_player_0_ability_code_1E60_player_0_diff', 'ability_code_1022_player_1', 'ability_code_1022_player_1_ability_code_1020_player_1_diff', 'ability_code_1022_player_1_ability_code_102A_player_1_diff', 'ability_code_1022_player_1_ability_code_13A0_player_1_diff', 'ability_code_1022_player_1_ability_code_16E2_player_1_diff', 'ability_code_1022_player_1_ability_code_1821_player_1_diff', 'ability_code_1022_player_1_ability_code_1822_player_1_diff', 'ability_code_1022_player_1_ability_code_1C60_player_1_diff', 'ability_code_1022_player_1_ability_code_1E60_player_1_diff', 'ability_code_1022_player_1_ability_code_1E60_player_1_diff_ability_code_1022_player_0_ability_code_1E60_player_0_diff_diff', 'ability_code_1023_diff', 'ability_code_1023_first_time_player_1', 'ability_code_1023_last_first_player_1', 'ability_code_1023_player_0_ability_code_102A_player_0_diff', 'ability_code_1023_player_0_ability_code_15E0_player_0_diff', 'ability_code_1023_player_0_ability_code_1821_player_0_diff', 'ability_code_1023_player_0_ability_code_1822_player_0_diff', 'ability_code_1023_player_0_ability_code_1C60_player_0_diff', 'ability_code_1023_player_0_ability_code_480_player_0_diff', 'ability_code_1023_player_1', 'ability_code_1023_player_1_ability_code_1020_player_1_diff', 'ability_code_1023_player_1_ability_code_1020_player_1_diff_ability_code_1023_player_0_ability_code_1020_player_0_diff_diff', 'ability_code_1023_player_1_ability_code_102A_player_1_diff', 'ability_code_1023_player_1_ability_code_1261_player_1_diff_ability_code_1023_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_1023_player_1_ability_code_16E2_player_1_diff', 'ability_code_1023_player_1_ability_code_16E2_player_1_diff_ability_code_1023_player_0_ability_code_16E2_player_0_diff_diff', 'ability_code_1023_player_1_ability_code_1822_player_1_diff', 'ability_code_1023_player_1_ability_code_1822_player_1_diff_ability_code_1023_player_0_ability_code_1822_player_0_diff_diff', 'ability_code_1023_player_1_ability_code_1C60_player_1_diff', 'ability_code_1023_player_1_ability_code_1E60_player_1_diff', 'ability_code_1023_player_1_ability_code_480_player_1_diff', 'ability_code_1023_player_1_ability_code_480_player_1_diff_ability_code_1023_player_0_ability_code_480_player_0_diff_diff', 'ability_code_1024_first_time_diff', 'ability_code_1024_first_time_player_0', 'ability_code_1024_player_1', 'ability_code_1025_diff', 'ability_code_1025_first_time_diff', 'ability_code_1025_first_time_player_0', 'ability_code_1025_first_time_player_1', 'ability_code_1025_player_1', 'ability_code_1026_first_time_diff', 'ability_code_1026_player_1', 'ability_code_1028_diff', 'ability_code_1028_player_0', 'ability_code_1028_player_1', 'ability_code_1029_diff', 'ability_code_1029_player_0', 'ability_code_1029_player_1', 'ability_code_102A_diff', 'ability_code_102A_first_time_player_1', 'ability_code_102A_player_0', 'ability_code_102A_player_0_ability_code_480_player_0_diff', 'ability_code_102A_player_1', 'ability_code_102A_player_1_ability_code_1261_player_1_diff', 'ability_code_102A_player_1_ability_code_480_player_1_diff', 'ability_code_102A_player_1_ability_code_480_player_1_diff_ability_code_102A_player_0_ability_code_480_player_0_diff_diff', 'ability_code_102B_first_time_player_0', 'ability_code_102B_first_time_player_1', 'ability_code_102B_player_1', 'ability_code_102D_first_time_diff', 'ability_code_102D_first_time_player_1', 'ability_code_102D_player_0', 'ability_code_102D_player_1', 'ability_code_102F_player_0', 'ability_code_102F_player_1', 'ability_code_103E_diff', 'ability_code_103E_player_0', 'ability_code_103E_player_1', 'ability_code_1060_player_1', 'ability_code_1080_diff', 'ability_code_1080_player_0', 'ability_code_1080_player_1', 'ability_code_1081_diff', 'ability_code_1081_player_0', 'ability_code_1081_player_1', 'ability_code_10E0_first_time_diff', 'ability_code_10E0_player_0', 'ability_code_10E0_player_1', 'ability_code_1100_diff', 'ability_code_1100_player_0', 'ability_code_1120_diff', 'ability_code_1120_player_0', 'ability_code_1120_player_1', 'ability_code_1121_diff', 'ability_code_1121_player_0', 'ability_code_1121_player_1', 'ability_code_1140_diff', 'ability_code_1140_player_0', 'ability_code_1140_player_1', 'ability_code_1142_diff', 'ability_code_1142_first_time_diff', 'ability_code_1142_player_0', 'ability_code_1142_player_1', 'ability_code_1143_diff', 'ability_code_1143_player_0', 'ability_code_1143_player_1', 'ability_code_1160_first_time_player_1', 'ability_code_1160_player_0', 'ability_code_1160_player_1', 'ability_code_1180_diff', 'ability_code_1180_player_0', 'ability_code_1180_player_1', 'ability_code_11A0_diff', 'ability_code_11A0_player_0', 'ability_code_11A0_player_1', 'ability_code_11C0_diff', 'ability_code_11C0_player_0', 'ability_code_11C0_player_1', 'ability_code_11E1_player_0', 'ability_code_11E1_player_1', 'ability_code_11E3_diff', 'ability_code_11E3_player_0', 'ability_code_11E3_player_1', 'ability_code_1201_diff', 'ability_code_1201_player_0', 'ability_code_1201_player_1', 'ability_code_1203_diff', 'ability_code_1203_player_0', 'ability_code_1203_player_1', 'ability_code_1204_diff', 'ability_code_1204_player_0', 'ability_code_1204_player_1', 'ability_code_1220_player_0', 'ability_code_1220_player_1', 'ability_code_1240_diff', 'ability_code_1240_player_0', 'ability_code_1240_player_1', 'ability_code_1260_diff', 'ability_code_1260_player_0', 'ability_code_1260_player_1', 'ability_code_1261_diff', 'ability_code_1261_first_time_diff', 'ability_code_1261_first_time_player_1', 'ability_code_1261_player_0', 'ability_code_1261_player_0_ability_code_480_player_0_diff', 'ability_code_1261_player_1', 'ability_code_1261_player_1_ability_code_480_player_1_diff', 'ability_code_127E_diff', 'ability_code_127E_player_0', 'ability_code_127E_player_1', 'ability_code_1280_diff', 'ability_code_1280_first_time_diff', 'ability_code_1280_player_0', 'ability_code_1280_player_1', 'ability_code_12A0_player_0', 'ability_code_12A0_player_1', 'ability_code_12A1_diff', 'ability_code_12A1_first_time_diff', 'ability_code_12A1_first_time_player_0', 'ability_code_12A1_player_0', 'ability_code_12A1_player_1', 'ability_code_12BE_diff', 'ability_code_12BE_player_0', 'ability_code_12BE_player_1', 'ability_code_12C0_first_time_diff', 'ability_code_12C0_player_0', 'ability_code_12C0_player_1', 'ability_code_12E0_diff', 'ability_code_12E0_player_0', 'ability_code_12E0_player_1', 'ability_code_12E1_first_time_diff', 'ability_code_12E1_player_1', 'ability_code_12FE_diff', 'ability_code_12FE_player_0', 'ability_code_12FE_player_1', 'ability_code_1300_diff', 'ability_code_1300_first_time_diff', 'ability_code_1300_first_time_player_0', 'ability_code_1300_first_time_player_1', 'ability_code_1300_player_0', 'ability_code_1300_player_1', 'ability_code_1320_first_time_diff', 'ability_code_1320_player_0', 'ability_code_1320_player_1', 'ability_code_1340_diff', 'ability_code_1340_first_time_diff', 'ability_code_1340_player_0', 'ability_code_1340_player_1', 'ability_code_1360_diff', 'ability_code_1360_first_time_player_1', 'ability_code_1360_player_0', 'ability_code_1360_player_0_ability_code_1020_player_0_diff', 'ability_code_1360_player_0_ability_code_1261_player_0_diff', 'ability_code_1360_player_0_ability_code_1821_player_0_diff', 'ability_code_1360_player_0_ability_code_1822_player_0_diff', 'ability_code_1360_player_0_ability_code_1E60_player_0_diff', 'ability_code_1360_player_0_ability_code_480_player_0_diff', 'ability_code_1360_player_0_ability_code_DE0_player_0_diff', 'ability_code_1360_player_1', 'ability_code_1360_player_1_ability_code_1020_player_1_diff', 'ability_code_1360_player_1_ability_code_1020_player_1_diff_ability_code_1360_player_0_ability_code_1020_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_1021_player_1_diff', 'ability_code_1360_player_1_ability_code_1022_player_1_diff', 'ability_code_1360_player_1_ability_code_1023_player_1_diff', 'ability_code_1360_player_1_ability_code_102A_player_1_diff', 'ability_code_1360_player_1_ability_code_1261_player_1_diff', 'ability_code_1360_player_1_ability_code_13E0_player_1_diff', 'ability_code_1360_player_1_ability_code_15E0_player_1_diff', 'ability_code_1360_player_1_ability_code_16E2_player_1_diff', 'ability_code_1360_player_1_ability_code_16E2_player_1_diff_ability_code_1360_player_0_ability_code_16E2_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_1820_player_1_diff', 'ability_code_1360_player_1_ability_code_1820_player_1_diff_ability_code_1360_player_0_ability_code_1820_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_1821_player_1_diff', 'ability_code_1360_player_1_ability_code_1822_player_1_diff', 'ability_code_1360_player_1_ability_code_1C60_player_1_diff', 'ability_code_1360_player_1_ability_code_1C60_player_1_diff_ability_code_1360_player_0_ability_code_1C60_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_1E60_player_1_diff', 'ability_code_1360_player_1_ability_code_1E60_player_1_diff_ability_code_1360_player_0_ability_code_1E60_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_DE0_player_1_diff_ability_code_1360_player_0_ability_code_DE0_player_0_diff_diff', 'ability_code_1380_first_time_player_1', 'ability_code_1380_player_0', 'ability_code_1380_player_1', 'ability_code_13A0_diff', 'ability_code_13A0_first_time_player_0', 'ability_code_13A0_first_time_player_1', 'ability_code_13A0_player_0_ability_code_102A_player_0_diff', 'ability_code_13A0_player_0_ability_code_1261_player_0_diff', 'ability_code_13A0_player_0_ability_code_15E0_player_0_diff', 'ability_code_13A0_player_0_ability_code_16E2_player_0_diff', 'ability_code_13A0_player_0_ability_code_1821_player_0_diff', 'ability_code_13A0_player_0_ability_code_1E60_player_0_diff', 'ability_code_13A0_player_0_ability_code_480_player_0_diff', 'ability_code_13A0_player_1', 'ability_code_13A0_player_1_ability_code_1020_player_1_diff_ability_code_13A0_player_0_ability_code_1020_player_0_diff_diff', 'ability_code_13A0_player_1_ability_code_1261_player_1_diff', 'ability_code_13A0_player_1_ability_code_15E0_player_1_diff', 'ability_code_13A0_player_1_ability_code_16E2_player_1_diff', 'ability_code_13A0_player_1_ability_code_16E2_player_1_diff_ability_code_13A0_player_0_ability_code_16E2_player_0_diff_diff', 'ability_code_13A0_player_1_ability_code_1821_player_1_diff', 'ability_code_13A0_player_1_ability_code_1C60_player_1_diff', 'ability_code_13A0_player_1_ability_code_1C60_player_1_diff_ability_code_13A0_player_0_ability_code_1C60_player_0_diff_diff', 'ability_code_13A0_player_1_ability_code_1E60_player_1_diff', 'ability_code_13A0_player_1_ability_code_1E60_player_1_diff_ability_code_13A0_player_0_ability_code_1E60_player_0_diff_diff', 'ability_code_13A0_player_1_ability_code_480_player_1_diff', 'ability_code_13A0_player_1_ability_code_480_player_1_diff_ability_code_13A0_player_0_ability_code_480_player_0_diff_diff', 'ability_code_13C0_player_0', 'ability_code_13C0_player_1', 'ability_code_13E0_diff', 'ability_code_13E0_first_time_diff', 'ability_code_13E0_first_time_player_1', 'ability_code_13E0_last_first_player_1', 'ability_code_13E0_player_0', 'ability_code_13E0_player_0_ability_code_1261_player_0_diff', 'ability_code_13E0_player_0_ability_code_13A0_player_0_diff', 'ability_code_13E0_player_0_ability_code_16E2_player_0_diff', 'ability_code_13E0_player_0_ability_code_1821_player_0_diff', 'ability_code_13E0_player_0_ability_code_1822_player_0_diff', 'ability_code_13E0_player_0_ability_code_1E60_player_0_diff', 'ability_code_13E0_player_0_ability_code_B40_player_0_diff', 'ability_code_13E0_player_0_ability_code_DE0_player_0_diff', 'ability_code_13E0_player_1_ability_code_1020_player_1_diff', 'ability_code_13E0_player_1_ability_code_1021_player_1_diff', 'ability_code_13E0_player_1_ability_code_1021_player_1_diff_ability_code_13E0_player_0_ability_code_1021_player_0_diff_diff', 'ability_code_13E0_player_1_ability_code_1022_player_1_diff', 'ability_code_13E0_player_1_ability_code_1023_player_1_diff_ability_code_13E0_player_0_ability_code_1023_player_0_diff_diff', 'ability_code_13E0_player_1_ability_code_102A_player_1_diff', 'ability_code_13E0_player_1_ability_code_102A_player_1_diff_ability_code_13E0_player_0_ability_code_102A_player_0_diff_diff', 'ability_code_13E0_player_1_ability_code_1261_player_1_diff', 'ability_code_13E0_player_1_ability_code_1261_player_1_diff_ability_code_13E0_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_13E0_player_1_ability_code_13A0_player_1_diff', 'ability_code_13E0_player_1_ability_code_15E0_player_1_diff', 'ability_code_13E0_player_1_ability_code_16E2_player_1_diff', 'ability_code_13E0_player_1_ability_code_1821_player_1_diff', 'ability_code_13E0_player_1_ability_code_1822_player_1_diff', 'ability_code_13E0_player_1_ability_code_1822_player_1_diff_ability_code_13E0_player_0_ability_code_1822_player_0_diff_diff', 'ability_code_13E0_player_1_ability_code_1C60_player_1_diff', 'ability_code_13E0_player_1_ability_code_1C60_player_1_diff_ability_code_13E0_player_0_ability_code_1C60_player_0_diff_diff', 'ability_code_13E0_player_1_ability_code_1E60_player_1_diff', 'ability_code_13E0_player_1_ability_code_480_player_1_diff', 'ability_code_13E0_player_1_ability_code_480_player_1_diff_ability_code_13E0_player_0_ability_code_480_player_0_diff_diff', 'ability_code_13E0_player_1_ability_code_DE0_player_1_diff', 'ability_code_13E1_diff', 'ability_code_13E1_player_0', 'ability_code_13E1_player_1', 'ability_code_13E2_diff', 'ability_code_13E2_player_0', 'ability_code_13E2_player_1', 'ability_code_13E3_first_time_diff', 'ability_code_13E3_player_1', 'ability_code_1401_diff', 'ability_code_1401_player_1', 'ability_code_1404_diff', 'ability_code_1404_player_0', 'ability_code_1404_player_1', 'ability_code_1405_first_time_diff', 'ability_code_1405_first_time_player_1', 'ability_code_1405_player_0', 'ability_code_1405_player_1', 'ability_code_1406_diff', 'ability_code_1406_player_0', 'ability_code_1406_player_1', 'ability_code_1407_diff', 'ability_code_1407_first_time_diff', 'ability_code_1407_first_time_player_0', 'ability_code_1407_first_time_player_1', 'ability_code_1407_player_0', 'ability_code_1418_diff', 'ability_code_1418_first_time_diff', 'ability_code_1418_player_0', 'ability_code_1418_player_1', 'ability_code_1420_diff', 'ability_code_1420_first_time_diff', 'ability_code_1420_first_time_player_0', 'ability_code_1420_player_0', 'ability_code_1420_player_1', 'ability_code_1421_player_0', 'ability_code_1421_player_1', 'ability_code_1422_player_1', 'ability_code_1423_diff', 'ability_code_1423_player_0', 'ability_code_1423_player_1', 'ability_code_1424_diff', 'ability_code_1424_first_time_diff', 'ability_code_1424_first_time_player_0', 'ability_code_1424_first_time_player_1', 'ability_code_1424_player_0', 'ability_code_1424_player_1', 'ability_code_1426_player_1', 'ability_code_1440_diff', 'ability_code_1440_player_0', 'ability_code_1440_player_1', 'ability_code_1441_diff', 'ability_code_1441_player_0', 'ability_code_1441_player_1', 'ability_code_1442_diff', 'ability_code_1442_first_time_diff', 'ability_code_1442_first_time_player_1', 'ability_code_1442_player_0', 'ability_code_1442_player_1', 'ability_code_1443_diff', 'ability_code_1443_player_0', 'ability_code_1443_player_1', 'ability_code_1445_diff', 'ability_code_1445_player_0', 'ability_code_1445_player_1', 'ability_code_1446_first_time_diff', 'ability_code_1446_first_time_player_1', 'ability_code_1446_player_0', 'ability_code_1446_player_1', 'ability_code_1447_diff', 'ability_code_1447_player_0', 'ability_code_1447_player_1', 'ability_code_1480_diff', 'ability_code_1480_player_0', 'ability_code_1480_player_1', 'ability_code_14A0_player_1', 'ability_code_14A1_diff', 'ability_code_14A1_player_0', 'ability_code_14A1_player_1', 'ability_code_14A2_diff', 'ability_code_14A2_player_0', 'ability_code_14A2_player_1', 'ability_code_14C1_diff', 'ability_code_14C1_player_0', 'ability_code_14C1_player_1', 'ability_code_14C4_diff', 'ability_code_14C4_player_0', 'ability_code_14C4_player_1', 'ability_code_14E0_diff', 'ability_code_14E0_player_0', 'ability_code_14E0_player_1', 'ability_code_14E3_diff', 'ability_code_14E3_player_0', 'ability_code_14E3_player_1', 'ability_code_14E9_diff', 'ability_code_14E9_player_0', 'ability_code_14E9_player_1', 'ability_code_14ED_diff', 'ability_code_14ED_player_0', 'ability_code_14ED_player_1', 'ability_code_14EE_diff', 'ability_code_14EE_player_0', 'ability_code_14EE_player_1', 'ability_code_14EF_diff', 'ability_code_14EF_player_0', 'ability_code_14EF_player_1', 'ability_code_1500_diff', 'ability_code_1500_player_0', 'ability_code_1500_player_1', 'ability_code_1525_diff', 'ability_code_1525_player_0', 'ability_code_1525_player_1', 'ability_code_1526_diff', 'ability_code_1526_player_0', 'ability_code_1526_player_1', 'ability_code_152B_diff', 'ability_code_152B_player_0', 'ability_code_152B_player_1', 'ability_code_152E_player_1', 'ability_code_152F_diff', 'ability_code_152F_player_0', 'ability_code_152F_player_1', 'ability_code_1541_first_time_diff', 'ability_code_1542_first_time_player_0', 'ability_code_1544_player_0', 'ability_code_1545_player_0', 'ability_code_1546_first_time_player_0', 'ability_code_1546_player_0', 'ability_code_1546_player_1', 'ability_code_1547_first_time_diff', 'ability_code_1547_first_time_player_1', 'ability_code_1547_player_1', 'ability_code_1549_player_0', 'ability_code_154A_player_0', 'ability_code_154A_player_1', 'ability_code_154B_player_0', 'ability_code_154C_player_0', 'ability_code_154D_first_time_diff', 'ability_code_154D_player_0', 'ability_code_154D_player_1', 'ability_code_154E_diff', 'ability_code_154E_player_0', 'ability_code_154E_player_1', 'ability_code_1562_player_0', 'ability_code_1563_diff', 'ability_code_1563_player_0', 'ability_code_1563_player_1', 'ability_code_1580_diff', 'ability_code_1580_first_time_diff', 'ability_code_1580_first_time_player_1', 'ability_code_1580_player_0', 'ability_code_1581_diff', 'ability_code_1581_first_time_diff', 'ability_code_1583_diff', 'ability_code_1583_player_1', 'ability_code_1584_diff', 'ability_code_1584_player_0', 'ability_code_1584_player_1', 'ability_code_1585_diff', 'ability_code_1585_player_0', 'ability_code_1586_first_time_diff', 'ability_code_1586_player_0', 'ability_code_15A0_diff', 'ability_code_15A0_player_1', 'ability_code_15A2_player_0', 'ability_code_15A4_player_0', 'ability_code_15A8_first_time_diff', 'ability_code_15A8_player_0', 'ability_code_15A8_player_1', 'ability_code_15A9_diff', 'ability_code_15A9_player_0', 'ability_code_15A9_player_1', 'ability_code_15C0_player_0', 'ability_code_15C0_player_1', 'ability_code_15C1_first_time_diff', 'ability_code_15C1_first_time_player_0', 'ability_code_15C1_first_time_player_1', 'ability_code_15C1_player_0', 'ability_code_15C2_player_0', 'ability_code_15C2_player_1', 'ability_code_15C3_diff', 'ability_code_15C3_first_time_diff', 'ability_code_15D2_diff', 'ability_code_15D2_player_0', 'ability_code_15D2_player_1', 'ability_code_15E0_diff', 'ability_code_15E0_first_time_diff', 'ability_code_15E0_player_0', 'ability_code_15E0_player_0_ability_code_1020_player_0_diff', 'ability_code_15E0_player_0_ability_code_102A_player_0_diff', 'ability_code_15E0_player_0_ability_code_1261_player_0_diff', 'ability_code_15E0_player_0_ability_code_16E2_player_0_diff', 'ability_code_15E0_player_0_ability_code_1E60_player_0_diff', 'ability_code_15E0_player_0_ability_code_480_player_0_diff', 'ability_code_15E0_player_1', 'ability_code_15E0_player_1_ability_code_102A_player_1_diff', 'ability_code_15E0_player_1_ability_code_102A_player_1_diff_ability_code_15E0_player_0_ability_code_102A_player_0_diff_diff', 'ability_code_15E0_player_1_ability_code_1261_player_1_diff', 'ability_code_15E0_player_1_ability_code_1261_player_1_diff_ability_code_15E0_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_15E0_player_1_ability_code_16E2_player_1_diff', 'ability_code_15E0_player_1_ability_code_1C60_player_1_diff', 'ability_code_15E0_player_1_ability_code_1C60_player_1_diff_ability_code_15E0_player_0_ability_code_1C60_player_0_diff_diff', 'ability_code_15E0_player_1_ability_code_480_player_1_diff_ability_code_15E0_player_0_ability_code_480_player_0_diff_diff', 'ability_code_1600_diff', 'ability_code_1600_player_1', 'ability_code_1680_diff', 'ability_code_1680_player_0', 'ability_code_1680_player_1', 'ability_code_1681_diff', 'ability_code_1681_player_0', 'ability_code_1681_player_1', 'ability_code_1683_player_0', 'ability_code_1684_diff', 'ability_code_1684_player_0', 'ability_code_1684_player_1', 'ability_code_1686_diff', 'ability_code_1686_player_0', 'ability_code_1686_player_1', 'ability_code_1687_diff', 'ability_code_1687_player_0', 'ability_code_1687_player_1', 'ability_code_16A1_diff', 'ability_code_16A1_player_0', 'ability_code_16A1_player_1', 'ability_code_16A2_diff', 'ability_code_16A2_player_0', 'ability_code_16A2_player_1', 'ability_code_16A5_diff', 'ability_code_16A5_player_0', 'ability_code_16A5_player_1', 'ability_code_16C4_diff', 'ability_code_16C4_player_0', 'ability_code_16C4_player_1', 'ability_code_16E0_player_0', 'ability_code_16E0_player_1', 'ability_code_16E2_diff', 'ability_code_16E2_first_time_diff', 'ability_code_16E2_player_0', 'ability_code_16E2_player_0_ability_code_102A_player_0_diff', 'ability_code_16E2_player_0_ability_code_1261_player_0_diff', 'ability_code_16E2_player_0_ability_code_480_player_0_diff', 'ability_code_16E2_player_1', 'ability_code_16E2_player_1_ability_code_102A_player_1_diff', 'ability_code_16E2_player_1_ability_code_102A_player_1_diff_ability_code_16E2_player_0_ability_code_102A_player_0_diff_diff', 'ability_code_16E2_player_1_ability_code_1261_player_1_diff', 'ability_code_16E2_player_1_ability_code_1261_player_1_diff_ability_code_16E2_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_16E2_player_1_ability_code_480_player_1_diff', 'ability_code_16E2_player_1_ability_code_480_player_1_diff_ability_code_16E2_player_0_ability_code_480_player_0_diff_diff', 'ability_code_16E3_diff', 'ability_code_16E3_first_time_diff', 'ability_code_16E3_first_time_player_1', 'ability_code_16E3_player_0', 'ability_code_16E4_diff', 'ability_code_16E4_first_time_diff', 'ability_code_16E4_first_time_player_1', 'ability_code_16E4_player_0', 'ability_code_16E4_player_1', 'ability_code_16E5_player_0', 'ability_code_16E6_player_0', 'ability_code_16E6_player_1', 'ability_code_16E7_diff', 'ability_code_16E7_player_0', 'ability_code_16E7_player_1', 'ability_code_16E8_diff', 'ability_code_16E8_player_0', 'ability_code_16E8_player_1', 'ability_code_16E9_diff', 'ability_code_16E9_player_0', 'ability_code_16E9_player_1', 'ability_code_16EA_first_time_diff', 'ability_code_16EA_player_0', 'ability_code_16ED_diff', 'ability_code_16ED_first_time_diff', 'ability_code_16ED_player_0', 'ability_code_16ED_player_1', 'ability_code_16EE_diff', 'ability_code_16EE_first_time_diff', 'ability_code_16EF_diff', 'ability_code_16EF_first_time_diff', 'ability_code_16EF_player_0', 'ability_code_1700_diff', 'ability_code_1700_player_0', 'ability_code_1700_player_1', 'ability_code_1701_player_1', 'ability_code_1720_diff', 'ability_code_1720_player_0', 'ability_code_1720_player_1', 'ability_code_1721_player_0', 'ability_code_1723_first_time_diff', 'ability_code_1723_first_time_player_0', 'ability_code_1723_player_0', 'ability_code_1723_player_1', 'ability_code_1724_diff', 'ability_code_1724_player_0', 'ability_code_1724_player_1', 'ability_code_1726_player_0', 'ability_code_1726_player_1', 'ability_code_1727_diff', 'ability_code_1727_player_0', 'ability_code_1727_player_1', 'ability_code_1740_diff', 'ability_code_1740_first_time_diff', 'ability_code_1740_first_time_player_1', 'ability_code_1740_player_0', 'ability_code_1740_player_1', 'ability_code_1741_diff', 'ability_code_1741_player_0', 'ability_code_1741_player_1', 'ability_code_1760_diff', 'ability_code_1760_player_0', 'ability_code_1760_player_1', 'ability_code_17A3_player_1', 'ability_code_17C1_diff', 'ability_code_17C1_first_time_diff', 'ability_code_17C1_player_0', 'ability_code_17C1_player_1', 'ability_code_17E1_diff', 'ability_code_17E1_player_0', 'ability_code_17E1_player_1', 'ability_code_1800_diff', 'ability_code_1800_player_0', 'ability_code_1800_player_1', 'ability_code_1803_diff', 'ability_code_1803_player_0', 'ability_code_1803_player_1', 'ability_code_1820_first_time_player_0', 'ability_code_1820_player_0', 'ability_code_1820_player_0_ability_code_1020_player_0_diff', 'ability_code_1820_player_0_ability_code_1021_player_0_diff', 'ability_code_1820_player_0_ability_code_1022_player_0_diff', 'ability_code_1820_player_0_ability_code_1023_player_0_diff', 'ability_code_1820_player_0_ability_code_102A_player_0_diff', 'ability_code_1820_player_0_ability_code_1261_player_0_diff', 'ability_code_1820_player_0_ability_code_13A0_player_0_diff', 'ability_code_1820_player_0_ability_code_13E0_player_0_diff', 'ability_code_1820_player_0_ability_code_15E0_player_0_diff', 'ability_code_1820_player_0_ability_code_1822_player_0_diff', 'ability_code_1820_player_0_ability_code_480_player_0_diff', 'ability_code_1820_player_0_ability_code_B40_player_0_diff', 'ability_code_1820_player_0_ability_code_DE0_player_0_diff', 'ability_code_1820_player_1_ability_code_1020_player_1_diff', 'ability_code_1820_player_1_ability_code_1020_player_1_diff_ability_code_1820_player_0_ability_code_1020_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_1021_player_1_diff', 'ability_code_1820_player_1_ability_code_1021_player_1_diff_ability_code_1820_player_0_ability_code_1021_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_1022_player_1_diff', 'ability_code_1820_player_1_ability_code_1022_player_1_diff_ability_code_1820_player_0_ability_code_1022_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_1023_player_1_diff', 'ability_code_1820_player_1_ability_code_1023_player_1_diff_ability_code_1820_player_0_ability_code_1023_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_102A_player_1_diff', 'ability_code_1820_player_1_ability_code_102A_player_1_diff_ability_code_1820_player_0_ability_code_102A_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_1261_player_1_diff', 'ability_code_1820_player_1_ability_code_1261_player_1_diff_ability_code_1820_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_13A0_player_1_diff', 'ability_code_1820_player_1_ability_code_13A0_player_1_diff_ability_code_1820_player_0_ability_code_13A0_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_13E0_player_1_diff', 'ability_code_1820_player_1_ability_code_15E0_player_1_diff_ability_code_1820_player_0_ability_code_15E0_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_16E2_player_1_diff', 'ability_code_1820_player_1_ability_code_1822_player_1_diff', 'ability_code_1820_player_1_ability_code_1822_player_1_diff_ability_code_1820_player_0_ability_code_1822_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_1C60_player_1_diff', 'ability_code_1820_player_1_ability_code_1C60_player_1_diff_ability_code_1820_player_0_ability_code_1C60_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_1E60_player_1_diff', 'ability_code_1820_player_1_ability_code_1E60_player_1_diff_ability_code_1820_player_0_ability_code_1E60_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_480_player_1_diff', 'ability_code_1820_player_1_ability_code_B40_player_1_diff', 'ability_code_1821_diff', 'ability_code_1821_first_time_player_0', 'ability_code_1821_player_0', 'ability_code_1821_player_0_ability_code_1020_player_0_diff', 'ability_code_1821_player_0_ability_code_102A_player_0_diff', 'ability_code_1821_player_0_ability_code_1261_player_0_diff', 'ability_code_1821_player_0_ability_code_15E0_player_0_diff', 'ability_code_1821_player_0_ability_code_16E2_player_0_diff', 'ability_code_1821_player_0_ability_code_1C60_player_0_diff', 'ability_code_1821_player_0_ability_code_1E60_player_0_diff', 'ability_code_1821_player_0_ability_code_480_player_0_diff', 'ability_code_1821_player_1', 'ability_code_1821_player_1_ability_code_1020_player_1_diff', 'ability_code_1821_player_1_ability_code_102A_player_1_diff', 'ability_code_1821_player_1_ability_code_1261_player_1_diff', 'ability_code_1821_player_1_ability_code_1261_player_1_diff_ability_code_1821_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_1821_player_1_ability_code_15E0_player_1_diff', 'ability_code_1821_player_1_ability_code_16E2_player_1_diff', 'ability_code_1821_player_1_ability_code_1C60_player_1_diff', 'ability_code_1822_diff', 'ability_code_1822_first_time_player_1', 'ability_code_1822_last_first_diff', 'ability_code_1822_last_first_player_1', 'ability_code_1822_player_0', 'ability_code_1822_player_0_ability_code_1020_player_0_diff', 'ability_code_1822_player_0_ability_code_102A_player_0_diff', 'ability_code_1822_player_0_ability_code_1261_player_0_diff', 'ability_code_1822_player_0_ability_code_13A0_player_0_diff', 'ability_code_1822_player_0_ability_code_15E0_player_0_diff', 'ability_code_1822_player_0_ability_code_1C60_player_0_diff', 'ability_code_1822_player_0_ability_code_480_player_0_diff', 'ability_code_1822_player_1', 'ability_code_1822_player_1_ability_code_1020_player_1_diff', 'ability_code_1822_player_1_ability_code_102A_player_1_diff', 'ability_code_1822_player_1_ability_code_102A_player_1_diff_ability_code_1822_player_0_ability_code_102A_player_0_diff_diff', 'ability_code_1822_player_1_ability_code_1261_player_1_diff', 'ability_code_1822_player_1_ability_code_1261_player_1_diff_ability_code_1822_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_1822_player_1_ability_code_13A0_player_1_diff', 'ability_code_1822_player_1_ability_code_13A0_player_1_diff_ability_code_1822_player_0_ability_code_13A0_player_0_diff_diff', 'ability_code_1822_player_1_ability_code_15E0_player_1_diff', 'ability_code_1822_player_1_ability_code_15E0_player_1_diff_ability_code_1822_player_0_ability_code_15E0_player_0_diff_diff', 'ability_code_1822_player_1_ability_code_16E2_player_1_diff', 'ability_code_1822_player_1_ability_code_1C60_player_1_diff', 'ability_code_1822_player_1_ability_code_1E60_player_1_diff_ability_code_1822_player_0_ability_code_1E60_player_0_diff_diff', 'ability_code_1822_player_1_ability_code_480_player_1_diff', 'ability_code_1822_player_1_ability_code_480_player_1_diff_ability_code_1822_player_0_ability_code_480_player_0_diff_diff', 'ability_code_1823_player_0', 'ability_code_1823_player_1', 'ability_code_1824_diff', 'ability_code_1824_player_1', 'ability_code_1829_first_time_diff', 'ability_code_1829_first_time_player_1', 'ability_code_182A_diff', 'ability_code_182A_player_0', 'ability_code_182A_player_1', 'ability_code_182B_diff', 'ability_code_182B_player_0', 'ability_code_182B_player_1', 'ability_code_182E_diff', 'ability_code_182E_player_0', 'ability_code_182E_player_1', 'ability_code_1860_diff', 'ability_code_1860_player_0', 'ability_code_1860_player_1', 'ability_code_1880_diff', 'ability_code_1880_player_0', 'ability_code_1880_player_1', 'ability_code_18A0_diff', 'ability_code_18A0_player_0', 'ability_code_18A0_player_1', 'ability_code_18C0_diff', 'ability_code_18C0_player_0', 'ability_code_18C0_player_1', 'ability_code_1920_diff', 'ability_code_1920_player_0', 'ability_code_1920_player_1', 'ability_code_1940_diff', 'ability_code_1940_player_0', 'ability_code_1940_player_1', 'ability_code_1960_diff', 'ability_code_1960_player_0', 'ability_code_1960_player_1', 'ability_code_1980_diff', 'ability_code_1980_player_0', 'ability_code_1980_player_1', 'ability_code_1A62_player_0', 'ability_code_1A62_player_1', 'ability_code_1A63_diff', 'ability_code_1A63_player_0', 'ability_code_1A63_player_1', 'ability_code_1AC0_diff', 'ability_code_1AC0_player_0', 'ability_code_1AC0_player_1', 'ability_code_1AC1_first_time_diff', 'ability_code_1AC1_first_time_player_1', 'ability_code_1AC1_player_0', 'ability_code_1AC1_player_1', 'ability_code_1AC3_player_0', 'ability_code_1AC3_player_1', 'ability_code_1AC4_diff', 'ability_code_1AC4_player_0', 'ability_code_1AC4_player_1', 'ability_code_1AC5_diff', 'ability_code_1AC5_player_0', 'ability_code_1AC6_diff', 'ability_code_1AC6_first_time_diff', 'ability_code_1AC6_first_time_player_0', 'ability_code_1AC6_player_0', 'ability_code_1AC6_player_1', 'ability_code_1AE0_diff', 'ability_code_1AE0_player_0', 'ability_code_1AE0_player_1', 'ability_code_1B00_diff', 'ability_code_1B00_player_0', 'ability_code_1B00_player_1', 'ability_code_1B21_diff', 'ability_code_1B21_player_0', 'ability_code_1B21_player_1', 'ability_code_1B40_diff', 'ability_code_1B40_player_0', 'ability_code_1B40_player_1', 'ability_code_1B60_diff', 'ability_code_1B60_player_1', 'ability_code_1BA0_diff', 'ability_code_1BA1_diff', 'ability_code_1BA1_player_0', 'ability_code_1BA1_player_1', 'ability_code_1BC0_player_1', 'ability_code_1BC1_diff', 'ability_code_1BC1_player_0', 'ability_code_1BC1_player_1', 'ability_code_1BE2_diff', 'ability_code_1BE2_player_0', 'ability_code_1BE2_player_1', 'ability_code_1C00_diff', 'ability_code_1C00_player_0', 'ability_code_1C60_diff', 'ability_code_1C60_first_time_diff', 'ability_code_1C60_first_time_player_1', 'ability_code_1C60_last_first_diff', 'ability_code_1C60_player_0', 'ability_code_1C60_player_0_ability_code_1020_player_0_diff', 'ability_code_1C60_player_0_ability_code_1261_player_0_diff', 'ability_code_1C60_player_0_ability_code_16E2_player_0_diff', 'ability_code_1C60_player_0_ability_code_1E60_player_0_diff', 'ability_code_1C60_player_0_ability_code_480_player_0_diff', 'ability_code_1C60_player_1', 'ability_code_1C60_player_1_ability_code_1020_player_1_diff', 'ability_code_1C60_player_1_ability_code_1020_player_1_diff_ability_code_1C60_player_0_ability_code_1020_player_0_diff_diff', 'ability_code_1C60_player_1_ability_code_102A_player_1_diff', 'ability_code_1C60_player_1_ability_code_102A_player_1_diff_ability_code_1C60_player_0_ability_code_102A_player_0_diff_diff', 'ability_code_1C60_player_1_ability_code_1261_player_1_diff_ability_code_1C60_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_1C60_player_1_ability_code_16E2_player_1_diff', 'ability_code_1C60_player_1_ability_code_16E2_player_1_diff_ability_code_1C60_player_0_ability_code_16E2_player_0_diff_diff', 'ability_code_1C60_player_1_ability_code_1E60_player_1_diff', 'ability_code_1C60_player_1_ability_code_480_player_1_diff', 'ability_code_1C60_player_1_ability_code_480_player_1_diff_ability_code_1C60_player_0_ability_code_480_player_0_diff_diff', 'ability_code_1C61_diff', 'ability_code_1C61_player_0', 'ability_code_1C61_player_1', 'ability_code_1C80_first_time_diff', 'ability_code_1C80_first_time_player_0', 'ability_code_1C80_first_time_player_1', 'ability_code_1C80_player_0', 'ability_code_1CA0_diff', 'ability_code_1CA0_player_0', 'ability_code_1CA0_player_1', 'ability_code_1CC0_diff', 'ability_code_1CC0_player_0', 'ability_code_1CC0_player_1', 'ability_code_1CE0_diff', 'ability_code_1CE0_player_0', 'ability_code_1CE0_player_1', 'ability_code_1D00_diff', 'ability_code_1D00_player_0', 'ability_code_1D00_player_1', 'ability_code_1D20_diff', 'ability_code_1D20_player_0', 'ability_code_1D20_player_1', 'ability_code_1D40_player_0', 'ability_code_1D40_player_1', 'ability_code_1D60_diff', 'ability_code_1D60_player_0', 'ability_code_1D60_player_1', 'ability_code_1D80_player_0', 'ability_code_1D81_diff', 'ability_code_1D81_player_0', 'ability_code_1D81_player_1', 'ability_code_1D83_diff', 'ability_code_1D83_player_0', 'ability_code_1D83_player_1', 'ability_code_1D86_diff', 'ability_code_1D86_first_time_player_0', 'ability_code_1D86_player_1', 'ability_code_1DA0_player_0', 'ability_code_1DA1_diff', 'ability_code_1DA1_player_1', 'ability_code_1DA2_player_0', 'ability_code_1DC0_diff', 'ability_code_1DC0_player_0', 'ability_code_1DC0_player_1', 'ability_code_1E20_diff', 'ability_code_1E20_player_0', 'ability_code_1E60_diff', 'ability_code_1E60_player_0', 'ability_code_1E60_player_0_ability_code_102A_player_0_diff', 'ability_code_1E60_player_0_ability_code_1261_player_0_diff', 'ability_code_1E60_player_0_ability_code_16E2_player_0_diff', 'ability_code_1E60_player_0_ability_code_480_player_0_diff', 'ability_code_1E60_player_1', 'ability_code_1E60_player_1_ability_code_102A_player_1_diff', 'ability_code_1E60_player_1_ability_code_1261_player_1_diff', 'ability_code_1E60_player_1_ability_code_1261_player_1_diff_ability_code_1E60_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_1E60_player_1_ability_code_16E2_player_1_diff', 'ability_code_1E60_player_1_ability_code_480_player_1_diff', 'ability_code_1EA0_diff', 'ability_code_1EA0_player_0', 'ability_code_1EA0_player_1', 'ability_code_1FC0_diff', 'ability_code_1FC0_player_0', 'ability_code_1FC0_player_1', 'ability_code_1FE0_diff', 'ability_code_1FE0_player_0', 'ability_code_1FE0_player_1', 'ability_code_2040_diff', 'ability_code_2040_player_0', 'ability_code_2040_player_1', 'ability_code_2060_diff', 'ability_code_2060_player_0', 'ability_code_2060_player_1', 'ability_code_2061_diff', 'ability_code_2061_player_0', 'ability_code_2061_player_1', 'ability_code_2080_first_time_diff', 'ability_code_2080_first_time_player_1', 'ability_code_20A0_diff', 'ability_code_20A0_player_0', 'ability_code_20C0_diff', 'ability_code_20C0_player_0', 'ability_code_20C0_player_1', 'ability_code_20E0_diff', 'ability_code_20E0_player_0', 'ability_code_20E0_player_1', 'ability_code_20E1_diff', 'ability_code_20E1_player_0', 'ability_code_20E1_player_1', 'ability_code_2100_diff', 'ability_code_2100_player_0', 'ability_code_2100_player_1', 'ability_code_2101_diff', 'ability_code_2101_player_0', 'ability_code_2101_player_1', 'ability_code_2120_first_time_diff', 'ability_code_2120_player_0', 'ability_code_2120_player_1', 'ability_code_213E_diff', 'ability_code_213E_player_0', 'ability_code_213E_player_1', 'ability_code_2140_diff', 'ability_code_2140_player_0', 'ability_code_2140_player_1', 'ability_code_2160_diff', 'ability_code_2160_player_0', 'ability_code_2160_player_1', 'ability_code_2180_diff', 'ability_code_2180_player_0', 'ability_code_2180_player_1', 'ability_code_22A0_diff', 'ability_code_22A0_first_time_diff', 'ability_code_22A0_player_0', 'ability_code_22A0_player_1', 'ability_code_22A1_diff', 'ability_code_22A1_player_0', 'ability_code_26C0_diff', 'ability_code_26C0_player_0', 'ability_code_26C0_player_1', 'ability_code_2720_diff', 'ability_code_2720_first_time_player_1', 'ability_code_2720_player_0', 'ability_code_2720_player_1', 'ability_code_2B00_player_0', 'ability_code_2B40_diff', 'ability_code_2B40_player_0', 'ability_code_2B40_player_1', 'ability_code_2C80_diff', 'ability_code_2C80_player_0', 'ability_code_2C80_player_1', 'ability_code_2DE0_diff', 'ability_code_2DE0_player_0', 'ability_code_2DE0_player_1', 'ability_code_2F60_diff', 'ability_code_2F60_player_0', 'ability_code_2F60_player_1', 'ability_code_3040_diff', 'ability_code_3040_first_time_diff', 'ability_code_3040_first_time_player_0', 'ability_code_3040_first_time_player_1', 'ability_code_3040_player_1', 'ability_code_3060_diff', 'ability_code_3060_player_0', 'ability_code_3060_player_1', 'ability_code_3120_diff', 'ability_code_3120_player_0', 'ability_code_3120_player_1', 'ability_code_3140_diff', 'ability_code_3140_player_0', 'ability_code_3140_player_1', 'ability_code_3160_diff', 'ability_code_3160_player_0', 'ability_code_3160_player_1', 'ability_code_31A0_player_0', 'ability_code_31A0_player_1', 'ability_code_3380_diff', 'ability_code_3380_player_0', 'ability_code_3380_player_1', 'ability_code_3480_diff', 'ability_code_3480_first_time_diff', 'ability_code_3480_first_time_player_0', 'ability_code_3480_player_0', 'ability_code_3480_player_1', 'ability_code_39A0_diff', 'ability_code_39A0_player_0', 'ability_code_39A0_player_1', 'ability_code_4020_diff', 'ability_code_4020_player_0', 'ability_code_4020_player_1', 'ability_code_4021_diff', 'ability_code_4021_player_0', 'ability_code_4021_player_1', 'ability_code_4040_diff', 'ability_code_4040_player_0', 'ability_code_4040_player_1', 'ability_code_40A0_diff', 'ability_code_40A0_player_0', 'ability_code_40A0_player_1', 'ability_code_4120_diff', 'ability_code_4120_player_0', 'ability_code_4120_player_1', 'ability_code_4160_diff', 'ability_code_4160_player_0', 'ability_code_4160_player_1', 'ability_code_41A0_diff', 'ability_code_41A0_player_0', 'ability_code_41A0_player_1', 'ability_code_41E0_diff', 'ability_code_41E0_player_0', 'ability_code_41E0_player_1', 'ability_code_4220_diff', 'ability_code_4220_player_0', 'ability_code_4220_player_1', 'ability_code_42A1_diff', 'ability_code_42A1_player_0', 'ability_code_42A1_player_1', 'ability_code_42C0_diff', 'ability_code_42C0_player_0', 'ability_code_42C1_diff', 'ability_code_42C1_player_0', 'ability_code_42C1_player_1', 'ability_code_43E0_diff', 'ability_code_43E0_player_0', 'ability_code_43E0_player_1', 'ability_code_480_diff', 'ability_code_480_first_time_diff', 'ability_code_480_first_time_player_0', 'ability_code_480_first_time_player_1', 'ability_code_480_player_0', 'ability_code_483_diff', 'ability_code_483_player_0', 'ability_code_483_player_1', 'ability_code_4AE0_diff', 'ability_code_4AE0_player_0', 'ability_code_4AE0_player_1', 'ability_code_4AFE_diff', 'ability_code_4AFE_player_0', 'ability_code_4AFE_player_1', 'ability_code_4B80_diff', 'ability_code_4B80_first_time_diff', 'ability_code_4B80_first_time_player_0', 'ability_code_4B80_player_0', 'ability_code_4B80_player_1', 'ability_code_4C0_diff', 'ability_code_4C0_player_0', 'ability_code_4C0_player_1', 'ability_code_4C1_diff', 'ability_code_4C1_first_time_diff', 'ability_code_4C1_first_time_player_1', 'ability_code_4C1_player_0', 'ability_code_4C1_player_1', 'ability_code_4C2_first_time_diff', 'ability_code_4C3_diff', 'ability_code_4C3_player_0', 'ability_code_4C3_player_1', 'ability_code_4C60_player_0', 'ability_code_4C60_player_1', 'ability_code_4C80_diff', 'ability_code_4C80_player_0', 'ability_code_4C80_player_1', 'ability_code_4E40_player_0', 'ability_code_4EA0_diff', 'ability_code_4EA0_player_0', 'ability_code_4EA0_player_1', 'ability_code_4EC0_diff', 'ability_code_4EC0_player_0', 'ability_code_4EC0_player_1', 'ability_code_55E0_diff', 'ability_code_55E0_player_0', 'ability_code_55E0_player_1', 'ability_code_5600_diff', 'ability_code_5600_player_0', 'ability_code_5600_player_1', 'ability_code_5640_player_0', 'ability_code_5640_player_1', 'ability_code_5641_diff', 'ability_code_5641_player_0', 'ability_code_5641_player_1', 'ability_code_56A0_diff', 'ability_code_56A0_player_0', 'ability_code_56A0_player_1', 'ability_code_5700_diff', 'ability_code_5700_player_0', 'ability_code_5700_player_1', 'ability_code_5A0_diff', 'ability_code_5A0_first_time_player_0', 'ability_code_5A0_player_0', 'ability_code_5A0_player_0_ability_code_1022_player_0_diff', 'ability_code_5A0_player_0_ability_code_1023_player_0_diff', 'ability_code_5A0_player_0_ability_code_102A_player_0_diff', 'ability_code_5A0_player_0_ability_code_1261_player_0_diff', 'ability_code_5A0_player_0_ability_code_13A0_player_0_diff', 'ability_code_5A0_player_0_ability_code_13E0_player_0_diff', 'ability_code_5A0_player_0_ability_code_15E0_player_0_diff', 'ability_code_5A0_player_0_ability_code_16E2_player_0_diff', 'ability_code_5A0_player_0_ability_code_1820_player_0_diff', 'ability_code_5A0_player_0_ability_code_1821_player_0_diff', 'ability_code_5A0_player_0_ability_code_1822_player_0_diff', 'ability_code_5A0_player_0_ability_code_1C60_player_0_diff', 'ability_code_5A0_player_0_ability_code_1E60_player_0_diff', 'ability_code_5A0_player_0_ability_code_480_player_0_diff', 'ability_code_5A0_player_0_ability_code_B40_player_0_diff', 'ability_code_5A0_player_1', 'ability_code_5A0_player_1_ability_code_1022_player_1_diff', 'ability_code_5A0_player_1_ability_code_1023_player_1_diff_ability_code_5A0_player_0_ability_code_1023_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_102A_player_1_diff', 'ability_code_5A0_player_1_ability_code_102A_player_1_diff_ability_code_5A0_player_0_ability_code_102A_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_13A0_player_1_diff', 'ability_code_5A0_player_1_ability_code_13A0_player_1_diff_ability_code_5A0_player_0_ability_code_13A0_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_16E2_player_1_diff', 'ability_code_5A0_player_1_ability_code_1820_player_1_diff', 'ability_code_5A0_player_1_ability_code_1820_player_1_diff_ability_code_5A0_player_0_ability_code_1820_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_1821_player_1_diff_ability_code_5A0_player_0_ability_code_1821_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_1822_player_1_diff', 'ability_code_5A0_player_1_ability_code_1822_player_1_diff_ability_code_5A0_player_0_ability_code_1822_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_1C60_player_1_diff', 'ability_code_5A0_player_1_ability_code_1C60_player_1_diff_ability_code_5A0_player_0_ability_code_1C60_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_1E60_player_1_diff_ability_code_5A0_player_0_ability_code_1E60_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_480_player_1_diff', 'ability_code_5A0_player_1_ability_code_B40_player_1_diff', 'ability_code_5A0_player_1_ability_code_DE0_player_1_diff_ability_code_5A0_player_0_ability_code_DE0_player_0_diff_diff', 'ability_code_780_diff', 'ability_code_780_player_0', 'ability_code_780_player_1', 'ability_code_7A0_diff', 'ability_code_7A0_player_0', 'ability_code_7A0_player_1', 'ability_code_7C0_diff', 'ability_code_7C0_player_0', 'ability_code_7C0_player_1', 'ability_code_7E0_diff', 'ability_code_7E0_player_0', 'ability_code_7E0_player_1', 'ability_code_820_diff', 'ability_code_820_player_0', 'ability_code_820_player_1', 'ability_code_880_diff', 'ability_code_880_player_0', 'ability_code_880_player_1', 'ability_code_8A0_player_0', 'ability_code_8A2_diff', 'ability_code_8A2_player_0', 'ability_code_8A2_player_1', 'ability_code_8E0_diff', 'ability_code_8E0_player_0', 'ability_code_8E0_player_1', 'ability_code_900_diff', 'ability_code_900_player_0', 'ability_code_900_player_1', 'ability_code_920_player_0', 'ability_code_920_player_1', 'ability_code_960_diff', 'ability_code_960_player_1', 'ability_code_9E0_diff', 'ability_code_9E0_player_0', 'ability_code_9E0_player_1', 'ability_code_A40_diff', 'ability_code_A40_player_0', 'ability_code_A40_player_1', 'ability_code_B00_diff', 'ability_code_B00_player_0', 'ability_code_B01_diff', 'ability_code_B01_player_0', 'ability_code_B01_player_1', 'ability_code_B40_diff', 'ability_code_B40_first_time_diff', 'ability_code_B40_first_time_player_1', 'ability_code_B40_last_first_diff', 'ability_code_B40_player_0', 'ability_code_B40_player_0_ability_code_1023_player_0_diff', 'ability_code_B40_player_0_ability_code_15E0_player_0_diff', 'ability_code_B40_player_0_ability_code_16E2_player_0_diff', 'ability_code_B40_player_0_ability_code_1821_player_0_diff', 'ability_code_B40_player_0_ability_code_1822_player_0_diff', 'ability_code_B40_player_0_ability_code_1E60_player_0_diff', 'ability_code_B40_player_0_ability_code_DE0_player_0_diff', 'ability_code_B40_player_1', 'ability_code_B40_player_1_ability_code_1022_player_1_diff', 'ability_code_B40_player_1_ability_code_1022_player_1_diff_ability_code_B40_player_0_ability_code_1022_player_0_diff_diff', 'ability_code_B40_player_1_ability_code_102A_player_1_diff', 'ability_code_B40_player_1_ability_code_1261_player_1_diff_ability_code_B40_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_B40_player_1_ability_code_13A0_player_1_diff', 'ability_code_B40_player_1_ability_code_15E0_player_1_diff', 'ability_code_B40_player_1_ability_code_16E2_player_1_diff', 'ability_code_B40_player_1_ability_code_16E2_player_1_diff_ability_code_B40_player_0_ability_code_16E2_player_0_diff_diff', 'ability_code_B40_player_1_ability_code_1821_player_1_diff', 'ability_code_B40_player_1_ability_code_1822_player_1_diff', 'ability_code_B40_player_1_ability_code_1822_player_1_diff_ability_code_B40_player_0_ability_code_1822_player_0_diff_diff', 'ability_code_B40_player_1_ability_code_1C60_player_1_diff', 'ability_code_B40_player_1_ability_code_1C60_player_1_diff_ability_code_B40_player_0_ability_code_1C60_player_0_diff_diff', 'ability_code_B40_player_1_ability_code_1E60_player_1_diff', 'ability_code_B40_player_1_ability_code_1E60_player_1_diff_ability_code_B40_player_0_ability_code_1E60_player_0_diff_diff', 'ability_code_B40_player_1_ability_code_480_player_1_diff', 'ability_code_B40_player_1_ability_code_DE0_player_1_diff', 'ability_code_B40_player_1_ability_code_DE0_player_1_diff_ability_code_B40_player_0_ability_code_DE0_player_0_diff_diff', 'ability_code_B60_player_0', 'ability_code_B60_player_1', 'ability_code_B61_diff', 'ability_code_B61_player_0', 'ability_code_B61_player_1', 'ability_code_BE0_diff', 'ability_code_BE0_player_0', 'ability_code_BE0_player_1', 'ability_code_CC0_player_0', 'ability_code_D00_diff', 'ability_code_D00_player_0', 'ability_code_D00_player_1', 'ability_code_D20_diff', 'ability_code_D20_player_0', 'ability_code_D20_player_1', 'ability_code_D40_player_0', 'ability_code_D41_player_0', 'ability_code_D61_diff', 'ability_code_D61_player_1', 'ability_code_D62_diff', 'ability_code_D62_player_0', 'ability_code_D62_player_1', 'ability_code_DE0_first_time_diff', 'ability_code_DE0_first_time_player_1', 'ability_code_DE0_player_0', 'ability_code_DE0_player_0_ability_code_1020_player_0_diff', 'ability_code_DE0_player_0_ability_code_1023_player_0_diff', 'ability_code_DE0_player_0_ability_code_102A_player_0_diff', 'ability_code_DE0_player_0_ability_code_1261_player_0_diff', 'ability_code_DE0_player_0_ability_code_13A0_player_0_diff', 'ability_code_DE0_player_0_ability_code_15E0_player_0_diff', 'ability_code_DE0_player_0_ability_code_16E2_player_0_diff', 'ability_code_DE0_player_0_ability_code_1821_player_0_diff', 'ability_code_DE0_player_0_ability_code_1822_player_0_diff', 'ability_code_DE0_player_0_ability_code_1C60_player_0_diff', 'ability_code_DE0_player_0_ability_code_480_player_0_diff', 'ability_code_DE0_player_1', 'ability_code_DE0_player_1_ability_code_1020_player_1_diff', 'ability_code_DE0_player_1_ability_code_1022_player_1_diff', 'ability_code_DE0_player_1_ability_code_1023_player_1_diff', 'ability_code_DE0_player_1_ability_code_102A_player_1_diff', 'ability_code_DE0_player_1_ability_code_102A_player_1_diff_ability_code_DE0_player_0_ability_code_102A_player_0_diff_diff', 'ability_code_DE0_player_1_ability_code_1261_player_1_diff', 'ability_code_DE0_player_1_ability_code_1261_player_1_diff_ability_code_DE0_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_DE0_player_1_ability_code_13A0_player_1_diff', 'ability_code_DE0_player_1_ability_code_13A0_player_1_diff_ability_code_DE0_player_0_ability_code_13A0_player_0_diff_diff', 'ability_code_DE0_player_1_ability_code_15E0_player_1_diff', 'ability_code_DE0_player_1_ability_code_15E0_player_1_diff_ability_code_DE0_player_0_ability_code_15E0_player_0_diff_diff', 'ability_code_DE0_player_1_ability_code_16E2_player_1_diff_ability_code_DE0_player_0_ability_code_16E2_player_0_diff_diff', 'ability_code_DE0_player_1_ability_code_1821_player_1_diff', 'ability_code_DE0_player_1_ability_code_1821_player_1_diff_ability_code_DE0_player_0_ability_code_1821_player_0_diff_diff', 'ability_code_DE0_player_1_ability_code_1C60_player_1_diff', 'ability_code_DE0_player_1_ability_code_1C60_player_1_diff_ability_code_DE0_player_0_ability_code_1C60_player_0_diff_diff', 'ability_code_DE0_player_1_ability_code_480_player_1_diff_ability_code_DE0_player_0_ability_code_480_player_0_diff_diff', 'ability_code_E00_diff', 'ability_code_E00_player_0', 'ability_code_E00_player_1', 'ability_code_E20_player_1', 'ability_code_EE0_diff', 'ability_code_EE0_player_0', 'ability_code_EE0_player_1', 'ability_code_EE1_diff', 'ability_code_EE1_player_0', 'ability_code_EE1_player_1', 'ability_code_F00_diff', 'ability_code_F00_player_0', 'ability_code_F00_player_1', 'ability_code_F01_diff', 'ability_code_F01_player_0', 'ability_code_F01_player_1', 'ability_code_F40_player_1', 'ability_code_F41_diff', 'ability_code_F41_player_0', 'ability_code_F41_player_1', 'ability_code_F60_diff', 'ability_code_F60_first_time_diff', 'ability_code_F60_first_time_player_1', 'ability_code_F60_player_1', 'ability_code_F61_first_time_player_0', 'ability_code_F61_first_time_player_1', 'ability_code_F61_player_0', 'ability_code_F61_player_1', 'ability_code_F80_first_time_diff', 'ability_code_F80_player_1', 'ability_code_F81_first_time_diff', 'ability_code_F81_player_0', 'ability_code_F81_player_1', 'ability_code_FC0_diff', 'ability_code_FC0_player_0', 'ability_code_FC0_player_1', 'ability_code_FC1_diff', 'ability_code_FC1_player_0', 'ability_code_FC1_player_1', 'ability_code_FE0_diff', 'ability_code_FE0_first_time_diff', 'ability_code_FE1_diff', 'ability_code_FE1_player_0', 'ability_code_FE1_player_1', 'all_unit_len_player_0', 'all_unit_len_player_0_BanelingCocoon_len_player_0_diff', 'all_unit_len_player_0_Barracks_len_player_0_diff', 'all_unit_len_player_0_Drone_len_player_0_diff', 'all_unit_len_player_0_Factory_len_player_0_diff', 'all_unit_len_player_0_MULE_len_player_0_diff', 'all_unit_len_player_0_Marine_len_player_0_diff', 'all_unit_len_player_0_OrbitalCommand_len_player_0_diff', 'all_unit_len_player_0_Overlord_len_player_0_diff', 'all_unit_len_player_0_Probe_len_player_0_diff', 'all_unit_len_player_0_Queen_len_player_0_diff', 'all_unit_len_player_0_Reaper_len_player_0_diff', 'all_unit_len_player_0_Roach_len_player_0_diff', 'all_unit_len_player_0_SCV_len_player_0_diff', 'all_unit_len_player_0_SupplyDepotLowered_len_player_0_diff', 'all_unit_len_player_1_BanelingCocoon_len_player_1_diff_all_unit_len_player_0_BanelingCocoon_len_player_0_diff_diff', 'all_unit_len_player_1_Barracks_len_player_1_diff_all_unit_len_player_0_Barracks_len_player_0_diff_diff', 'all_unit_len_player_1_Factory_len_player_1_diff_all_unit_len_player_0_Factory_len_player_0_diff_diff', 'all_unit_len_player_1_Hatchery_len_player_1_diff_all_unit_len_player_0_Hatchery_len_player_0_diff_diff', 'all_unit_len_player_1_Hellion_len_player_1_diff_all_unit_len_player_0_Hellion_len_player_0_diff_diff', 'all_unit_len_player_1_MULE_len_player_1_diff', 'all_unit_len_player_1_OrbitalCommand_len_player_1_diff_all_unit_len_player_0_OrbitalCommand_len_player_0_diff_diff', 'all_unit_len_player_1_Overlord_len_player_1_diff', 'all_unit_len_player_1_Queen_len_player_1_diff_all_unit_len_player_0_Queen_len_player_0_diff_diff', 'all_unit_len_player_1_Reaper_len_player_1_diff_all_unit_len_player_0_Reaper_len_player_0_diff_diff', 'all_unit_len_player_1_Roach_len_player_1_diff_all_unit_len_player_0_Roach_len_player_0_diff_diff', 'all_unit_len_player_1_SCV_len_player_1_diff', 'all_unit_len_player_1_Starport_len_player_1_diff', 'all_unit_len_player_1_Starport_len_player_1_diff_all_unit_len_player_0_Starport_len_player_0_diff_diff', 'all_unit_len_player_1_SupplyDepotLowered_len_player_1_diff', 'all_unit_len_player_1_SupplyDepotLowered_len_player_1_diff_all_unit_len_player_0_SupplyDepotLowered_len_player_0_diff_diff', 'camera_pos_x_max_player_1', 'camera_pos_x_mean_player_0', 'camera_pos_x_mean_player_1', 'camera_pos_x_min_player_1', 'camera_pos_y_max_player_0', 'camera_pos_y_max_player_1', 'camera_pos_y_min_player_0', 'game_id_time_max_div_nunique', 'species_player_0', 'species_player_1', 'species_war_kind', 'time_count_player_diff', 'time_sum_player_diff', 'under_1_Ability_player_0', 'under_1_AddToControlGroup_player_1', 'under_1_Camera_diff', 'under_1_Camera_player_1', 'under_1_ControlGroup_player_0', 'under_1_ControlGroup_player_1', 'under_1_Right Click_player_0', 'under_1_Selection_diff', 'under_1_Selection_player_1', 'under_3_Ability_player_0', 'under_3_AddToControlGroup_diff', 'under_3_AddToControlGroup_player_1', 'under_3_Camera_diff', 'under_3_Camera_player_0', 'under_3_Camera_player_1', 'under_3_ControlGroup_player_0', 'under_3_ControlGroup_player_1', 'under_3_GetControlGroup_player_1', 'under_3_Right Click_player_0', 'under_3_Selection_diff', 'under_3_SetControlGroup_player_1', 'under_5_Ability_diff', 'under_5_AddToControlGroup_diff', 'under_5_AddToControlGroup_player_0', 'under_5_AddToControlGroup_player_1', 'under_5_Camera_player_1', 'under_5_ControlGroup_diff', 'under_5_ControlGroup_player_0', 'under_5_ControlGroup_player_1', 'under_5_Right Click_player_1', 'under_5_Selection_player_0', 'under_5_SetControlGroup_diff', 'under_5_SetControlGroup_player_0', 'under_5_SetControlGroup_player_1', 'work_diff', 'worker_count_player_0', 'worker_count_player_1']
bad_feature_fold4 = ['Ability_player_0_isnull', 'Ability_player_1_isnull', 'AddToControlGroup_player_0_isnull', 'AddToControlGroup_player_1', 'AdeptPiercingWeapon_len_player_0', 'Adept_len_player_0', 'Adept_len_player_1', 'Adept_right_click_target_count_diff', 'Adept_right_click_target_count_player_0', 'AiurLightBridgeAbandonedNE10Out_len_player_0', 'AiurLightBridgeAbandonedNE10Out_len_player_1', 'AiurLightBridgeAbandonedNE8Out_len_player_0', 'AiurLightBridgeAbandonedNE8Out_len_player_1', 'AiurLightBridgeAbandonedNE8_len_player_0', 'AiurLightBridgeAbandonedNE8_len_player_1', 'AiurLightBridgeNE10Out_len_player_0', 'AiurLightBridgeNE10Out_len_player_1', 'AiurLightBridgeNE8_len_player_0', 'AiurLightBridgeNE8_len_player_1', 'Archon_len_player_0', 'Archon_len_player_1', 'Armory_len_player_0', 'Armory_len_player_1', 'Assimilator_len_player_0', 'AutoTurret_len_player_0', 'AutoTurret_len_player_1', 'BanelingBurrowed_len_player_0', 'BanelingBurrowed_len_player_1', 'BanelingCocoon_len_player_0', 'BanelingCocoon_len_player_0_Hellion_len_player_0_diff', 'BanelingCocoon_len_player_0_MULE_len_player_0_diff', 'BanelingCocoon_len_player_0_Overlord_len_player_0_diff', 'BanelingCocoon_len_player_0_Queen_len_player_0_diff', 'BanelingCocoon_len_player_0_Reaper_len_player_0_diff', 'BanelingCocoon_len_player_0_Starport_len_player_0_diff', 'BanelingCocoon_len_player_1_Factory_len_player_1_diff', 'BanelingCocoon_len_player_1_Hatchery_len_player_1_diff_BanelingCocoon_len_player_0_Hatchery_len_player_0_diff_diff', 'BanelingCocoon_len_player_1_Hellion_len_player_1_diff', 'BanelingCocoon_len_player_1_MULE_len_player_1_diff', 'BanelingCocoon_len_player_1_MULE_len_player_1_diff_BanelingCocoon_len_player_0_MULE_len_player_0_diff_diff', 'BanelingCocoon_len_player_1_Overlord_len_player_1_diff', 'BanelingCocoon_len_player_1_Queen_len_player_1_diff', 'BanelingCocoon_len_player_1_Queen_len_player_1_diff_BanelingCocoon_len_player_0_Queen_len_player_0_diff_diff', 'BanelingCocoon_len_player_1_Reaper_len_player_1_diff', 'BanelingCocoon_len_player_1_Starport_len_player_1_diff', 'BanelingCocoon_len_player_1_Starport_len_player_1_diff_BanelingCocoon_len_player_0_Starport_len_player_0_diff_diff', 'BanelingCocoon_right_click_target_count_diff', 'BanelingCocoon_right_click_target_count_player_0', 'BanelingNest_len_player_1', 'Baneling_len_player_0', 'Baneling_right_click_target_count_diff', 'Baneling_right_click_target_count_player_0', 'Baneling_right_click_target_count_player_1', 'Banshee_len_player_1', 'BarracksReactor_len_player_0', 'BarracksReactor_len_player_1', 'BarracksTechLab_len_player_0', 'BarracksTechLab_len_player_1', 'Barracks_len_player_0_BanelingCocoon_len_player_0_diff', 'Barracks_len_player_0_Hellion_len_player_0_diff', 'Barracks_len_player_0_Overlord_len_player_0_diff', 'Barracks_len_player_0_Queen_len_player_0_diff', 'Barracks_len_player_0_Reaper_len_player_0_diff', 'Barracks_len_player_1', 'Barracks_len_player_1_BanelingCocoon_len_player_1_diff', 'Barracks_len_player_1_Hatchery_len_player_1_diff', 'Barracks_len_player_1_Hatchery_len_player_1_diff_Barracks_len_player_0_Hatchery_len_player_0_diff_diff', 'Barracks_len_player_1_Hellion_len_player_1_diff', 'Barracks_len_player_1_MULE_len_player_1_diff', 'Barracks_len_player_1_OrbitalCommand_len_player_1_diff', 'Barracks_len_player_1_Overlord_len_player_1_diff', 'Barracks_len_player_1_Overlord_len_player_1_diff_Barracks_len_player_0_Overlord_len_player_0_diff_diff', 'Barracks_len_player_1_Queen_len_player_1_diff', 'Barracks_len_player_1_Queen_len_player_1_diff_Barracks_len_player_0_Queen_len_player_0_diff_diff', 'Barracks_len_player_1_Reaper_len_player_1_diff', 'Barracks_len_player_1_Reaper_len_player_1_diff_Barracks_len_player_0_Reaper_len_player_0_diff_diff', 'Barracks_len_player_1_Roach_len_player_1_diff', 'Barracks_len_player_1_Starport_len_player_1_diff', 'Barracks_len_player_1_Starport_len_player_1_diff_Barracks_len_player_0_Starport_len_player_0_diff_diff', 'Barracks_len_player_1_SupplyDepotLowered_len_player_1_diff_Barracks_len_player_0_SupplyDepotLowered_len_player_0_diff_diff', 'Barracks_right_click_target_count_diff', 'Barracks_right_click_target_count_player_1', 'Battlecruiser_len_player_0', 'Battlecruiser_len_player_1', 'Beacon_TerranSmall_len_player_0', 'Beacon_TerranSmall_len_player_1', 'Beacon_Terran_len_player_0', 'Beacon_Terran_len_player_1', 'BroodLordAWeapon_len_player_0', 'BroodLordAWeapon_len_player_1', 'BroodLordWeapon_len_player_0', 'BroodLordWeapon_len_player_1', 'BroodLord_len_player_0', 'BroodLord_len_player_1', 'Broodling_len_player_0', 'Broodling_len_player_1', 'Bunker_len_player_1', 'Bunker_right_click_target_count_player_0', 'BypassArmorDrone_len_player_0', 'BypassArmorDrone_len_player_1', 'Camera_player_0_isnull', 'Camera_player_1', 'Camera_player_1_isnull', 'Carrier_len_player_0', 'Carrier_len_player_1', 'ChangelingMarineShield_len_player_1', 'ChangelingMarine_len_player_0', 'ChangelingMarine_len_player_1', 'ChangelingZealot_len_player_0', 'ChangelingZealot_len_player_1', 'ChangelingZerglingWings_len_player_0', 'ChangelingZerglingWings_len_player_1', 'ChangelingZergling_len_player_0', 'ChangelingZergling_len_player_1', 'Changeling_len_player_0', 'Changeling_len_player_1', 'Colossus_len_player_0', 'CommandCenter_len_player_1', 'ControlGroup_player_0', 'ControlGroup_player_0_ControlGroup_player_1_diff', 'ControlGroup_player_0_isnull', 'ControlGroup_player_1', 'ControlGroup_player_1_isnull', 'Corruptor_len_player_0', 'Corruptor_len_player_1', 'CreepOnlyBlocker4x4_len_player_0', 'CreepOnlyBlocker4x4_len_player_1', 'CreepOnlyBlocker4x4_right_click_target_count_player_0', 'CreepOnlyBlocker4x4_right_click_target_count_player_1', 'CreepTumorBurrowed_len_player_0', 'CreepTumorBurrowed_len_player_1', 'CreepTumorBurrowed_right_click_target_count_player_0', 'CreepTumorQueen_len_player_0', 'CreepTumorQueen_len_player_1', 'CreepTumor_len_player_0', 'CreepTumor_len_player_1', 'CyberneticsCore_len_player_0', 'CyberneticsCore_right_click_target_count_diff', 'CyberneticsCore_right_click_target_count_player_0', 'CyberneticsCore_right_click_target_count_player_1', 'CycloneMissileLargeAir_len_player_0', 'Cyclone_len_player_0', 'Cyclone_len_player_1', 'DarkShrine_len_player_0', 'DarkShrine_len_player_1', 'DarkTemplar_len_player_0', 'DestructibleIce2x4Horizontal_len_player_0', 'DestructibleIce2x4Horizontal_len_player_1', 'DestructibleIce2x4Vertical_len_player_0', 'DestructibleIce2x4Vertical_len_player_1', 'DestructibleIce2x4Vertical_right_click_target_count_player_0', 'DestructibleIce2x4Vertical_right_click_target_count_player_1', 'DestructibleIce2x6Horizontal_len_player_0', 'DestructibleIce2x6Horizontal_len_player_1', 'DestructibleIce2x6Vertical_len_player_0', 'DestructibleIce2x6Vertical_len_player_1', 'DestructibleSignsConstruction_len_player_1', 'DestructibleSignsConstruction_right_click_target_count_player_1', 'DestructibleSignsFunny_len_player_0', 'DestructibleSignsFunny_right_click_target_count_player_0', 'DestructibleSignsFunny_right_click_target_count_player_1', 'DestructibleSignsIcons_len_player_1', 'DestructibleSignsIcons_right_click_target_count_diff', 'DestructibleSignsIcons_right_click_target_count_player_0', 'DestructibleSignsIcons_right_click_target_count_player_1', 'Disruptor_len_player_0', 'Disruptor_len_player_1', 'DroneBurrowed_len_player_0', 'Drone_len_player_0_BanelingCocoon_len_player_0_diff', 'Drone_len_player_0_Barracks_len_player_0_diff', 'Drone_len_player_0_Hatchery_len_player_0_diff', 'Drone_len_player_0_Hellion_len_player_0_diff', 'Drone_len_player_0_Marine_len_player_0_diff', 'Drone_len_player_0_Overlord_len_player_0_diff', 'Drone_len_player_0_Probe_len_player_0_diff', 'Drone_len_player_0_Roach_len_player_0_diff', 'Drone_len_player_0_SupplyDepotLowered_len_player_0_diff', 'Drone_len_player_0_Zergling_len_player_0_diff', 'Drone_len_player_1_Barracks_len_player_1_diff_Drone_len_player_0_Barracks_len_player_0_diff_diff', 'Drone_len_player_1_Factory_len_player_1_diff', 'Drone_len_player_1_Factory_len_player_1_diff_Drone_len_player_0_Factory_len_player_0_diff_diff', 'Drone_len_player_1_Hatchery_len_player_1_diff', 'Drone_len_player_1_MULE_len_player_1_diff', 'Drone_len_player_1_MULE_len_player_1_diff_Drone_len_player_0_MULE_len_player_0_diff_diff', 'Drone_len_player_1_Overlord_len_player_1_diff', 'Drone_len_player_1_Overlord_len_player_1_diff_Drone_len_player_0_Overlord_len_player_0_diff_diff', 'Drone_len_player_1_Queen_len_player_1_diff', 'Drone_len_player_1_Queen_len_player_1_diff_Drone_len_player_0_Queen_len_player_0_diff_diff', 'Drone_len_player_1_Reaper_len_player_1_diff', 'Drone_len_player_1_Reaper_len_player_1_diff_Drone_len_player_0_Reaper_len_player_0_diff_diff', 'Drone_len_player_1_Roach_len_player_1_diff', 'Drone_len_player_1_Roach_len_player_1_diff_Drone_len_player_0_Roach_len_player_0_diff_diff', 'Drone_len_player_1_Starport_len_player_1_diff', 'Drone_len_player_1_SupplyDepotLowered_len_player_1_diff_Drone_len_player_0_SupplyDepotLowered_len_player_0_diff_diff', 'Drone_len_player_1_Zergling_len_player_1_diff_Drone_len_player_0_Zergling_len_player_0_diff_diff', 'Egg_len_player_0', 'Egg_len_player_0_Drone_len_player_0_diff', 'Egg_len_player_0_Hellion_len_player_0_diff', 'Egg_len_player_0_MULE_len_player_0_diff', 'Egg_len_player_0_Marine_len_player_0_diff', 'Egg_len_player_0_Probe_len_player_0_diff', 'Egg_len_player_0_Reaper_len_player_0_diff', 'Egg_len_player_0_Starport_len_player_0_diff', 'Egg_len_player_0_SupplyDepotLowered_len_player_0_diff', 'Egg_len_player_0_Zergling_len_player_0_diff', 'Egg_len_player_1_BanelingCocoon_len_player_1_diff_Egg_len_player_0_BanelingCocoon_len_player_0_diff_diff', 'Egg_len_player_1_Barracks_len_player_1_diff', 'Egg_len_player_1_Barracks_len_player_1_diff_Egg_len_player_0_Barracks_len_player_0_diff_diff', 'Egg_len_player_1_Drone_len_player_1_diff_Egg_len_player_0_Drone_len_player_0_diff_diff', 'Egg_len_player_1_Factory_len_player_1_diff', 'Egg_len_player_1_Factory_len_player_1_diff_Egg_len_player_0_Factory_len_player_0_diff_diff', 'Egg_len_player_1_Hellion_len_player_1_diff', 'Egg_len_player_1_Hellion_len_player_1_diff_Egg_len_player_0_Hellion_len_player_0_diff_diff', 'Egg_len_player_1_MULE_len_player_1_diff', 'Egg_len_player_1_MULE_len_player_1_diff_Egg_len_player_0_MULE_len_player_0_diff_diff', 'Egg_len_player_1_Marine_len_player_1_diff', 'Egg_len_player_1_OrbitalCommand_len_player_1_diff', 'Egg_len_player_1_Probe_len_player_1_diff', 'Egg_len_player_1_Queen_len_player_1_diff', 'Egg_len_player_1_Reaper_len_player_1_diff', 'Egg_len_player_1_Roach_len_player_1_diff_Egg_len_player_0_Roach_len_player_0_diff_diff', 'Egg_len_player_1_SCV_len_player_1_diff', 'Egg_len_player_1_Starport_len_player_1_diff', 'Egg_len_player_1_Starport_len_player_1_diff_Egg_len_player_0_Starport_len_player_0_diff_diff', 'Egg_len_player_1_SupplyDepotLowered_len_player_1_diff', 'EngineeringBay_len_player_1', 'EngineeringBay_right_click_target_count_diff', 'EngineeringBay_right_click_target_count_player_0', 'EngineeringBay_right_click_target_count_player_1', 'ExtendingBridgeNEWide10Out_len_player_1', 'ExtendingBridgeNEWide10_len_player_0', 'ExtendingBridgeNEWide10_len_player_1', 'ExtendingBridgeNEWide8Out_len_player_0', 'ExtendingBridgeNEWide8Out_len_player_1', 'ExtendingBridgeNWWide8Out_len_player_0', 'ExtendingBridgeNWWide8Out_len_player_1', 'ExtendingBridgeNWWide8_len_player_0', 'ExtendingBridgeNWWide8_len_player_1', 'Extractor_len_player_1', 'Extractor_right_click_target_count_diff', 'Extractor_right_click_target_count_player_0', 'Extractor_right_click_target_count_player_1', 'FactoryReactor_len_player_0', 'FactoryReactor_len_player_1', 'FactoryTechLab_len_player_1', 'Factory_len_player_0_Starport_len_player_0_diff', 'Factory_len_player_1', 'Factory_len_player_1_Hatchery_len_player_1_diff_Factory_len_player_0_Hatchery_len_player_0_diff_diff', 'Factory_len_player_1_Hellion_len_player_1_diff_Factory_len_player_0_Hellion_len_player_0_diff_diff', 'Factory_len_player_1_MULE_len_player_1_diff', 'Factory_len_player_1_Reaper_len_player_1_diff', 'Factory_len_player_1_Roach_len_player_1_diff', 'Factory_right_click_target_count_player_0', 'FleetBeacon_len_player_0', 'Forge_len_player_0', 'Forge_right_click_target_count_player_1', 'FungalGrowthMissile_len_player_0', 'FungalGrowthMissile_len_player_1', 'FungalGrowthMissile_right_click_target_count_diff', 'FungalGrowthMissile_right_click_target_count_player_0', 'FungalGrowthMissile_right_click_target_count_player_1', 'FusionCore_len_player_0', 'FusionCore_len_player_1', 'Gateway_len_player_0', 'GetControlGroup_player_0_isnull', 'GetControlGroup_player_1', 'GetControlGroup_player_1_isnull', 'GhostAcademy_len_player_0', 'GhostAcademy_len_player_1', 'Ghost_len_player_0', 'Ghost_len_player_1', 'GlaiveWurmBounceWeapon_len_player_0', 'GlaiveWurmBounceWeapon_len_player_1', 'GreaterSpire_len_player_0', 'GreaterSpire_len_player_1', 'Hatchery_len_player_0', 'Hatchery_len_player_0_Reaper_len_player_0_diff', 'Hatchery_len_player_0_Roach_len_player_0_diff', 'Hatchery_len_player_0_Starport_len_player_0_diff', 'Hatchery_len_player_1', 'Hatchery_len_player_1_Roach_len_player_1_diff', 'Hatchery_len_player_1_Starport_len_player_1_diff', 'Hatchery_len_player_1_Starport_len_player_1_diff_Hatchery_len_player_0_Starport_len_player_0_diff_diff', 'Hatchery_right_click_target_count_diff', 'Hatchery_right_click_target_count_player_0', 'Hellion_len_player_0', 'Hellion_len_player_0_Hatchery_len_player_0_diff', 'Hellion_len_player_0_MULE_len_player_0_diff', 'Hellion_len_player_0_Reaper_len_player_0_diff', 'Hellion_len_player_0_Roach_len_player_0_diff', 'Hellion_len_player_1', 'Hellion_len_player_1_Hatchery_len_player_1_diff', 'Hellion_len_player_1_Hatchery_len_player_1_diff_Hellion_len_player_0_Hatchery_len_player_0_diff_diff', 'Hellion_len_player_1_MULE_len_player_1_diff', 'Hellion_len_player_1_MULE_len_player_1_diff_Hellion_len_player_0_MULE_len_player_0_diff_diff', 'Hellion_len_player_1_Reaper_len_player_1_diff', 'Hellion_len_player_1_Reaper_len_player_1_diff_Hellion_len_player_0_Reaper_len_player_0_diff_diff', 'Hellion_len_player_1_Roach_len_player_1_diff', 'Hellion_len_player_1_Roach_len_player_1_diff_Hellion_len_player_0_Roach_len_player_0_diff_diff', 'Hellion_right_click_target_count_diff', 'Hellion_right_click_target_count_player_0', 'Hellion_right_click_target_count_player_1', 'HighTemplar_len_player_0', 'HighTemplar_len_player_1', 'Hive_len_player_0', 'Hive_right_click_target_count_diff', 'Hive_right_click_target_count_player_0', 'Hive_right_click_target_count_player_1', 'HydraliskDen_len_player_1', 'Hydralisk_len_player_1', 'Immortal_len_player_0', 'Immortal_len_player_1', 'InfestationPit_len_player_1', 'InfestorBurrowed_len_player_1', 'Infestor_len_player_1', 'Lair_right_click_target_count_diff', 'Lair_right_click_target_count_player_0', 'Larva_len_player_1', 'LocustMPFlying_len_player_0', 'LocustMPFlying_len_player_1', 'LurkerEgg_len_player_0', 'LurkerEgg_len_player_1', 'Lurker_len_player_0', 'Lurker_len_player_1', 'MULE_len_player_0', 'MULE_len_player_0_Hatchery_len_player_0_diff', 'MULE_len_player_0_Reaper_len_player_0_diff', 'MULE_len_player_0_Roach_len_player_0_diff', 'MULE_len_player_1', 'MULE_len_player_1_Hatchery_len_player_1_diff', 'MULE_len_player_1_Hatchery_len_player_1_diff_MULE_len_player_0_Hatchery_len_player_0_diff_diff', 'MULE_len_player_1_Reaper_len_player_1_diff', 'MULE_len_player_1_Reaper_len_player_1_diff_MULE_len_player_0_Reaper_len_player_0_diff_diff', 'MULE_len_player_1_Roach_len_player_1_diff', 'MULE_len_player_1_Roach_len_player_1_diff_MULE_len_player_0_Roach_len_player_0_diff_diff', 'MULE_len_player_1_Starport_len_player_1_diff', 'MULE_right_click_target_count_diff', 'MULE_right_click_target_count_player_0', 'MULE_right_click_target_count_player_1', 'Marauder_len_player_1', 'Marine_len_player_0', 'Marine_len_player_0_BanelingCocoon_len_player_0_diff', 'Marine_len_player_0_Hatchery_len_player_0_diff', 'Marine_len_player_0_Hellion_len_player_0_diff', 'Marine_len_player_0_MULE_len_player_0_diff', 'Marine_len_player_0_Overlord_len_player_0_diff', 'Marine_len_player_0_Probe_len_player_0_diff', 'Marine_len_player_0_Queen_len_player_0_diff', 'Marine_len_player_0_Roach_len_player_0_diff', 'Marine_len_player_0_Starport_len_player_0_diff', 'Marine_len_player_1', 'Marine_len_player_1_BanelingCocoon_len_player_1_diff', 'Marine_len_player_1_BanelingCocoon_len_player_1_diff_Marine_len_player_0_BanelingCocoon_len_player_0_diff_diff', 'Marine_len_player_1_Barracks_len_player_1_diff', 'Marine_len_player_1_Barracks_len_player_1_diff_Marine_len_player_0_Barracks_len_player_0_diff_diff', 'Marine_len_player_1_Factory_len_player_1_diff', 'Marine_len_player_1_Hatchery_len_player_1_diff', 'Marine_len_player_1_Hatchery_len_player_1_diff_Marine_len_player_0_Hatchery_len_player_0_diff_diff', 'Marine_len_player_1_Hellion_len_player_1_diff', 'Marine_len_player_1_MULE_len_player_1_diff', 'Marine_len_player_1_MULE_len_player_1_diff_Marine_len_player_0_MULE_len_player_0_diff_diff', 'Marine_len_player_1_OrbitalCommand_len_player_1_diff', 'Marine_len_player_1_OrbitalCommand_len_player_1_diff_Marine_len_player_0_OrbitalCommand_len_player_0_diff_diff', 'Marine_len_player_1_Overlord_len_player_1_diff_Marine_len_player_0_Overlord_len_player_0_diff_diff', 'Marine_len_player_1_Probe_len_player_1_diff_Marine_len_player_0_Probe_len_player_0_diff_diff', 'Marine_len_player_1_Queen_len_player_1_diff', 'Marine_len_player_1_Queen_len_player_1_diff_Marine_len_player_0_Queen_len_player_0_diff_diff', 'Marine_len_player_1_Reaper_len_player_1_diff', 'Marine_len_player_1_Reaper_len_player_1_diff_Marine_len_player_0_Reaper_len_player_0_diff_diff', 'Marine_len_player_1_Roach_len_player_1_diff', 'Marine_len_player_1_Starport_len_player_1_diff_Marine_len_player_0_Starport_len_player_0_diff_diff', 'Marine_len_player_1_SupplyDepotLowered_len_player_1_diff', 'Marine_len_player_1_Zergling_len_player_1_diff_Marine_len_player_0_Zergling_len_player_0_diff_diff', 'Marine_right_click_target_count_diff', 'Marine_right_click_target_count_player_1', 'Medivac_len_player_0', 'Medivac_len_player_1', 'Medivac_right_click_target_count_player_0', 'MissileTurret_len_player_0', 'MissileTurret_len_player_1', 'Mutalisk_len_player_0', 'Mutalisk_len_player_1', 'Nexus_len_player_0', 'Nexus_right_click_target_count_diff', 'None_len_player_0', 'None_len_player_1', 'None_right_click_target_count_diff', 'NydusNetwork_len_player_1', 'NydusWorm_len_player_0', 'NydusWorm_len_player_1', 'Observer_len_player_0', 'OrbitalCommand_len_player_0_BanelingCocoon_len_player_0_diff', 'OrbitalCommand_len_player_0_Hellion_len_player_0_diff', 'OrbitalCommand_len_player_0_Queen_len_player_0_diff', 'OrbitalCommand_len_player_0_Reaper_len_player_0_diff', 'OrbitalCommand_len_player_0_SupplyDepotLowered_len_player_0_diff', 'OrbitalCommand_len_player_1_BanelingCocoon_len_player_1_diff_OrbitalCommand_len_player_0_BanelingCocoon_len_player_0_diff_diff', 'OrbitalCommand_len_player_1_Factory_len_player_1_diff', 'OrbitalCommand_len_player_1_Hatchery_len_player_1_diff', 'OrbitalCommand_len_player_1_Hellion_len_player_1_diff', 'OrbitalCommand_len_player_1_MULE_len_player_1_diff', 'OrbitalCommand_len_player_1_MULE_len_player_1_diff_OrbitalCommand_len_player_0_MULE_len_player_0_diff_diff', 'OrbitalCommand_len_player_1_Overlord_len_player_1_diff_OrbitalCommand_len_player_0_Overlord_len_player_0_diff_diff', 'OrbitalCommand_len_player_1_Queen_len_player_1_diff', 'OrbitalCommand_len_player_1_Queen_len_player_1_diff_OrbitalCommand_len_player_0_Queen_len_player_0_diff_diff', 'OrbitalCommand_len_player_1_Roach_len_player_1_diff', 'OrbitalCommand_len_player_1_Starport_len_player_1_diff', 'OrbitalCommand_len_player_1_Starport_len_player_1_diff_OrbitalCommand_len_player_0_Starport_len_player_0_diff_diff', 'OrbitalCommand_len_player_1_SupplyDepotLowered_len_player_1_diff', 'OrbitalCommand_right_click_target_count_diff', 'OverlordGenerateCreepKeybind_len_player_0', 'OverlordGenerateCreepKeybind_len_player_1', 'Overlord_len_player_0', 'Overlord_len_player_0_Hellion_len_player_0_diff', 'Overlord_len_player_0_MULE_len_player_0_diff', 'Overlord_len_player_0_Queen_len_player_0_diff', 'Overlord_len_player_0_Reaper_len_player_0_diff', 'Overlord_len_player_1', 'Overlord_len_player_1_Factory_len_player_1_diff', 'Overlord_len_player_1_Factory_len_player_1_diff_Overlord_len_player_0_Factory_len_player_0_diff_diff', 'Overlord_len_player_1_Hellion_len_player_1_diff', 'Overlord_len_player_1_MULE_len_player_1_diff', 'Overlord_len_player_1_MULE_len_player_1_diff_Overlord_len_player_0_MULE_len_player_0_diff_diff', 'Overlord_len_player_1_Queen_len_player_1_diff', 'Overlord_len_player_1_Reaper_len_player_1_diff', 'Overlord_len_player_1_Roach_len_player_1_diff', 'Overlord_len_player_1_Starport_len_player_1_diff_Overlord_len_player_0_Starport_len_player_0_diff_diff', 'Overlord_right_click_target_count_diff', 'Overlord_right_click_target_count_player_0', 'Overlord_right_click_target_count_player_1', 'OverseerCocoon_len_player_0', 'OverseerCocoon_len_player_1', 'Overseer_len_player_0', 'Phoenix_len_player_0', 'PhotonCannon_len_player_0', 'PhotonCannon_len_player_1', 'PhotonCannon_right_click_target_count_diff', 'PhotonCannon_right_click_target_count_player_0', 'PhysicsCapsule_len_player_0', 'PhysicsCapsule_len_player_1', 'PhysicsCapsule_right_click_target_count_diff', 'PhysicsCapsule_right_click_target_count_player_0', 'PhysicsCapsule_right_click_target_count_player_1', 'PlanetaryFortress_len_player_1', 'Probe_len_player_0', 'Probe_len_player_0_Hellion_len_player_0_diff', 'Probe_len_player_0_MULE_len_player_0_diff', 'Probe_len_player_0_Roach_len_player_0_diff', 'Probe_len_player_0_Starport_len_player_0_diff', 'Probe_len_player_1', 'Probe_len_player_1_BanelingCocoon_len_player_1_diff', 'Probe_len_player_1_Barracks_len_player_1_diff', 'Probe_len_player_1_Factory_len_player_1_diff', 'Probe_len_player_1_Hatchery_len_player_1_diff', 'Probe_len_player_1_Hatchery_len_player_1_diff_Probe_len_player_0_Hatchery_len_player_0_diff_diff', 'Probe_len_player_1_Hellion_len_player_1_diff', 'Probe_len_player_1_MULE_len_player_1_diff', 'Probe_len_player_1_OrbitalCommand_len_player_1_diff', 'Probe_len_player_1_OrbitalCommand_len_player_1_diff_Probe_len_player_0_OrbitalCommand_len_player_0_diff_diff', 'Probe_len_player_1_Overlord_len_player_1_diff', 'Probe_len_player_1_Queen_len_player_1_diff', 'Probe_len_player_1_Reaper_len_player_1_diff', 'Probe_len_player_1_Reaper_len_player_1_diff_Probe_len_player_0_Reaper_len_player_0_diff_diff', 'Probe_len_player_1_Roach_len_player_1_diff', 'Probe_len_player_1_Starport_len_player_1_diff', 'Probe_len_player_1_Starport_len_player_1_diff_Probe_len_player_0_Starport_len_player_0_diff_diff', 'Probe_len_player_1_SupplyDepotLowered_len_player_1_diff', 'Pylon_len_player_0', 'Pylon_len_player_1', 'Pylon_right_click_target_count_diff', 'QueenBurrowed_len_player_0', 'QueenBurrowed_len_player_1', 'Queen_len_player_0', 'Queen_len_player_0_Factory_len_player_0_diff', 'Queen_len_player_0_Hellion_len_player_0_diff', 'Queen_len_player_0_MULE_len_player_0_diff', 'Queen_len_player_0_Reaper_len_player_0_diff', 'Queen_len_player_1', 'Queen_len_player_1_Factory_len_player_1_diff', 'Queen_len_player_1_Factory_len_player_1_diff_Queen_len_player_0_Factory_len_player_0_diff_diff', 'Queen_len_player_1_Hatchery_len_player_1_diff', 'Queen_len_player_1_Hellion_len_player_1_diff', 'Queen_len_player_1_MULE_len_player_1_diff', 'Queen_len_player_1_Reaper_len_player_1_diff', 'Queen_len_player_1_Roach_len_player_1_diff', 'Queen_len_player_1_Starport_len_player_1_diff', 'Queen_right_click_target_count_diff', 'Queen_right_click_target_count_player_0', 'Queen_right_click_target_count_player_1', 'Ravager_len_player_0', 'Ravager_len_player_1', 'Raven_len_player_0', 'Raven_len_player_1', 'Reactor_len_player_0', 'Reactor_len_player_1', 'Reaper_len_player_0', 'Reaper_len_player_0_Roach_len_player_0_diff', 'Reaper_len_player_0_Starport_len_player_0_diff', 'Reaper_len_player_1', 'Reaper_len_player_1_Roach_len_player_1_diff', 'Reaper_len_player_1_Roach_len_player_1_diff_Reaper_len_player_0_Roach_len_player_0_diff_diff', 'Reaper_len_player_1_Starport_len_player_1_diff', 'Reaper_right_click_target_count_player_0', 'Reaper_right_click_target_count_player_1', 'Refinery_len_player_0', 'Refinery_len_player_1', 'Refinery_right_click_target_count_player_1', 'RichMineralField750_len_player_0', 'RichMineralField750_len_player_1', 'RichMineralField_len_player_0', 'RichMineralField_len_player_1', 'Right Click_player_0_isnull', 'Right Click_player_1', 'Right Click_player_1_isnull', 'RoachBurrowed_len_player_0', 'RoachBurrowed_len_player_1', 'RoachWarren_len_player_0', 'Roach_len_player_0', 'Roach_len_player_0_Starport_len_player_0_diff', 'Roach_len_player_1', 'Roach_len_player_1_Starport_len_player_1_diff', 'Roach_right_click_target_count_diff', 'Roach_right_click_target_count_player_0', 'Roach_right_click_target_count_player_1', 'RoboticsBay_len_player_0', 'RoboticsFacility_len_player_0', 'Rocks2x2NonConjoined_len_player_1', 'SCV_len_player_0_BanelingCocoon_len_player_0_diff', 'SCV_len_player_0_Barracks_len_player_0_diff', 'SCV_len_player_0_Drone_len_player_0_diff', 'SCV_len_player_0_Hatchery_len_player_0_diff', 'SCV_len_player_0_OrbitalCommand_len_player_0_diff', 'SCV_len_player_0_Overlord_len_player_0_diff', 'SCV_len_player_0_Probe_len_player_0_diff', 'SCV_len_player_0_Queen_len_player_0_diff', 'SCV_len_player_0_Roach_len_player_0_diff', 'SCV_len_player_0_SupplyDepotLowered_len_player_0_diff', 'SCV_len_player_0_Zergling_len_player_0_diff', 'SCV_len_player_1', 'SCV_len_player_1_BanelingCocoon_len_player_1_diff', 'SCV_len_player_1_Barracks_len_player_1_diff', 'SCV_len_player_1_Drone_len_player_1_diff', 'SCV_len_player_1_Factory_len_player_1_diff', 'SCV_len_player_1_Factory_len_player_1_diff_SCV_len_player_0_Factory_len_player_0_diff_diff', 'SCV_len_player_1_Hatchery_len_player_1_diff', 'SCV_len_player_1_Hatchery_len_player_1_diff_SCV_len_player_0_Hatchery_len_player_0_diff_diff', 'SCV_len_player_1_Hellion_len_player_1_diff', 'SCV_len_player_1_MULE_len_player_1_diff', 'SCV_len_player_1_Marine_len_player_1_diff', 'SCV_len_player_1_Marine_len_player_1_diff_SCV_len_player_0_Marine_len_player_0_diff_diff', 'SCV_len_player_1_OrbitalCommand_len_player_1_diff', 'SCV_len_player_1_OrbitalCommand_len_player_1_diff_SCV_len_player_0_OrbitalCommand_len_player_0_diff_diff', 'SCV_len_player_1_Overlord_len_player_1_diff', 'SCV_len_player_1_Overlord_len_player_1_diff_SCV_len_player_0_Overlord_len_player_0_diff_diff', 'SCV_len_player_1_Probe_len_player_1_diff_SCV_len_player_0_Probe_len_player_0_diff_diff', 'SCV_len_player_1_Queen_len_player_1_diff', 'SCV_len_player_1_Queen_len_player_1_diff_SCV_len_player_0_Queen_len_player_0_diff_diff', 'SCV_len_player_1_Reaper_len_player_1_diff', 'SCV_len_player_1_Roach_len_player_1_diff', 'SCV_len_player_1_Starport_len_player_1_diff', 'SCV_len_player_1_Starport_len_player_1_diff_SCV_len_player_0_Starport_len_player_0_diff_diff', 'SCV_len_player_1_SupplyDepotLowered_len_player_1_diff', 'SCV_len_player_1_Zergling_len_player_1_diff', 'SCV_len_player_1_Zergling_len_player_1_diff_SCV_len_player_0_Zergling_len_player_0_diff_diff', 'SCV_right_click_target_count_player_0', 'Selection_player_0_isnull', 'Selection_player_1_isnull', 'SensorTower_len_player_0', 'SensorTower_len_player_1', 'Sentry_len_player_0', 'SetControlGroup_player_0_SetControlGroup_player_1_diff', 'SetControlGroup_player_0_isnull', 'SetControlGroup_player_1', 'SetControlGroup_player_1_isnull', 'SiegeTankSieged_len_player_1', 'SiegeTankSieged_right_click_target_count_diff', 'SiegeTankSieged_right_click_target_count_player_0', 'SiegeTank_right_click_target_count_player_0', 'SiegeTank_right_click_target_count_player_1', 'SpawningPool_len_player_1', 'SpawningPool_right_click_target_count_diff', 'SpawningPool_right_click_target_count_player_0', 'SpawningPool_right_click_target_count_player_1', 'SpineCrawlerUprooted_len_player_0', 'SpineCrawlerUprooted_len_player_1', 'SpineCrawler_len_player_0', 'SpineCrawler_len_player_1', 'Spire_len_player_0', 'Spire_len_player_1', 'SporeCrawlerUprooted_len_player_0', 'SporeCrawlerUprooted_len_player_1', 'SporeCrawler_len_player_1', 'Stalker_len_player_0', 'Stalker_len_player_1', 'Stalker_right_click_target_count_diff', 'Stalker_right_click_target_count_player_0', 'Stalker_right_click_target_count_player_1', 'StarportReactor_len_player_0', 'StarportReactor_len_player_1', 'StarportTechLab_len_player_0', 'StarportTechLab_len_player_1', 'Starport_len_player_0', 'Starport_len_player_1', 'Starport_right_click_target_count_diff', 'Starport_right_click_target_count_player_0', 'SupplyDepotLowered_len_player_0', 'SupplyDepotLowered_len_player_0_BanelingCocoon_len_player_0_diff', 'SupplyDepotLowered_len_player_0_Factory_len_player_0_diff', 'SupplyDepotLowered_len_player_0_Hellion_len_player_0_diff', 'SupplyDepotLowered_len_player_0_Overlord_len_player_0_diff', 'SupplyDepotLowered_len_player_0_Starport_len_player_0_diff', 'SupplyDepotLowered_len_player_1', 'SupplyDepotLowered_len_player_1_BanelingCocoon_len_player_1_diff', 'SupplyDepotLowered_len_player_1_Factory_len_player_1_diff', 'SupplyDepotLowered_len_player_1_Hatchery_len_player_1_diff', 'SupplyDepotLowered_len_player_1_Hatchery_len_player_1_diff_SupplyDepotLowered_len_player_0_Hatchery_len_player_0_diff_diff', 'SupplyDepotLowered_len_player_1_Hellion_len_player_1_diff', 'SupplyDepotLowered_len_player_1_MULE_len_player_1_diff', 'SupplyDepotLowered_len_player_1_Overlord_len_player_1_diff', 'SupplyDepotLowered_len_player_1_Overlord_len_player_1_diff_SupplyDepotLowered_len_player_0_Overlord_len_player_0_diff_diff', 'SupplyDepotLowered_len_player_1_Queen_len_player_1_diff', 'SupplyDepotLowered_len_player_1_Queen_len_player_1_diff_SupplyDepotLowered_len_player_0_Queen_len_player_0_diff_diff', 'SupplyDepotLowered_len_player_1_Reaper_len_player_1_diff_SupplyDepotLowered_len_player_0_Reaper_len_player_0_diff_diff', 'SupplyDepotLowered_len_player_1_Roach_len_player_1_diff', 'SupplyDepotLowered_len_player_1_Starport_len_player_1_diff', 'SupplyDepotLowered_len_player_1_Starport_len_player_1_diff_SupplyDepotLowered_len_player_0_Starport_len_player_0_diff_diff', 'SupplyDepotLowered_right_click_target_count_player_0', 'SupplyDepot_right_click_target_count_diff', 'SupplyDepot_right_click_target_count_player_1', 'SwarmHost_len_player_1', 'TechLab_len_player_1', 'TemplarArchive_len_player_0', 'TemplarArchive_len_player_1', 'Thor_len_player_0', 'Thor_len_player_1', 'TwilightCouncil_len_player_0', 'UltraliskCavern_len_player_1', 'VikingAssault_len_player_0', 'VikingAssault_len_player_1', 'Viking_len_player_0', 'Viking_len_player_1', 'VoidRay_len_player_0', 'WarpGate_len_player_0', 'WarpGate_right_click_target_count_diff', 'WarpGate_right_click_target_count_player_0', 'WarpGate_right_click_target_count_player_1', 'WarpPrismPhasing_len_player_0', 'WarpPrismPhasing_len_player_1', 'WarpPrism_len_player_0', 'Weapon_len_player_0', 'Weapon_len_player_1', 'XelNagaDestructibleBlocker8W_len_player_0', 'XelNagaDestructibleBlocker8W_len_player_1', 'XelNagaTower_len_player_0', 'XelNagaTower_len_player_1', 'XelNagaTower_right_click_target_count_diff', 'XelNagaTower_right_click_target_count_player_0', 'XelNagaTower_right_click_target_count_player_1', 'Zealot_len_player_0', 'Zealot_right_click_target_count_diff', 'Zealot_right_click_target_count_player_0', 'ZerglingBurrowed_len_player_0', 'ZerglingBurrowed_len_player_1', 'Zergling_len_player_0', 'Zergling_len_player_0_Barracks_len_player_0_diff', 'Zergling_len_player_0_Hellion_len_player_0_diff', 'Zergling_len_player_0_MULE_len_player_0_diff', 'Zergling_len_player_0_Probe_len_player_0_diff', 'Zergling_len_player_0_Queen_len_player_0_diff', 'Zergling_len_player_0_Starport_len_player_0_diff', 'Zergling_len_player_1', 'Zergling_len_player_1_BanelingCocoon_len_player_1_diff', 'Zergling_len_player_1_BanelingCocoon_len_player_1_diff_Zergling_len_player_0_BanelingCocoon_len_player_0_diff_diff', 'Zergling_len_player_1_Barracks_len_player_1_diff', 'Zergling_len_player_1_Factory_len_player_1_diff', 'Zergling_len_player_1_Factory_len_player_1_diff_Zergling_len_player_0_Factory_len_player_0_diff_diff', 'Zergling_len_player_1_Hatchery_len_player_1_diff', 'Zergling_len_player_1_Hellion_len_player_1_diff', 'Zergling_len_player_1_Hellion_len_player_1_diff_Zergling_len_player_0_Hellion_len_player_0_diff_diff', 'Zergling_len_player_1_MULE_len_player_1_diff', 'Zergling_len_player_1_MULE_len_player_1_diff_Zergling_len_player_0_MULE_len_player_0_diff_diff', 'Zergling_len_player_1_OrbitalCommand_len_player_1_diff', 'Zergling_len_player_1_OrbitalCommand_len_player_1_diff_Zergling_len_player_0_OrbitalCommand_len_player_0_diff_diff', 'Zergling_len_player_1_Reaper_len_player_1_diff', 'Zergling_len_player_1_Reaper_len_player_1_diff_Zergling_len_player_0_Reaper_len_player_0_diff_diff', 'Zergling_len_player_1_Roach_len_player_1_diff', 'Zergling_len_player_1_Starport_len_player_1_diff', 'Zergling_len_player_1_SupplyDepotLowered_len_player_1_diff_Zergling_len_player_0_SupplyDepotLowered_len_player_0_diff_diff', 'ability_code_1000_diff', 'ability_code_1000_player_0', 'ability_code_1000_player_1', 'ability_code_1020_diff', 'ability_code_1020_player_0_ability_code_102A_player_0_diff', 'ability_code_1020_player_0_ability_code_16E2_player_0_diff', 'ability_code_1020_player_0_ability_code_1E60_player_0_diff', 'ability_code_1020_player_1', 'ability_code_1020_player_1_ability_code_102A_player_1_diff_ability_code_1020_player_0_ability_code_102A_player_0_diff_diff', 'ability_code_1020_player_1_ability_code_1261_player_1_diff', 'ability_code_1020_player_1_ability_code_16E2_player_1_diff', 'ability_code_1020_player_1_ability_code_1E60_player_1_diff', 'ability_code_1020_player_1_ability_code_1E60_player_1_diff_ability_code_1020_player_0_ability_code_1E60_player_0_diff_diff', 'ability_code_1020_player_1_ability_code_480_player_1_diff', 'ability_code_1021_diff', 'ability_code_1021_first_time_player_1', 'ability_code_1021_player_0', 'ability_code_1021_player_0_ability_code_1023_player_0_diff', 'ability_code_1021_player_0_ability_code_102A_player_0_diff', 'ability_code_1021_player_0_ability_code_13A0_player_0_diff', 'ability_code_1021_player_0_ability_code_16E2_player_0_diff', 'ability_code_1021_player_0_ability_code_1822_player_0_diff', 'ability_code_1021_player_0_ability_code_1E60_player_0_diff', 'ability_code_1021_player_0_ability_code_B40_player_0_diff', 'ability_code_1021_player_1_ability_code_1020_player_1_diff', 'ability_code_1021_player_1_ability_code_1020_player_1_diff_ability_code_1021_player_0_ability_code_1020_player_0_diff_diff', 'ability_code_1021_player_1_ability_code_1022_player_1_diff', 'ability_code_1021_player_1_ability_code_1022_player_1_diff_ability_code_1021_player_0_ability_code_1022_player_0_diff_diff', 'ability_code_1021_player_1_ability_code_1023_player_1_diff_ability_code_1021_player_0_ability_code_1023_player_0_diff_diff', 'ability_code_1021_player_1_ability_code_102A_player_1_diff', 'ability_code_1021_player_1_ability_code_102A_player_1_diff_ability_code_1021_player_0_ability_code_102A_player_0_diff_diff', 'ability_code_1021_player_1_ability_code_1261_player_1_diff_ability_code_1021_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_1021_player_1_ability_code_13A0_player_1_diff', 'ability_code_1021_player_1_ability_code_13A0_player_1_diff_ability_code_1021_player_0_ability_code_13A0_player_0_diff_diff', 'ability_code_1021_player_1_ability_code_16E2_player_1_diff', 'ability_code_1021_player_1_ability_code_16E2_player_1_diff_ability_code_1021_player_0_ability_code_16E2_player_0_diff_diff', 'ability_code_1021_player_1_ability_code_1821_player_1_diff_ability_code_1021_player_0_ability_code_1821_player_0_diff_diff', 'ability_code_1021_player_1_ability_code_1822_player_1_diff', 'ability_code_1021_player_1_ability_code_1822_player_1_diff_ability_code_1021_player_0_ability_code_1822_player_0_diff_diff', 'ability_code_1021_player_1_ability_code_1C60_player_1_diff', 'ability_code_1021_player_1_ability_code_1E60_player_1_diff', 'ability_code_1021_player_1_ability_code_1E60_player_1_diff_ability_code_1021_player_0_ability_code_1E60_player_0_diff_diff', 'ability_code_1021_player_1_ability_code_480_player_1_diff', 'ability_code_1021_player_1_ability_code_480_player_1_diff_ability_code_1021_player_0_ability_code_480_player_0_diff_diff', 'ability_code_1021_player_1_ability_code_B40_player_1_diff', 'ability_code_1021_player_1_ability_code_B40_player_1_diff_ability_code_1021_player_0_ability_code_B40_player_0_diff_diff', 'ability_code_1021_player_1_ability_code_DE0_player_1_diff', 'ability_code_1021_player_1_ability_code_DE0_player_1_diff_ability_code_1021_player_0_ability_code_DE0_player_0_diff_diff', 'ability_code_1022_diff', 'ability_code_1022_first_time_diff', 'ability_code_1022_first_time_player_1', 'ability_code_1022_player_0_ability_code_1023_player_0_diff', 'ability_code_1022_player_0_ability_code_102A_player_0_diff', 'ability_code_1022_player_0_ability_code_13A0_player_0_diff', 'ability_code_1022_player_0_ability_code_16E2_player_0_diff', 'ability_code_1022_player_0_ability_code_1821_player_0_diff', 'ability_code_1022_player_0_ability_code_1822_player_0_diff', 'ability_code_1022_player_0_ability_code_1E60_player_0_diff', 'ability_code_1022_player_0_ability_code_480_player_0_diff', 'ability_code_1022_player_1', 'ability_code_1022_player_1_ability_code_1020_player_1_diff', 'ability_code_1022_player_1_ability_code_1023_player_1_diff', 'ability_code_1022_player_1_ability_code_1023_player_1_diff_ability_code_1022_player_0_ability_code_1023_player_0_diff_diff', 'ability_code_1022_player_1_ability_code_102A_player_1_diff_ability_code_1022_player_0_ability_code_102A_player_0_diff_diff', 'ability_code_1022_player_1_ability_code_1261_player_1_diff', 'ability_code_1022_player_1_ability_code_13A0_player_1_diff', 'ability_code_1022_player_1_ability_code_13A0_player_1_diff_ability_code_1022_player_0_ability_code_13A0_player_0_diff_diff', 'ability_code_1022_player_1_ability_code_15E0_player_1_diff_ability_code_1022_player_0_ability_code_15E0_player_0_diff_diff', 'ability_code_1022_player_1_ability_code_16E2_player_1_diff', 'ability_code_1022_player_1_ability_code_16E2_player_1_diff_ability_code_1022_player_0_ability_code_16E2_player_0_diff_diff', 'ability_code_1022_player_1_ability_code_1821_player_1_diff', 'ability_code_1022_player_1_ability_code_1822_player_1_diff', 'ability_code_1022_player_1_ability_code_1C60_player_1_diff', 'ability_code_1022_player_1_ability_code_1C60_player_1_diff_ability_code_1022_player_0_ability_code_1C60_player_0_diff_diff', 'ability_code_1022_player_1_ability_code_1E60_player_1_diff_ability_code_1022_player_0_ability_code_1E60_player_0_diff_diff', 'ability_code_1022_player_1_ability_code_480_player_1_diff', 'ability_code_1022_player_1_ability_code_480_player_1_diff_ability_code_1022_player_0_ability_code_480_player_0_diff_diff', 'ability_code_1023_diff', 'ability_code_1023_first_time_diff', 'ability_code_1023_last_first_diff', 'ability_code_1023_player_0', 'ability_code_1023_player_0_ability_code_1020_player_0_diff', 'ability_code_1023_player_0_ability_code_102A_player_0_diff', 'ability_code_1023_player_0_ability_code_1261_player_0_diff', 'ability_code_1023_player_0_ability_code_15E0_player_0_diff', 'ability_code_1023_player_0_ability_code_16E2_player_0_diff', 'ability_code_1023_player_0_ability_code_1821_player_0_diff', 'ability_code_1023_player_0_ability_code_1822_player_0_diff', 'ability_code_1023_player_0_ability_code_1C60_player_0_diff', 'ability_code_1023_player_0_ability_code_1E60_player_0_diff', 'ability_code_1023_player_0_ability_code_480_player_0_diff', 'ability_code_1023_player_1', 'ability_code_1023_player_1_ability_code_1020_player_1_diff', 'ability_code_1023_player_1_ability_code_1020_player_1_diff_ability_code_1023_player_0_ability_code_1020_player_0_diff_diff', 'ability_code_1023_player_1_ability_code_102A_player_1_diff', 'ability_code_1023_player_1_ability_code_102A_player_1_diff_ability_code_1023_player_0_ability_code_102A_player_0_diff_diff', 'ability_code_1023_player_1_ability_code_1261_player_1_diff', 'ability_code_1023_player_1_ability_code_1261_player_1_diff_ability_code_1023_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_1023_player_1_ability_code_13A0_player_1_diff', 'ability_code_1023_player_1_ability_code_15E0_player_1_diff', 'ability_code_1023_player_1_ability_code_16E2_player_1_diff', 'ability_code_1023_player_1_ability_code_1821_player_1_diff', 'ability_code_1023_player_1_ability_code_1821_player_1_diff_ability_code_1023_player_0_ability_code_1821_player_0_diff_diff', 'ability_code_1023_player_1_ability_code_1822_player_1_diff', 'ability_code_1023_player_1_ability_code_1822_player_1_diff_ability_code_1023_player_0_ability_code_1822_player_0_diff_diff', 'ability_code_1023_player_1_ability_code_1C60_player_1_diff', 'ability_code_1023_player_1_ability_code_1C60_player_1_diff_ability_code_1023_player_0_ability_code_1C60_player_0_diff_diff', 'ability_code_1023_player_1_ability_code_1E60_player_1_diff', 'ability_code_1023_player_1_ability_code_480_player_1_diff', 'ability_code_1023_player_1_ability_code_480_player_1_diff_ability_code_1023_player_0_ability_code_480_player_0_diff_diff', 'ability_code_1024_diff', 'ability_code_1024_first_time_diff', 'ability_code_1024_first_time_player_0', 'ability_code_1024_player_1', 'ability_code_1025_first_time_diff', 'ability_code_1025_first_time_player_0', 'ability_code_1025_player_0', 'ability_code_1025_player_1', 'ability_code_1026_diff', 'ability_code_1026_first_time_diff', 'ability_code_1026_player_0', 'ability_code_1026_player_1', 'ability_code_1028_diff', 'ability_code_1028_player_0', 'ability_code_1028_player_1', 'ability_code_1029_diff', 'ability_code_1029_player_0', 'ability_code_1029_player_1', 'ability_code_102A_diff', 'ability_code_102A_first_time_diff', 'ability_code_102A_player_0', 'ability_code_102A_player_0_ability_code_1261_player_0_diff', 'ability_code_102A_player_0_ability_code_480_player_0_diff', 'ability_code_102A_player_1', 'ability_code_102A_player_1_ability_code_1261_player_1_diff', 'ability_code_102A_player_1_ability_code_1261_player_1_diff_ability_code_102A_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_102A_player_1_ability_code_480_player_1_diff', 'ability_code_102A_player_1_ability_code_480_player_1_diff_ability_code_102A_player_0_ability_code_480_player_0_diff_diff', 'ability_code_102B_diff', 'ability_code_102B_first_time_diff', 'ability_code_102B_first_time_player_1', 'ability_code_102B_player_0', 'ability_code_102B_player_1', 'ability_code_102D_first_time_diff', 'ability_code_102D_player_1', 'ability_code_102F_diff', 'ability_code_102F_player_0', 'ability_code_102F_player_1', 'ability_code_103E_diff', 'ability_code_103E_player_0', 'ability_code_103E_player_1', 'ability_code_1060_player_0', 'ability_code_1060_player_1', 'ability_code_1080_diff', 'ability_code_1080_player_0', 'ability_code_1080_player_1', 'ability_code_1081_diff', 'ability_code_1081_player_0', 'ability_code_1081_player_1', 'ability_code_10E0_diff', 'ability_code_10E0_first_time_diff', 'ability_code_10E0_first_time_player_0', 'ability_code_10E0_first_time_player_1', 'ability_code_10E0_player_0', 'ability_code_10E0_player_1', 'ability_code_1100_diff', 'ability_code_1100_player_0', 'ability_code_1100_player_1', 'ability_code_1120_diff', 'ability_code_1120_player_0', 'ability_code_1120_player_1', 'ability_code_1121_diff', 'ability_code_1121_player_0', 'ability_code_1121_player_1', 'ability_code_1140_diff', 'ability_code_1140_player_0', 'ability_code_1140_player_1', 'ability_code_1142_diff', 'ability_code_1142_first_time_diff', 'ability_code_1142_first_time_player_1', 'ability_code_1142_player_0', 'ability_code_1142_player_1', 'ability_code_1143_diff', 'ability_code_1143_player_0', 'ability_code_1143_player_1', 'ability_code_1160_diff', 'ability_code_1160_first_time_diff', 'ability_code_1160_first_time_player_0', 'ability_code_1160_first_time_player_1', 'ability_code_1160_player_1', 'ability_code_1180_diff', 'ability_code_1180_player_0', 'ability_code_1180_player_1', 'ability_code_11A0_diff', 'ability_code_11A0_player_0', 'ability_code_11A0_player_1', 'ability_code_11C0_diff', 'ability_code_11C0_player_0', 'ability_code_11C0_player_1', 'ability_code_11E1_diff', 'ability_code_11E1_player_1', 'ability_code_11E3_player_0', 'ability_code_11E3_player_1', 'ability_code_1201_diff', 'ability_code_1201_player_0', 'ability_code_1201_player_1', 'ability_code_1203_diff', 'ability_code_1203_player_0', 'ability_code_1203_player_1', 'ability_code_1204_diff', 'ability_code_1204_player_0', 'ability_code_1204_player_1', 'ability_code_1220_diff', 'ability_code_1220_player_0', 'ability_code_1220_player_1', 'ability_code_1240_diff', 'ability_code_1240_player_0', 'ability_code_1240_player_1', 'ability_code_1260_first_time_diff', 'ability_code_1260_player_1', 'ability_code_1261_diff', 'ability_code_1261_player_0', 'ability_code_1261_player_0_ability_code_480_player_0_diff', 'ability_code_1261_player_1', 'ability_code_1261_player_1_ability_code_480_player_1_diff_ability_code_1261_player_0_ability_code_480_player_0_diff_diff', 'ability_code_127E_diff', 'ability_code_127E_player_0', 'ability_code_127E_player_1', 'ability_code_1280_first_time_diff', 'ability_code_1280_first_time_player_1', 'ability_code_1280_player_0', 'ability_code_1280_player_1', 'ability_code_12A0_player_0', 'ability_code_12A0_player_1', 'ability_code_12A1_diff', 'ability_code_12A1_first_time_diff', 'ability_code_12A1_player_0', 'ability_code_12A1_player_1', 'ability_code_12BE_diff', 'ability_code_12BE_player_0', 'ability_code_12BE_player_1', 'ability_code_12C0_diff', 'ability_code_12C0_first_time_diff', 'ability_code_12C0_first_time_player_1', 'ability_code_12C0_player_0', 'ability_code_12C0_player_1', 'ability_code_12E0_diff', 'ability_code_12E0_player_0', 'ability_code_12E0_player_1', 'ability_code_12E1_diff', 'ability_code_12E1_first_time_diff', 'ability_code_12E1_first_time_player_1', 'ability_code_12E1_player_0', 'ability_code_12E1_player_1', 'ability_code_12FE_diff', 'ability_code_12FE_player_0', 'ability_code_12FE_player_1', 'ability_code_1300_diff', 'ability_code_1300_first_time_diff', 'ability_code_1300_first_time_player_0', 'ability_code_1300_first_time_player_1', 'ability_code_1300_player_0', 'ability_code_1300_player_1', 'ability_code_1320_first_time_diff', 'ability_code_1320_player_0', 'ability_code_1320_player_1', 'ability_code_1340_first_time_diff', 'ability_code_1340_first_time_player_1', 'ability_code_1340_player_0', 'ability_code_1340_player_1', 'ability_code_1360_diff', 'ability_code_1360_first_time_player_1', 'ability_code_1360_last_first_player_0', 'ability_code_1360_player_0', 'ability_code_1360_player_0_ability_code_1020_player_0_diff', 'ability_code_1360_player_0_ability_code_1022_player_0_diff', 'ability_code_1360_player_0_ability_code_102A_player_0_diff', 'ability_code_1360_player_0_ability_code_1261_player_0_diff', 'ability_code_1360_player_0_ability_code_13A0_player_0_diff', 'ability_code_1360_player_0_ability_code_13E0_player_0_diff', 'ability_code_1360_player_0_ability_code_16E2_player_0_diff', 'ability_code_1360_player_0_ability_code_1821_player_0_diff', 'ability_code_1360_player_0_ability_code_1822_player_0_diff', 'ability_code_1360_player_0_ability_code_1C60_player_0_diff', 'ability_code_1360_player_0_ability_code_1E60_player_0_diff', 'ability_code_1360_player_0_ability_code_480_player_0_diff', 'ability_code_1360_player_0_ability_code_B40_player_0_diff', 'ability_code_1360_player_1', 'ability_code_1360_player_1_ability_code_1020_player_1_diff', 'ability_code_1360_player_1_ability_code_1020_player_1_diff_ability_code_1360_player_0_ability_code_1020_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_1021_player_1_diff', 'ability_code_1360_player_1_ability_code_1021_player_1_diff_ability_code_1360_player_0_ability_code_1021_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_1022_player_1_diff_ability_code_1360_player_0_ability_code_1022_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_1023_player_1_diff', 'ability_code_1360_player_1_ability_code_1023_player_1_diff_ability_code_1360_player_0_ability_code_1023_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_102A_player_1_diff_ability_code_1360_player_0_ability_code_102A_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_1261_player_1_diff_ability_code_1360_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_13A0_player_1_diff', 'ability_code_1360_player_1_ability_code_13A0_player_1_diff_ability_code_1360_player_0_ability_code_13A0_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_13E0_player_1_diff', 'ability_code_1360_player_1_ability_code_13E0_player_1_diff_ability_code_1360_player_0_ability_code_13E0_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_15E0_player_1_diff', 'ability_code_1360_player_1_ability_code_16E2_player_1_diff', 'ability_code_1360_player_1_ability_code_16E2_player_1_diff_ability_code_1360_player_0_ability_code_16E2_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_1820_player_1_diff', 'ability_code_1360_player_1_ability_code_1821_player_1_diff', 'ability_code_1360_player_1_ability_code_1822_player_1_diff', 'ability_code_1360_player_1_ability_code_1822_player_1_diff_ability_code_1360_player_0_ability_code_1822_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_1C60_player_1_diff', 'ability_code_1360_player_1_ability_code_1C60_player_1_diff_ability_code_1360_player_0_ability_code_1C60_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_1E60_player_1_diff', 'ability_code_1360_player_1_ability_code_1E60_player_1_diff_ability_code_1360_player_0_ability_code_1E60_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_480_player_1_diff', 'ability_code_1360_player_1_ability_code_480_player_1_diff_ability_code_1360_player_0_ability_code_480_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_B40_player_1_diff', 'ability_code_1360_player_1_ability_code_B40_player_1_diff_ability_code_1360_player_0_ability_code_B40_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_DE0_player_1_diff', 'ability_code_1360_player_1_ability_code_DE0_player_1_diff_ability_code_1360_player_0_ability_code_DE0_player_0_diff_diff', 'ability_code_1380_diff', 'ability_code_1380_first_time_diff', 'ability_code_1380_first_time_player_0', 'ability_code_1380_player_0', 'ability_code_1380_player_1', 'ability_code_13A0_diff', 'ability_code_13A0_first_time_player_0', 'ability_code_13A0_first_time_player_1', 'ability_code_13A0_player_0', 'ability_code_13A0_player_0_ability_code_1261_player_0_diff', 'ability_code_13A0_player_0_ability_code_15E0_player_0_diff', 'ability_code_13A0_player_0_ability_code_16E2_player_0_diff', 'ability_code_13A0_player_0_ability_code_1821_player_0_diff', 'ability_code_13A0_player_0_ability_code_1C60_player_0_diff', 'ability_code_13A0_player_0_ability_code_1E60_player_0_diff', 'ability_code_13A0_player_1', 'ability_code_13A0_player_1_ability_code_1020_player_1_diff', 'ability_code_13A0_player_1_ability_code_102A_player_1_diff', 'ability_code_13A0_player_1_ability_code_15E0_player_1_diff', 'ability_code_13A0_player_1_ability_code_16E2_player_1_diff', 'ability_code_13A0_player_1_ability_code_16E2_player_1_diff_ability_code_13A0_player_0_ability_code_16E2_player_0_diff_diff', 'ability_code_13A0_player_1_ability_code_1821_player_1_diff', 'ability_code_13A0_player_1_ability_code_1821_player_1_diff_ability_code_13A0_player_0_ability_code_1821_player_0_diff_diff', 'ability_code_13A0_player_1_ability_code_1C60_player_1_diff', 'ability_code_13A0_player_1_ability_code_1C60_player_1_diff_ability_code_13A0_player_0_ability_code_1C60_player_0_diff_diff', 'ability_code_13A0_player_1_ability_code_1E60_player_1_diff', 'ability_code_13A0_player_1_ability_code_1E60_player_1_diff_ability_code_13A0_player_0_ability_code_1E60_player_0_diff_diff', 'ability_code_13A0_player_1_ability_code_480_player_1_diff', 'ability_code_13A0_player_1_ability_code_480_player_1_diff_ability_code_13A0_player_0_ability_code_480_player_0_diff_diff', 'ability_code_13C0_diff', 'ability_code_13C0_first_time_diff', 'ability_code_13C0_first_time_player_0', 'ability_code_13C0_player_0', 'ability_code_13C0_player_1', 'ability_code_13E0_diff', 'ability_code_13E0_first_time_player_0', 'ability_code_13E0_first_time_player_1', 'ability_code_13E0_last_first_diff', 'ability_code_13E0_last_first_player_1', 'ability_code_13E0_player_0', 'ability_code_13E0_player_0_ability_code_1021_player_0_diff', 'ability_code_13E0_player_0_ability_code_1022_player_0_diff', 'ability_code_13E0_player_0_ability_code_1023_player_0_diff', 'ability_code_13E0_player_0_ability_code_102A_player_0_diff', 'ability_code_13E0_player_0_ability_code_1261_player_0_diff', 'ability_code_13E0_player_0_ability_code_15E0_player_0_diff', 'ability_code_13E0_player_0_ability_code_16E2_player_0_diff', 'ability_code_13E0_player_0_ability_code_1821_player_0_diff', 'ability_code_13E0_player_0_ability_code_1822_player_0_diff', 'ability_code_13E0_player_0_ability_code_1C60_player_0_diff', 'ability_code_13E0_player_0_ability_code_1E60_player_0_diff', 'ability_code_13E0_player_0_ability_code_480_player_0_diff', 'ability_code_13E0_player_0_ability_code_B40_player_0_diff', 'ability_code_13E0_player_0_ability_code_DE0_player_0_diff', 'ability_code_13E0_player_1', 'ability_code_13E0_player_1_ability_code_1020_player_1_diff', 'ability_code_13E0_player_1_ability_code_1020_player_1_diff_ability_code_13E0_player_0_ability_code_1020_player_0_diff_diff', 'ability_code_13E0_player_1_ability_code_1021_player_1_diff', 'ability_code_13E0_player_1_ability_code_1021_player_1_diff_ability_code_13E0_player_0_ability_code_1021_player_0_diff_diff', 'ability_code_13E0_player_1_ability_code_1023_player_1_diff_ability_code_13E0_player_0_ability_code_1023_player_0_diff_diff', 'ability_code_13E0_player_1_ability_code_102A_player_1_diff', 'ability_code_13E0_player_1_ability_code_1261_player_1_diff', 'ability_code_13E0_player_1_ability_code_1261_player_1_diff_ability_code_13E0_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_13E0_player_1_ability_code_13A0_player_1_diff_ability_code_13E0_player_0_ability_code_13A0_player_0_diff_diff', 'ability_code_13E0_player_1_ability_code_15E0_player_1_diff', 'ability_code_13E0_player_1_ability_code_16E2_player_1_diff', 'ability_code_13E0_player_1_ability_code_16E2_player_1_diff_ability_code_13E0_player_0_ability_code_16E2_player_0_diff_diff', 'ability_code_13E0_player_1_ability_code_1821_player_1_diff', 'ability_code_13E0_player_1_ability_code_1821_player_1_diff_ability_code_13E0_player_0_ability_code_1821_player_0_diff_diff', 'ability_code_13E0_player_1_ability_code_1822_player_1_diff', 'ability_code_13E0_player_1_ability_code_1822_player_1_diff_ability_code_13E0_player_0_ability_code_1822_player_0_diff_diff', 'ability_code_13E0_player_1_ability_code_1C60_player_1_diff_ability_code_13E0_player_0_ability_code_1C60_player_0_diff_diff', 'ability_code_13E0_player_1_ability_code_1E60_player_1_diff', 'ability_code_13E0_player_1_ability_code_1E60_player_1_diff_ability_code_13E0_player_0_ability_code_1E60_player_0_diff_diff', 'ability_code_13E0_player_1_ability_code_480_player_1_diff', 'ability_code_13E0_player_1_ability_code_480_player_1_diff_ability_code_13E0_player_0_ability_code_480_player_0_diff_diff', 'ability_code_13E0_player_1_ability_code_B40_player_1_diff', 'ability_code_13E0_player_1_ability_code_B40_player_1_diff_ability_code_13E0_player_0_ability_code_B40_player_0_diff_diff', 'ability_code_13E0_player_1_ability_code_DE0_player_1_diff', 'ability_code_13E1_diff', 'ability_code_13E1_first_time_player_1', 'ability_code_13E1_player_0', 'ability_code_13E1_player_1', 'ability_code_13E2_diff', 'ability_code_13E2_player_0', 'ability_code_13E2_player_1', 'ability_code_13E3_diff', 'ability_code_13E3_first_time_diff', 'ability_code_13E3_first_time_player_0', 'ability_code_13E3_player_0', 'ability_code_13E3_player_1', 'ability_code_1401_first_time_diff', 'ability_code_1401_player_0', 'ability_code_1401_player_1', 'ability_code_1404_diff', 'ability_code_1404_player_0', 'ability_code_1404_player_1', 'ability_code_1405_diff', 'ability_code_1405_first_time_diff', 'ability_code_1405_player_0', 'ability_code_1405_player_1', 'ability_code_1406_diff', 'ability_code_1406_player_0', 'ability_code_1406_player_1', 'ability_code_1407_diff', 'ability_code_1407_first_time_diff', 'ability_code_1407_first_time_player_0', 'ability_code_1407_first_time_player_1', 'ability_code_1407_player_0', 'ability_code_1407_player_1', 'ability_code_1418_diff', 'ability_code_1418_first_time_diff', 'ability_code_1418_first_time_player_1', 'ability_code_1418_player_0', 'ability_code_1418_player_1', 'ability_code_1420_diff', 'ability_code_1420_player_1', 'ability_code_1421_diff', 'ability_code_1421_player_0', 'ability_code_1421_player_1', 'ability_code_1422_player_0', 'ability_code_1422_player_1', 'ability_code_1423_diff', 'ability_code_1423_player_0', 'ability_code_1423_player_1', 'ability_code_1424_diff', 'ability_code_1424_first_time_player_0', 'ability_code_1424_first_time_player_1', 'ability_code_1424_player_0', 'ability_code_1424_player_1', 'ability_code_1426_diff', 'ability_code_1426_player_0', 'ability_code_1426_player_1', 'ability_code_1440_diff', 'ability_code_1440_player_0', 'ability_code_1440_player_1', 'ability_code_1441_diff', 'ability_code_1441_player_0', 'ability_code_1441_player_1', 'ability_code_1442_diff', 'ability_code_1442_first_time_diff', 'ability_code_1442_first_time_player_1', 'ability_code_1442_player_0', 'ability_code_1442_player_1', 'ability_code_1443_diff', 'ability_code_1443_player_0', 'ability_code_1443_player_1', 'ability_code_1445_diff', 'ability_code_1445_player_0', 'ability_code_1445_player_1', 'ability_code_1446_first_time_diff', 'ability_code_1446_first_time_player_1', 'ability_code_1446_player_0', 'ability_code_1446_player_1', 'ability_code_1447_diff', 'ability_code_1447_player_0', 'ability_code_1447_player_1', 'ability_code_1480_diff', 'ability_code_1480_player_0', 'ability_code_1480_player_1', 'ability_code_14A0_first_time_diff', 'ability_code_14A0_player_0', 'ability_code_14A0_player_1', 'ability_code_14A1_diff', 'ability_code_14A1_first_time_player_0', 'ability_code_14A1_player_0', 'ability_code_14A1_player_1', 'ability_code_14A2_diff', 'ability_code_14A2_player_0', 'ability_code_14A2_player_1', 'ability_code_14C1_diff', 'ability_code_14C1_player_0', 'ability_code_14C1_player_1', 'ability_code_14C4_diff', 'ability_code_14C4_player_0', 'ability_code_14C4_player_1', 'ability_code_14E0_diff', 'ability_code_14E0_player_0', 'ability_code_14E0_player_1', 'ability_code_14E3_diff', 'ability_code_14E3_player_0', 'ability_code_14E3_player_1', 'ability_code_14E9_diff', 'ability_code_14E9_player_0', 'ability_code_14E9_player_1', 'ability_code_14ED_diff', 'ability_code_14ED_player_0', 'ability_code_14ED_player_1', 'ability_code_14EE_diff', 'ability_code_14EE_player_0', 'ability_code_14EE_player_1', 'ability_code_14EF_diff', 'ability_code_14EF_player_0', 'ability_code_14EF_player_1', 'ability_code_1500_diff', 'ability_code_1500_player_0', 'ability_code_1500_player_1', 'ability_code_1525_diff', 'ability_code_1525_player_0', 'ability_code_1525_player_1', 'ability_code_1526_diff', 'ability_code_1526_player_0', 'ability_code_1526_player_1', 'ability_code_152B_diff', 'ability_code_152B_player_0', 'ability_code_152B_player_1', 'ability_code_152E_player_1', 'ability_code_152F_diff', 'ability_code_152F_player_0', 'ability_code_152F_player_1', 'ability_code_1540_first_time_diff', 'ability_code_1540_player_0', 'ability_code_1541_first_time_diff', 'ability_code_1542_first_time_player_0', 'ability_code_1542_first_time_player_1', 'ability_code_1542_player_0', 'ability_code_1543_diff', 'ability_code_1543_first_time_diff', 'ability_code_1543_first_time_player_0', 'ability_code_1543_player_0', 'ability_code_1543_player_1', 'ability_code_1544_first_time_diff', 'ability_code_1544_player_0', 'ability_code_1544_player_1', 'ability_code_1545_player_0', 'ability_code_1546_first_time_diff', 'ability_code_1546_player_0', 'ability_code_1547_first_time_diff', 'ability_code_1547_first_time_player_0', 'ability_code_1547_first_time_player_1', 'ability_code_1547_player_1', 'ability_code_1549_diff', 'ability_code_1549_first_time_diff', 'ability_code_1549_player_0', 'ability_code_1549_player_1', 'ability_code_154A_player_0', 'ability_code_154A_player_1', 'ability_code_154B_player_0', 'ability_code_154C_player_0', 'ability_code_154C_player_1', 'ability_code_154D_diff', 'ability_code_154D_first_time_diff', 'ability_code_154D_first_time_player_1', 'ability_code_154D_player_0', 'ability_code_154D_player_1', 'ability_code_154E_diff', 'ability_code_154E_first_time_diff', 'ability_code_154E_player_0', 'ability_code_154E_player_1', 'ability_code_1562_diff', 'ability_code_1562_player_0', 'ability_code_1562_player_1', 'ability_code_1563_diff', 'ability_code_1563_player_0', 'ability_code_1563_player_1', 'ability_code_1580_first_time_diff', 'ability_code_1580_first_time_player_0', 'ability_code_1580_player_0', 'ability_code_1580_player_1', 'ability_code_1581_diff', 'ability_code_1581_first_time_diff', 'ability_code_1581_first_time_player_0', 'ability_code_1581_player_0', 'ability_code_1581_player_1', 'ability_code_1583_diff', 'ability_code_1583_player_1', 'ability_code_1584_diff', 'ability_code_1584_player_0', 'ability_code_1584_player_1', 'ability_code_1585_diff', 'ability_code_1585_player_0', 'ability_code_1585_player_1', 'ability_code_1586_first_time_player_0', 'ability_code_1586_player_0', 'ability_code_15A0_player_0', 'ability_code_15A0_player_1', 'ability_code_15A2_player_0', 'ability_code_15A4_player_0', 'ability_code_15A4_player_1', 'ability_code_15A8_first_time_diff', 'ability_code_15A8_first_time_player_0', 'ability_code_15A8_player_0', 'ability_code_15A8_player_1', 'ability_code_15A9_diff', 'ability_code_15A9_player_0', 'ability_code_15A9_player_1', 'ability_code_15C0_diff', 'ability_code_15C0_player_0', 'ability_code_15C0_player_1', 'ability_code_15C1_diff', 'ability_code_15C1_first_time_diff', 'ability_code_15C1_first_time_player_0', 'ability_code_15C1_first_time_player_1', 'ability_code_15C1_player_0', 'ability_code_15C1_player_1', 'ability_code_15C2_player_0', 'ability_code_15C2_player_1', 'ability_code_15C3_first_time_diff', 'ability_code_15C3_player_0', 'ability_code_15D2_diff', 'ability_code_15D2_player_0', 'ability_code_15D2_player_1', 'ability_code_15E0_diff', 'ability_code_15E0_player_0', 'ability_code_15E0_player_0_ability_code_1020_player_0_diff', 'ability_code_15E0_player_0_ability_code_102A_player_0_diff', 'ability_code_15E0_player_0_ability_code_1261_player_0_diff', 'ability_code_15E0_player_0_ability_code_16E2_player_0_diff', 'ability_code_15E0_player_0_ability_code_1C60_player_0_diff', 'ability_code_15E0_player_0_ability_code_1E60_player_0_diff', 'ability_code_15E0_player_1', 'ability_code_15E0_player_1_ability_code_1020_player_1_diff_ability_code_15E0_player_0_ability_code_1020_player_0_diff_diff', 'ability_code_15E0_player_1_ability_code_102A_player_1_diff', 'ability_code_15E0_player_1_ability_code_1261_player_1_diff', 'ability_code_15E0_player_1_ability_code_16E2_player_1_diff', 'ability_code_15E0_player_1_ability_code_1C60_player_1_diff', 'ability_code_15E0_player_1_ability_code_1E60_player_1_diff', 'ability_code_1600_diff', 'ability_code_1600_player_1', 'ability_code_1680_diff', 'ability_code_1680_player_0', 'ability_code_1680_player_1', 'ability_code_1681_diff', 'ability_code_1681_player_0', 'ability_code_1681_player_1', 'ability_code_1683_player_0', 'ability_code_1684_diff', 'ability_code_1684_player_0', 'ability_code_1684_player_1', 'ability_code_1686_player_0', 'ability_code_1686_player_1', 'ability_code_1687_diff', 'ability_code_1687_player_0', 'ability_code_1687_player_1', 'ability_code_16A1_diff', 'ability_code_16A1_player_0', 'ability_code_16A1_player_1', 'ability_code_16A2_diff', 'ability_code_16A2_player_0', 'ability_code_16A2_player_1', 'ability_code_16A5_player_0', 'ability_code_16A5_player_1', 'ability_code_16C4_diff', 'ability_code_16C4_player_0', 'ability_code_16C4_player_1', 'ability_code_16E0_first_time_diff', 'ability_code_16E0_player_0', 'ability_code_16E0_player_1', 'ability_code_16E2_diff', 'ability_code_16E2_first_time_player_1', 'ability_code_16E2_player_0', 'ability_code_16E2_player_0_ability_code_102A_player_0_diff', 'ability_code_16E2_player_0_ability_code_1261_player_0_diff', 'ability_code_16E2_player_1', 'ability_code_16E2_player_1_ability_code_102A_player_1_diff', 'ability_code_16E2_player_1_ability_code_102A_player_1_diff_ability_code_16E2_player_0_ability_code_102A_player_0_diff_diff', 'ability_code_16E2_player_1_ability_code_1261_player_1_diff', 'ability_code_16E2_player_1_ability_code_1261_player_1_diff_ability_code_16E2_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_16E2_player_1_ability_code_480_player_1_diff', 'ability_code_16E3_player_0', 'ability_code_16E3_player_1', 'ability_code_16E4_diff', 'ability_code_16E4_first_time_diff', 'ability_code_16E4_first_time_player_0', 'ability_code_16E4_first_time_player_1', 'ability_code_16E4_player_0', 'ability_code_16E4_player_1', 'ability_code_16E5_player_1', 'ability_code_16E6_diff', 'ability_code_16E6_player_0', 'ability_code_16E6_player_1', 'ability_code_16E7_diff', 'ability_code_16E7_player_0', 'ability_code_16E7_player_1', 'ability_code_16E8_player_0', 'ability_code_16E8_player_1', 'ability_code_16E9_diff', 'ability_code_16E9_player_0', 'ability_code_16E9_player_1', 'ability_code_16EA_diff', 'ability_code_16EA_first_time_diff', 'ability_code_16EA_first_time_player_0', 'ability_code_16EA_player_0', 'ability_code_16EA_player_1', 'ability_code_16ED_player_0', 'ability_code_16ED_player_1', 'ability_code_16EE_first_time_diff', 'ability_code_16EE_first_time_player_0', 'ability_code_16EE_player_0', 'ability_code_16EF_diff', 'ability_code_16EF_first_time_diff', 'ability_code_16EF_first_time_player_0', 'ability_code_16EF_player_1', 'ability_code_1700_diff', 'ability_code_1700_player_0', 'ability_code_1700_player_1', 'ability_code_1701_diff', 'ability_code_1701_player_0', 'ability_code_1701_player_1', 'ability_code_1720_diff', 'ability_code_1720_player_0', 'ability_code_1721_player_0', 'ability_code_1723_diff', 'ability_code_1723_first_time_diff', 'ability_code_1723_first_time_player_0', 'ability_code_1723_player_0', 'ability_code_1723_player_1', 'ability_code_1724_diff', 'ability_code_1724_player_0', 'ability_code_1724_player_1', 'ability_code_1726_player_0', 'ability_code_1726_player_1', 'ability_code_1727_diff', 'ability_code_1727_player_0', 'ability_code_1727_player_1', 'ability_code_1740_player_0', 'ability_code_1740_player_1', 'ability_code_1741_diff', 'ability_code_1741_player_0', 'ability_code_1741_player_1', 'ability_code_1760_player_0', 'ability_code_1760_player_1', 'ability_code_17A1_diff', 'ability_code_17A1_player_0', 'ability_code_17A1_player_1', 'ability_code_17A3_player_1', 'ability_code_17C1_diff', 'ability_code_17C1_first_time_diff', 'ability_code_17C1_player_0', 'ability_code_17C1_player_1', 'ability_code_17E1_diff', 'ability_code_17E1_player_1', 'ability_code_1800_diff', 'ability_code_1800_player_0', 'ability_code_1800_player_1', 'ability_code_1803_diff', 'ability_code_1803_player_0', 'ability_code_1803_player_1', 'ability_code_1820_diff', 'ability_code_1820_first_time_diff', 'ability_code_1820_player_0', 'ability_code_1820_player_0_ability_code_1021_player_0_diff', 'ability_code_1820_player_0_ability_code_1022_player_0_diff', 'ability_code_1820_player_0_ability_code_1023_player_0_diff', 'ability_code_1820_player_0_ability_code_102A_player_0_diff', 'ability_code_1820_player_0_ability_code_1261_player_0_diff', 'ability_code_1820_player_0_ability_code_13A0_player_0_diff', 'ability_code_1820_player_0_ability_code_13E0_player_0_diff', 'ability_code_1820_player_0_ability_code_15E0_player_0_diff', 'ability_code_1820_player_0_ability_code_16E2_player_0_diff', 'ability_code_1820_player_0_ability_code_1822_player_0_diff', 'ability_code_1820_player_0_ability_code_1C60_player_0_diff', 'ability_code_1820_player_0_ability_code_480_player_0_diff', 'ability_code_1820_player_0_ability_code_B40_player_0_diff', 'ability_code_1820_player_0_ability_code_DE0_player_0_diff', 'ability_code_1820_player_1_ability_code_1020_player_1_diff', 'ability_code_1820_player_1_ability_code_1021_player_1_diff', 'ability_code_1820_player_1_ability_code_1021_player_1_diff_ability_code_1820_player_0_ability_code_1021_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_1022_player_1_diff', 'ability_code_1820_player_1_ability_code_1022_player_1_diff_ability_code_1820_player_0_ability_code_1022_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_1023_player_1_diff', 'ability_code_1820_player_1_ability_code_1023_player_1_diff_ability_code_1820_player_0_ability_code_1023_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_102A_player_1_diff', 'ability_code_1820_player_1_ability_code_1261_player_1_diff', 'ability_code_1820_player_1_ability_code_1261_player_1_diff_ability_code_1820_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_13A0_player_1_diff', 'ability_code_1820_player_1_ability_code_13E0_player_1_diff', 'ability_code_1820_player_1_ability_code_13E0_player_1_diff_ability_code_1820_player_0_ability_code_13E0_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_15E0_player_1_diff', 'ability_code_1820_player_1_ability_code_15E0_player_1_diff_ability_code_1820_player_0_ability_code_15E0_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_16E2_player_1_diff', 'ability_code_1820_player_1_ability_code_16E2_player_1_diff_ability_code_1820_player_0_ability_code_16E2_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_1821_player_1_diff', 'ability_code_1820_player_1_ability_code_1821_player_1_diff_ability_code_1820_player_0_ability_code_1821_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_1822_player_1_diff', 'ability_code_1820_player_1_ability_code_1C60_player_1_diff', 'ability_code_1820_player_1_ability_code_1C60_player_1_diff_ability_code_1820_player_0_ability_code_1C60_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_1E60_player_1_diff', 'ability_code_1820_player_1_ability_code_1E60_player_1_diff_ability_code_1820_player_0_ability_code_1E60_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_480_player_1_diff', 'ability_code_1820_player_1_ability_code_480_player_1_diff_ability_code_1820_player_0_ability_code_480_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_B40_player_1_diff_ability_code_1820_player_0_ability_code_B40_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_DE0_player_1_diff', 'ability_code_1821_first_time_diff', 'ability_code_1821_first_time_player_1', 'ability_code_1821_player_0', 'ability_code_1821_player_0_ability_code_1020_player_0_diff', 'ability_code_1821_player_0_ability_code_102A_player_0_diff', 'ability_code_1821_player_0_ability_code_1261_player_0_diff', 'ability_code_1821_player_0_ability_code_15E0_player_0_diff', 'ability_code_1821_player_0_ability_code_1C60_player_0_diff', 'ability_code_1821_player_0_ability_code_1E60_player_0_diff', 'ability_code_1821_player_1', 'ability_code_1821_player_1_ability_code_1020_player_1_diff', 'ability_code_1821_player_1_ability_code_102A_player_1_diff', 'ability_code_1821_player_1_ability_code_1261_player_1_diff', 'ability_code_1821_player_1_ability_code_1261_player_1_diff_ability_code_1821_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_1821_player_1_ability_code_15E0_player_1_diff', 'ability_code_1821_player_1_ability_code_1C60_player_1_diff', 'ability_code_1821_player_1_ability_code_1C60_player_1_diff_ability_code_1821_player_0_ability_code_1C60_player_0_diff_diff', 'ability_code_1821_player_1_ability_code_1E60_player_1_diff', 'ability_code_1821_player_1_ability_code_1E60_player_1_diff_ability_code_1821_player_0_ability_code_1E60_player_0_diff_diff', 'ability_code_1822_diff', 'ability_code_1822_first_time_player_0', 'ability_code_1822_first_time_player_1', 'ability_code_1822_last_first_player_0', 'ability_code_1822_player_0', 'ability_code_1822_player_0_ability_code_1020_player_0_diff', 'ability_code_1822_player_0_ability_code_1261_player_0_diff', 'ability_code_1822_player_0_ability_code_13A0_player_0_diff', 'ability_code_1822_player_0_ability_code_15E0_player_0_diff', 'ability_code_1822_player_0_ability_code_16E2_player_0_diff', 'ability_code_1822_player_0_ability_code_1C60_player_0_diff', 'ability_code_1822_player_0_ability_code_1E60_player_0_diff', 'ability_code_1822_player_1', 'ability_code_1822_player_1_ability_code_1020_player_1_diff', 'ability_code_1822_player_1_ability_code_1020_player_1_diff_ability_code_1822_player_0_ability_code_1020_player_0_diff_diff', 'ability_code_1822_player_1_ability_code_102A_player_1_diff', 'ability_code_1822_player_1_ability_code_102A_player_1_diff_ability_code_1822_player_0_ability_code_102A_player_0_diff_diff', 'ability_code_1822_player_1_ability_code_1261_player_1_diff', 'ability_code_1822_player_1_ability_code_13A0_player_1_diff', 'ability_code_1822_player_1_ability_code_13A0_player_1_diff_ability_code_1822_player_0_ability_code_13A0_player_0_diff_diff', 'ability_code_1822_player_1_ability_code_15E0_player_1_diff', 'ability_code_1822_player_1_ability_code_16E2_player_1_diff', 'ability_code_1822_player_1_ability_code_1821_player_1_diff', 'ability_code_1822_player_1_ability_code_1821_player_1_diff_ability_code_1822_player_0_ability_code_1821_player_0_diff_diff', 'ability_code_1822_player_1_ability_code_1C60_player_1_diff', 'ability_code_1822_player_1_ability_code_1C60_player_1_diff_ability_code_1822_player_0_ability_code_1C60_player_0_diff_diff', 'ability_code_1822_player_1_ability_code_1E60_player_1_diff_ability_code_1822_player_0_ability_code_1E60_player_0_diff_diff', 'ability_code_1822_player_1_ability_code_480_player_1_diff_ability_code_1822_player_0_ability_code_480_player_0_diff_diff', 'ability_code_1823_diff', 'ability_code_1823_player_0', 'ability_code_1823_player_1', 'ability_code_1824_diff', 'ability_code_1824_player_0', 'ability_code_1824_player_1', 'ability_code_1829_first_time_diff', 'ability_code_182A_diff', 'ability_code_182A_player_0', 'ability_code_182A_player_1', 'ability_code_182B_diff', 'ability_code_182B_player_0', 'ability_code_182B_player_1', 'ability_code_182E_diff', 'ability_code_182E_player_0', 'ability_code_182E_player_1', 'ability_code_1860_diff', 'ability_code_1860_player_0', 'ability_code_1860_player_1', 'ability_code_1880_diff', 'ability_code_1880_player_0', 'ability_code_1880_player_1', 'ability_code_18A0_diff', 'ability_code_18A0_player_0', 'ability_code_18A0_player_1', 'ability_code_18C0_diff', 'ability_code_18C0_player_0', 'ability_code_18C0_player_1', 'ability_code_1920_diff', 'ability_code_1920_player_0', 'ability_code_1920_player_1', 'ability_code_1940_diff', 'ability_code_1940_player_0', 'ability_code_1940_player_1', 'ability_code_1960_diff', 'ability_code_1960_player_0', 'ability_code_1960_player_1', 'ability_code_1980_diff', 'ability_code_1980_player_0', 'ability_code_1980_player_1', 'ability_code_1A62_diff', 'ability_code_1A62_player_0', 'ability_code_1A62_player_1', 'ability_code_1A63_diff', 'ability_code_1A63_player_0', 'ability_code_1A63_player_1', 'ability_code_1AC0_diff', 'ability_code_1AC0_player_1', 'ability_code_1AC1_diff', 'ability_code_1AC1_first_time_player_1', 'ability_code_1AC1_player_0', 'ability_code_1AC1_player_1', 'ability_code_1AC3_diff', 'ability_code_1AC3_player_0', 'ability_code_1AC3_player_1', 'ability_code_1AC4_diff', 'ability_code_1AC4_player_0', 'ability_code_1AC4_player_1', 'ability_code_1AC5_diff', 'ability_code_1AC5_player_0', 'ability_code_1AC5_player_1', 'ability_code_1AC6_diff', 'ability_code_1AC6_first_time_diff', 'ability_code_1AC6_first_time_player_0', 'ability_code_1AC6_player_0', 'ability_code_1AC6_player_1', 'ability_code_1AE0_diff', 'ability_code_1AE0_player_0', 'ability_code_1AE0_player_1', 'ability_code_1B00_diff', 'ability_code_1B00_player_0', 'ability_code_1B00_player_1', 'ability_code_1B21_diff', 'ability_code_1B21_player_0', 'ability_code_1B21_player_1', 'ability_code_1B40_diff', 'ability_code_1B40_player_0', 'ability_code_1B40_player_1', 'ability_code_1B60_diff', 'ability_code_1B60_player_1', 'ability_code_1BA0_player_0', 'ability_code_1BA0_player_1', 'ability_code_1BA1_diff', 'ability_code_1BA1_player_0', 'ability_code_1BA1_player_1', 'ability_code_1BC0_diff', 'ability_code_1BC0_player_0', 'ability_code_1BC0_player_1', 'ability_code_1BC1_diff', 'ability_code_1BC1_player_0', 'ability_code_1BC1_player_1', 'ability_code_1BE2_diff', 'ability_code_1BE2_player_0', 'ability_code_1BE2_player_1', 'ability_code_1C00_diff', 'ability_code_1C00_player_0', 'ability_code_1C00_player_1', 'ability_code_1C60_diff', 'ability_code_1C60_first_time_diff', 'ability_code_1C60_first_time_player_1', 'ability_code_1C60_last_first_diff', 'ability_code_1C60_last_first_player_1', 'ability_code_1C60_player_0', 'ability_code_1C60_player_0_ability_code_102A_player_0_diff', 'ability_code_1C60_player_0_ability_code_16E2_player_0_diff', 'ability_code_1C60_player_0_ability_code_1E60_player_0_diff', 'ability_code_1C60_player_0_ability_code_480_player_0_diff', 'ability_code_1C60_player_1', 'ability_code_1C60_player_1_ability_code_1020_player_1_diff', 'ability_code_1C60_player_1_ability_code_102A_player_1_diff', 'ability_code_1C60_player_1_ability_code_1261_player_1_diff', 'ability_code_1C60_player_1_ability_code_16E2_player_1_diff', 'ability_code_1C60_player_1_ability_code_16E2_player_1_diff_ability_code_1C60_player_0_ability_code_16E2_player_0_diff_diff', 'ability_code_1C60_player_1_ability_code_1E60_player_1_diff', 'ability_code_1C60_player_1_ability_code_1E60_player_1_diff_ability_code_1C60_player_0_ability_code_1E60_player_0_diff_diff', 'ability_code_1C60_player_1_ability_code_480_player_1_diff', 'ability_code_1C61_diff', 'ability_code_1C61_player_0', 'ability_code_1C61_player_1', 'ability_code_1C80_diff', 'ability_code_1C80_first_time_diff', 'ability_code_1C80_first_time_player_0', 'ability_code_1C80_player_0', 'ability_code_1C80_player_1', 'ability_code_1CA0_diff', 'ability_code_1CA0_player_0', 'ability_code_1CA0_player_1', 'ability_code_1CC0_diff', 'ability_code_1CC0_player_0', 'ability_code_1CC0_player_1', 'ability_code_1CE0_diff', 'ability_code_1CE0_player_0', 'ability_code_1CE0_player_1', 'ability_code_1D00_diff', 'ability_code_1D00_player_0', 'ability_code_1D00_player_1', 'ability_code_1D20_player_0', 'ability_code_1D20_player_1', 'ability_code_1D40_diff', 'ability_code_1D40_player_0', 'ability_code_1D40_player_1', 'ability_code_1D60_diff', 'ability_code_1D60_player_0', 'ability_code_1D60_player_1', 'ability_code_1D80_player_0', 'ability_code_1D81_diff', 'ability_code_1D81_player_0', 'ability_code_1D81_player_1', 'ability_code_1D83_diff', 'ability_code_1D83_player_0', 'ability_code_1D83_player_1', 'ability_code_1D86_diff', 'ability_code_1D86_first_time_player_0', 'ability_code_1D86_player_0', 'ability_code_1D86_player_1', 'ability_code_1DA0_player_0', 'ability_code_1DA1_diff', 'ability_code_1DA1_player_0', 'ability_code_1DA1_player_1', 'ability_code_1DA2_diff', 'ability_code_1DA2_player_0', 'ability_code_1DA2_player_1', 'ability_code_1DC0_diff', 'ability_code_1DC0_player_0', 'ability_code_1DC0_player_1', 'ability_code_1E20_diff', 'ability_code_1E20_player_0', 'ability_code_1E60_first_time_diff', 'ability_code_1E60_player_0', 'ability_code_1E60_player_0_ability_code_102A_player_0_diff', 'ability_code_1E60_player_0_ability_code_1261_player_0_diff', 'ability_code_1E60_player_0_ability_code_16E2_player_0_diff', 'ability_code_1E60_player_0_ability_code_480_player_0_diff', 'ability_code_1E60_player_1', 'ability_code_1E60_player_1_ability_code_102A_player_1_diff', 'ability_code_1E60_player_1_ability_code_1261_player_1_diff', 'ability_code_1E60_player_1_ability_code_1261_player_1_diff_ability_code_1E60_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_1E60_player_1_ability_code_16E2_player_1_diff', 'ability_code_1E60_player_1_ability_code_480_player_1_diff_ability_code_1E60_player_0_ability_code_480_player_0_diff_diff', 'ability_code_1EA0_diff', 'ability_code_1EA0_player_0', 'ability_code_1EA0_player_1', 'ability_code_1FC0_diff', 'ability_code_1FC0_player_0', 'ability_code_1FC0_player_1', 'ability_code_1FE0_diff', 'ability_code_1FE0_player_0', 'ability_code_1FE0_player_1', 'ability_code_2040_diff', 'ability_code_2040_player_0', 'ability_code_2040_player_1', 'ability_code_2060_diff', 'ability_code_2060_player_0', 'ability_code_2060_player_1', 'ability_code_2061_diff', 'ability_code_2061_player_0', 'ability_code_2061_player_1', 'ability_code_2080_diff', 'ability_code_2080_first_time_diff', 'ability_code_2080_player_1', 'ability_code_20A0_player_1', 'ability_code_20C0_diff', 'ability_code_20C0_player_0', 'ability_code_20C0_player_1', 'ability_code_20E0_diff', 'ability_code_20E0_player_0', 'ability_code_20E0_player_1', 'ability_code_20E1_diff', 'ability_code_20E1_player_0', 'ability_code_20E1_player_1', 'ability_code_2100_diff', 'ability_code_2100_player_0', 'ability_code_2100_player_1', 'ability_code_2101_diff', 'ability_code_2101_player_0', 'ability_code_2101_player_1', 'ability_code_2120_diff', 'ability_code_2120_first_time_diff', 'ability_code_2120_first_time_player_0', 'ability_code_2120_player_0', 'ability_code_2120_player_1', 'ability_code_213E_diff', 'ability_code_213E_player_0', 'ability_code_213E_player_1', 'ability_code_2140_diff', 'ability_code_2140_player_0', 'ability_code_2140_player_1', 'ability_code_2160_diff', 'ability_code_2160_player_0', 'ability_code_2160_player_1', 'ability_code_2180_diff', 'ability_code_2180_player_0', 'ability_code_2180_player_1', 'ability_code_22A0_diff', 'ability_code_22A0_first_time_diff', 'ability_code_22A0_first_time_player_0', 'ability_code_22A0_first_time_player_1', 'ability_code_22A0_player_0', 'ability_code_22A0_player_1', 'ability_code_22A1_player_1', 'ability_code_26C0_diff', 'ability_code_26C0_player_0', 'ability_code_26C0_player_1', 'ability_code_2720_diff', 'ability_code_2720_first_time_player_0', 'ability_code_2720_player_0', 'ability_code_2720_player_1', 'ability_code_2B00_player_0', 'ability_code_2B00_player_1', 'ability_code_2B40_diff', 'ability_code_2B40_player_0', 'ability_code_2B40_player_1', 'ability_code_2C80_diff', 'ability_code_2C80_player_0', 'ability_code_2C80_player_1', 'ability_code_2DE0_diff', 'ability_code_2DE0_player_0', 'ability_code_2DE0_player_1', 'ability_code_2F60_diff', 'ability_code_2F60_player_0', 'ability_code_2F60_player_1', 'ability_code_3040_diff', 'ability_code_3040_first_time_diff', 'ability_code_3040_first_time_player_1', 'ability_code_3040_player_0', 'ability_code_3040_player_1', 'ability_code_3060_diff', 'ability_code_3060_player_0', 'ability_code_3060_player_1', 'ability_code_3120_diff', 'ability_code_3120_player_0', 'ability_code_3120_player_1', 'ability_code_3140_diff', 'ability_code_3140_player_0', 'ability_code_3140_player_1', 'ability_code_3160_diff', 'ability_code_3160_player_0', 'ability_code_3160_player_1', 'ability_code_31A0_first_time_diff', 'ability_code_31A0_first_time_player_0', 'ability_code_31A0_first_time_player_1', 'ability_code_31A0_player_0', 'ability_code_31A0_player_1', 'ability_code_3380_diff', 'ability_code_3380_player_0', 'ability_code_3380_player_1', 'ability_code_3480_diff', 'ability_code_3480_first_time_diff', 'ability_code_3480_first_time_player_0', 'ability_code_3480_player_0', 'ability_code_3480_player_1', 'ability_code_39A0_diff', 'ability_code_39A0_player_0', 'ability_code_39A0_player_1', 'ability_code_4020_diff', 'ability_code_4020_player_0', 'ability_code_4020_player_1', 'ability_code_4021_diff', 'ability_code_4021_player_0', 'ability_code_4021_player_1', 'ability_code_4040_diff', 'ability_code_4040_player_0', 'ability_code_4040_player_1', 'ability_code_40A0_diff', 'ability_code_40A0_player_0', 'ability_code_40A0_player_1', 'ability_code_4120_diff', 'ability_code_4120_player_0', 'ability_code_4120_player_1', 'ability_code_4160_player_0', 'ability_code_4160_player_1', 'ability_code_41A0_diff', 'ability_code_41A0_player_0', 'ability_code_41A0_player_1', 'ability_code_41E0_diff', 'ability_code_41E0_player_0', 'ability_code_41E0_player_1', 'ability_code_4220_diff', 'ability_code_4220_player_0', 'ability_code_4220_player_1', 'ability_code_42A1_diff', 'ability_code_42A1_player_0', 'ability_code_42A1_player_1', 'ability_code_42C1_diff', 'ability_code_42C1_player_0', 'ability_code_42C1_player_1', 'ability_code_43E0_diff', 'ability_code_43E0_player_0', 'ability_code_43E0_player_1', 'ability_code_480_diff', 'ability_code_480_first_time_diff', 'ability_code_480_first_time_player_0', 'ability_code_480_first_time_player_1', 'ability_code_480_player_0', 'ability_code_480_player_1', 'ability_code_483_diff', 'ability_code_483_player_0', 'ability_code_483_player_1', 'ability_code_4AE0_diff', 'ability_code_4AE0_player_0', 'ability_code_4AE0_player_1', 'ability_code_4AFE_diff', 'ability_code_4AFE_player_0', 'ability_code_4AFE_player_1', 'ability_code_4B80_diff', 'ability_code_4B80_first_time_diff', 'ability_code_4B80_player_0', 'ability_code_4C0_diff', 'ability_code_4C0_player_0', 'ability_code_4C0_player_1', 'ability_code_4C1_diff', 'ability_code_4C1_first_time_player_0', 'ability_code_4C1_player_1', 'ability_code_4C2_first_time_diff', 'ability_code_4C3_diff', 'ability_code_4C3_player_0', 'ability_code_4C3_player_1', 'ability_code_4C60_player_0', 'ability_code_4C60_player_1', 'ability_code_4C80_diff', 'ability_code_4C80_player_0', 'ability_code_4C80_player_1', 'ability_code_4E40_diff', 'ability_code_4E40_first_time_diff', 'ability_code_4E40_first_time_player_1', 'ability_code_4E40_player_0', 'ability_code_4E40_player_1', 'ability_code_4EA0_diff', 'ability_code_4EA0_player_0', 'ability_code_4EA0_player_1', 'ability_code_4EC0_diff', 'ability_code_4EC0_player_0', 'ability_code_4EC0_player_1', 'ability_code_55E0_diff', 'ability_code_55E0_player_0', 'ability_code_55E0_player_1', 'ability_code_5600_diff', 'ability_code_5600_player_0', 'ability_code_5600_player_1', 'ability_code_5640_diff', 'ability_code_5640_player_0', 'ability_code_5640_player_1', 'ability_code_5641_diff', 'ability_code_5641_player_0', 'ability_code_5641_player_1', 'ability_code_56A0_diff', 'ability_code_56A0_player_0', 'ability_code_56A0_player_1', 'ability_code_5700_diff', 'ability_code_5700_player_0', 'ability_code_5700_player_1', 'ability_code_5A0_diff', 'ability_code_5A0_first_time_diff', 'ability_code_5A0_first_time_player_1', 'ability_code_5A0_last_first_player_1', 'ability_code_5A0_player_0', 'ability_code_5A0_player_0_ability_code_1020_player_0_diff', 'ability_code_5A0_player_0_ability_code_1021_player_0_diff', 'ability_code_5A0_player_0_ability_code_1022_player_0_diff', 'ability_code_5A0_player_0_ability_code_1023_player_0_diff', 'ability_code_5A0_player_0_ability_code_102A_player_0_diff', 'ability_code_5A0_player_0_ability_code_1261_player_0_diff', 'ability_code_5A0_player_0_ability_code_1360_player_0_diff', 'ability_code_5A0_player_0_ability_code_13A0_player_0_diff', 'ability_code_5A0_player_0_ability_code_13E0_player_0_diff', 'ability_code_5A0_player_0_ability_code_15E0_player_0_diff', 'ability_code_5A0_player_0_ability_code_1820_player_0_diff', 'ability_code_5A0_player_0_ability_code_1C60_player_0_diff', 'ability_code_5A0_player_0_ability_code_1E60_player_0_diff', 'ability_code_5A0_player_0_ability_code_480_player_0_diff', 'ability_code_5A0_player_0_ability_code_B40_player_0_diff', 'ability_code_5A0_player_1', 'ability_code_5A0_player_1_ability_code_1021_player_1_diff', 'ability_code_5A0_player_1_ability_code_1021_player_1_diff_ability_code_5A0_player_0_ability_code_1021_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_1022_player_1_diff', 'ability_code_5A0_player_1_ability_code_1023_player_1_diff_ability_code_5A0_player_0_ability_code_1023_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_102A_player_1_diff', 'ability_code_5A0_player_1_ability_code_1261_player_1_diff', 'ability_code_5A0_player_1_ability_code_1261_player_1_diff_ability_code_5A0_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_1360_player_1_diff', 'ability_code_5A0_player_1_ability_code_13A0_player_1_diff', 'ability_code_5A0_player_1_ability_code_13A0_player_1_diff_ability_code_5A0_player_0_ability_code_13A0_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_13E0_player_1_diff_ability_code_5A0_player_0_ability_code_13E0_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_16E2_player_1_diff', 'ability_code_5A0_player_1_ability_code_1822_player_1_diff', 'ability_code_5A0_player_1_ability_code_1822_player_1_diff_ability_code_5A0_player_0_ability_code_1822_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_1C60_player_1_diff', 'ability_code_5A0_player_1_ability_code_1C60_player_1_diff_ability_code_5A0_player_0_ability_code_1C60_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_1E60_player_1_diff', 'ability_code_5A0_player_1_ability_code_480_player_1_diff_ability_code_5A0_player_0_ability_code_480_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_B40_player_1_diff', 'ability_code_5A0_player_1_ability_code_B40_player_1_diff_ability_code_5A0_player_0_ability_code_B40_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_DE0_player_1_diff', 'ability_code_780_diff', 'ability_code_780_player_0', 'ability_code_780_player_1', 'ability_code_7A0_diff', 'ability_code_7A0_player_0', 'ability_code_7A0_player_1', 'ability_code_7C0_diff', 'ability_code_7C0_player_0', 'ability_code_7C0_player_1', 'ability_code_7E0_diff', 'ability_code_7E0_player_0', 'ability_code_7E0_player_1', 'ability_code_820_diff', 'ability_code_820_player_0', 'ability_code_820_player_1', 'ability_code_880_diff', 'ability_code_880_player_0', 'ability_code_880_player_1', 'ability_code_8A0_player_0', 'ability_code_8A0_player_1', 'ability_code_8A2_diff', 'ability_code_8A2_player_0', 'ability_code_8A2_player_1', 'ability_code_8E0_diff', 'ability_code_8E0_player_0', 'ability_code_8E0_player_1', 'ability_code_900_diff', 'ability_code_900_player_0', 'ability_code_900_player_1', 'ability_code_920_first_time_diff', 'ability_code_920_player_0', 'ability_code_960_diff', 'ability_code_960_player_1', 'ability_code_9E0_diff', 'ability_code_9E0_player_0', 'ability_code_9E0_player_1', 'ability_code_A40_diff', 'ability_code_A40_player_0', 'ability_code_A40_player_1', 'ability_code_B00_diff', 'ability_code_B00_player_0', 'ability_code_B01_diff', 'ability_code_B01_player_0', 'ability_code_B01_player_1', 'ability_code_B40_diff', 'ability_code_B40_first_time_player_1', 'ability_code_B40_last_first_diff', 'ability_code_B40_last_first_player_1', 'ability_code_B40_player_0', 'ability_code_B40_player_0_ability_code_1020_player_0_diff', 'ability_code_B40_player_0_ability_code_1023_player_0_diff', 'ability_code_B40_player_0_ability_code_1261_player_0_diff', 'ability_code_B40_player_0_ability_code_13A0_player_0_diff', 'ability_code_B40_player_0_ability_code_15E0_player_0_diff', 'ability_code_B40_player_0_ability_code_16E2_player_0_diff', 'ability_code_B40_player_0_ability_code_1821_player_0_diff', 'ability_code_B40_player_0_ability_code_1822_player_0_diff', 'ability_code_B40_player_0_ability_code_1C60_player_0_diff', 'ability_code_B40_player_0_ability_code_1E60_player_0_diff', 'ability_code_B40_player_0_ability_code_DE0_player_0_diff', 'ability_code_B40_player_1', 'ability_code_B40_player_1_ability_code_1023_player_1_diff_ability_code_B40_player_0_ability_code_1023_player_0_diff_diff', 'ability_code_B40_player_1_ability_code_102A_player_1_diff', 'ability_code_B40_player_1_ability_code_1261_player_1_diff', 'ability_code_B40_player_1_ability_code_13A0_player_1_diff', 'ability_code_B40_player_1_ability_code_15E0_player_1_diff', 'ability_code_B40_player_1_ability_code_16E2_player_1_diff', 'ability_code_B40_player_1_ability_code_16E2_player_1_diff_ability_code_B40_player_0_ability_code_16E2_player_0_diff_diff', 'ability_code_B40_player_1_ability_code_1821_player_1_diff', 'ability_code_B40_player_1_ability_code_1821_player_1_diff_ability_code_B40_player_0_ability_code_1821_player_0_diff_diff', 'ability_code_B40_player_1_ability_code_1822_player_1_diff', 'ability_code_B40_player_1_ability_code_1822_player_1_diff_ability_code_B40_player_0_ability_code_1822_player_0_diff_diff', 'ability_code_B40_player_1_ability_code_1C60_player_1_diff_ability_code_B40_player_0_ability_code_1C60_player_0_diff_diff', 'ability_code_B40_player_1_ability_code_1E60_player_1_diff', 'ability_code_B40_player_1_ability_code_1E60_player_1_diff_ability_code_B40_player_0_ability_code_1E60_player_0_diff_diff', 'ability_code_B40_player_1_ability_code_DE0_player_1_diff', 'ability_code_B60_diff', 'ability_code_B60_player_0', 'ability_code_B61_diff', 'ability_code_B61_player_0', 'ability_code_B61_player_1', 'ability_code_BE0_diff', 'ability_code_BE0_player_0', 'ability_code_CC0_player_0', 'ability_code_CC0_player_1', 'ability_code_D00_diff', 'ability_code_D00_player_0', 'ability_code_D00_player_1', 'ability_code_D20_player_0', 'ability_code_D20_player_1', 'ability_code_D40_diff', 'ability_code_D40_player_0', 'ability_code_D40_player_1', 'ability_code_D41_player_0', 'ability_code_D61_diff', 'ability_code_D61_player_1', 'ability_code_D62_diff', 'ability_code_D62_player_0', 'ability_code_D62_player_1', 'ability_code_DE0_diff', 'ability_code_DE0_player_0', 'ability_code_DE0_player_0_ability_code_1020_player_0_diff', 'ability_code_DE0_player_0_ability_code_1022_player_0_diff', 'ability_code_DE0_player_0_ability_code_1023_player_0_diff', 'ability_code_DE0_player_0_ability_code_102A_player_0_diff', 'ability_code_DE0_player_0_ability_code_1261_player_0_diff', 'ability_code_DE0_player_0_ability_code_13A0_player_0_diff', 'ability_code_DE0_player_0_ability_code_15E0_player_0_diff', 'ability_code_DE0_player_0_ability_code_16E2_player_0_diff', 'ability_code_DE0_player_0_ability_code_1C60_player_0_diff', 'ability_code_DE0_player_0_ability_code_480_player_0_diff', 'ability_code_DE0_player_1', 'ability_code_DE0_player_1_ability_code_1020_player_1_diff', 'ability_code_DE0_player_1_ability_code_1022_player_1_diff', 'ability_code_DE0_player_1_ability_code_1022_player_1_diff_ability_code_DE0_player_0_ability_code_1022_player_0_diff_diff', 'ability_code_DE0_player_1_ability_code_1023_player_1_diff', 'ability_code_DE0_player_1_ability_code_1023_player_1_diff_ability_code_DE0_player_0_ability_code_1023_player_0_diff_diff', 'ability_code_DE0_player_1_ability_code_102A_player_1_diff', 'ability_code_DE0_player_1_ability_code_1261_player_1_diff', 'ability_code_DE0_player_1_ability_code_1261_player_1_diff_ability_code_DE0_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_DE0_player_1_ability_code_13A0_player_1_diff', 'ability_code_DE0_player_1_ability_code_13A0_player_1_diff_ability_code_DE0_player_0_ability_code_13A0_player_0_diff_diff', 'ability_code_DE0_player_1_ability_code_15E0_player_1_diff', 'ability_code_DE0_player_1_ability_code_16E2_player_1_diff', 'ability_code_DE0_player_1_ability_code_1821_player_1_diff_ability_code_DE0_player_0_ability_code_1821_player_0_diff_diff', 'ability_code_DE0_player_1_ability_code_1822_player_1_diff', 'ability_code_DE0_player_1_ability_code_1822_player_1_diff_ability_code_DE0_player_0_ability_code_1822_player_0_diff_diff', 'ability_code_DE0_player_1_ability_code_1C60_player_1_diff', 'ability_code_DE0_player_1_ability_code_1C60_player_1_diff_ability_code_DE0_player_0_ability_code_1C60_player_0_diff_diff', 'ability_code_DE0_player_1_ability_code_1E60_player_1_diff_ability_code_DE0_player_0_ability_code_1E60_player_0_diff_diff', 'ability_code_DE0_player_1_ability_code_480_player_1_diff', 'ability_code_E00_diff', 'ability_code_E00_player_0', 'ability_code_E00_player_1', 'ability_code_E20_diff', 'ability_code_E20_player_0', 'ability_code_E20_player_1', 'ability_code_E80_first_time_diff', 'ability_code_E80_player_0', 'ability_code_EE0_diff', 'ability_code_EE0_player_0', 'ability_code_EE0_player_1', 'ability_code_EE1_diff', 'ability_code_EE1_player_0', 'ability_code_EE1_player_1', 'ability_code_F00_diff', 'ability_code_F00_player_0', 'ability_code_F00_player_1', 'ability_code_F01_diff', 'ability_code_F01_player_0', 'ability_code_F40_diff', 'ability_code_F40_player_1', 'ability_code_F41_diff', 'ability_code_F41_player_0', 'ability_code_F41_player_1', 'ability_code_F60_diff', 'ability_code_F60_player_0', 'ability_code_F60_player_1', 'ability_code_F61_diff', 'ability_code_F61_first_time_diff', 'ability_code_F61_player_0', 'ability_code_F80_first_time_player_1', 'ability_code_F80_player_1', 'ability_code_F81_diff', 'ability_code_F81_first_time_diff', 'ability_code_F81_player_0', 'ability_code_F81_player_1', 'ability_code_FC0_diff', 'ability_code_FC0_player_0', 'ability_code_FC0_player_1', 'ability_code_FC1_diff', 'ability_code_FC1_player_0', 'ability_code_FC1_player_1', 'ability_code_FE0_first_time_diff', 'ability_code_FE1_diff', 'ability_code_FE1_player_0', 'ability_code_FE1_player_1', 'all_unit_diff', 'all_unit_len_player_0', 'all_unit_len_player_0_BanelingCocoon_len_player_0_diff', 'all_unit_len_player_0_Factory_len_player_0_diff', 'all_unit_len_player_0_Hatchery_len_player_0_diff', 'all_unit_len_player_0_Hellion_len_player_0_diff', 'all_unit_len_player_0_Marine_len_player_0_diff', 'all_unit_len_player_0_OrbitalCommand_len_player_0_diff', 'all_unit_len_player_0_Overlord_len_player_0_diff', 'all_unit_len_player_0_Probe_len_player_0_diff', 'all_unit_len_player_0_Queen_len_player_0_diff', 'all_unit_len_player_0_SCV_len_player_0_diff', 'all_unit_len_player_0_Starport_len_player_0_diff', 'all_unit_len_player_0_SupplyDepotLowered_len_player_0_diff', 'all_unit_len_player_1_BanelingCocoon_len_player_1_diff', 'all_unit_len_player_1_BanelingCocoon_len_player_1_diff_all_unit_len_player_0_BanelingCocoon_len_player_0_diff_diff', 'all_unit_len_player_1_Barracks_len_player_1_diff', 'all_unit_len_player_1_Barracks_len_player_1_diff_all_unit_len_player_0_Barracks_len_player_0_diff_diff', 'all_unit_len_player_1_Drone_len_player_1_diff', 'all_unit_len_player_1_Egg_len_player_1_diff', 'all_unit_len_player_1_Egg_len_player_1_diff_all_unit_len_player_0_Egg_len_player_0_diff_diff', 'all_unit_len_player_1_Hellion_len_player_1_diff_all_unit_len_player_0_Hellion_len_player_0_diff_diff', 'all_unit_len_player_1_MULE_len_player_1_diff', 'all_unit_len_player_1_Marine_len_player_1_diff_all_unit_len_player_0_Marine_len_player_0_diff_diff', 'all_unit_len_player_1_OrbitalCommand_len_player_1_diff', 'all_unit_len_player_1_Overlord_len_player_1_diff', 'all_unit_len_player_1_Probe_len_player_1_diff_all_unit_len_player_0_Probe_len_player_0_diff_diff', 'all_unit_len_player_1_Queen_len_player_1_diff', 'all_unit_len_player_1_Queen_len_player_1_diff_all_unit_len_player_0_Queen_len_player_0_diff_diff', 'all_unit_len_player_1_Reaper_len_player_1_diff', 'all_unit_len_player_1_Roach_len_player_1_diff', 'all_unit_len_player_1_Roach_len_player_1_diff_all_unit_len_player_0_Roach_len_player_0_diff_diff', 'all_unit_len_player_1_SCV_len_player_1_diff', 'all_unit_len_player_1_SCV_len_player_1_diff_all_unit_len_player_0_SCV_len_player_0_diff_diff', 'all_unit_len_player_1_Starport_len_player_1_diff', 'all_unit_len_player_1_SupplyDepotLowered_len_player_1_diff', 'all_unit_len_player_1_SupplyDepotLowered_len_player_1_diff_all_unit_len_player_0_SupplyDepotLowered_len_player_0_diff_diff', 'all_unit_len_player_1_Zergling_len_player_1_diff', 'camera_pos_x_max_player_1', 'camera_pos_x_mean_player_0', 'camera_pos_x_mean_player_1', 'camera_pos_y_max_player_0', 'camera_pos_y_max_player_1', 'camera_pos_y_mean_player_0', 'camera_pos_y_mean_player_1', 'camera_pos_y_min_player_0', 'game_id_time_max_div_nunique', 'species_player_0', 'species_player_1', 'species_war_kind', 'time_count_player_diff', 'under_1_Ability_diff', 'under_1_AddToControlGroup_player_0', 'under_1_AddToControlGroup_player_1', 'under_1_Camera_player_1', 'under_1_ControlGroup_diff', 'under_1_ControlGroup_player_0', 'under_1_ControlGroup_player_1', 'under_1_GetControlGroup_player_0', 'under_1_Right Click_diff', 'under_1_Right Click_player_0', 'under_1_Selection_diff', 'under_1_SetControlGroup_diff', 'under_3_AddToControlGroup_diff', 'under_3_AddToControlGroup_player_0', 'under_3_AddToControlGroup_player_1', 'under_3_ControlGroup_diff', 'under_3_ControlGroup_player_1', 'under_3_GetControlGroup_player_1', 'under_3_Right Click_diff', 'under_3_Right Click_player_1', 'under_3_Selection_diff', 'under_3_SetControlGroup_diff', 'under_3_SetControlGroup_player_0', 'under_3_SetControlGroup_player_1', 'under_5_Ability_diff', 'under_5_Ability_player_0', 'under_5_AddToControlGroup_diff', 'under_5_Camera_diff', 'under_5_Camera_player_0', 'under_5_Camera_player_1', 'under_5_GetControlGroup_player_0', 'under_5_GetControlGroup_player_1', 'under_5_Right Click_diff', 'under_5_Right Click_player_0', 'under_5_Right Click_player_1', 'under_5_Selection_player_1', 'under_5_SetControlGroup_diff', 'under_5_SetControlGroup_player_1', 'worker_count_player_1']
bad_feature_fold5 = ['Ability_player_0_isnull', 'Ability_player_1_isnull', 'AddToControlGroup_player_0_AddToControlGroup_player_1_diff', 'AddToControlGroup_player_0_isnull', 'AddToControlGroup_player_1', 'AddToControlGroup_player_1_isnull', 'AdeptPiercingWeapon_len_player_0', 'Adept_len_player_0', 'Adept_right_click_target_count_player_0', 'Adept_right_click_target_count_player_1', 'AiurLightBridgeAbandonedNE10Out_len_player_0', 'AiurLightBridgeAbandonedNE10Out_len_player_1', 'AiurLightBridgeAbandonedNE8Out_len_player_0', 'AiurLightBridgeAbandonedNE8Out_len_player_1', 'AiurLightBridgeAbandonedNE8_len_player_0', 'AiurLightBridgeAbandonedNE8_len_player_1', 'AiurLightBridgeNE10Out_len_player_0', 'AiurLightBridgeNE10Out_len_player_1', 'AiurLightBridgeNE8_len_player_0', 'AiurLightBridgeNE8_len_player_1', 'Archon_len_player_0', 'Archon_len_player_1', 'Armory_len_player_1', 'Assimilator_len_player_0', 'Assimilator_len_player_1', 'Assimilator_right_click_target_count_player_1', 'AutoTurret_len_player_0', 'AutoTurret_len_player_1', 'BanelingBurrowed_len_player_0', 'BanelingBurrowed_len_player_1', 'BanelingCocoon_len_player_0', 'BanelingCocoon_len_player_0_Hellion_len_player_0_diff', 'BanelingCocoon_len_player_0_MULE_len_player_0_diff', 'BanelingCocoon_len_player_0_Overlord_len_player_0_diff', 'BanelingCocoon_len_player_0_Reaper_len_player_0_diff', 'BanelingCocoon_len_player_0_Roach_len_player_0_diff', 'BanelingCocoon_len_player_1', 'BanelingCocoon_len_player_1_Factory_len_player_1_diff', 'BanelingCocoon_len_player_1_Factory_len_player_1_diff_BanelingCocoon_len_player_0_Factory_len_player_0_diff_diff', 'BanelingCocoon_len_player_1_Hatchery_len_player_1_diff', 'BanelingCocoon_len_player_1_Hatchery_len_player_1_diff_BanelingCocoon_len_player_0_Hatchery_len_player_0_diff_diff', 'BanelingCocoon_len_player_1_Hellion_len_player_1_diff', 'BanelingCocoon_len_player_1_Hellion_len_player_1_diff_BanelingCocoon_len_player_0_Hellion_len_player_0_diff_diff', 'BanelingCocoon_len_player_1_MULE_len_player_1_diff', 'BanelingCocoon_len_player_1_MULE_len_player_1_diff_BanelingCocoon_len_player_0_MULE_len_player_0_diff_diff', 'BanelingCocoon_len_player_1_Overlord_len_player_1_diff', 'BanelingCocoon_len_player_1_Overlord_len_player_1_diff_BanelingCocoon_len_player_0_Overlord_len_player_0_diff_diff', 'BanelingCocoon_len_player_1_Queen_len_player_1_diff', 'BanelingCocoon_len_player_1_Reaper_len_player_1_diff', 'BanelingCocoon_len_player_1_Reaper_len_player_1_diff_BanelingCocoon_len_player_0_Reaper_len_player_0_diff_diff', 'BanelingCocoon_len_player_1_Roach_len_player_1_diff', 'BanelingCocoon_len_player_1_Starport_len_player_1_diff', 'BanelingCocoon_right_click_target_count_player_0', 'BanelingCocoon_right_click_target_count_player_1', 'BanelingNest_len_player_0', 'BanelingNest_len_player_1', 'Baneling_len_player_0', 'Baneling_len_player_1', 'Baneling_right_click_target_count_diff', 'Baneling_right_click_target_count_player_0', 'Baneling_right_click_target_count_player_1', 'Banshee_len_player_0', 'Banshee_len_player_1', 'BarracksReactor_len_player_0', 'BarracksReactor_len_player_1', 'BarracksTechLab_len_player_1', 'Barracks_len_player_0', 'Barracks_len_player_0_BanelingCocoon_len_player_0_diff', 'Barracks_len_player_0_Factory_len_player_0_diff', 'Barracks_len_player_0_Hatchery_len_player_0_diff', 'Barracks_len_player_0_Overlord_len_player_0_diff', 'Barracks_len_player_0_Queen_len_player_0_diff', 'Barracks_len_player_0_Reaper_len_player_0_diff', 'Barracks_len_player_0_Roach_len_player_0_diff', 'Barracks_len_player_0_Starport_len_player_0_diff', 'Barracks_len_player_1_BanelingCocoon_len_player_1_diff', 'Barracks_len_player_1_Hatchery_len_player_1_diff', 'Barracks_len_player_1_Hatchery_len_player_1_diff_Barracks_len_player_0_Hatchery_len_player_0_diff_diff', 'Barracks_len_player_1_Hellion_len_player_1_diff', 'Barracks_len_player_1_Hellion_len_player_1_diff_Barracks_len_player_0_Hellion_len_player_0_diff_diff', 'Barracks_len_player_1_MULE_len_player_1_diff', 'Barracks_len_player_1_Overlord_len_player_1_diff', 'Barracks_len_player_1_Overlord_len_player_1_diff_Barracks_len_player_0_Overlord_len_player_0_diff_diff', 'Barracks_len_player_1_Queen_len_player_1_diff', 'Barracks_len_player_1_Reaper_len_player_1_diff', 'Barracks_len_player_1_Roach_len_player_1_diff', 'Barracks_len_player_1_Roach_len_player_1_diff_Barracks_len_player_0_Roach_len_player_0_diff_diff', 'Barracks_len_player_1_Starport_len_player_1_diff_Barracks_len_player_0_Starport_len_player_0_diff_diff', 'Barracks_len_player_1_SupplyDepotLowered_len_player_1_diff', 'Barracks_right_click_target_count_diff', 'Battlecruiser_len_player_0', 'Battlecruiser_len_player_1', 'Beacon_TerranSmall_len_player_0', 'Beacon_TerranSmall_len_player_1', 'Beacon_Terran_len_player_0', 'Beacon_Terran_len_player_1', 'BroodLordAWeapon_len_player_0', 'BroodLordAWeapon_len_player_1', 'BroodLordWeapon_len_player_0', 'BroodLordWeapon_len_player_1', 'BroodLord_len_player_0', 'BroodLord_len_player_1', 'Broodling_len_player_0', 'Broodling_len_player_1', 'Bunker_len_player_0', 'Bunker_len_player_1', 'Bunker_right_click_target_count_diff', 'BypassArmorDrone_len_player_0', 'BypassArmorDrone_len_player_1', 'Camera_player_0_Camera_player_1_diff', 'Camera_player_0_isnull', 'Camera_player_1_isnull', 'Carrier_len_player_0', 'Carrier_len_player_1', 'ChangelingMarineShield_len_player_1', 'ChangelingMarine_len_player_0', 'ChangelingMarine_len_player_1', 'ChangelingZealot_len_player_0', 'ChangelingZealot_len_player_1', 'ChangelingZerglingWings_len_player_0', 'ChangelingZerglingWings_len_player_1', 'ChangelingZergling_len_player_0', 'ChangelingZergling_len_player_1', 'Changeling_len_player_0', 'Changeling_len_player_1', 'Colossus_len_player_0', 'Colossus_len_player_1', 'CommandCenterFlying_len_player_1', 'CommandCenter_len_player_1', 'ControlGroup_player_0_ControlGroup_player_1_diff', 'ControlGroup_player_0_isnull', 'ControlGroup_player_1_isnull', 'Corruptor_len_player_0', 'Corruptor_len_player_1', 'CreepOnlyBlocker4x4_len_player_0', 'CreepOnlyBlocker4x4_len_player_1', 'CreepOnlyBlocker4x4_right_click_target_count_diff', 'CreepOnlyBlocker4x4_right_click_target_count_player_0', 'CreepOnlyBlocker4x4_right_click_target_count_player_1', 'CreepTumorBurrowed_len_player_1', 'CreepTumorBurrowed_right_click_target_count_player_0', 'CreepTumorBurrowed_right_click_target_count_player_1', 'CreepTumorQueen_len_player_0', 'CreepTumorQueen_len_player_1', 'CreepTumor_len_player_0', 'CreepTumor_len_player_1', 'CyberneticsCore_len_player_0', 'CyberneticsCore_len_player_1', 'CyberneticsCore_right_click_target_count_diff', 'CyberneticsCore_right_click_target_count_player_0', 'CyberneticsCore_right_click_target_count_player_1', 'CycloneMissileLargeAir_len_player_0', 'CycloneMissileLargeAir_len_player_1', 'Cyclone_len_player_0', 'Cyclone_len_player_1', 'DarkShrine_len_player_0', 'DarkShrine_len_player_1', 'DarkTemplar_len_player_0', 'DarkTemplar_len_player_1', 'DestructibleIce2x4Horizontal_len_player_0', 'DestructibleIce2x4Horizontal_len_player_1', 'DestructibleIce2x4Vertical_len_player_0', 'DestructibleIce2x4Vertical_len_player_1', 'DestructibleIce2x4Vertical_right_click_target_count_diff', 'DestructibleIce2x4Vertical_right_click_target_count_player_0', 'DestructibleIce2x4Vertical_right_click_target_count_player_1', 'DestructibleIce2x6Horizontal_len_player_0', 'DestructibleIce2x6Horizontal_len_player_1', 'DestructibleIce2x6Vertical_len_player_0', 'DestructibleIce2x6Vertical_len_player_1', 'DestructibleSignsConstruction_len_player_0', 'DestructibleSignsConstruction_len_player_1', 'DestructibleSignsConstruction_right_click_target_count_diff', 'DestructibleSignsConstruction_right_click_target_count_player_1', 'DestructibleSignsFunny_len_player_0', 'DestructibleSignsIcons_len_player_0', 'DestructibleSignsIcons_len_player_1', 'DestructibleSignsIcons_right_click_target_count_diff', 'DestructibleSignsIcons_right_click_target_count_player_0', 'DestructibleSignsIcons_right_click_target_count_player_1', 'Disruptor_len_player_0', 'Disruptor_len_player_1', 'DroneBurrowed_len_player_0', 'Drone_len_player_0', 'Drone_len_player_0_Barracks_len_player_0_diff', 'Drone_len_player_0_Hellion_len_player_0_diff', 'Drone_len_player_0_MULE_len_player_0_diff', 'Drone_len_player_0_Marine_len_player_0_diff', 'Drone_len_player_0_Overlord_len_player_0_diff', 'Drone_len_player_0_Probe_len_player_0_diff', 'Drone_len_player_0_Reaper_len_player_0_diff', 'Drone_len_player_0_Roach_len_player_0_diff', 'Drone_len_player_1', 'Drone_len_player_1_BanelingCocoon_len_player_1_diff', 'Drone_len_player_1_Barracks_len_player_1_diff', 'Drone_len_player_1_Factory_len_player_1_diff_Drone_len_player_0_Factory_len_player_0_diff_diff', 'Drone_len_player_1_Hatchery_len_player_1_diff', 'Drone_len_player_1_Hatchery_len_player_1_diff_Drone_len_player_0_Hatchery_len_player_0_diff_diff', 'Drone_len_player_1_Hellion_len_player_1_diff', 'Drone_len_player_1_Hellion_len_player_1_diff_Drone_len_player_0_Hellion_len_player_0_diff_diff', 'Drone_len_player_1_MULE_len_player_1_diff', 'Drone_len_player_1_MULE_len_player_1_diff_Drone_len_player_0_MULE_len_player_0_diff_diff', 'Drone_len_player_1_Marine_len_player_1_diff', 'Drone_len_player_1_OrbitalCommand_len_player_1_diff', 'Drone_len_player_1_Overlord_len_player_1_diff', 'Drone_len_player_1_Overlord_len_player_1_diff_Drone_len_player_0_Overlord_len_player_0_diff_diff', 'Drone_len_player_1_Queen_len_player_1_diff', 'Drone_len_player_1_Queen_len_player_1_diff_Drone_len_player_0_Queen_len_player_0_diff_diff', 'Drone_len_player_1_Reaper_len_player_1_diff', 'Drone_len_player_1_Reaper_len_player_1_diff_Drone_len_player_0_Reaper_len_player_0_diff_diff', 'Drone_len_player_1_Roach_len_player_1_diff', 'Drone_len_player_1_Starport_len_player_1_diff', 'Drone_len_player_1_Starport_len_player_1_diff_Drone_len_player_0_Starport_len_player_0_diff_diff', 'Drone_len_player_1_SupplyDepotLowered_len_player_1_diff', 'Drone_len_player_1_SupplyDepotLowered_len_player_1_diff_Drone_len_player_0_SupplyDepotLowered_len_player_0_diff_diff', 'Drone_len_player_1_Zergling_len_player_1_diff', 'Drone_len_player_1_Zergling_len_player_1_diff_Drone_len_player_0_Zergling_len_player_0_diff_diff', 'Drone_right_click_target_count_player_1', 'Egg_len_player_0_Factory_len_player_0_diff', 'Egg_len_player_0_Hellion_len_player_0_diff', 'Egg_len_player_0_MULE_len_player_0_diff', 'Egg_len_player_0_Marine_len_player_0_diff', 'Egg_len_player_0_OrbitalCommand_len_player_0_diff', 'Egg_len_player_0_Overlord_len_player_0_diff', 'Egg_len_player_0_Reaper_len_player_0_diff', 'Egg_len_player_0_SupplyDepotLowered_len_player_0_diff', 'Egg_len_player_1', 'Egg_len_player_1_Factory_len_player_1_diff_Egg_len_player_0_Factory_len_player_0_diff_diff', 'Egg_len_player_1_Hatchery_len_player_1_diff', 'Egg_len_player_1_Hellion_len_player_1_diff', 'Egg_len_player_1_MULE_len_player_1_diff', 'Egg_len_player_1_Marine_len_player_1_diff', 'Egg_len_player_1_OrbitalCommand_len_player_1_diff', 'Egg_len_player_1_OrbitalCommand_len_player_1_diff_Egg_len_player_0_OrbitalCommand_len_player_0_diff_diff', 'Egg_len_player_1_Queen_len_player_1_diff', 'Egg_len_player_1_Starport_len_player_1_diff', 'Egg_len_player_1_Starport_len_player_1_diff_Egg_len_player_0_Starport_len_player_0_diff_diff', 'Egg_len_player_1_SupplyDepotLowered_len_player_1_diff', 'EngineeringBay_len_player_0', 'EngineeringBay_len_player_1', 'EngineeringBay_right_click_target_count_diff', 'EngineeringBay_right_click_target_count_player_0', 'EngineeringBay_right_click_target_count_player_1', 'EvolutionChamber_len_player_0', 'ExtendingBridgeNEWide10Out_len_player_0', 'ExtendingBridgeNEWide10Out_len_player_1', 'ExtendingBridgeNEWide10_len_player_0', 'ExtendingBridgeNEWide10_len_player_1', 'ExtendingBridgeNEWide8Out_len_player_0', 'ExtendingBridgeNEWide8Out_len_player_1', 'ExtendingBridgeNWWide8Out_len_player_0', 'ExtendingBridgeNWWide8Out_len_player_1', 'ExtendingBridgeNWWide8_len_player_0', 'ExtendingBridgeNWWide8_len_player_1', 'Extractor_len_player_0', 'Extractor_right_click_target_count_player_1', 'FactoryReactor_len_player_0', 'FactoryReactor_len_player_1', 'FactoryTechLab_len_player_0', 'FactoryTechLab_len_player_1', 'Factory_len_player_0', 'Factory_len_player_0_Hatchery_len_player_0_diff', 'Factory_len_player_0_Roach_len_player_0_diff', 'Factory_len_player_1', 'Factory_len_player_1_Hatchery_len_player_1_diff_Factory_len_player_0_Hatchery_len_player_0_diff_diff', 'Factory_len_player_1_Hellion_len_player_1_diff', 'Factory_len_player_1_MULE_len_player_1_diff', 'Factory_len_player_1_Reaper_len_player_1_diff', 'Factory_len_player_1_Roach_len_player_1_diff', 'Factory_len_player_1_Starport_len_player_1_diff', 'Factory_right_click_target_count_diff', 'Factory_right_click_target_count_player_0', 'Factory_right_click_target_count_player_1', 'FleetBeacon_len_player_0', 'FleetBeacon_len_player_1', 'Forge_len_player_0', 'Forge_len_player_1', 'Forge_right_click_target_count_diff', 'Forge_right_click_target_count_player_1', 'FungalGrowthMissile_len_player_1', 'FungalGrowthMissile_right_click_target_count_diff', 'FungalGrowthMissile_right_click_target_count_player_0', 'FungalGrowthMissile_right_click_target_count_player_1', 'FusionCore_len_player_0', 'FusionCore_len_player_1', 'Gateway_len_player_0', 'GetControlGroup_player_0_isnull', 'GetControlGroup_player_1_isnull', 'GhostAcademy_len_player_0', 'GhostAcademy_len_player_1', 'Ghost_len_player_0', 'Ghost_len_player_1', 'GlaiveWurmBounceWeapon_len_player_0', 'GlaiveWurmBounceWeapon_len_player_1', 'GreaterSpire_len_player_0', 'GreaterSpire_len_player_1', 'Hatchery_len_player_0', 'Hatchery_len_player_0_Reaper_len_player_0_diff', 'Hatchery_len_player_0_Starport_len_player_0_diff', 'Hatchery_len_player_1', 'Hatchery_len_player_1_Reaper_len_player_1_diff', 'Hatchery_len_player_1_Reaper_len_player_1_diff_Hatchery_len_player_0_Reaper_len_player_0_diff_diff', 'Hatchery_len_player_1_Roach_len_player_1_diff', 'Hatchery_len_player_1_Starport_len_player_1_diff', 'Hatchery_len_player_1_Starport_len_player_1_diff_Hatchery_len_player_0_Starport_len_player_0_diff_diff', 'Hatchery_right_click_target_count_player_0', 'Hellion_len_player_0', 'Hellion_len_player_0_Hatchery_len_player_0_diff', 'Hellion_len_player_0_MULE_len_player_0_diff', 'Hellion_len_player_0_Reaper_len_player_0_diff', 'Hellion_len_player_0_Roach_len_player_0_diff', 'Hellion_len_player_1', 'Hellion_len_player_1_Hatchery_len_player_1_diff', 'Hellion_len_player_1_MULE_len_player_1_diff', 'Hellion_len_player_1_Reaper_len_player_1_diff', 'Hellion_len_player_1_Reaper_len_player_1_diff_Hellion_len_player_0_Reaper_len_player_0_diff_diff', 'Hellion_len_player_1_Roach_len_player_1_diff', 'Hellion_len_player_1_Starport_len_player_1_diff', 'Hellion_right_click_target_count_diff', 'Hellion_right_click_target_count_player_0', 'Hellion_right_click_target_count_player_1', 'HighTemplar_len_player_0', 'HighTemplar_len_player_1', 'Hive_len_player_0', 'Hive_right_click_target_count_diff', 'Hive_right_click_target_count_player_0', 'Hive_right_click_target_count_player_1', 'HydraliskDen_len_player_0', 'HydraliskDen_len_player_1', 'Hydralisk_len_player_0', 'Hydralisk_len_player_1', 'Immortal_len_player_1', 'InfestationPit_len_player_0', 'InfestationPit_len_player_1', 'InfestorBurrowed_len_player_1', 'Infestor_len_player_1', 'Lair_right_click_target_count_diff', 'Lair_right_click_target_count_player_1', 'Larva_len_player_1', 'LocustMPFlying_len_player_0', 'LocustMPFlying_len_player_1', 'LurkerEgg_len_player_0', 'LurkerEgg_len_player_1', 'Lurker_len_player_0', 'Lurker_len_player_1', 'MULE_len_player_0', 'MULE_len_player_0_Hatchery_len_player_0_diff', 'MULE_len_player_0_Reaper_len_player_0_diff', 'MULE_len_player_0_Roach_len_player_0_diff', 'MULE_len_player_1', 'MULE_len_player_1_Hatchery_len_player_1_diff', 'MULE_len_player_1_Hatchery_len_player_1_diff_MULE_len_player_0_Hatchery_len_player_0_diff_diff', 'MULE_len_player_1_Reaper_len_player_1_diff', 'MULE_len_player_1_Reaper_len_player_1_diff_MULE_len_player_0_Reaper_len_player_0_diff_diff', 'MULE_len_player_1_Roach_len_player_1_diff', 'MULE_len_player_1_Starport_len_player_1_diff', 'MULE_right_click_target_count_diff', 'MULE_right_click_target_count_player_0', 'MULE_right_click_target_count_player_1', 'Marine_len_player_0', 'Marine_len_player_0_Hatchery_len_player_0_diff', 'Marine_len_player_0_Hellion_len_player_0_diff', 'Marine_len_player_0_OrbitalCommand_len_player_0_diff', 'Marine_len_player_0_Overlord_len_player_0_diff', 'Marine_len_player_0_Queen_len_player_0_diff', 'Marine_len_player_0_Reaper_len_player_0_diff', 'Marine_len_player_0_Roach_len_player_0_diff', 'Marine_len_player_0_Starport_len_player_0_diff', 'Marine_len_player_0_Zergling_len_player_0_diff', 'Marine_len_player_1', 'Marine_len_player_1_BanelingCocoon_len_player_1_diff', 'Marine_len_player_1_Factory_len_player_1_diff', 'Marine_len_player_1_Hatchery_len_player_1_diff_Marine_len_player_0_Hatchery_len_player_0_diff_diff', 'Marine_len_player_1_Hellion_len_player_1_diff', 'Marine_len_player_1_Hellion_len_player_1_diff_Marine_len_player_0_Hellion_len_player_0_diff_diff', 'Marine_len_player_1_MULE_len_player_1_diff_Marine_len_player_0_MULE_len_player_0_diff_diff', 'Marine_len_player_1_OrbitalCommand_len_player_1_diff', 'Marine_len_player_1_Overlord_len_player_1_diff_Marine_len_player_0_Overlord_len_player_0_diff_diff', 'Marine_len_player_1_Probe_len_player_1_diff', 'Marine_len_player_1_Probe_len_player_1_diff_Marine_len_player_0_Probe_len_player_0_diff_diff', 'Marine_len_player_1_Queen_len_player_1_diff', 'Marine_len_player_1_Queen_len_player_1_diff_Marine_len_player_0_Queen_len_player_0_diff_diff', 'Marine_len_player_1_Reaper_len_player_1_diff', 'Marine_len_player_1_Roach_len_player_1_diff', 'Marine_len_player_1_Starport_len_player_1_diff_Marine_len_player_0_Starport_len_player_0_diff_diff', 'Marine_len_player_1_SupplyDepotLowered_len_player_1_diff_Marine_len_player_0_SupplyDepotLowered_len_player_0_diff_diff', 'Marine_len_player_1_Zergling_len_player_1_diff', 'Marine_right_click_target_count_player_0', 'Marine_right_click_target_count_player_1', 'Medivac_len_player_0', 'Medivac_len_player_1', 'Medivac_right_click_target_count_player_0', 'MissileTurret_len_player_0', 'MissileTurret_len_player_1', 'Mutalisk_len_player_0', 'Mutalisk_len_player_1', 'Nexus_right_click_target_count_diff', 'None_len_player_0', 'None_len_player_1', 'None_right_click_target_count_player_1', 'NydusNetwork_len_player_0', 'NydusNetwork_len_player_1', 'NydusWorm_len_player_0', 'NydusWorm_len_player_1', 'Observer_len_player_0', 'OrbitalCommand_len_player_0_Factory_len_player_0_diff', 'OrbitalCommand_len_player_0_Hellion_len_player_0_diff', 'OrbitalCommand_len_player_1', 'OrbitalCommand_len_player_1_BanelingCocoon_len_player_1_diff', 'OrbitalCommand_len_player_1_Factory_len_player_1_diff', 'OrbitalCommand_len_player_1_Hatchery_len_player_1_diff', 'OrbitalCommand_len_player_1_Hellion_len_player_1_diff', 'OrbitalCommand_len_player_1_Hellion_len_player_1_diff_OrbitalCommand_len_player_0_Hellion_len_player_0_diff_diff', 'OrbitalCommand_len_player_1_MULE_len_player_1_diff', 'OrbitalCommand_len_player_1_Overlord_len_player_1_diff', 'OrbitalCommand_len_player_1_Overlord_len_player_1_diff_OrbitalCommand_len_player_0_Overlord_len_player_0_diff_diff', 'OrbitalCommand_len_player_1_Queen_len_player_1_diff', 'OrbitalCommand_len_player_1_Queen_len_player_1_diff_OrbitalCommand_len_player_0_Queen_len_player_0_diff_diff', 'OrbitalCommand_len_player_1_Roach_len_player_1_diff', 'OrbitalCommand_len_player_1_Starport_len_player_1_diff', 'OrbitalCommand_len_player_1_SupplyDepotLowered_len_player_1_diff', 'OrbitalCommand_len_player_1_SupplyDepotLowered_len_player_1_diff_OrbitalCommand_len_player_0_SupplyDepotLowered_len_player_0_diff_diff', 'OrbitalCommand_right_click_target_count_diff', 'OrbitalCommand_right_click_target_count_player_1', 'OverlordGenerateCreepKeybind_len_player_0', 'OverlordGenerateCreepKeybind_len_player_1', 'Overlord_len_player_0', 'Overlord_len_player_0_Hatchery_len_player_0_diff', 'Overlord_len_player_0_Hellion_len_player_0_diff', 'Overlord_len_player_0_MULE_len_player_0_diff', 'Overlord_len_player_0_Reaper_len_player_0_diff', 'Overlord_len_player_0_Starport_len_player_0_diff', 'Overlord_len_player_1', 'Overlord_len_player_1_Factory_len_player_1_diff', 'Overlord_len_player_1_Hatchery_len_player_1_diff', 'Overlord_len_player_1_Hellion_len_player_1_diff', 'Overlord_len_player_1_Hellion_len_player_1_diff_Overlord_len_player_0_Hellion_len_player_0_diff_diff', 'Overlord_len_player_1_MULE_len_player_1_diff', 'Overlord_len_player_1_MULE_len_player_1_diff_Overlord_len_player_0_MULE_len_player_0_diff_diff', 'Overlord_len_player_1_Queen_len_player_1_diff_Overlord_len_player_0_Queen_len_player_0_diff_diff', 'Overlord_len_player_1_Reaper_len_player_1_diff', 'Overlord_len_player_1_Reaper_len_player_1_diff_Overlord_len_player_0_Reaper_len_player_0_diff_diff', 'Overlord_len_player_1_Starport_len_player_1_diff', 'Overlord_len_player_1_Starport_len_player_1_diff_Overlord_len_player_0_Starport_len_player_0_diff_diff', 'Overlord_right_click_target_count_diff', 'Overlord_right_click_target_count_player_0', 'Overlord_right_click_target_count_player_1', 'OverseerCocoon_len_player_0', 'OverseerCocoon_len_player_1', 'Overseer_len_player_0', 'Overseer_len_player_1', 'Phoenix_len_player_0', 'Phoenix_len_player_1', 'PhotonCannon_len_player_0', 'PhotonCannon_len_player_1', 'PhotonCannon_right_click_target_count_player_0', 'PhotonCannon_right_click_target_count_player_1', 'PhysicsCapsule_len_player_0', 'PhysicsCapsule_len_player_1', 'PhysicsCapsule_right_click_target_count_diff', 'PhysicsCapsule_right_click_target_count_player_0', 'PhysicsCapsule_right_click_target_count_player_1', 'PlanetaryFortress_len_player_1', 'Probe_len_player_0', 'Probe_len_player_0_BanelingCocoon_len_player_0_diff', 'Probe_len_player_0_Barracks_len_player_0_diff', 'Probe_len_player_0_Factory_len_player_0_diff', 'Probe_len_player_0_Hellion_len_player_0_diff', 'Probe_len_player_0_MULE_len_player_0_diff', 'Probe_len_player_0_Overlord_len_player_0_diff', 'Probe_len_player_0_Queen_len_player_0_diff', 'Probe_len_player_0_Reaper_len_player_0_diff', 'Probe_len_player_0_Roach_len_player_0_diff', 'Probe_len_player_0_Starport_len_player_0_diff', 'Probe_len_player_0_SupplyDepotLowered_len_player_0_diff', 'Probe_len_player_1', 'Probe_len_player_1_BanelingCocoon_len_player_1_diff', 'Probe_len_player_1_BanelingCocoon_len_player_1_diff_Probe_len_player_0_BanelingCocoon_len_player_0_diff_diff', 'Probe_len_player_1_Barracks_len_player_1_diff', 'Probe_len_player_1_Barracks_len_player_1_diff_Probe_len_player_0_Barracks_len_player_0_diff_diff', 'Probe_len_player_1_Factory_len_player_1_diff_Probe_len_player_0_Factory_len_player_0_diff_diff', 'Probe_len_player_1_Hatchery_len_player_1_diff', 'Probe_len_player_1_Hellion_len_player_1_diff', 'Probe_len_player_1_Hellion_len_player_1_diff_Probe_len_player_0_Hellion_len_player_0_diff_diff', 'Probe_len_player_1_MULE_len_player_1_diff', 'Probe_len_player_1_MULE_len_player_1_diff_Probe_len_player_0_MULE_len_player_0_diff_diff', 'Probe_len_player_1_OrbitalCommand_len_player_1_diff', 'Probe_len_player_1_OrbitalCommand_len_player_1_diff_Probe_len_player_0_OrbitalCommand_len_player_0_diff_diff', 'Probe_len_player_1_Overlord_len_player_1_diff', 'Probe_len_player_1_Queen_len_player_1_diff', 'Probe_len_player_1_Reaper_len_player_1_diff_Probe_len_player_0_Reaper_len_player_0_diff_diff', 'Probe_len_player_1_Roach_len_player_1_diff', 'Probe_len_player_1_Roach_len_player_1_diff_Probe_len_player_0_Roach_len_player_0_diff_diff', 'Probe_len_player_1_Starport_len_player_1_diff', 'Probe_len_player_1_Starport_len_player_1_diff_Probe_len_player_0_Starport_len_player_0_diff_diff', 'Probe_len_player_1_SupplyDepotLowered_len_player_1_diff', 'Pylon_len_player_0', 'Pylon_len_player_1', 'QueenBurrowed_len_player_0', 'QueenBurrowed_len_player_1', 'Queen_len_player_0', 'Queen_len_player_0_Factory_len_player_0_diff', 'Queen_len_player_0_Hatchery_len_player_0_diff', 'Queen_len_player_0_Hellion_len_player_0_diff', 'Queen_len_player_0_MULE_len_player_0_diff', 'Queen_len_player_0_Reaper_len_player_0_diff', 'Queen_len_player_0_Roach_len_player_0_diff', 'Queen_len_player_0_Starport_len_player_0_diff', 'Queen_len_player_1', 'Queen_len_player_1_Factory_len_player_1_diff', 'Queen_len_player_1_Hatchery_len_player_1_diff', 'Queen_len_player_1_Hatchery_len_player_1_diff_Queen_len_player_0_Hatchery_len_player_0_diff_diff', 'Queen_len_player_1_Hellion_len_player_1_diff', 'Queen_len_player_1_Hellion_len_player_1_diff_Queen_len_player_0_Hellion_len_player_0_diff_diff', 'Queen_len_player_1_MULE_len_player_1_diff', 'Queen_len_player_1_Reaper_len_player_1_diff', 'Queen_len_player_1_Reaper_len_player_1_diff_Queen_len_player_0_Reaper_len_player_0_diff_diff', 'Queen_len_player_1_Starport_len_player_1_diff', 'Queen_len_player_1_Starport_len_player_1_diff_Queen_len_player_0_Starport_len_player_0_diff_diff', 'Queen_right_click_target_count_diff', 'Queen_right_click_target_count_player_0', 'Queen_right_click_target_count_player_1', 'Ravager_len_player_0', 'Raven_len_player_0', 'Raven_len_player_1', 'Reactor_len_player_0', 'Reactor_len_player_1', 'Reaper_len_player_0', 'Reaper_len_player_0_Roach_len_player_0_diff', 'Reaper_len_player_0_Starport_len_player_0_diff', 'Reaper_len_player_1', 'Reaper_len_player_1_Roach_len_player_1_diff', 'Reaper_len_player_1_Roach_len_player_1_diff_Reaper_len_player_0_Roach_len_player_0_diff_diff', 'Reaper_len_player_1_Starport_len_player_1_diff', 'Reaper_len_player_1_Starport_len_player_1_diff_Reaper_len_player_0_Starport_len_player_0_diff_diff', 'Reaper_right_click_target_count_diff', 'Reaper_right_click_target_count_player_1', 'Refinery_len_player_0', 'Refinery_len_player_1', 'Refinery_right_click_target_count_diff', 'Refinery_right_click_target_count_player_0', 'RichMineralField750_len_player_0', 'RichMineralField750_len_player_1', 'RichMineralField_len_player_0', 'RichMineralField_len_player_1', 'Right Click_player_0_Right Click_player_1_diff', 'Right Click_player_0_isnull', 'Right Click_player_1', 'Right Click_player_1_isnull', 'RoachBurrowed_len_player_0', 'RoachBurrowed_len_player_1', 'RoachWarren_len_player_0', 'RoachWarren_len_player_1', 'Roach_len_player_0', 'Roach_len_player_0_Starport_len_player_0_diff', 'Roach_len_player_1', 'Roach_len_player_1_Starport_len_player_1_diff', 'Roach_right_click_target_count_diff', 'Roach_right_click_target_count_player_0', 'Roach_right_click_target_count_player_1', 'RoboticsBay_len_player_0', 'RoboticsBay_len_player_1', 'RoboticsFacility_len_player_0', 'Rocks2x2NonConjoined_len_player_0', 'Rocks2x2NonConjoined_len_player_1', 'SCV_len_player_0', 'SCV_len_player_0_BanelingCocoon_len_player_0_diff', 'SCV_len_player_0_Drone_len_player_0_diff', 'SCV_len_player_0_Factory_len_player_0_diff', 'SCV_len_player_0_Hatchery_len_player_0_diff', 'SCV_len_player_0_MULE_len_player_0_diff', 'SCV_len_player_0_Marine_len_player_0_diff', 'SCV_len_player_0_OrbitalCommand_len_player_0_diff', 'SCV_len_player_0_Overlord_len_player_0_diff', 'SCV_len_player_0_Probe_len_player_0_diff', 'SCV_len_player_0_Queen_len_player_0_diff', 'SCV_len_player_0_Reaper_len_player_0_diff', 'SCV_len_player_0_Roach_len_player_0_diff', 'SCV_len_player_0_Starport_len_player_0_diff', 'SCV_len_player_0_SupplyDepotLowered_len_player_0_diff', 'SCV_len_player_0_Zergling_len_player_0_diff', 'SCV_len_player_1', 'SCV_len_player_1_BanelingCocoon_len_player_1_diff', 'SCV_len_player_1_BanelingCocoon_len_player_1_diff_SCV_len_player_0_BanelingCocoon_len_player_0_diff_diff', 'SCV_len_player_1_Barracks_len_player_1_diff', 'SCV_len_player_1_Drone_len_player_1_diff', 'SCV_len_player_1_Factory_len_player_1_diff', 'SCV_len_player_1_Factory_len_player_1_diff_SCV_len_player_0_Factory_len_player_0_diff_diff', 'SCV_len_player_1_Hatchery_len_player_1_diff', 'SCV_len_player_1_Hellion_len_player_1_diff', 'SCV_len_player_1_Hellion_len_player_1_diff_SCV_len_player_0_Hellion_len_player_0_diff_diff', 'SCV_len_player_1_MULE_len_player_1_diff', 'SCV_len_player_1_MULE_len_player_1_diff_SCV_len_player_0_MULE_len_player_0_diff_diff', 'SCV_len_player_1_Marine_len_player_1_diff', 'SCV_len_player_1_Marine_len_player_1_diff_SCV_len_player_0_Marine_len_player_0_diff_diff', 'SCV_len_player_1_OrbitalCommand_len_player_1_diff', 'SCV_len_player_1_Probe_len_player_1_diff_SCV_len_player_0_Probe_len_player_0_diff_diff', 'SCV_len_player_1_Queen_len_player_1_diff', 'SCV_len_player_1_Queen_len_player_1_diff_SCV_len_player_0_Queen_len_player_0_diff_diff', 'SCV_len_player_1_Reaper_len_player_1_diff', 'SCV_len_player_1_Roach_len_player_1_diff', 'SCV_len_player_1_Roach_len_player_1_diff_SCV_len_player_0_Roach_len_player_0_diff_diff', 'SCV_len_player_1_Starport_len_player_1_diff_SCV_len_player_0_Starport_len_player_0_diff_diff', 'SCV_len_player_1_SupplyDepotLowered_len_player_1_diff', 'SCV_len_player_1_SupplyDepotLowered_len_player_1_diff_SCV_len_player_0_SupplyDepotLowered_len_player_0_diff_diff', 'SCV_len_player_1_Zergling_len_player_1_diff', 'SCV_len_player_1_Zergling_len_player_1_diff_SCV_len_player_0_Zergling_len_player_0_diff_diff', 'SCV_right_click_target_count_player_0', 'Selection_player_0', 'Selection_player_0_isnull', 'Selection_player_1_isnull', 'SensorTower_len_player_0', 'SensorTower_len_player_1', 'SetControlGroup_player_0_isnull', 'SetControlGroup_player_1', 'SetControlGroup_player_1_isnull', 'SiegeTankSieged_len_player_1', 'SiegeTankSieged_right_click_target_count_diff', 'SiegeTankSieged_right_click_target_count_player_0', 'SiegeTank_len_player_1', 'SiegeTank_right_click_target_count_diff', 'SiegeTank_right_click_target_count_player_0', 'SiegeTank_right_click_target_count_player_1', 'SpawningPool_len_player_1', 'SpawningPool_right_click_target_count_diff', 'SpawningPool_right_click_target_count_player_0', 'SpawningPool_right_click_target_count_player_1', 'SpineCrawlerUprooted_len_player_0', 'SpineCrawlerUprooted_len_player_1', 'SpineCrawler_len_player_1', 'Spire_len_player_0', 'Spire_len_player_1', 'SporeCrawlerUprooted_len_player_0', 'SporeCrawlerUprooted_len_player_1', 'SporeCrawler_len_player_0', 'SporeCrawler_len_player_1', 'Stalker_len_player_0', 'Stalker_right_click_target_count_diff', 'Stalker_right_click_target_count_player_0', 'StarportReactor_len_player_0', 'StarportReactor_len_player_1', 'StarportTechLab_len_player_0', 'StarportTechLab_len_player_1', 'Starport_len_player_0', 'Starport_len_player_1', 'Starport_right_click_target_count_diff', 'Starport_right_click_target_count_player_0', 'Starport_right_click_target_count_player_1', 'SupplyDepotLowered_len_player_0', 'SupplyDepotLowered_len_player_0_BanelingCocoon_len_player_0_diff', 'SupplyDepotLowered_len_player_0_Factory_len_player_0_diff', 'SupplyDepotLowered_len_player_1', 'SupplyDepotLowered_len_player_1_BanelingCocoon_len_player_1_diff', 'SupplyDepotLowered_len_player_1_BanelingCocoon_len_player_1_diff_SupplyDepotLowered_len_player_0_BanelingCocoon_len_player_0_diff_diff', 'SupplyDepotLowered_len_player_1_Factory_len_player_1_diff', 'SupplyDepotLowered_len_player_1_Factory_len_player_1_diff_SupplyDepotLowered_len_player_0_Factory_len_player_0_diff_diff', 'SupplyDepotLowered_len_player_1_Hatchery_len_player_1_diff', 'SupplyDepotLowered_len_player_1_Hatchery_len_player_1_diff_SupplyDepotLowered_len_player_0_Hatchery_len_player_0_diff_diff', 'SupplyDepotLowered_len_player_1_Hellion_len_player_1_diff', 'SupplyDepotLowered_len_player_1_MULE_len_player_1_diff', 'SupplyDepotLowered_len_player_1_Queen_len_player_1_diff', 'SupplyDepotLowered_len_player_1_Queen_len_player_1_diff_SupplyDepotLowered_len_player_0_Queen_len_player_0_diff_diff', 'SupplyDepotLowered_len_player_1_Reaper_len_player_1_diff', 'SupplyDepotLowered_len_player_1_Roach_len_player_1_diff', 'SupplyDepotLowered_len_player_1_Starport_len_player_1_diff', 'SupplyDepotLowered_right_click_target_count_diff', 'SupplyDepotLowered_right_click_target_count_player_0', 'SupplyDepotLowered_right_click_target_count_player_1', 'SupplyDepot_right_click_target_count_player_1', 'SwarmHost_len_player_1', 'TechLab_len_player_1', 'TemplarArchive_len_player_0', 'TemplarArchive_len_player_1', 'Thor_len_player_0', 'Thor_len_player_1', 'TwilightCouncil_len_player_0', 'TwilightCouncil_len_player_1', 'UltraliskCavern_len_player_1', 'VikingAssault_len_player_0', 'VikingAssault_len_player_1', 'Viking_len_player_0', 'Viking_len_player_1', 'VoidRay_len_player_0', 'VoidRay_len_player_1', 'WarpGate_len_player_0', 'WarpGate_right_click_target_count_player_0', 'WarpPrismPhasing_len_player_0', 'WarpPrismPhasing_len_player_1', 'WarpPrism_len_player_0', 'WarpPrism_len_player_1', 'Weapon_len_player_0', 'Weapon_len_player_1', 'XelNagaDestructibleBlocker8W_len_player_0', 'XelNagaDestructibleBlocker8W_len_player_1', 'XelNagaTower_len_player_0', 'XelNagaTower_len_player_1', 'XelNagaTower_right_click_target_count_diff', 'XelNagaTower_right_click_target_count_player_0', 'XelNagaTower_right_click_target_count_player_1', 'Zealot_len_player_0', 'Zealot_len_player_1', 'Zealot_right_click_target_count_diff', 'Zealot_right_click_target_count_player_0', 'Zealot_right_click_target_count_player_1', 'ZerglingBurrowed_len_player_0', 'ZerglingBurrowed_len_player_1', 'Zergling_len_player_0', 'Zergling_len_player_0_Barracks_len_player_0_diff', 'Zergling_len_player_0_Hatchery_len_player_0_diff', 'Zergling_len_player_0_Hellion_len_player_0_diff', 'Zergling_len_player_0_MULE_len_player_0_diff', 'Zergling_len_player_0_OrbitalCommand_len_player_0_diff', 'Zergling_len_player_0_Overlord_len_player_0_diff', 'Zergling_len_player_0_Probe_len_player_0_diff', 'Zergling_len_player_0_Queen_len_player_0_diff', 'Zergling_len_player_0_Reaper_len_player_0_diff', 'Zergling_len_player_0_Roach_len_player_0_diff', 'Zergling_len_player_0_Starport_len_player_0_diff', 'Zergling_len_player_1', 'Zergling_len_player_1_BanelingCocoon_len_player_1_diff', 'Zergling_len_player_1_BanelingCocoon_len_player_1_diff_Zergling_len_player_0_BanelingCocoon_len_player_0_diff_diff', 'Zergling_len_player_1_Barracks_len_player_1_diff', 'Zergling_len_player_1_Barracks_len_player_1_diff_Zergling_len_player_0_Barracks_len_player_0_diff_diff', 'Zergling_len_player_1_Factory_len_player_1_diff', 'Zergling_len_player_1_Factory_len_player_1_diff_Zergling_len_player_0_Factory_len_player_0_diff_diff', 'Zergling_len_player_1_Hatchery_len_player_1_diff', 'Zergling_len_player_1_Hatchery_len_player_1_diff_Zergling_len_player_0_Hatchery_len_player_0_diff_diff', 'Zergling_len_player_1_Hellion_len_player_1_diff', 'Zergling_len_player_1_MULE_len_player_1_diff', 'Zergling_len_player_1_MULE_len_player_1_diff_Zergling_len_player_0_MULE_len_player_0_diff_diff', 'Zergling_len_player_1_OrbitalCommand_len_player_1_diff', 'Zergling_len_player_1_OrbitalCommand_len_player_1_diff_Zergling_len_player_0_OrbitalCommand_len_player_0_diff_diff', 'Zergling_len_player_1_Overlord_len_player_1_diff', 'Zergling_len_player_1_Overlord_len_player_1_diff_Zergling_len_player_0_Overlord_len_player_0_diff_diff', 'Zergling_len_player_1_Probe_len_player_1_diff', 'Zergling_len_player_1_Queen_len_player_1_diff_Zergling_len_player_0_Queen_len_player_0_diff_diff', 'Zergling_len_player_1_Reaper_len_player_1_diff', 'Zergling_len_player_1_Reaper_len_player_1_diff_Zergling_len_player_0_Reaper_len_player_0_diff_diff', 'Zergling_len_player_1_Roach_len_player_1_diff', 'Zergling_len_player_1_Roach_len_player_1_diff_Zergling_len_player_0_Roach_len_player_0_diff_diff', 'Zergling_len_player_1_Starport_len_player_1_diff', 'Zergling_len_player_1_SupplyDepotLowered_len_player_1_diff', 'Zergling_len_player_1_SupplyDepotLowered_len_player_1_diff_Zergling_len_player_0_SupplyDepotLowered_len_player_0_diff_diff', 'Zergling_right_click_target_count_player_1', 'ability_code_1000_diff', 'ability_code_1000_player_0', 'ability_code_1000_player_1', 'ability_code_1020_first_time_diff', 'ability_code_1020_player_0_ability_code_1261_player_0_diff', 'ability_code_1020_player_1', 'ability_code_1020_player_1_ability_code_102A_player_1_diff', 'ability_code_1020_player_1_ability_code_102A_player_1_diff_ability_code_1020_player_0_ability_code_102A_player_0_diff_diff', 'ability_code_1020_player_1_ability_code_1261_player_1_diff', 'ability_code_1020_player_1_ability_code_16E2_player_1_diff', 'ability_code_1020_player_1_ability_code_1E60_player_1_diff', 'ability_code_1020_player_1_ability_code_1E60_player_1_diff_ability_code_1020_player_0_ability_code_1E60_player_0_diff_diff', 'ability_code_1020_player_1_ability_code_480_player_1_diff', 'ability_code_1021_diff', 'ability_code_1021_first_time_diff', 'ability_code_1021_first_time_player_0', 'ability_code_1021_first_time_player_1', 'ability_code_1021_last_first_diff', 'ability_code_1021_player_0_ability_code_1022_player_0_diff', 'ability_code_1021_player_0_ability_code_1023_player_0_diff', 'ability_code_1021_player_0_ability_code_15E0_player_0_diff', 'ability_code_1021_player_0_ability_code_16E2_player_0_diff', 'ability_code_1021_player_0_ability_code_1821_player_0_diff', 'ability_code_1021_player_0_ability_code_1822_player_0_diff', 'ability_code_1021_player_0_ability_code_1C60_player_0_diff', 'ability_code_1021_player_0_ability_code_1E60_player_0_diff', 'ability_code_1021_player_0_ability_code_DE0_player_0_diff', 'ability_code_1021_player_1', 'ability_code_1021_player_1_ability_code_1020_player_1_diff', 'ability_code_1021_player_1_ability_code_1020_player_1_diff_ability_code_1021_player_0_ability_code_1020_player_0_diff_diff', 'ability_code_1021_player_1_ability_code_1022_player_1_diff', 'ability_code_1021_player_1_ability_code_1022_player_1_diff_ability_code_1021_player_0_ability_code_1022_player_0_diff_diff', 'ability_code_1021_player_1_ability_code_1023_player_1_diff', 'ability_code_1021_player_1_ability_code_1023_player_1_diff_ability_code_1021_player_0_ability_code_1023_player_0_diff_diff', 'ability_code_1021_player_1_ability_code_102A_player_1_diff', 'ability_code_1021_player_1_ability_code_102A_player_1_diff_ability_code_1021_player_0_ability_code_102A_player_0_diff_diff', 'ability_code_1021_player_1_ability_code_1261_player_1_diff', 'ability_code_1021_player_1_ability_code_13A0_player_1_diff', 'ability_code_1021_player_1_ability_code_15E0_player_1_diff_ability_code_1021_player_0_ability_code_15E0_player_0_diff_diff', 'ability_code_1021_player_1_ability_code_16E2_player_1_diff', 'ability_code_1021_player_1_ability_code_16E2_player_1_diff_ability_code_1021_player_0_ability_code_16E2_player_0_diff_diff', 'ability_code_1021_player_1_ability_code_1821_player_1_diff', 'ability_code_1021_player_1_ability_code_1822_player_1_diff', 'ability_code_1021_player_1_ability_code_1822_player_1_diff_ability_code_1021_player_0_ability_code_1822_player_0_diff_diff', 'ability_code_1021_player_1_ability_code_1C60_player_1_diff', 'ability_code_1021_player_1_ability_code_1E60_player_1_diff', 'ability_code_1021_player_1_ability_code_480_player_1_diff', 'ability_code_1021_player_1_ability_code_B40_player_1_diff_ability_code_1021_player_0_ability_code_B40_player_0_diff_diff', 'ability_code_1021_player_1_ability_code_DE0_player_1_diff', 'ability_code_1021_player_1_ability_code_DE0_player_1_diff_ability_code_1021_player_0_ability_code_DE0_player_0_diff_diff', 'ability_code_1022_diff', 'ability_code_1022_first_time_diff', 'ability_code_1022_first_time_player_1', 'ability_code_1022_last_first_diff', 'ability_code_1022_last_first_player_1', 'ability_code_1022_player_0', 'ability_code_1022_player_0_ability_code_1020_player_0_diff', 'ability_code_1022_player_0_ability_code_1023_player_0_diff', 'ability_code_1022_player_0_ability_code_102A_player_0_diff', 'ability_code_1022_player_0_ability_code_1261_player_0_diff', 'ability_code_1022_player_0_ability_code_13A0_player_0_diff', 'ability_code_1022_player_0_ability_code_15E0_player_0_diff', 'ability_code_1022_player_0_ability_code_16E2_player_0_diff', 'ability_code_1022_player_0_ability_code_1822_player_0_diff', 'ability_code_1022_player_0_ability_code_1C60_player_0_diff', 'ability_code_1022_player_0_ability_code_1E60_player_0_diff', 'ability_code_1022_player_1', 'ability_code_1022_player_1_ability_code_1020_player_1_diff', 'ability_code_1022_player_1_ability_code_1023_player_1_diff', 'ability_code_1022_player_1_ability_code_1023_player_1_diff_ability_code_1022_player_0_ability_code_1023_player_0_diff_diff', 'ability_code_1022_player_1_ability_code_102A_player_1_diff', 'ability_code_1022_player_1_ability_code_102A_player_1_diff_ability_code_1022_player_0_ability_code_102A_player_0_diff_diff', 'ability_code_1022_player_1_ability_code_1261_player_1_diff', 'ability_code_1022_player_1_ability_code_13A0_player_1_diff', 'ability_code_1022_player_1_ability_code_13A0_player_1_diff_ability_code_1022_player_0_ability_code_13A0_player_0_diff_diff', 'ability_code_1022_player_1_ability_code_15E0_player_1_diff', 'ability_code_1022_player_1_ability_code_15E0_player_1_diff_ability_code_1022_player_0_ability_code_15E0_player_0_diff_diff', 'ability_code_1022_player_1_ability_code_16E2_player_1_diff', 'ability_code_1022_player_1_ability_code_16E2_player_1_diff_ability_code_1022_player_0_ability_code_16E2_player_0_diff_diff', 'ability_code_1022_player_1_ability_code_1821_player_1_diff', 'ability_code_1022_player_1_ability_code_1821_player_1_diff_ability_code_1022_player_0_ability_code_1821_player_0_diff_diff', 'ability_code_1022_player_1_ability_code_1822_player_1_diff', 'ability_code_1022_player_1_ability_code_1822_player_1_diff_ability_code_1022_player_0_ability_code_1822_player_0_diff_diff', 'ability_code_1022_player_1_ability_code_1C60_player_1_diff', 'ability_code_1022_player_1_ability_code_1C60_player_1_diff_ability_code_1022_player_0_ability_code_1C60_player_0_diff_diff', 'ability_code_1022_player_1_ability_code_1E60_player_1_diff', 'ability_code_1022_player_1_ability_code_1E60_player_1_diff_ability_code_1022_player_0_ability_code_1E60_player_0_diff_diff', 'ability_code_1022_player_1_ability_code_480_player_1_diff', 'ability_code_1023_diff', 'ability_code_1023_first_time_diff', 'ability_code_1023_first_time_player_0', 'ability_code_1023_player_0', 'ability_code_1023_player_0_ability_code_102A_player_0_diff', 'ability_code_1023_player_0_ability_code_13A0_player_0_diff', 'ability_code_1023_player_0_ability_code_15E0_player_0_diff', 'ability_code_1023_player_0_ability_code_16E2_player_0_diff', 'ability_code_1023_player_0_ability_code_1821_player_0_diff', 'ability_code_1023_player_0_ability_code_1822_player_0_diff', 'ability_code_1023_player_0_ability_code_1E60_player_0_diff', 'ability_code_1023_player_0_ability_code_480_player_0_diff', 'ability_code_1023_player_1', 'ability_code_1023_player_1_ability_code_1020_player_1_diff', 'ability_code_1023_player_1_ability_code_1020_player_1_diff_ability_code_1023_player_0_ability_code_1020_player_0_diff_diff', 'ability_code_1023_player_1_ability_code_102A_player_1_diff', 'ability_code_1023_player_1_ability_code_102A_player_1_diff_ability_code_1023_player_0_ability_code_102A_player_0_diff_diff', 'ability_code_1023_player_1_ability_code_1261_player_1_diff', 'ability_code_1023_player_1_ability_code_1261_player_1_diff_ability_code_1023_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_1023_player_1_ability_code_13A0_player_1_diff', 'ability_code_1023_player_1_ability_code_13A0_player_1_diff_ability_code_1023_player_0_ability_code_13A0_player_0_diff_diff', 'ability_code_1023_player_1_ability_code_15E0_player_1_diff', 'ability_code_1023_player_1_ability_code_15E0_player_1_diff_ability_code_1023_player_0_ability_code_15E0_player_0_diff_diff', 'ability_code_1023_player_1_ability_code_16E2_player_1_diff', 'ability_code_1023_player_1_ability_code_16E2_player_1_diff_ability_code_1023_player_0_ability_code_16E2_player_0_diff_diff', 'ability_code_1023_player_1_ability_code_1821_player_1_diff', 'ability_code_1023_player_1_ability_code_1821_player_1_diff_ability_code_1023_player_0_ability_code_1821_player_0_diff_diff', 'ability_code_1023_player_1_ability_code_1822_player_1_diff', 'ability_code_1023_player_1_ability_code_1822_player_1_diff_ability_code_1023_player_0_ability_code_1822_player_0_diff_diff', 'ability_code_1023_player_1_ability_code_1E60_player_1_diff', 'ability_code_1023_player_1_ability_code_480_player_1_diff', 'ability_code_1023_player_1_ability_code_480_player_1_diff_ability_code_1023_player_0_ability_code_480_player_0_diff_diff', 'ability_code_1024_diff', 'ability_code_1024_first_time_diff', 'ability_code_1024_first_time_player_0', 'ability_code_1024_first_time_player_1', 'ability_code_1024_player_0', 'ability_code_1024_player_1', 'ability_code_1025_first_time_diff', 'ability_code_1025_first_time_player_1', 'ability_code_1025_player_0', 'ability_code_1025_player_1', 'ability_code_1026_diff', 'ability_code_1026_first_time_diff', 'ability_code_1026_first_time_player_0', 'ability_code_1026_first_time_player_1', 'ability_code_1026_player_0', 'ability_code_1026_player_1', 'ability_code_1028_diff', 'ability_code_1028_player_0', 'ability_code_1028_player_1', 'ability_code_1029_diff', 'ability_code_1029_player_0', 'ability_code_1029_player_1', 'ability_code_102A_diff', 'ability_code_102A_player_0', 'ability_code_102A_player_0_ability_code_480_player_0_diff', 'ability_code_102A_player_1', 'ability_code_102A_player_1_ability_code_1261_player_1_diff', 'ability_code_102A_player_1_ability_code_1261_player_1_diff_ability_code_102A_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_102A_player_1_ability_code_480_player_1_diff', 'ability_code_102A_player_1_ability_code_480_player_1_diff_ability_code_102A_player_0_ability_code_480_player_0_diff_diff', 'ability_code_102B_diff', 'ability_code_102B_first_time_player_0', 'ability_code_102B_player_0', 'ability_code_102B_player_1', 'ability_code_102D_first_time_diff', 'ability_code_102D_first_time_player_1', 'ability_code_102D_player_0', 'ability_code_102D_player_1', 'ability_code_102F_player_0', 'ability_code_102F_player_1', 'ability_code_103E_diff', 'ability_code_103E_player_0', 'ability_code_103E_player_1', 'ability_code_1060_player_0', 'ability_code_1060_player_1', 'ability_code_1080_diff', 'ability_code_1080_player_0', 'ability_code_1080_player_1', 'ability_code_1081_diff', 'ability_code_1081_player_0', 'ability_code_1081_player_1', 'ability_code_10E0_first_time_diff', 'ability_code_10E0_first_time_player_1', 'ability_code_10E0_player_1', 'ability_code_1100_diff', 'ability_code_1100_player_0', 'ability_code_1100_player_1', 'ability_code_1120_diff', 'ability_code_1120_player_0', 'ability_code_1120_player_1', 'ability_code_1121_diff', 'ability_code_1121_player_0', 'ability_code_1121_player_1', 'ability_code_1140_diff', 'ability_code_1140_player_0', 'ability_code_1140_player_1', 'ability_code_1142_diff', 'ability_code_1142_first_time_diff', 'ability_code_1142_first_time_player_1', 'ability_code_1142_player_0', 'ability_code_1142_player_1', 'ability_code_1143_diff', 'ability_code_1143_player_0', 'ability_code_1143_player_1', 'ability_code_1160_diff', 'ability_code_1160_first_time_diff', 'ability_code_1160_first_time_player_1', 'ability_code_1160_player_0', 'ability_code_1160_player_1', 'ability_code_1180_diff', 'ability_code_1180_player_0', 'ability_code_1180_player_1', 'ability_code_11A0_diff', 'ability_code_11A0_player_0', 'ability_code_11A0_player_1', 'ability_code_11C0_diff', 'ability_code_11C0_player_0', 'ability_code_11C0_player_1', 'ability_code_11E1_player_0', 'ability_code_11E1_player_1', 'ability_code_11E3_diff', 'ability_code_11E3_player_0', 'ability_code_11E3_player_1', 'ability_code_1201_diff', 'ability_code_1201_player_0', 'ability_code_1201_player_1', 'ability_code_1203_diff', 'ability_code_1203_player_0', 'ability_code_1203_player_1', 'ability_code_1204_diff', 'ability_code_1204_player_0', 'ability_code_1204_player_1', 'ability_code_1220_diff', 'ability_code_1220_player_0', 'ability_code_1220_player_1', 'ability_code_1240_diff', 'ability_code_1240_player_0', 'ability_code_1240_player_1', 'ability_code_1260_diff', 'ability_code_1260_first_time_player_1', 'ability_code_1260_player_0', 'ability_code_1260_player_1', 'ability_code_1261_diff', 'ability_code_1261_first_time_diff', 'ability_code_1261_player_0', 'ability_code_1261_player_0_ability_code_480_player_0_diff', 'ability_code_1261_player_1', 'ability_code_1261_player_1_ability_code_480_player_1_diff', 'ability_code_1261_player_1_ability_code_480_player_1_diff_ability_code_1261_player_0_ability_code_480_player_0_diff_diff', 'ability_code_127E_diff', 'ability_code_127E_player_0', 'ability_code_127E_player_1', 'ability_code_1280_diff', 'ability_code_1280_first_time_diff', 'ability_code_1280_first_time_player_0', 'ability_code_1280_first_time_player_1', 'ability_code_1280_player_0', 'ability_code_1280_player_1', 'ability_code_12A0_diff', 'ability_code_12A0_first_time_diff', 'ability_code_12A0_player_0', 'ability_code_12A0_player_1', 'ability_code_12A1_diff', 'ability_code_12A1_first_time_diff', 'ability_code_12A1_first_time_player_1', 'ability_code_12A1_player_0', 'ability_code_12A1_player_1', 'ability_code_12BE_diff', 'ability_code_12BE_player_0', 'ability_code_12BE_player_1', 'ability_code_12C0_diff', 'ability_code_12C0_first_time_diff', 'ability_code_12C0_first_time_player_1', 'ability_code_12C0_player_0', 'ability_code_12C0_player_1', 'ability_code_12E0_diff', 'ability_code_12E0_player_0', 'ability_code_12E0_player_1', 'ability_code_12E1_diff', 'ability_code_12E1_first_time_diff', 'ability_code_12E1_player_0', 'ability_code_12E1_player_1', 'ability_code_12FE_diff', 'ability_code_12FE_player_0', 'ability_code_12FE_player_1', 'ability_code_1300_diff', 'ability_code_1300_first_time_diff', 'ability_code_1300_first_time_player_0', 'ability_code_1300_first_time_player_1', 'ability_code_1300_player_0', 'ability_code_1300_player_1', 'ability_code_1320_diff', 'ability_code_1320_first_time_diff', 'ability_code_1320_first_time_player_0', 'ability_code_1320_first_time_player_1', 'ability_code_1320_player_0', 'ability_code_1320_player_1', 'ability_code_1340_first_time_diff', 'ability_code_1340_first_time_player_0', 'ability_code_1340_first_time_player_1', 'ability_code_1340_player_0', 'ability_code_1340_player_1', 'ability_code_1360_diff', 'ability_code_1360_first_time_player_0', 'ability_code_1360_last_first_player_0', 'ability_code_1360_player_0_ability_code_1020_player_0_diff', 'ability_code_1360_player_0_ability_code_1021_player_0_diff', 'ability_code_1360_player_0_ability_code_1022_player_0_diff', 'ability_code_1360_player_0_ability_code_102A_player_0_diff', 'ability_code_1360_player_0_ability_code_1261_player_0_diff', 'ability_code_1360_player_0_ability_code_13A0_player_0_diff', 'ability_code_1360_player_0_ability_code_13E0_player_0_diff', 'ability_code_1360_player_0_ability_code_15E0_player_0_diff', 'ability_code_1360_player_0_ability_code_16E2_player_0_diff', 'ability_code_1360_player_0_ability_code_1821_player_0_diff', 'ability_code_1360_player_0_ability_code_1822_player_0_diff', 'ability_code_1360_player_0_ability_code_1E60_player_0_diff', 'ability_code_1360_player_0_ability_code_480_player_0_diff', 'ability_code_1360_player_0_ability_code_DE0_player_0_diff', 'ability_code_1360_player_1', 'ability_code_1360_player_1_ability_code_1020_player_1_diff', 'ability_code_1360_player_1_ability_code_1020_player_1_diff_ability_code_1360_player_0_ability_code_1020_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_1021_player_1_diff', 'ability_code_1360_player_1_ability_code_1021_player_1_diff_ability_code_1360_player_0_ability_code_1021_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_1022_player_1_diff', 'ability_code_1360_player_1_ability_code_1022_player_1_diff_ability_code_1360_player_0_ability_code_1022_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_1023_player_1_diff', 'ability_code_1360_player_1_ability_code_102A_player_1_diff', 'ability_code_1360_player_1_ability_code_102A_player_1_diff_ability_code_1360_player_0_ability_code_102A_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_1261_player_1_diff', 'ability_code_1360_player_1_ability_code_13A0_player_1_diff', 'ability_code_1360_player_1_ability_code_13A0_player_1_diff_ability_code_1360_player_0_ability_code_13A0_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_13E0_player_1_diff_ability_code_1360_player_0_ability_code_13E0_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_15E0_player_1_diff', 'ability_code_1360_player_1_ability_code_15E0_player_1_diff_ability_code_1360_player_0_ability_code_15E0_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_16E2_player_1_diff', 'ability_code_1360_player_1_ability_code_1820_player_1_diff', 'ability_code_1360_player_1_ability_code_1821_player_1_diff', 'ability_code_1360_player_1_ability_code_1821_player_1_diff_ability_code_1360_player_0_ability_code_1821_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_1822_player_1_diff', 'ability_code_1360_player_1_ability_code_1C60_player_1_diff', 'ability_code_1360_player_1_ability_code_1C60_player_1_diff_ability_code_1360_player_0_ability_code_1C60_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_1E60_player_1_diff', 'ability_code_1360_player_1_ability_code_1E60_player_1_diff_ability_code_1360_player_0_ability_code_1E60_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_B40_player_1_diff', 'ability_code_1360_player_1_ability_code_B40_player_1_diff_ability_code_1360_player_0_ability_code_B40_player_0_diff_diff', 'ability_code_1360_player_1_ability_code_DE0_player_1_diff', 'ability_code_1360_player_1_ability_code_DE0_player_1_diff_ability_code_1360_player_0_ability_code_DE0_player_0_diff_diff', 'ability_code_1380_diff', 'ability_code_1380_first_time_diff', 'ability_code_1380_first_time_player_0', 'ability_code_1380_first_time_player_1', 'ability_code_1380_player_0', 'ability_code_1380_player_1', 'ability_code_13A0_diff', 'ability_code_13A0_first_time_diff', 'ability_code_13A0_first_time_player_0', 'ability_code_13A0_first_time_player_1', 'ability_code_13A0_player_0', 'ability_code_13A0_player_0_ability_code_1020_player_0_diff', 'ability_code_13A0_player_0_ability_code_102A_player_0_diff', 'ability_code_13A0_player_0_ability_code_1261_player_0_diff', 'ability_code_13A0_player_0_ability_code_15E0_player_0_diff', 'ability_code_13A0_player_0_ability_code_16E2_player_0_diff', 'ability_code_13A0_player_0_ability_code_1821_player_0_diff', 'ability_code_13A0_player_0_ability_code_1C60_player_0_diff', 'ability_code_13A0_player_0_ability_code_1E60_player_0_diff', 'ability_code_13A0_player_1', 'ability_code_13A0_player_1_ability_code_1020_player_1_diff', 'ability_code_13A0_player_1_ability_code_1020_player_1_diff_ability_code_13A0_player_0_ability_code_1020_player_0_diff_diff', 'ability_code_13A0_player_1_ability_code_102A_player_1_diff', 'ability_code_13A0_player_1_ability_code_102A_player_1_diff_ability_code_13A0_player_0_ability_code_102A_player_0_diff_diff', 'ability_code_13A0_player_1_ability_code_1261_player_1_diff', 'ability_code_13A0_player_1_ability_code_1261_player_1_diff_ability_code_13A0_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_13A0_player_1_ability_code_15E0_player_1_diff', 'ability_code_13A0_player_1_ability_code_16E2_player_1_diff', 'ability_code_13A0_player_1_ability_code_1821_player_1_diff', 'ability_code_13A0_player_1_ability_code_1821_player_1_diff_ability_code_13A0_player_0_ability_code_1821_player_0_diff_diff', 'ability_code_13A0_player_1_ability_code_1C60_player_1_diff', 'ability_code_13A0_player_1_ability_code_1E60_player_1_diff', 'ability_code_13A0_player_1_ability_code_1E60_player_1_diff_ability_code_13A0_player_0_ability_code_1E60_player_0_diff_diff', 'ability_code_13A0_player_1_ability_code_480_player_1_diff', 'ability_code_13A0_player_1_ability_code_480_player_1_diff_ability_code_13A0_player_0_ability_code_480_player_0_diff_diff', 'ability_code_13C0_diff', 'ability_code_13C0_first_time_diff', 'ability_code_13C0_first_time_player_0', 'ability_code_13C0_first_time_player_1', 'ability_code_13C0_player_0', 'ability_code_13C0_player_1', 'ability_code_13E0_diff', 'ability_code_13E0_first_time_diff', 'ability_code_13E0_last_first_diff', 'ability_code_13E0_last_first_player_0', 'ability_code_13E0_player_0', 'ability_code_13E0_player_0_ability_code_1020_player_0_diff', 'ability_code_13E0_player_0_ability_code_1021_player_0_diff', 'ability_code_13E0_player_0_ability_code_1022_player_0_diff', 'ability_code_13E0_player_0_ability_code_1261_player_0_diff', 'ability_code_13E0_player_0_ability_code_13A0_player_0_diff', 'ability_code_13E0_player_0_ability_code_15E0_player_0_diff', 'ability_code_13E0_player_0_ability_code_16E2_player_0_diff', 'ability_code_13E0_player_0_ability_code_1821_player_0_diff', 'ability_code_13E0_player_0_ability_code_1822_player_0_diff', 'ability_code_13E0_player_0_ability_code_1E60_player_0_diff', 'ability_code_13E0_player_0_ability_code_DE0_player_0_diff', 'ability_code_13E0_player_1', 'ability_code_13E0_player_1_ability_code_1020_player_1_diff', 'ability_code_13E0_player_1_ability_code_1021_player_1_diff', 'ability_code_13E0_player_1_ability_code_1021_player_1_diff_ability_code_13E0_player_0_ability_code_1021_player_0_diff_diff', 'ability_code_13E0_player_1_ability_code_1022_player_1_diff', 'ability_code_13E0_player_1_ability_code_1023_player_1_diff', 'ability_code_13E0_player_1_ability_code_102A_player_1_diff', 'ability_code_13E0_player_1_ability_code_1261_player_1_diff', 'ability_code_13E0_player_1_ability_code_1261_player_1_diff_ability_code_13E0_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_13E0_player_1_ability_code_13A0_player_1_diff', 'ability_code_13E0_player_1_ability_code_15E0_player_1_diff', 'ability_code_13E0_player_1_ability_code_16E2_player_1_diff', 'ability_code_13E0_player_1_ability_code_16E2_player_1_diff_ability_code_13E0_player_0_ability_code_16E2_player_0_diff_diff', 'ability_code_13E0_player_1_ability_code_1821_player_1_diff', 'ability_code_13E0_player_1_ability_code_1821_player_1_diff_ability_code_13E0_player_0_ability_code_1821_player_0_diff_diff', 'ability_code_13E0_player_1_ability_code_1822_player_1_diff', 'ability_code_13E0_player_1_ability_code_1822_player_1_diff_ability_code_13E0_player_0_ability_code_1822_player_0_diff_diff', 'ability_code_13E0_player_1_ability_code_1C60_player_1_diff', 'ability_code_13E0_player_1_ability_code_1E60_player_1_diff', 'ability_code_13E0_player_1_ability_code_480_player_1_diff', 'ability_code_13E0_player_1_ability_code_480_player_1_diff_ability_code_13E0_player_0_ability_code_480_player_0_diff_diff', 'ability_code_13E0_player_1_ability_code_B40_player_1_diff', 'ability_code_13E0_player_1_ability_code_B40_player_1_diff_ability_code_13E0_player_0_ability_code_B40_player_0_diff_diff', 'ability_code_13E0_player_1_ability_code_DE0_player_1_diff', 'ability_code_13E0_player_1_ability_code_DE0_player_1_diff_ability_code_13E0_player_0_ability_code_DE0_player_0_diff_diff', 'ability_code_13E1_diff', 'ability_code_13E1_player_0', 'ability_code_13E1_player_1', 'ability_code_13E2_diff', 'ability_code_13E2_player_0', 'ability_code_13E2_player_1', 'ability_code_13E3_diff', 'ability_code_13E3_first_time_diff', 'ability_code_13E3_first_time_player_0', 'ability_code_13E3_player_0', 'ability_code_13E3_player_1', 'ability_code_1401_first_time_diff', 'ability_code_1401_player_0', 'ability_code_1401_player_1', 'ability_code_1404_diff', 'ability_code_1404_player_0', 'ability_code_1404_player_1', 'ability_code_1405_diff', 'ability_code_1405_first_time_diff', 'ability_code_1405_first_time_player_1', 'ability_code_1405_player_0', 'ability_code_1405_player_1', 'ability_code_1406_diff', 'ability_code_1406_player_0', 'ability_code_1406_player_1', 'ability_code_1407_diff', 'ability_code_1407_first_time_diff', 'ability_code_1407_first_time_player_1', 'ability_code_1407_player_0', 'ability_code_1407_player_1', 'ability_code_1418_diff', 'ability_code_1418_first_time_diff', 'ability_code_1418_first_time_player_0', 'ability_code_1418_player_0', 'ability_code_1418_player_1', 'ability_code_1420_first_time_diff', 'ability_code_1420_first_time_player_1', 'ability_code_1420_player_0', 'ability_code_1420_player_1', 'ability_code_1421_diff', 'ability_code_1421_player_1', 'ability_code_1422_diff', 'ability_code_1422_player_1', 'ability_code_1423_diff', 'ability_code_1423_player_0', 'ability_code_1423_player_1', 'ability_code_1424_diff', 'ability_code_1424_first_time_diff', 'ability_code_1424_first_time_player_1', 'ability_code_1424_player_0', 'ability_code_1424_player_1', 'ability_code_1426_player_0', 'ability_code_1426_player_1', 'ability_code_1440_diff', 'ability_code_1440_player_0', 'ability_code_1440_player_1', 'ability_code_1441_diff', 'ability_code_1441_player_0', 'ability_code_1441_player_1', 'ability_code_1442_diff', 'ability_code_1442_first_time_diff', 'ability_code_1442_player_0', 'ability_code_1442_player_1', 'ability_code_1443_diff', 'ability_code_1443_player_0', 'ability_code_1443_player_1', 'ability_code_1445_diff', 'ability_code_1445_player_0', 'ability_code_1445_player_1', 'ability_code_1446_first_time_diff', 'ability_code_1446_first_time_player_1', 'ability_code_1446_player_0', 'ability_code_1446_player_1', 'ability_code_1447_diff', 'ability_code_1447_player_0', 'ability_code_1447_player_1', 'ability_code_1480_diff', 'ability_code_1480_player_0', 'ability_code_1480_player_1', 'ability_code_14A0_diff', 'ability_code_14A0_player_0', 'ability_code_14A0_player_1', 'ability_code_14A1_diff', 'ability_code_14A1_first_time_diff', 'ability_code_14A1_player_0', 'ability_code_14A1_player_1', 'ability_code_14A2_diff', 'ability_code_14A2_player_0', 'ability_code_14A2_player_1', 'ability_code_14C1_diff', 'ability_code_14C1_player_0', 'ability_code_14C1_player_1', 'ability_code_14C4_diff', 'ability_code_14C4_player_0', 'ability_code_14C4_player_1', 'ability_code_14E0_diff', 'ability_code_14E0_player_0', 'ability_code_14E0_player_1', 'ability_code_14E3_diff', 'ability_code_14E3_player_0', 'ability_code_14E3_player_1', 'ability_code_14E9_diff', 'ability_code_14E9_player_0', 'ability_code_14E9_player_1', 'ability_code_14ED_diff', 'ability_code_14ED_player_0', 'ability_code_14ED_player_1', 'ability_code_14EE_diff', 'ability_code_14EE_player_0', 'ability_code_14EE_player_1', 'ability_code_14EF_diff', 'ability_code_14EF_player_0', 'ability_code_14EF_player_1', 'ability_code_1500_diff', 'ability_code_1500_player_0', 'ability_code_1500_player_1', 'ability_code_1525_diff', 'ability_code_1525_player_0', 'ability_code_1525_player_1', 'ability_code_1526_diff', 'ability_code_1526_player_0', 'ability_code_1526_player_1', 'ability_code_152B_diff', 'ability_code_152B_player_0', 'ability_code_152B_player_1', 'ability_code_152E_player_0', 'ability_code_152E_player_1', 'ability_code_152F_diff', 'ability_code_152F_player_0', 'ability_code_152F_player_1', 'ability_code_1540_first_time_diff', 'ability_code_1540_first_time_player_0', 'ability_code_1540_player_0', 'ability_code_1540_player_1', 'ability_code_1541_first_time_diff', 'ability_code_1541_first_time_player_1', 'ability_code_1541_player_0', 'ability_code_1541_player_1', 'ability_code_1542_first_time_diff', 'ability_code_1542_first_time_player_0', 'ability_code_1542_player_0', 'ability_code_1543_first_time_diff', 'ability_code_1543_first_time_player_0', 'ability_code_1544_diff', 'ability_code_1544_first_time_diff', 'ability_code_1544_first_time_player_0', 'ability_code_1544_first_time_player_1', 'ability_code_1544_player_0', 'ability_code_1544_player_1', 'ability_code_1545_player_0', 'ability_code_1545_player_1', 'ability_code_1546_diff', 'ability_code_1546_first_time_diff', 'ability_code_1546_first_time_player_0', 'ability_code_1546_player_0', 'ability_code_1546_player_1', 'ability_code_1547_diff', 'ability_code_1547_first_time_diff', 'ability_code_1547_player_0', 'ability_code_1549_first_time_diff', 'ability_code_1549_player_0', 'ability_code_1549_player_1', 'ability_code_154A_player_0', 'ability_code_154A_player_1', 'ability_code_154B_diff', 'ability_code_154B_player_0', 'ability_code_154B_player_1', 'ability_code_154C_diff', 'ability_code_154C_player_0', 'ability_code_154C_player_1', 'ability_code_154D_diff', 'ability_code_154D_first_time_diff', 'ability_code_154D_first_time_player_0', 'ability_code_154D_player_0', 'ability_code_154D_player_1', 'ability_code_154E_diff', 'ability_code_154E_first_time_diff', 'ability_code_154E_first_time_player_0', 'ability_code_154E_player_0', 'ability_code_154E_player_1', 'ability_code_1562_diff', 'ability_code_1562_player_0', 'ability_code_1562_player_1', 'ability_code_1563_diff', 'ability_code_1563_player_0', 'ability_code_1563_player_1', 'ability_code_1580_diff', 'ability_code_1580_first_time_diff', 'ability_code_1580_first_time_player_0', 'ability_code_1580_player_0', 'ability_code_1580_player_1', 'ability_code_1581_diff', 'ability_code_1581_first_time_diff', 'ability_code_1581_first_time_player_0', 'ability_code_1581_first_time_player_1', 'ability_code_1581_player_0', 'ability_code_1581_player_1', 'ability_code_1583_diff', 'ability_code_1583_player_1', 'ability_code_1584_diff', 'ability_code_1584_player_0', 'ability_code_1584_player_1', 'ability_code_1585_diff', 'ability_code_1585_player_0', 'ability_code_1585_player_1', 'ability_code_1586_diff', 'ability_code_1586_first_time_player_0', 'ability_code_1586_player_0', 'ability_code_15A0_diff', 'ability_code_15A0_player_0', 'ability_code_15A0_player_1', 'ability_code_15A2_player_0', 'ability_code_15A2_player_1', 'ability_code_15A4_player_0', 'ability_code_15A8_diff', 'ability_code_15A8_first_time_diff', 'ability_code_15A8_player_0', 'ability_code_15A8_player_1', 'ability_code_15A9_diff', 'ability_code_15A9_player_0', 'ability_code_15A9_player_1', 'ability_code_15C0_diff', 'ability_code_15C0_player_0', 'ability_code_15C0_player_1', 'ability_code_15C1_diff', 'ability_code_15C1_first_time_diff', 'ability_code_15C1_first_time_player_0', 'ability_code_15C1_first_time_player_1', 'ability_code_15C1_player_0', 'ability_code_15C1_player_1', 'ability_code_15C2_diff', 'ability_code_15C2_player_0', 'ability_code_15C2_player_1', 'ability_code_15C3_diff', 'ability_code_15C3_first_time_diff', 'ability_code_15C3_first_time_player_0', 'ability_code_15C3_player_1', 'ability_code_15D2_diff', 'ability_code_15D2_player_0', 'ability_code_15D2_player_1', 'ability_code_15E0_diff', 'ability_code_15E0_player_0', 'ability_code_15E0_player_0_ability_code_1020_player_0_diff', 'ability_code_15E0_player_0_ability_code_102A_player_0_diff', 'ability_code_15E0_player_0_ability_code_1261_player_0_diff', 'ability_code_15E0_player_0_ability_code_16E2_player_0_diff', 'ability_code_15E0_player_0_ability_code_1C60_player_0_diff', 'ability_code_15E0_player_0_ability_code_1E60_player_0_diff', 'ability_code_15E0_player_0_ability_code_480_player_0_diff', 'ability_code_15E0_player_1', 'ability_code_15E0_player_1_ability_code_1020_player_1_diff', 'ability_code_15E0_player_1_ability_code_1020_player_1_diff_ability_code_15E0_player_0_ability_code_1020_player_0_diff_diff', 'ability_code_15E0_player_1_ability_code_102A_player_1_diff', 'ability_code_15E0_player_1_ability_code_1261_player_1_diff', 'ability_code_15E0_player_1_ability_code_1261_player_1_diff_ability_code_15E0_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_15E0_player_1_ability_code_16E2_player_1_diff', 'ability_code_15E0_player_1_ability_code_16E2_player_1_diff_ability_code_15E0_player_0_ability_code_16E2_player_0_diff_diff', 'ability_code_15E0_player_1_ability_code_1C60_player_1_diff', 'ability_code_15E0_player_1_ability_code_1C60_player_1_diff_ability_code_15E0_player_0_ability_code_1C60_player_0_diff_diff', 'ability_code_15E0_player_1_ability_code_1E60_player_1_diff', 'ability_code_15E0_player_1_ability_code_1E60_player_1_diff_ability_code_15E0_player_0_ability_code_1E60_player_0_diff_diff', 'ability_code_1600_diff', 'ability_code_1600_player_1', 'ability_code_1680_diff', 'ability_code_1680_player_0', 'ability_code_1680_player_1', 'ability_code_1681_diff', 'ability_code_1681_player_0', 'ability_code_1681_player_1', 'ability_code_1683_diff', 'ability_code_1683_player_0', 'ability_code_1684_diff', 'ability_code_1684_player_0', 'ability_code_1684_player_1', 'ability_code_1686_diff', 'ability_code_1686_player_0', 'ability_code_1686_player_1', 'ability_code_1687_diff', 'ability_code_1687_player_0', 'ability_code_1687_player_1', 'ability_code_16A1_diff', 'ability_code_16A1_player_0', 'ability_code_16A1_player_1', 'ability_code_16A2_diff', 'ability_code_16A2_player_0', 'ability_code_16A2_player_1', 'ability_code_16A5_diff', 'ability_code_16A5_player_0', 'ability_code_16A5_player_1', 'ability_code_16C4_diff', 'ability_code_16C4_player_0', 'ability_code_16C4_player_1', 'ability_code_16E0_first_time_diff', 'ability_code_16E0_first_time_player_0', 'ability_code_16E0_player_0', 'ability_code_16E0_player_1', 'ability_code_16E2_diff', 'ability_code_16E2_player_0', 'ability_code_16E2_player_0_ability_code_102A_player_0_diff', 'ability_code_16E2_player_0_ability_code_1261_player_0_diff', 'ability_code_16E2_player_0_ability_code_480_player_0_diff', 'ability_code_16E2_player_1', 'ability_code_16E2_player_1_ability_code_102A_player_1_diff', 'ability_code_16E2_player_1_ability_code_102A_player_1_diff_ability_code_16E2_player_0_ability_code_102A_player_0_diff_diff', 'ability_code_16E2_player_1_ability_code_1261_player_1_diff', 'ability_code_16E2_player_1_ability_code_1261_player_1_diff_ability_code_16E2_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_16E2_player_1_ability_code_480_player_1_diff', 'ability_code_16E3_diff', 'ability_code_16E3_first_time_diff', 'ability_code_16E3_first_time_player_1', 'ability_code_16E3_player_0', 'ability_code_16E3_player_1', 'ability_code_16E4_diff', 'ability_code_16E4_first_time_diff', 'ability_code_16E4_first_time_player_1', 'ability_code_16E4_player_0', 'ability_code_16E4_player_1', 'ability_code_16E5_diff', 'ability_code_16E5_player_0', 'ability_code_16E5_player_1', 'ability_code_16E6_player_0', 'ability_code_16E6_player_1', 'ability_code_16E7_diff', 'ability_code_16E7_player_0', 'ability_code_16E7_player_1', 'ability_code_16E8_player_0', 'ability_code_16E8_player_1', 'ability_code_16E9_diff', 'ability_code_16E9_player_0', 'ability_code_16E9_player_1', 'ability_code_16EA_diff', 'ability_code_16EA_first_time_diff', 'ability_code_16EA_player_0', 'ability_code_16EA_player_1', 'ability_code_16ED_diff', 'ability_code_16ED_first_time_diff', 'ability_code_16ED_player_0', 'ability_code_16ED_player_1', 'ability_code_16EE_diff', 'ability_code_16EE_first_time_diff', 'ability_code_16EE_player_0', 'ability_code_16EE_player_1', 'ability_code_16EF_diff', 'ability_code_16EF_first_time_diff', 'ability_code_16EF_first_time_player_0', 'ability_code_16EF_player_0', 'ability_code_16EF_player_1', 'ability_code_1700_diff', 'ability_code_1700_player_0', 'ability_code_1700_player_1', 'ability_code_1701_diff', 'ability_code_1701_player_0', 'ability_code_1701_player_1', 'ability_code_1720_diff', 'ability_code_1720_player_0', 'ability_code_1720_player_1', 'ability_code_1721_diff', 'ability_code_1721_player_0', 'ability_code_1723_first_time_diff', 'ability_code_1723_first_time_player_0', 'ability_code_1723_player_0', 'ability_code_1723_player_1', 'ability_code_1724_diff', 'ability_code_1724_player_0', 'ability_code_1724_player_1', 'ability_code_1726_player_0', 'ability_code_1726_player_1', 'ability_code_1727_diff', 'ability_code_1727_player_0', 'ability_code_1727_player_1', 'ability_code_1740_first_time_player_1', 'ability_code_1740_player_0', 'ability_code_1740_player_1', 'ability_code_1741_diff', 'ability_code_1741_player_0', 'ability_code_1741_player_1', 'ability_code_1760_player_0', 'ability_code_17A1_player_0', 'ability_code_17A1_player_1', 'ability_code_17A3_player_0', 'ability_code_17A3_player_1', 'ability_code_17C1_first_time_diff', 'ability_code_17C1_player_0', 'ability_code_17C1_player_1', 'ability_code_17E1_diff', 'ability_code_17E1_player_0', 'ability_code_17E1_player_1', 'ability_code_1800_diff', 'ability_code_1800_player_0', 'ability_code_1800_player_1', 'ability_code_1803_diff', 'ability_code_1803_player_0', 'ability_code_1803_player_1', 'ability_code_1820_diff', 'ability_code_1820_first_time_player_0', 'ability_code_1820_first_time_player_1', 'ability_code_1820_player_0', 'ability_code_1820_player_0_ability_code_1020_player_0_diff', 'ability_code_1820_player_0_ability_code_1021_player_0_diff', 'ability_code_1820_player_0_ability_code_1022_player_0_diff', 'ability_code_1820_player_0_ability_code_1023_player_0_diff', 'ability_code_1820_player_0_ability_code_102A_player_0_diff', 'ability_code_1820_player_0_ability_code_1261_player_0_diff', 'ability_code_1820_player_0_ability_code_13A0_player_0_diff', 'ability_code_1820_player_0_ability_code_13E0_player_0_diff', 'ability_code_1820_player_0_ability_code_15E0_player_0_diff', 'ability_code_1820_player_0_ability_code_1822_player_0_diff', 'ability_code_1820_player_0_ability_code_1C60_player_0_diff', 'ability_code_1820_player_0_ability_code_480_player_0_diff', 'ability_code_1820_player_0_ability_code_DE0_player_0_diff', 'ability_code_1820_player_1', 'ability_code_1820_player_1_ability_code_1020_player_1_diff', 'ability_code_1820_player_1_ability_code_1020_player_1_diff_ability_code_1820_player_0_ability_code_1020_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_1021_player_1_diff', 'ability_code_1820_player_1_ability_code_1021_player_1_diff_ability_code_1820_player_0_ability_code_1021_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_1022_player_1_diff', 'ability_code_1820_player_1_ability_code_1022_player_1_diff_ability_code_1820_player_0_ability_code_1022_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_1023_player_1_diff', 'ability_code_1820_player_1_ability_code_1023_player_1_diff_ability_code_1820_player_0_ability_code_1023_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_102A_player_1_diff', 'ability_code_1820_player_1_ability_code_102A_player_1_diff_ability_code_1820_player_0_ability_code_102A_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_1261_player_1_diff', 'ability_code_1820_player_1_ability_code_1261_player_1_diff_ability_code_1820_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_13A0_player_1_diff', 'ability_code_1820_player_1_ability_code_13A0_player_1_diff_ability_code_1820_player_0_ability_code_13A0_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_13E0_player_1_diff', 'ability_code_1820_player_1_ability_code_13E0_player_1_diff_ability_code_1820_player_0_ability_code_13E0_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_15E0_player_1_diff', 'ability_code_1820_player_1_ability_code_15E0_player_1_diff_ability_code_1820_player_0_ability_code_15E0_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_16E2_player_1_diff', 'ability_code_1820_player_1_ability_code_16E2_player_1_diff_ability_code_1820_player_0_ability_code_16E2_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_1821_player_1_diff', 'ability_code_1820_player_1_ability_code_1C60_player_1_diff', 'ability_code_1820_player_1_ability_code_1C60_player_1_diff_ability_code_1820_player_0_ability_code_1C60_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_1E60_player_1_diff_ability_code_1820_player_0_ability_code_1E60_player_0_diff_diff', 'ability_code_1820_player_1_ability_code_480_player_1_diff', 'ability_code_1820_player_1_ability_code_B40_player_1_diff', 'ability_code_1820_player_1_ability_code_B40_player_1_diff_ability_code_1820_player_0_ability_code_B40_player_0_diff_diff', 'ability_code_1821_diff', 'ability_code_1821_first_time_diff', 'ability_code_1821_player_0', 'ability_code_1821_player_0_ability_code_1020_player_0_diff', 'ability_code_1821_player_0_ability_code_102A_player_0_diff', 'ability_code_1821_player_0_ability_code_1261_player_0_diff', 'ability_code_1821_player_0_ability_code_15E0_player_0_diff', 'ability_code_1821_player_0_ability_code_1C60_player_0_diff', 'ability_code_1821_player_1', 'ability_code_1821_player_1_ability_code_1020_player_1_diff', 'ability_code_1821_player_1_ability_code_102A_player_1_diff', 'ability_code_1821_player_1_ability_code_102A_player_1_diff_ability_code_1821_player_0_ability_code_102A_player_0_diff_diff', 'ability_code_1821_player_1_ability_code_1261_player_1_diff', 'ability_code_1821_player_1_ability_code_1261_player_1_diff_ability_code_1821_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_1821_player_1_ability_code_15E0_player_1_diff', 'ability_code_1821_player_1_ability_code_16E2_player_1_diff', 'ability_code_1821_player_1_ability_code_16E2_player_1_diff_ability_code_1821_player_0_ability_code_16E2_player_0_diff_diff', 'ability_code_1821_player_1_ability_code_1C60_player_1_diff', 'ability_code_1821_player_1_ability_code_1E60_player_1_diff', 'ability_code_1821_player_1_ability_code_480_player_1_diff', 'ability_code_1822_diff', 'ability_code_1822_player_0', 'ability_code_1822_player_0_ability_code_1020_player_0_diff', 'ability_code_1822_player_0_ability_code_102A_player_0_diff', 'ability_code_1822_player_0_ability_code_1261_player_0_diff', 'ability_code_1822_player_0_ability_code_13A0_player_0_diff', 'ability_code_1822_player_0_ability_code_15E0_player_0_diff', 'ability_code_1822_player_0_ability_code_16E2_player_0_diff', 'ability_code_1822_player_0_ability_code_1821_player_0_diff', 'ability_code_1822_player_0_ability_code_1C60_player_0_diff', 'ability_code_1822_player_0_ability_code_1E60_player_0_diff', 'ability_code_1822_player_0_ability_code_480_player_0_diff', 'ability_code_1822_player_1', 'ability_code_1822_player_1_ability_code_1020_player_1_diff', 'ability_code_1822_player_1_ability_code_102A_player_1_diff', 'ability_code_1822_player_1_ability_code_102A_player_1_diff_ability_code_1822_player_0_ability_code_102A_player_0_diff_diff', 'ability_code_1822_player_1_ability_code_1261_player_1_diff', 'ability_code_1822_player_1_ability_code_1261_player_1_diff_ability_code_1822_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_1822_player_1_ability_code_13A0_player_1_diff', 'ability_code_1822_player_1_ability_code_15E0_player_1_diff', 'ability_code_1822_player_1_ability_code_15E0_player_1_diff_ability_code_1822_player_0_ability_code_15E0_player_0_diff_diff', 'ability_code_1822_player_1_ability_code_16E2_player_1_diff', 'ability_code_1822_player_1_ability_code_16E2_player_1_diff_ability_code_1822_player_0_ability_code_16E2_player_0_diff_diff', 'ability_code_1822_player_1_ability_code_1821_player_1_diff', 'ability_code_1822_player_1_ability_code_1C60_player_1_diff', 'ability_code_1822_player_1_ability_code_1C60_player_1_diff_ability_code_1822_player_0_ability_code_1C60_player_0_diff_diff', 'ability_code_1822_player_1_ability_code_1E60_player_1_diff', 'ability_code_1822_player_1_ability_code_1E60_player_1_diff_ability_code_1822_player_0_ability_code_1E60_player_0_diff_diff', 'ability_code_1822_player_1_ability_code_480_player_1_diff_ability_code_1822_player_0_ability_code_480_player_0_diff_diff', 'ability_code_1823_diff', 'ability_code_1823_player_0', 'ability_code_1823_player_1', 'ability_code_1824_diff', 'ability_code_1824_player_0', 'ability_code_1824_player_1', 'ability_code_1829_first_time_diff', 'ability_code_182A_diff', 'ability_code_182A_player_0', 'ability_code_182A_player_1', 'ability_code_182B_diff', 'ability_code_182B_player_0', 'ability_code_182B_player_1', 'ability_code_182E_diff', 'ability_code_182E_player_0', 'ability_code_182E_player_1', 'ability_code_1860_diff', 'ability_code_1860_player_0', 'ability_code_1860_player_1', 'ability_code_1880_diff', 'ability_code_1880_player_0', 'ability_code_1880_player_1', 'ability_code_18A0_diff', 'ability_code_18A0_player_0', 'ability_code_18A0_player_1', 'ability_code_18C0_diff', 'ability_code_18C0_player_0', 'ability_code_18C0_player_1', 'ability_code_1920_diff', 'ability_code_1920_player_0', 'ability_code_1920_player_1', 'ability_code_1940_diff', 'ability_code_1940_player_0', 'ability_code_1940_player_1', 'ability_code_1960_diff', 'ability_code_1960_player_0', 'ability_code_1960_player_1', 'ability_code_1980_diff', 'ability_code_1980_player_0', 'ability_code_1980_player_1', 'ability_code_1A62_diff', 'ability_code_1A62_player_0', 'ability_code_1A62_player_1', 'ability_code_1A63_diff', 'ability_code_1A63_player_0', 'ability_code_1A63_player_1', 'ability_code_1AC0_diff', 'ability_code_1AC0_player_0', 'ability_code_1AC0_player_1', 'ability_code_1AC1_first_time_diff', 'ability_code_1AC1_first_time_player_0', 'ability_code_1AC1_player_0', 'ability_code_1AC1_player_1', 'ability_code_1AC3_diff', 'ability_code_1AC3_player_0', 'ability_code_1AC3_player_1', 'ability_code_1AC4_diff', 'ability_code_1AC4_player_0', 'ability_code_1AC4_player_1', 'ability_code_1AC5_diff', 'ability_code_1AC5_player_0', 'ability_code_1AC5_player_1', 'ability_code_1AC6_diff', 'ability_code_1AC6_first_time_diff', 'ability_code_1AC6_first_time_player_0', 'ability_code_1AC6_player_0', 'ability_code_1AC6_player_1', 'ability_code_1AE0_diff', 'ability_code_1AE0_player_0', 'ability_code_1AE0_player_1', 'ability_code_1B00_diff', 'ability_code_1B00_player_0', 'ability_code_1B00_player_1', 'ability_code_1B21_diff', 'ability_code_1B21_player_0', 'ability_code_1B21_player_1', 'ability_code_1B40_diff', 'ability_code_1B40_player_0', 'ability_code_1B40_player_1', 'ability_code_1B60_diff', 'ability_code_1B60_player_1', 'ability_code_1BA0_diff', 'ability_code_1BA0_player_0', 'ability_code_1BA0_player_1', 'ability_code_1BA1_diff', 'ability_code_1BA1_player_0', 'ability_code_1BA1_player_1', 'ability_code_1BC0_player_0', 'ability_code_1BC0_player_1', 'ability_code_1BC1_diff', 'ability_code_1BC1_player_0', 'ability_code_1BC1_player_1', 'ability_code_1BE2_diff', 'ability_code_1BE2_player_0', 'ability_code_1BE2_player_1', 'ability_code_1C00_diff', 'ability_code_1C00_player_0', 'ability_code_1C00_player_1', 'ability_code_1C60_diff', 'ability_code_1C60_first_time_diff', 'ability_code_1C60_last_first_diff', 'ability_code_1C60_last_first_player_0', 'ability_code_1C60_player_0', 'ability_code_1C60_player_0_ability_code_1020_player_0_diff', 'ability_code_1C60_player_0_ability_code_102A_player_0_diff', 'ability_code_1C60_player_0_ability_code_1261_player_0_diff', 'ability_code_1C60_player_0_ability_code_16E2_player_0_diff', 'ability_code_1C60_player_0_ability_code_1E60_player_0_diff', 'ability_code_1C60_player_0_ability_code_480_player_0_diff', 'ability_code_1C60_player_1', 'ability_code_1C60_player_1_ability_code_1020_player_1_diff', 'ability_code_1C60_player_1_ability_code_1020_player_1_diff_ability_code_1C60_player_0_ability_code_1020_player_0_diff_diff', 'ability_code_1C60_player_1_ability_code_102A_player_1_diff', 'ability_code_1C60_player_1_ability_code_102A_player_1_diff_ability_code_1C60_player_0_ability_code_102A_player_0_diff_diff', 'ability_code_1C60_player_1_ability_code_1261_player_1_diff', 'ability_code_1C60_player_1_ability_code_16E2_player_1_diff', 'ability_code_1C60_player_1_ability_code_16E2_player_1_diff_ability_code_1C60_player_0_ability_code_16E2_player_0_diff_diff', 'ability_code_1C60_player_1_ability_code_1E60_player_1_diff', 'ability_code_1C60_player_1_ability_code_1E60_player_1_diff_ability_code_1C60_player_0_ability_code_1E60_player_0_diff_diff', 'ability_code_1C60_player_1_ability_code_480_player_1_diff', 'ability_code_1C60_player_1_ability_code_480_player_1_diff_ability_code_1C60_player_0_ability_code_480_player_0_diff_diff', 'ability_code_1C61_diff', 'ability_code_1C61_player_0', 'ability_code_1C61_player_1', 'ability_code_1C80_first_time_diff', 'ability_code_1C80_player_0', 'ability_code_1CA0_diff', 'ability_code_1CA0_player_0', 'ability_code_1CA0_player_1', 'ability_code_1CC0_diff', 'ability_code_1CC0_player_0', 'ability_code_1CC0_player_1', 'ability_code_1CE0_diff', 'ability_code_1CE0_player_0', 'ability_code_1CE0_player_1', 'ability_code_1D00_diff', 'ability_code_1D00_player_0', 'ability_code_1D00_player_1', 'ability_code_1D20_diff', 'ability_code_1D20_player_0', 'ability_code_1D20_player_1', 'ability_code_1D40_diff', 'ability_code_1D40_player_0', 'ability_code_1D40_player_1', 'ability_code_1D60_diff', 'ability_code_1D60_player_0', 'ability_code_1D60_player_1', 'ability_code_1D80_player_0', 'ability_code_1D81_diff', 'ability_code_1D81_player_0', 'ability_code_1D81_player_1', 'ability_code_1D83_diff', 'ability_code_1D83_player_0', 'ability_code_1D83_player_1', 'ability_code_1D86_diff', 'ability_code_1D86_first_time_player_0', 'ability_code_1D86_player_0', 'ability_code_1D86_player_1', 'ability_code_1DA0_player_0', 'ability_code_1DA0_player_1', 'ability_code_1DA1_diff', 'ability_code_1DA1_player_0', 'ability_code_1DA1_player_1', 'ability_code_1DA2_diff', 'ability_code_1DA2_player_0', 'ability_code_1DA2_player_1', 'ability_code_1DC0_diff', 'ability_code_1DC0_player_0', 'ability_code_1DC0_player_1', 'ability_code_1E20_diff', 'ability_code_1E20_player_0', 'ability_code_1E60_diff', 'ability_code_1E60_first_time_diff', 'ability_code_1E60_first_time_player_1', 'ability_code_1E60_player_0', 'ability_code_1E60_player_0_ability_code_102A_player_0_diff', 'ability_code_1E60_player_0_ability_code_1261_player_0_diff', 'ability_code_1E60_player_0_ability_code_16E2_player_0_diff', 'ability_code_1E60_player_0_ability_code_480_player_0_diff', 'ability_code_1E60_player_1', 'ability_code_1E60_player_1_ability_code_102A_player_1_diff', 'ability_code_1E60_player_1_ability_code_102A_player_1_diff_ability_code_1E60_player_0_ability_code_102A_player_0_diff_diff', 'ability_code_1E60_player_1_ability_code_1261_player_1_diff', 'ability_code_1E60_player_1_ability_code_1261_player_1_diff_ability_code_1E60_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_1E60_player_1_ability_code_16E2_player_1_diff', 'ability_code_1E60_player_1_ability_code_480_player_1_diff', 'ability_code_1EA0_diff', 'ability_code_1EA0_player_0', 'ability_code_1EA0_player_1', 'ability_code_1FC0_diff', 'ability_code_1FC0_player_0', 'ability_code_1FC0_player_1', 'ability_code_1FE0_diff', 'ability_code_1FE0_player_0', 'ability_code_1FE0_player_1', 'ability_code_2040_diff', 'ability_code_2040_player_0', 'ability_code_2040_player_1', 'ability_code_2060_diff', 'ability_code_2060_player_0', 'ability_code_2060_player_1', 'ability_code_2061_diff', 'ability_code_2061_player_0', 'ability_code_2061_player_1', 'ability_code_2080_first_time_diff', 'ability_code_2080_player_0', 'ability_code_20A0_player_0', 'ability_code_20A0_player_1', 'ability_code_20C0_diff', 'ability_code_20C0_player_0', 'ability_code_20C0_player_1', 'ability_code_20E0_diff', 'ability_code_20E0_player_0', 'ability_code_20E0_player_1', 'ability_code_20E1_diff', 'ability_code_20E1_player_0', 'ability_code_20E1_player_1', 'ability_code_2100_diff', 'ability_code_2100_player_0', 'ability_code_2100_player_1', 'ability_code_2101_diff', 'ability_code_2101_player_0', 'ability_code_2101_player_1', 'ability_code_2120_diff', 'ability_code_2120_first_time_diff', 'ability_code_2120_first_time_player_0', 'ability_code_2120_player_0', 'ability_code_2120_player_1', 'ability_code_213E_diff', 'ability_code_213E_player_0', 'ability_code_213E_player_1', 'ability_code_2140_diff', 'ability_code_2140_player_0', 'ability_code_2140_player_1', 'ability_code_2160_diff', 'ability_code_2160_player_0', 'ability_code_2160_player_1', 'ability_code_2180_diff', 'ability_code_2180_player_0', 'ability_code_2180_player_1', 'ability_code_22A0_diff', 'ability_code_22A0_first_time_diff', 'ability_code_22A0_first_time_player_0', 'ability_code_22A0_first_time_player_1', 'ability_code_22A0_player_0', 'ability_code_22A0_player_1', 'ability_code_22A1_player_0', 'ability_code_22A1_player_1', 'ability_code_26C0_diff', 'ability_code_26C0_player_0', 'ability_code_26C0_player_1', 'ability_code_2720_diff', 'ability_code_2720_first_time_diff', 'ability_code_2720_first_time_player_0', 'ability_code_2720_player_0', 'ability_code_2720_player_1', 'ability_code_2B00_diff', 'ability_code_2B00_player_0', 'ability_code_2B00_player_1', 'ability_code_2B40_diff', 'ability_code_2B40_player_0', 'ability_code_2B40_player_1', 'ability_code_2C80_diff', 'ability_code_2C80_player_0', 'ability_code_2C80_player_1', 'ability_code_2DE0_diff', 'ability_code_2DE0_player_0', 'ability_code_2DE0_player_1', 'ability_code_2F60_diff', 'ability_code_2F60_player_0', 'ability_code_2F60_player_1', 'ability_code_3040_diff', 'ability_code_3040_first_time_diff', 'ability_code_3040_first_time_player_0', 'ability_code_3040_first_time_player_1', 'ability_code_3040_player_0', 'ability_code_3040_player_1', 'ability_code_3060_diff', 'ability_code_3060_player_0', 'ability_code_3060_player_1', 'ability_code_3120_diff', 'ability_code_3120_player_0', 'ability_code_3120_player_1', 'ability_code_3140_diff', 'ability_code_3140_player_0', 'ability_code_3140_player_1', 'ability_code_3160_diff', 'ability_code_3160_player_0', 'ability_code_3160_player_1', 'ability_code_31A0_diff', 'ability_code_31A0_first_time_player_1', 'ability_code_31A0_player_0', 'ability_code_31A0_player_1', 'ability_code_3380_player_0', 'ability_code_3380_player_1', 'ability_code_3480_diff', 'ability_code_3480_first_time_diff', 'ability_code_3480_first_time_player_0', 'ability_code_3480_player_0', 'ability_code_3480_player_1', 'ability_code_39A0_diff', 'ability_code_39A0_player_0', 'ability_code_39A0_player_1', 'ability_code_4020_diff', 'ability_code_4020_player_0', 'ability_code_4020_player_1', 'ability_code_4021_diff', 'ability_code_4021_player_0', 'ability_code_4021_player_1', 'ability_code_4040_diff', 'ability_code_4040_player_0', 'ability_code_4040_player_1', 'ability_code_40A0_diff', 'ability_code_40A0_player_0', 'ability_code_40A0_player_1', 'ability_code_4120_diff', 'ability_code_4120_player_0', 'ability_code_4120_player_1', 'ability_code_4160_player_0', 'ability_code_4160_player_1', 'ability_code_41A0_diff', 'ability_code_41A0_player_0', 'ability_code_41A0_player_1', 'ability_code_41E0_diff', 'ability_code_41E0_player_0', 'ability_code_41E0_player_1', 'ability_code_4220_diff', 'ability_code_4220_player_0', 'ability_code_4220_player_1', 'ability_code_42A1_diff', 'ability_code_42A1_player_0', 'ability_code_42A1_player_1', 'ability_code_42C0_player_0', 'ability_code_42C1_diff', 'ability_code_42C1_player_0', 'ability_code_42C1_player_1', 'ability_code_43E0_diff', 'ability_code_43E0_player_0', 'ability_code_43E0_player_1', 'ability_code_480_diff', 'ability_code_480_first_time_diff', 'ability_code_480_first_time_player_0', 'ability_code_480_player_0', 'ability_code_480_player_1', 'ability_code_483_diff', 'ability_code_483_player_0', 'ability_code_483_player_1', 'ability_code_4AE0_diff', 'ability_code_4AE0_player_0', 'ability_code_4AE0_player_1', 'ability_code_4AFE_diff', 'ability_code_4AFE_player_0', 'ability_code_4AFE_player_1', 'ability_code_4B80_diff', 'ability_code_4B80_first_time_diff', 'ability_code_4B80_first_time_player_1', 'ability_code_4B80_player_0', 'ability_code_4B80_player_1', 'ability_code_4C0_diff', 'ability_code_4C0_player_0', 'ability_code_4C0_player_1', 'ability_code_4C1_diff', 'ability_code_4C1_first_time_diff', 'ability_code_4C1_first_time_player_0', 'ability_code_4C1_first_time_player_1', 'ability_code_4C1_player_0', 'ability_code_4C1_player_1', 'ability_code_4C2_diff', 'ability_code_4C2_first_time_diff', 'ability_code_4C2_first_time_player_0', 'ability_code_4C2_player_1', 'ability_code_4C3_diff', 'ability_code_4C3_player_0', 'ability_code_4C3_player_1', 'ability_code_4C60_diff', 'ability_code_4C60_player_0', 'ability_code_4C60_player_1', 'ability_code_4C80_diff', 'ability_code_4C80_player_0', 'ability_code_4C80_player_1', 'ability_code_4E40_first_time_diff', 'ability_code_4E40_first_time_player_0', 'ability_code_4E40_player_1', 'ability_code_4EA0_diff', 'ability_code_4EA0_player_0', 'ability_code_4EA0_player_1', 'ability_code_4EC0_diff', 'ability_code_4EC0_player_0', 'ability_code_4EC0_player_1', 'ability_code_55E0_diff', 'ability_code_55E0_player_0', 'ability_code_55E0_player_1', 'ability_code_5600_diff', 'ability_code_5600_player_0', 'ability_code_5600_player_1', 'ability_code_5640_diff', 'ability_code_5640_player_0', 'ability_code_5640_player_1', 'ability_code_5641_diff', 'ability_code_5641_player_0', 'ability_code_5641_player_1', 'ability_code_56A0_diff', 'ability_code_56A0_player_0', 'ability_code_56A0_player_1', 'ability_code_5700_diff', 'ability_code_5700_player_0', 'ability_code_5700_player_1', 'ability_code_5A0_first_time_diff', 'ability_code_5A0_first_time_player_0', 'ability_code_5A0_player_0', 'ability_code_5A0_player_0_ability_code_1021_player_0_diff', 'ability_code_5A0_player_0_ability_code_102A_player_0_diff', 'ability_code_5A0_player_0_ability_code_1261_player_0_diff', 'ability_code_5A0_player_0_ability_code_13A0_player_0_diff', 'ability_code_5A0_player_0_ability_code_1820_player_0_diff', 'ability_code_5A0_player_0_ability_code_1821_player_0_diff', 'ability_code_5A0_player_0_ability_code_1822_player_0_diff', 'ability_code_5A0_player_0_ability_code_1C60_player_0_diff', 'ability_code_5A0_player_0_ability_code_1E60_player_0_diff', 'ability_code_5A0_player_0_ability_code_480_player_0_diff', 'ability_code_5A0_player_0_ability_code_B40_player_0_diff', 'ability_code_5A0_player_0_ability_code_DE0_player_0_diff', 'ability_code_5A0_player_1', 'ability_code_5A0_player_1_ability_code_1020_player_1_diff_ability_code_5A0_player_0_ability_code_1020_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_1021_player_1_diff', 'ability_code_5A0_player_1_ability_code_1021_player_1_diff_ability_code_5A0_player_0_ability_code_1021_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_1022_player_1_diff', 'ability_code_5A0_player_1_ability_code_1022_player_1_diff_ability_code_5A0_player_0_ability_code_1022_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_1023_player_1_diff', 'ability_code_5A0_player_1_ability_code_1023_player_1_diff_ability_code_5A0_player_0_ability_code_1023_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_1261_player_1_diff', 'ability_code_5A0_player_1_ability_code_1261_player_1_diff_ability_code_5A0_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_13A0_player_1_diff', 'ability_code_5A0_player_1_ability_code_13A0_player_1_diff_ability_code_5A0_player_0_ability_code_13A0_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_13E0_player_1_diff', 'ability_code_5A0_player_1_ability_code_13E0_player_1_diff_ability_code_5A0_player_0_ability_code_13E0_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_16E2_player_1_diff', 'ability_code_5A0_player_1_ability_code_16E2_player_1_diff_ability_code_5A0_player_0_ability_code_16E2_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_1820_player_1_diff', 'ability_code_5A0_player_1_ability_code_1820_player_1_diff_ability_code_5A0_player_0_ability_code_1820_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_1821_player_1_diff', 'ability_code_5A0_player_1_ability_code_1821_player_1_diff_ability_code_5A0_player_0_ability_code_1821_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_1E60_player_1_diff_ability_code_5A0_player_0_ability_code_1E60_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_480_player_1_diff', 'ability_code_5A0_player_1_ability_code_480_player_1_diff_ability_code_5A0_player_0_ability_code_480_player_0_diff_diff', 'ability_code_5A0_player_1_ability_code_B40_player_1_diff', 'ability_code_5A0_player_1_ability_code_DE0_player_1_diff', 'ability_code_780_diff', 'ability_code_780_player_0', 'ability_code_780_player_1', 'ability_code_7A0_diff', 'ability_code_7A0_player_0', 'ability_code_7A0_player_1', 'ability_code_7C0_diff', 'ability_code_7C0_player_0', 'ability_code_7C0_player_1', 'ability_code_7E0_diff', 'ability_code_7E0_player_0', 'ability_code_7E0_player_1', 'ability_code_820_diff', 'ability_code_820_player_0', 'ability_code_820_player_1', 'ability_code_880_diff', 'ability_code_880_player_0', 'ability_code_880_player_1', 'ability_code_8A0_player_0', 'ability_code_8A0_player_1', 'ability_code_8A2_diff', 'ability_code_8A2_player_0', 'ability_code_8A2_player_1', 'ability_code_8E0_diff', 'ability_code_8E0_player_0', 'ability_code_8E0_player_1', 'ability_code_900_diff', 'ability_code_900_player_0', 'ability_code_900_player_1', 'ability_code_920_player_0', 'ability_code_920_player_1', 'ability_code_960_diff', 'ability_code_960_player_1', 'ability_code_9E0_diff', 'ability_code_9E0_player_0', 'ability_code_9E0_player_1', 'ability_code_A40_diff', 'ability_code_A40_player_0', 'ability_code_A40_player_1', 'ability_code_B00_diff', 'ability_code_B00_player_0', 'ability_code_B01_diff', 'ability_code_B01_player_0', 'ability_code_B01_player_1', 'ability_code_B40_diff', 'ability_code_B40_first_time_diff', 'ability_code_B40_first_time_player_1', 'ability_code_B40_last_first_diff', 'ability_code_B40_last_first_player_1', 'ability_code_B40_player_0', 'ability_code_B40_player_0_ability_code_1023_player_0_diff', 'ability_code_B40_player_0_ability_code_102A_player_0_diff', 'ability_code_B40_player_0_ability_code_15E0_player_0_diff', 'ability_code_B40_player_0_ability_code_16E2_player_0_diff', 'ability_code_B40_player_0_ability_code_1821_player_0_diff', 'ability_code_B40_player_0_ability_code_1822_player_0_diff', 'ability_code_B40_player_0_ability_code_1C60_player_0_diff', 'ability_code_B40_player_0_ability_code_1E60_player_0_diff', 'ability_code_B40_player_0_ability_code_480_player_0_diff', 'ability_code_B40_player_0_ability_code_DE0_player_0_diff', 'ability_code_B40_player_1', 'ability_code_B40_player_1_ability_code_1020_player_1_diff', 'ability_code_B40_player_1_ability_code_1022_player_1_diff', 'ability_code_B40_player_1_ability_code_1023_player_1_diff', 'ability_code_B40_player_1_ability_code_102A_player_1_diff', 'ability_code_B40_player_1_ability_code_102A_player_1_diff_ability_code_B40_player_0_ability_code_102A_player_0_diff_diff', 'ability_code_B40_player_1_ability_code_1261_player_1_diff', 'ability_code_B40_player_1_ability_code_1261_player_1_diff_ability_code_B40_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_B40_player_1_ability_code_13A0_player_1_diff', 'ability_code_B40_player_1_ability_code_13A0_player_1_diff_ability_code_B40_player_0_ability_code_13A0_player_0_diff_diff', 'ability_code_B40_player_1_ability_code_15E0_player_1_diff', 'ability_code_B40_player_1_ability_code_15E0_player_1_diff_ability_code_B40_player_0_ability_code_15E0_player_0_diff_diff', 'ability_code_B40_player_1_ability_code_16E2_player_1_diff', 'ability_code_B40_player_1_ability_code_16E2_player_1_diff_ability_code_B40_player_0_ability_code_16E2_player_0_diff_diff', 'ability_code_B40_player_1_ability_code_1821_player_1_diff', 'ability_code_B40_player_1_ability_code_1821_player_1_diff_ability_code_B40_player_0_ability_code_1821_player_0_diff_diff', 'ability_code_B40_player_1_ability_code_1822_player_1_diff', 'ability_code_B40_player_1_ability_code_1822_player_1_diff_ability_code_B40_player_0_ability_code_1822_player_0_diff_diff', 'ability_code_B40_player_1_ability_code_1C60_player_1_diff', 'ability_code_B40_player_1_ability_code_1C60_player_1_diff_ability_code_B40_player_0_ability_code_1C60_player_0_diff_diff', 'ability_code_B40_player_1_ability_code_1E60_player_1_diff', 'ability_code_B40_player_1_ability_code_480_player_1_diff', 'ability_code_B40_player_1_ability_code_480_player_1_diff_ability_code_B40_player_0_ability_code_480_player_0_diff_diff', 'ability_code_B40_player_1_ability_code_DE0_player_1_diff', 'ability_code_B40_player_1_ability_code_DE0_player_1_diff_ability_code_B40_player_0_ability_code_DE0_player_0_diff_diff', 'ability_code_B60_player_0', 'ability_code_B60_player_1', 'ability_code_B61_diff', 'ability_code_B61_player_0', 'ability_code_B61_player_1', 'ability_code_BE0_diff', 'ability_code_BE0_player_0', 'ability_code_CC0_player_0', 'ability_code_CC0_player_1', 'ability_code_D00_diff', 'ability_code_D00_player_0', 'ability_code_D00_player_1', 'ability_code_D20_diff', 'ability_code_D20_player_0', 'ability_code_D20_player_1', 'ability_code_D40_diff', 'ability_code_D40_player_0', 'ability_code_D40_player_1', 'ability_code_D41_player_0', 'ability_code_D61_player_1', 'ability_code_D62_diff', 'ability_code_D62_player_0', 'ability_code_D62_player_1', 'ability_code_DE0_diff', 'ability_code_DE0_first_time_player_0', 'ability_code_DE0_player_0', 'ability_code_DE0_player_0_ability_code_1020_player_0_diff', 'ability_code_DE0_player_0_ability_code_1022_player_0_diff', 'ability_code_DE0_player_0_ability_code_1023_player_0_diff', 'ability_code_DE0_player_0_ability_code_102A_player_0_diff', 'ability_code_DE0_player_0_ability_code_1261_player_0_diff', 'ability_code_DE0_player_0_ability_code_13A0_player_0_diff', 'ability_code_DE0_player_0_ability_code_15E0_player_0_diff', 'ability_code_DE0_player_0_ability_code_16E2_player_0_diff', 'ability_code_DE0_player_0_ability_code_1822_player_0_diff', 'ability_code_DE0_player_0_ability_code_1C60_player_0_diff', 'ability_code_DE0_player_0_ability_code_1E60_player_0_diff', 'ability_code_DE0_player_0_ability_code_480_player_0_diff', 'ability_code_DE0_player_1', 'ability_code_DE0_player_1_ability_code_1020_player_1_diff', 'ability_code_DE0_player_1_ability_code_1022_player_1_diff', 'ability_code_DE0_player_1_ability_code_1022_player_1_diff_ability_code_DE0_player_0_ability_code_1022_player_0_diff_diff', 'ability_code_DE0_player_1_ability_code_1023_player_1_diff', 'ability_code_DE0_player_1_ability_code_1023_player_1_diff_ability_code_DE0_player_0_ability_code_1023_player_0_diff_diff', 'ability_code_DE0_player_1_ability_code_102A_player_1_diff', 'ability_code_DE0_player_1_ability_code_102A_player_1_diff_ability_code_DE0_player_0_ability_code_102A_player_0_diff_diff', 'ability_code_DE0_player_1_ability_code_1261_player_1_diff', 'ability_code_DE0_player_1_ability_code_1261_player_1_diff_ability_code_DE0_player_0_ability_code_1261_player_0_diff_diff', 'ability_code_DE0_player_1_ability_code_13A0_player_1_diff', 'ability_code_DE0_player_1_ability_code_13A0_player_1_diff_ability_code_DE0_player_0_ability_code_13A0_player_0_diff_diff', 'ability_code_DE0_player_1_ability_code_15E0_player_1_diff', 'ability_code_DE0_player_1_ability_code_15E0_player_1_diff_ability_code_DE0_player_0_ability_code_15E0_player_0_diff_diff', 'ability_code_DE0_player_1_ability_code_16E2_player_1_diff', 'ability_code_DE0_player_1_ability_code_16E2_player_1_diff_ability_code_DE0_player_0_ability_code_16E2_player_0_diff_diff', 'ability_code_DE0_player_1_ability_code_1821_player_1_diff', 'ability_code_DE0_player_1_ability_code_1822_player_1_diff', 'ability_code_DE0_player_1_ability_code_1822_player_1_diff_ability_code_DE0_player_0_ability_code_1822_player_0_diff_diff', 'ability_code_DE0_player_1_ability_code_1C60_player_1_diff', 'ability_code_DE0_player_1_ability_code_1C60_player_1_diff_ability_code_DE0_player_0_ability_code_1C60_player_0_diff_diff', 'ability_code_DE0_player_1_ability_code_1E60_player_1_diff', 'ability_code_DE0_player_1_ability_code_1E60_player_1_diff_ability_code_DE0_player_0_ability_code_1E60_player_0_diff_diff', 'ability_code_DE0_player_1_ability_code_480_player_1_diff', 'ability_code_E00_diff', 'ability_code_E00_player_0', 'ability_code_E00_player_1', 'ability_code_E20_diff', 'ability_code_E20_player_0', 'ability_code_E20_player_1', 'ability_code_E80_first_time_diff', 'ability_code_E80_first_time_player_0', 'ability_code_E80_player_0', 'ability_code_EE0_diff', 'ability_code_EE0_player_0', 'ability_code_EE0_player_1', 'ability_code_EE1_diff', 'ability_code_EE1_player_0', 'ability_code_EE1_player_1', 'ability_code_F00_diff', 'ability_code_F00_player_0', 'ability_code_F00_player_1', 'ability_code_F01_diff', 'ability_code_F01_player_0', 'ability_code_F01_player_1', 'ability_code_F40_player_0', 'ability_code_F40_player_1', 'ability_code_F41_diff', 'ability_code_F41_player_0', 'ability_code_F41_player_1', 'ability_code_F60_diff', 'ability_code_F60_first_time_player_1', 'ability_code_F60_player_0', 'ability_code_F60_player_1', 'ability_code_F61_diff', 'ability_code_F61_first_time_diff', 'ability_code_F61_first_time_player_0', 'ability_code_F61_first_time_player_1', 'ability_code_F61_player_0', 'ability_code_F61_player_1', 'ability_code_F80_diff', 'ability_code_F80_first_time_diff', 'ability_code_F80_first_time_player_1', 'ability_code_F80_player_0', 'ability_code_F80_player_1', 'ability_code_F81_diff', 'ability_code_F81_first_time_diff', 'ability_code_F81_first_time_player_1', 'ability_code_F81_player_0', 'ability_code_F81_player_1', 'ability_code_FC0_diff', 'ability_code_FC0_player_0', 'ability_code_FC0_player_1', 'ability_code_FC1_diff', 'ability_code_FC1_player_0', 'ability_code_FC1_player_1', 'ability_code_FE0_first_time_diff', 'ability_code_FE0_player_0', 'ability_code_FE1_diff', 'ability_code_FE1_player_0', 'ability_code_FE1_player_1', 'all_unit_diff', 'all_unit_len_player_0', 'all_unit_len_player_0_Barracks_len_player_0_diff', 'all_unit_len_player_0_Egg_len_player_0_diff', 'all_unit_len_player_0_Factory_len_player_0_diff', 'all_unit_len_player_0_Hellion_len_player_0_diff', 'all_unit_len_player_0_MULE_len_player_0_diff', 'all_unit_len_player_0_Marine_len_player_0_diff', 'all_unit_len_player_0_OrbitalCommand_len_player_0_diff', 'all_unit_len_player_0_Overlord_len_player_0_diff', 'all_unit_len_player_0_Probe_len_player_0_diff', 'all_unit_len_player_0_Reaper_len_player_0_diff', 'all_unit_len_player_0_SCV_len_player_0_diff', 'all_unit_len_player_0_Starport_len_player_0_diff', 'all_unit_len_player_0_SupplyDepotLowered_len_player_0_diff', 'all_unit_len_player_0_Zergling_len_player_0_diff', 'all_unit_len_player_1', 'all_unit_len_player_1_Drone_len_player_1_diff', 'all_unit_len_player_1_Egg_len_player_1_diff', 'all_unit_len_player_1_Factory_len_player_1_diff_all_unit_len_player_0_Factory_len_player_0_diff_diff', 'all_unit_len_player_1_Hatchery_len_player_1_diff_all_unit_len_player_0_Hatchery_len_player_0_diff_diff', 'all_unit_len_player_1_Hellion_len_player_1_diff', 'all_unit_len_player_1_MULE_len_player_1_diff', 'all_unit_len_player_1_MULE_len_player_1_diff_all_unit_len_player_0_MULE_len_player_0_diff_diff', 'all_unit_len_player_1_Marine_len_player_1_diff_all_unit_len_player_0_Marine_len_player_0_diff_diff', 'all_unit_len_player_1_OrbitalCommand_len_player_1_diff', 'all_unit_len_player_1_Overlord_len_player_1_diff', 'all_unit_len_player_1_Queen_len_player_1_diff', 'all_unit_len_player_1_Reaper_len_player_1_diff_all_unit_len_player_0_Reaper_len_player_0_diff_diff', 'all_unit_len_player_1_Starport_len_player_1_diff', 'all_unit_len_player_1_SupplyDepotLowered_len_player_1_diff', 'build_count_player_0', 'build_count_player_1', 'camera_pos_x_max_player_1', 'camera_pos_x_mean_player_1', 'camera_pos_x_min_player_0', 'camera_pos_y_max_player_1', 'camera_pos_y_min_player_0', 'game_id_time_max_div_nunique', 'species_player_0', 'species_player_1', 'species_war_kind', 'time_count_player_diff', 'time_std_player_0', 'under_1_Ability_diff', 'under_1_Ability_player_0', 'under_1_AddToControlGroup_diff', 'under_1_AddToControlGroup_player_0', 'under_1_AddToControlGroup_player_1', 'under_1_ControlGroup_diff', 'under_1_ControlGroup_player_0', 'under_1_ControlGroup_player_1', 'under_1_GetControlGroup_diff', 'under_1_GetControlGroup_player_0', 'under_1_GetControlGroup_player_1', 'under_1_SetControlGroup_diff', 'under_1_SetControlGroup_player_0', 'under_1_SetControlGroup_player_1', 'under_3_Ability_player_0', 'under_3_AddToControlGroup_diff', 'under_3_Camera_diff', 'under_3_ControlGroup_diff', 'under_3_ControlGroup_player_0', 'under_3_GetControlGroup_player_0', 'under_3_GetControlGroup_player_1', 'under_3_Right Click_diff', 'under_3_SetControlGroup_diff', 'under_3_SetControlGroup_player_0', 'under_3_SetControlGroup_player_1', 'under_5_Ability_diff', 'under_5_AddToControlGroup_player_0', 'under_5_AddToControlGroup_player_1', 'under_5_Camera_player_0', 'under_5_ControlGroup_diff', 'under_5_ControlGroup_player_0', 'under_5_ControlGroup_player_1', 'under_5_GetControlGroup_player_0', 'under_5_Selection_diff', 'under_5_SetControlGroup_player_0', 'under_5_SetControlGroup_player_1', 'work_diff']

In [13]:

fold_perm_dict = {0:['perm_importance_swap2_1fold_last2.csv',6.00E-07],
                  1:['perm_importance_swap2_2fold_last2.csv',4.80E-06],
                  2:['perm_importance_swap2_3fold_last2.csv',4.00E-06],
                  3:['perm_importance_swap2_4fold_last2.csv',7.00E-06],
                  4:['perm_importance_swap2_5fold_last2.csv',2.00E-05]}


def get_train_columns(train_df, test_df, useless, perm_file, cutoff):
    fi_df = pd.read_csv(perm_file)
    # cutoff보다 작은 Feature는 사용하지 않음
    bad_feature = list(fi_df.loc[fi_df['perm_importance']<cutoff,'feature'].values) 
    train_columns = [c for c in train_df.columns if c not in ['pseudo','aug','swap','winner','player_0','player_1','game_id','oof',]+
                     useless+bad_feature]

In [14]:
def get_train_columns(train_df, test_df, useless, bad_feature):
    train_columns = [c for c in train_df.columns if c not in ['pseudo','aug','swap','winner','player_0','player_1','game_id','oof',]+
                     useless+bad_feature]
    print(len(train_columns))
    # train_df와 test_df에 공통으로 있는 column만 사용
    # 일부 unit은 test에 존재하지 않음
    train_columns = sorted(set(train_columns).intersection(set(test_df.columns)))
    print(len(train_columns))
    return train_columns

In [15]:
def merge_data(source, df):
    """
    merge_data함수에 넘겨지는 source는 game_id, player column을 가져야 합니다.
    각 항목을 player_0, player_1로 dataframe에 merge 시키는 역할
    
    source data
    game_id player ability_count
    1       0      10
    1       1      20
    
    merge되었을 때 df 값
    game_id player_0 player_1 ability_count_player_0 ability_count_player_1
    0       0        1        10                     20
    """
    new_c = [c for c in source.columns if c not in ['game_id','player']]
    
    source.columns = ['game_id', 'player_0'] + [f'{c}_player_0'for c in new_c]
    df = df.merge(source, on=['game_id','player_0'], how='left')
    
    source.columns = ['game_id', 'player_1'] + [f'{c}_player_1'for c in new_c]
    df = df.merge(source, on=['game_id','player_1'], how='left')
    return df

In [16]:
def train_test_label_encoding(train_df, test_df, col1, col2):
    """
    train, test, player_0, player_1에 나누어져 있는 값을 label encoding
    """
    set1 = set(train_df[col1].dropna().unique())
    set2 = set(train_df[col2].dropna().unique())
    set3 = set(test_df[col1].dropna().unique())
    set4 = set(test_df[col2].dropna().unique())
    unique_label = sorted(set1 | set2 | set3 | set4)
    label_map = dict(zip(unique_label, np.arange(len(unique_label))))

    train_df[col1] = train_df[col1].map(label_map)
    train_df[col2] = train_df[col2].map(label_map)
    test_df[col1] = test_df[col1].map(label_map)
    test_df[col2] = test_df[col2].map(label_map)
    return train_df, test_df

In [17]:
def need_swap(row):
    """
    player swap이 필요한지 확인
    Z vs T -> T vs Z
    P vs T -> T vs P
    P vs Z -> Z vs P
    """
    if row['species_player_0'] == 'Z' and row['species_player_1'] == 'T':
        return 1
    elif row['species_player_0'] == 'P' and row['species_player_1'] == 'T':
        return 1
    elif row['species_player_0'] == 'P' and row['species_player_1'] == 'Z':
        return 1
    else:
        return 0

def get_default_preprocessing(data, is_train):
    """
    기본 변환
    """
    df = pd.DataFrame(data['game_id'].unique(), columns=['game_id'])
    df['player_0'] = 0
    df['player_1'] = 1
    
    # swap이 필요한지 계산
    species_temp = data.drop_duplicates(['game_id','player'])[['game_id','player','species']]
    df_temp = merge_data(species_temp, df) 
    df_temp['swap'] = df_temp.apply(lambda row: need_swap(row), axis=1)
    if 'swap' in data.columns:
        del data['swap']
    
    # 기존 Data에 Swap이 필요한지 merge해주고
    data = data.merge(df_temp[['game_id','swap']],on='game_id',how='left')
    # xor 연산으로 swap이 1일 때 player swap
    # xor 0^1 -> 0
    data.loc[data['swap']==1,'player'] = data.loc[data['swap']==1,'player']^1
    if is_train:
        # 현재 항상 player 1이 이길때 winner값이 결정되는데 player가 바뀌었으니 winner값도 swap합니다.
        data.loc[data['swap']==1,'winner'] = data.loc[data['swap']==1,'winner']^1
        # winner 추가
        winner = data.groupby(['game_id'])['winner'].max()
        df['winner'] = df['game_id'].map(winner)
    
    # swap 여부 추가
    df = df.merge(df_temp[['game_id','swap']],on='game_id',how='left')
    # swap된 종족 추가
    species_temp = data.drop_duplicates(['game_id','player'])[['game_id','player','species']]
    df = merge_data(species_temp, df)
    
    # 종족 label encoding 'T':0,'Z':1,'P':2
    # 종족 전쟁 유형도 추가
    df['species_player_0'] = df['species_player_0'].map({'T':0,'Z':1,'P':2})
    df['species_player_1'] = df['species_player_1'].map({'T':0,'Z':1,'P':2})
    df['species_war_kind'] = df['species_player_0'].astype(str) + '_' + df['species_player_1'].astype(str)
    df['species_war_kind'] = df['species_war_kind'].map({'0_0':0,'0_1':1,'0_2':2, '1_1':3,'1_2':4,'2_2':5})
    
    return df, data

In [18]:
def make_time_feautre(data, df):
    """
    time feature 추가
    """
    # game_id별 게임 시간, time의 unique한 값
    time_temp = data.groupby(['game_id'])['time'].agg(['max','nunique']).reset_index()
    time_temp.columns = ['game_id','game_id_time_max','game_id_time_nunique']
    df = df.merge(time_temp, on='game_id', how='left')
    
    # game_id length
    df['game_id_length'] = df['game_id'].map(data['game_id'].value_counts())
    
    # player 별 time std, count
    player_time = data.groupby(['game_id','player'])['time'].agg(['std','count','sum']).reset_index()
    player_time.columns = ['game_id','player'] + [c[0]+'_'+c[1] for c in itertools.product(['time'],['std','count','sum'])]
    df = merge_data(player_time, df)
    
    # std, count, sum의 diff    
    df['time_count_player_diff'] =  df['time_count_player_1'] - df['time_count_player_0']
    df['time_sum_player_diff'] = df['time_sum_player_1'] - df['time_sum_player_0']
    df['time_std_player_diff'] = df['time_std_player_1'] - df['time_std_player_0']
    
    # 시간은 긴데 time log가 별로 없거나 특정 시간에만 time 값이 몰려 있는 경우
    df['game_id_time_max_div_nunique'] = df['game_id_time_max']/df['game_id_time_nunique']
    return df

In [19]:
def make_is_null_feat(df,c1,c2, nullv, is_null_feature=True):
    """
    null인지 아닌지 feature 추가
    """
    if is_null_feature is True:
        df[f'{c1}_isnull'] = df[c1].isnull()
        df[f'{c2}_isnull'] = df[c2].isnull()
    df[c1] = df[c1].fillna(nullv)
    df[c2] = df[c2].fillna(nullv)
    return df

def make_event_count(data, df):
    # species가 game_id, player에서 모두 동일하니까.. species count하면 event의 개수를 구할 수 있음
    event_df = data.groupby(['game_id','player','event'])['species'].count().reset_index()
    
    # 각 event별 count 개수 추가
    for e in event_df['event'].unique():
        temp = event_df.query("event == @e")
        del temp['event']
        temp.columns = ['game_id','player',e]
        df = merge_data(temp, df)
        
        # null값 0으로 채우고 원래 null은 따로 is_null feature 추가
        # 도움 될 수 있고 안될 수 있는데 어차피 feature selection 할거라서 몽땅 추가함
        new_p0_c = f'{e}_player_0'
        new_p1_c = f'{e}_player_1'
        df = make_is_null_feat(df,new_p0_c,new_p1_c,0)
    
    player_0_col = [f'{c}_player_0' for c in event_df['event'].unique()]
    player_1_col = [f'{c}_player_1' for c in event_df['event'].unique()]
    # player_1과 0의 event count diff 추가
    for c1, c2 in zip(player_0_col, player_1_col):
        df[f'{c1}_{c2}_diff'] = df[c1] - df[c2]
        
    return df

In [20]:
def get_ablility_code(x):
    try:
        return x.split('-')[0].strip('( )')
    except:
        return np.nan

def make_ability_code_feautre(data, df, count=100):
    # ability code별 count 개수
    ability_code_temp = data.groupby(['game_id','player','ability_code'])['species'].count().reset_index()
    
    # param으로 넘겨받은 count만큼 상위 n개를 선택하여 loop돌면서 feature 추가
    for ab in tqdm(ability_code_temp['ability_code'].value_counts().head(count).index):
        temp = ability_code_temp.loc[ability_code_temp['ability_code']==ab]
        del temp['ability_code']
        new_c = f'ability_code_{ab}'
        temp.columns = ['game_id','player',new_c]
        # player 별 ability code count 추가
        df = merge_data(temp, df)
        
        new_p0_c = f'{new_c}_player_0'
        new_p1_c = f'{new_c}_player_1'
        # null 0으로 채우고 is null feature 추가
        df = make_is_null_feat(df,new_p0_c,new_p1_c,0,False)
        # ability code count diff 추가
        df[f'{new_c}_diff'] = df[f'{new_c}_player_1'] - df[f'{new_c}_player_0']
        
    # build ability만 추출하여 build sum값 추가
    data.loc[(data['event']=='Ability'),'is_Build'] = data.loc[(data['event']=='Ability'),'event_contents'].apply(lambda x: 1 if x.find('Build')!=-1 else 0)
    ability_build_count = data.groupby(['game_id','player'])['is_Build'].sum().reset_index()
    ability_build_count.columns = ['game_id','player','build_count']
    df = merge_data(ability_build_count, df)
    
    # build diff
    df['build_count_diff'] = df['build_count_player_1'] - df['build_count_player_0']
    return df

In [21]:
def read_set(x):
    try:
        return literal_eval(x)
    except:
        return np.nan
    
def get_safe_len(x):
    try:
        return len(x)
    except:
        return 0
     
def make_selection_unit_feature(data, df, org_unit_df):
    unit_df = org_unit_df.copy()
    unit_list = unit_df.columns[2:]
    
    # 위에서 만든 selection 기반 unit count df에서 unit개수만큼 loop돌면서 len 추가
    for c in tqdm(unit_list):
        # unit_df[c] = unit_df[c].apply(lambda x: read_set(x))
        unit_df[f'{c}_len'] = unit_df[c].apply(lambda x: get_safe_len(x))
    
    for c in ['SCV_len','Drone_len','Probe_len']:
        if c not in unit_df.columns:
            unit_df[c] = 0
    
    # 일꾼 수 추가
    # unit_df는 swap이 안되어 있기 때문에 player swap 수행
    unit_df['worker_count'] = unit_df['SCV_len']+unit_df['Drone_len']+unit_df['Probe_len']    
    unit_df = unit_df.merge(df[['game_id','swap']],on='game_id',how='left')
    unit_df.loc[unit_df['swap']==1,'player'] = unit_df.loc[unit_df['swap']==1,'player']^1
    temp = unit_df[['game_id','player','worker_count']]
    df = merge_data(temp, df)
    # 일꾼 수 diff
    df['work_diff'] = df['worker_count_player_1']-df['worker_count_player_0']
    
    # 모든 unit 개수
    len_feature = sorted([c for c in unit_df.columns if c.find('len')!=-1])       
    unit_df['all_unit_len'] = unit_df[len_feature].sum(axis=1)
    temp = unit_df[['game_id','player','all_unit_len']]
    df = merge_data(temp, df)
    # 모든 unit 개수 diff
    df['all_unit_diff'] = df['all_unit_len_player_1']-df['all_unit_len_player_0']
    
    # 모든 unit별 개수 feature 추가
    temp = unit_df[['game_id','player']+len_feature]
    temp.fillna(0, inplace=True)
    df = merge_data(temp, df)
        
    return df

In [22]:
def make_event_count_time(data, df, time):
    """
    위에 event count feature와 동일.. 다만 특정 시간 이전의 event count만 계산
    """
    event_df = data.loc[data['time']<time].groupby(['game_id','player','event'])['species'].count().reset_index()
    
    for e in event_df['event'].unique():
        temp = event_df.query("event == @e")
        del temp['event']
        new_c = f'under_{time}_{e}'
        temp.columns = ['game_id','player',new_c]
        df = merge_data(temp, df)
        new_c_p0 = f'{new_c}_player_0'
        new_c_p1 = f'{new_c}_player_1'
        df[new_c_p0] = df[new_c_p0].fillna(0)
        df[new_c_p1] = df[new_c_p1].fillna(0)
        df[f'{new_c}_diff'] = df[new_c_p1] - df[new_c_p0]     
    return df

In [23]:
train.loc[train['event']=='Ability','ability_code'] = train.loc[train['event']=='Ability','event_contents'].apply(lambda x: get_ablility_code(x))
test.loc[test['event']=='Ability','ability_code'] = test.loc[test['event']=='Ability','event_contents'].apply(lambda x: get_ablility_code(x))

In [24]:
train_df, train = get_default_preprocessing(train, is_train=True)
train_df = make_time_feautre(train, train_df)
train_df = make_event_count(train, train_df)
train_df = make_ability_code_feautre(train, train_df, 350)

100%|██████████| 350/350 [00:57<00:00,  6.10it/s]


In [25]:
test_df, test = get_default_preprocessing(test, is_train=False)
test_df = make_time_feautre(test, test_df)
test_df = make_event_count(test, test_df)
test_df = make_ability_code_feautre(test, test_df, 350)

100%|██████████| 350/350 [00:47<00:00,  7.37it/s]


In [26]:
train_df = make_selection_unit_feature(train, train_df, train_unit_df)
train_df = make_event_count_time(train, train_df, 1)
train_df = make_event_count_time(train, train_df, 3)
train_df = make_event_count_time(train, train_df, 5)

100%|██████████| 174/174 [00:04<00:00, 37.72it/s]
C:\Users\User\anaconda3\lib\site-packages\pandas\core\frame.py:4147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [27]:
test_df = make_selection_unit_feature(test, test_df, test_unit_df)
test_df = make_event_count_time(test, test_df, 1)
test_df = make_event_count_time(test, test_df, 3)
test_df = make_event_count_time(test, test_df, 5)

100%|██████████| 173/173 [00:04<00:00, 42.21it/s]


In [28]:
def get_right_click_target(x, index):
    if x.find('Target') != -1:
        return x.split(';')[0].split(' ')[index].strip(' ')
    else:
        return np.nan
train.loc[train['event']=='Right Click','right_click_target'] = train.loc[train['event']=='Right Click','event_contents'].apply(lambda x: get_right_click_target(x, 1))
test.loc[test['event']=='Right Click','right_click_target'] = test.loc[test['event']=='Right Click','event_contents'].apply(lambda x: get_right_click_target(x, 1))

In [29]:
def make_right_click_target_feature(data, df, click_list=None):
    # 이전까지는 species로 했는데 사실 int형 변수 하나 추가하고 groupby하는 것이 더 빠릅니다.
    # right click target count 개수 추가
    data['item']=1
    temp = data.loc[data['event']=='Right Click'].groupby(['game_id','player','right_click_target'])['item'].sum().reset_index()
    
    # click_list를 train, test 동일하게 하게 위하여 test의 경우 click_list를 parameter로 받습니다.
    if click_list is None:
        click_list = temp['right_click_target'].value_counts()[:50].index # 상위 50개만!
    
    for c in tqdm(click_list):
        temp2 = temp.query('right_click_target==@c')
        del temp2['right_click_target']
        new_c = f'{c}_right_click_target_count'
        temp2.columns = ['game_id','player',new_c]
        # right click target의 count값 추가
        df = merge_data(temp2, df)
        new_c_p0 = f'{new_c}_player_0'
        new_c_p1 = f'{new_c}_player_1'
        # null은 0, count니까
        df[new_c_p0] = df[new_c_p0].fillna(0)
        df[new_c_p1] = df[new_c_p1].fillna(0)
        # diff feature 추가
        df[f'{c}_right_click_target_count_diff'] = df[f'{c}_right_click_target_count_player_0'] - df[f'{c}_right_click_target_count_player_1']
        
    return df, click_list

In [30]:
train_df, click_list = make_right_click_target_feature(train, train_df, None)
test_df, click_list = make_right_click_target_feature(test, test_df, click_list)

100%|██████████| 50/50 [00:14<00:00,  3.53it/s]


In [31]:
def get_camera_pos(x, pos):
        try:
            return float(x.strip('at ( )').split(',')[pos])
        except BaseException as e:
            return np.nan
        
train.loc[train['event']=='Camera','camera_pos_x'] = train.loc[train['event']=='Camera','event_contents'].apply(lambda x: get_camera_pos(x,0))
train.loc[train['event']=='Camera','camera_pos_y'] = train.loc[train['event']=='Camera','event_contents'].apply(lambda x: get_camera_pos(x,1))
test.loc[test['event']=='Camera','camera_pos_x'] = test.loc[test['event']=='Camera','event_contents'].apply(lambda x: get_camera_pos(x,0))
test.loc[test['event']=='Camera','camera_pos_y'] = test.loc[test['event']=='Camera','event_contents'].apply(lambda x: get_camera_pos(x,1))

In [32]:
temp = train.groupby(['game_id','player'])[['camera_pos_x','camera_pos_y']].agg(['std','min','max','mean']).reset_index()
temp.columns = ['game_id','player'] + [c[0]+'_'+c[1] for c in itertools.product(['camera_pos_x','camera_pos_y'],['std','min','max','mean'])]
train_df = merge_data(temp, train_df)

In [33]:
temp = test.groupby(['game_id','player'])[['camera_pos_x','camera_pos_y']].agg(['std','min','max','mean']).reset_index()
temp.columns = ['game_id','player'] + [c[0]+'_'+c[1] for c in itertools.product(['camera_pos_x','camera_pos_y'],['std','min','max','mean'])]
test_df = merge_data(temp, test_df)

In [34]:
ability_player_0 = list()
for c in train_df.columns:
    if c.find('ability_code')!=-1 and c.find('player_0')!=-1:
        ability_player_0.append(c)

In [35]:
ability_player_0[:10]

['ability_code_5A0_player_0',
 'ability_code_1360_player_0',
 'ability_code_1021_player_0',
 'ability_code_1022_player_0',
 'ability_code_1023_player_0',
 'ability_code_1820_player_0',
 'ability_code_13E0_player_0',
 'ability_code_1822_player_0',
 'ability_code_B40_player_0',
 'ability_code_1C60_player_0']

In [36]:
ablity_code_column = train_df[ability_player_0].sum().sort_values(ascending=False).head(20).index
print(ablity_code_column)

Index(['ability_code_5A0_player_0', 'ability_code_1360_player_0',
       'ability_code_1820_player_0', 'ability_code_13E0_player_0',
       'ability_code_1021_player_0', 'ability_code_B40_player_0',
       'ability_code_DE0_player_0', 'ability_code_1022_player_0',
       'ability_code_1023_player_0', 'ability_code_1822_player_0',
       'ability_code_13A0_player_0', 'ability_code_1821_player_0',
       'ability_code_15E0_player_0', 'ability_code_1C60_player_0',
       'ability_code_1020_player_0', 'ability_code_1E60_player_0',
       'ability_code_16E2_player_0', 'ability_code_102A_player_0',
       'ability_code_1261_player_0', 'ability_code_480_player_0'],
      dtype='object')


In [37]:
for c in itertools.combinations(ablity_code_column,2):
    new_c0 = f'{c[0]}_{c[1]}_diff'
    c1 = c[0][:-1] + '1'
    c2 = c[1][:-1] + '1'
    new_c1 = f'{c1}_{c2}_diff'
    
    # combination diff feature
    train_df[new_c0] = train_df[c[1]] - train_df[c[0]] # player 0
    train_df[new_c1] = train_df[c2] - train_df[c1] # player 1
    test_df[new_c0] = test_df[c[1]] - test_df[c[0]] # player 0
    test_df[new_c1] = test_df[c2] - test_df[c1] # player 1
    
    # 그리고 그 player1, player0을 또 다시 diff
    new_c2 = f'{new_c1}_{new_c0}_diff'
    train_df[new_c2] = train_df[new_c1] - train_df[new_c0]
    test_df[new_c2] = test_df[new_c1] - test_df[new_c0]

In [38]:
ability_len_0 = list()
for c in train_df.columns:
    if c.find('len')!=-1 and c.find('player_0')!=-1:
        ability_len_0.append(c)

In [39]:
ablity_len_column = train_df[ability_len_0].sum().sort_values(ascending=False).head(20).index
ablity_len_column

Index(['all_unit_len_player_0', 'Egg_len_player_0', 'SCV_len_player_0',
       'Drone_len_player_0', 'Marine_len_player_0', 'Zergling_len_player_0',
       'Probe_len_player_0', 'Barracks_len_player_0',
       'OrbitalCommand_len_player_0', 'SupplyDepotLowered_len_player_0',
       'BanelingCocoon_len_player_0', 'Overlord_len_player_0',
       'Queen_len_player_0', 'Factory_len_player_0', 'Hellion_len_player_0',
       'MULE_len_player_0', 'Hatchery_len_player_0', 'Reaper_len_player_0',
       'Roach_len_player_0', 'Starport_len_player_0'],
      dtype='object')

In [40]:
for c in itertools.combinations(ablity_len_column,2):
    new_c0 = f'{c[0]}_{c[1]}_diff'
    c1 = c[0][:-1] + '1'
    c2 = c[1][:-1] + '1'
    new_c1 = f'{c1}_{c2}_diff'
    train_df[new_c0] = train_df[c[1]] - train_df[c[0]]
    train_df[new_c1] = train_df[c2] - train_df[c1]
    test_df[new_c0] = test_df[c[1]] - test_df[c[0]]
    test_df[new_c1] = test_df[c2] - test_df[c1]
    
    new_c2 = f'{new_c1}_{new_c0}_diff'
    train_df[new_c2] = train_df[new_c1] - train_df[new_c0]
    test_df[new_c2] = test_df[new_c1] - test_df[new_c0]

In [41]:
def make_ability_code_time_feautre(data, df, count=100):
    """
    Ability code가 언제 처음 발생했는지 추가하기 위한 코드입니다.
    """
    # ability_code의 time first값을 group by해서 구하고
    ability_code_temp = data.groupby(['game_id','player','ability_code'])['time'].first().reset_index()
    
    # 넘겨 받은 count값을 활용하여 상위 n개만 선택하여 첫번째 ability code 시간을 구합니다.
    # 마찬가지로 diff도 추가합니다.
    for ab in tqdm(ability_code_temp['ability_code'].value_counts().head(count).index):
        temp = ability_code_temp.loc[ability_code_temp['ability_code']==ab]
        del temp['ability_code']
        new_c = f'ability_code_{ab}_first_time'
        temp.columns = ['game_id','player',new_c]
        df = merge_data(temp, df)
        new_p0_c = f'{new_c}_player_0'
        new_p1_c = f'{new_c}_player_1'
        df[f'{new_c}_diff'] = df[f'{new_c}_player_1'] - df[f'{new_c}_player_0']
    return df

In [42]:
train_df = make_ability_code_time_feautre(train, train_df, 100)
test_df = make_ability_code_time_feautre(test, test_df, 100)

100%|██████████| 100/100 [00:50<00:00,  1.99it/s]


In [43]:
def make_ability_code_last_time_feautre(data, df, count=100):
    # 위와 동일한데 마지막 ability code의 시간을 추출합니다.
    ability_code_temp = data.groupby(['game_id','player','ability_code'])['time'].last().reset_index()

    for ab in tqdm(ability_code_temp['ability_code'].value_counts().head(count).index):
        temp = ability_code_temp.loc[ability_code_temp['ability_code']==ab]
        del temp['ability_code']
        new_c = f'ability_code_{ab}_last_first'
        temp.columns = ['game_id','player',new_c]
        df = merge_data(temp, df)
        new_p0_c = f'{new_c}_player_0'
        new_p1_c = f'{new_c}_player_1'
        df[f'{new_c}_diff'] = df[f'{new_c}_player_1'] - df[f'{new_c}_player_0']
        #df[f'{new_c}_div'] = df[f'{new_c}_player_1']/df[f'{new_c}_player_0']
    return df

In [44]:
train_df = make_ability_code_last_time_feautre(train, train_df, 10)
test_df = make_ability_code_last_time_feautre(test, test_df, 10)

100%|██████████| 10/10 [00:06<00:00,  1.54it/s]


In [45]:
useless = []
for c in train_df.columns:
    try:
        if train_df[c].nunique()<2:
            useless.append(c)
            print("useless", c)
    except:
        print("exception:", c)

useless player_0
useless player_1
useless ability_code_2B40_player_1
useless ability_code_26C0_player_0
useless ability_code_16A1_player_0
useless ability_code_1BC1_player_1
useless ability_code_1203_player_1
useless ability_code_1860_player_0
useless ability_code_1583_player_0
useless ability_code_56A0_player_1
useless ability_code_1880_player_0
useless ability_code_1E20_player_1
useless ability_code_3180_player_0
useless ability_code_1687_player_0
useless ability_code_17C0_player_0
useless ability_code_1682_player_0
useless ability_code_1B60_player_0
useless ability_code_1B23_player_0
useless ability_code_1B00_player_0
useless ability_code_EA2_player_0
useless ability_code_11E0_player_1
useless ability_code_18A0_player_0
useless ability_code_22C0_player_1
useless ability_code_840_player_1
useless BroodLord_len_player_0
useless ChangelingMarineShield_len_player_0
useless ExtendingBridgeNWWide8_len_player_0
useless InfestorBurrowed_len_player_0
useless Infestor_len_player_0
useless Ult

In [51]:
train_columns = get_train_columns(x_train, x_test, useless, bad_feature_list)
trn_x, trn_y = x_train.iloc[trn_index][train_columns], y_train.iloc[trn_index]
val_x, val_y = x_train.iloc[val_index][train_columns], y_train.iloc[val_index]

935
898
['Ability_player_0', 'Ability_player_0_Ability_player_1_diff', 'Ability_player_1', 'AddToControlGroup_player_0', 'AddToControlGroup_player_0_AddToControlGroup_player_1_diff', 'AddToControlGroup_player_1_isnull', 'AdeptPiercingWeapon_len_player_1', 'Adept_right_click_target_count_player_1', 'Assimilator_len_player_1', 'Assimilator_right_click_target_count_diff', 'Assimilator_right_click_target_count_player_0', 'Assimilator_right_click_target_count_player_1', 'BanelingCocoon_len_player_0_Factory_len_player_0_diff', 'BanelingCocoon_len_player_0_Hatchery_len_player_0_diff', 'BanelingCocoon_len_player_0_Roach_len_player_0_diff', 'BanelingCocoon_len_player_1', 'BanelingCocoon_len_player_1_Factory_len_player_1_diff_BanelingCocoon_len_player_0_Factory_len_player_0_diff_diff', 'BanelingCocoon_len_player_1_Hatchery_len_player_1_diff', 'BanelingCocoon_len_player_1_Hellion_len_player_1_diff_BanelingCocoon_len_player_0_Hellion_len_player_0_diff_diff', 'BanelingCocoon_len_player_1_Overlord_l

In [ ]:
 

## 모델 생성
model = Sequential()

model.add(LSTM(64, return_sequences=True, input_shape=(x_train.shape[1],1)))
model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.5))
# model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
# mse, binary_crossentropy
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['acc'])
model.summary()

# 트레이닝 값으로 학습 # validation_split=0.2 validation_data=(x_test, y_test)
hist = model.fit(x_train, y_train, validation_split=0.2
                 , batch_size=10, epochs=10)
results = model.evaluate(x_test, y_test)

In [ ]:
#====================================================================#

In [46]:
# 위에서 선언한 나쁜 feature를 가져오고
fold_perm_dict = {0:bad_feature_fold1,
                  1:bad_feature_fold2,
                  2:bad_feature_fold3,
                  3:bad_feature_fold4,
                  4:bad_feature_fold5}

train_columns = [c for c in train_df.columns if c not in ['pseudo','aug','swap','winner','player_0','player_1','game_id','oof',]+
                 useless]
category_col = [] # category feature 사용 안함
print(len(train_columns))
train_columns = sorted(set(train_columns).intersection(set(test_df.columns)))
print(len(train_columns))

# parameter는 hyperopt로 찾음..
lgb_param = {'objective': 'binary', 
             'boosting_type': 'gbdt', 
             'subsample_freq': 7, 
             'num_leaves': 62, 
             'min_data_in_leaf': 40, 
             'subsample_for_bin': 34000, 
             'max_depth': 11, 
             'feature_fraction': 0.749778, 
             'bagging_fraction': 0.893392, 
             'lambda_l1': 0.640773, 
             'lambda_l2': 1.371190, 
             'min_child_weight': 16.668750, 
             'nthread': 44, 
             'metric': 'auc', 
             'learning_rate': 0.021498, 
             'min_sum_hessian_in_leaf': 8, 
             'seed': 42, 
             #'drop_rate': 0.646244, 
             #'skip_drop': 0.592465, 
             #'max_drop': 65,
             'n_estimators': 4000}

3139
3102


In [47]:
x_train = train_df.copy()

y_train = x_train['winner']
x_test = test_df.copy()

oof_train = np.zeros((len(x_train),))
oof_test = np.zeros((len(x_test),))
auc_score_list= []   
NFOLD = 5
SEED = 42

# StratifiedKFold 사용 5fold, seed 42
# stratfied는 winner값
folds = StratifiedKFold(n_splits=NFOLD, shuffle=True, random_state=42)
for fold_, (trn_index, val_index) in enumerate(folds.split(x_train, y_train)):
    print(f"{fold_+1} FOLD Start!!")
    # fold마다 permutation importance를 다르게 적용하여 학습한 column이 매 fold마다 다릅니다.
    # LB 0.765 -> 0.766
    bad_feature_list = fold_perm_dict[fold_].copy()
    train_columns = get_train_columns(x_train, x_test, useless, bad_feature_list)
    
    trn_x, trn_y = x_train.iloc[trn_index][train_columns], y_train.iloc[trn_index]
    val_x, val_y = x_train.iloc[val_index][train_columns], y_train.iloc[val_index]
    dtrain = lgbm.Dataset(trn_x, label=trn_y, silent=True)
    dcross = lgbm.Dataset(val_x, label=val_y, silent=True)
    
    clf = lgbm.train(lgb_param, train_set=dtrain, valid_sets=[dtrain, dcross], 
                       verbose_eval=200, early_stopping_rounds=200, categorical_feature=category_col)
    
    val_pred = clf.predict(val_x)
    oof_train[val_index] = val_pred
    
    _score = roc_auc_score(val_y, val_pred)
    auc_score_list.append(_score)
    print(f"{fold_+1} FOLD AUC: ", _score)
    
    oof_test += clf.predict(x_test[train_columns])/NFOLD

1 FOLD Start!!
1189
1152


C:\Users\User\anaconda3\lib\site-packages\lightgbm\engine.py:151: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\User\anaconda3\lib\site-packages\lightgbm\basic.py:1554: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is []
  warnings.warn('categorical_feature in Dataset is overridden.\n'


[LightGBM] [Warning] min_sum_hessian_in_leaf is set=8, min_child_weight=16.66875 will be ignored. Current value: min_sum_hessian_in_leaf=8
[LightGBM] [Info] Number of positive: 7766, number of negative: 7782
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.147633 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 88903
[LightGBM] [Info] Number of data points in the train set: 15548, number of used features: 1143
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=8, min_child_weight=16.66875 will be ignored. Current value: min_sum_hessian_in_leaf=8
Training until validation scores don't improve for 200 rounds
[200]	training's auc: 0.91818	valid_1's auc: 0.723679
[400]	training's auc: 0.974396	valid_1's auc: 0.73382
[600]	training's auc: 0.992115	valid_1's auc: 0.738259
[800]	training's auc: 0.997678	valid_1's auc: 0.739655
[1000]	training's auc: 0.999373	valid_1's auc: 0.740056
[1200]	training's auc:

C:\Users\User\anaconda3\lib\site-packages\lightgbm\engine.py:151: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\User\anaconda3\lib\site-packages\lightgbm\basic.py:1554: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is []
  warnings.warn('categorical_feature in Dataset is overridden.\n'


[LightGBM] [Warning] min_sum_hessian_in_leaf is set=8, min_child_weight=16.66875 will be ignored. Current value: min_sum_hessian_in_leaf=8
[LightGBM] [Info] Number of positive: 7766, number of negative: 7783
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.140261 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 93113
[LightGBM] [Info] Number of data points in the train set: 15549, number of used features: 1065
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=8, min_child_weight=16.66875 will be ignored. Current value: min_sum_hessian_in_leaf=8
Training until validation scores don't improve for 200 rounds
[200]	training's auc: 0.922296	valid_1's auc: 0.727537
[400]	training's auc: 0.976543	valid_1's auc: 0.73739
[600]	training's auc: 0.993026	valid_1's auc: 0.740854
[800]	training's auc: 0.998067	valid_1's auc: 0.741388
Early stopping, best iteration is:
[717]	training's auc: 0.996593	valid_1's a

C:\Users\User\anaconda3\lib\site-packages\lightgbm\engine.py:151: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\User\anaconda3\lib\site-packages\lightgbm\basic.py:1554: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is []
  warnings.warn('categorical_feature in Dataset is overridden.\n'


[LightGBM] [Warning] min_sum_hessian_in_leaf is set=8, min_child_weight=16.66875 will be ignored. Current value: min_sum_hessian_in_leaf=8
[LightGBM] [Info] Number of positive: 7766, number of negative: 7783
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.135414 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 91790
[LightGBM] [Info] Number of data points in the train set: 15549, number of used features: 1132
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=8, min_child_weight=16.66875 will be ignored. Current value: min_sum_hessian_in_leaf=8
Training until validation scores don't improve for 200 rounds
[200]	training's auc: 0.919393	valid_1's auc: 0.732301
[400]	training's auc: 0.975032	valid_1's auc: 0.737616
[600]	training's auc: 0.992436	valid_1's auc: 0.740645
[800]	training's auc: 0.997669	valid_1's auc: 0.742015
[1000]	training's auc: 0.999398	valid_1's auc: 0.742828
Early stopping, best

C:\Users\User\anaconda3\lib\site-packages\lightgbm\engine.py:151: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\User\anaconda3\lib\site-packages\lightgbm\basic.py:1554: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is []
  warnings.warn('categorical_feature in Dataset is overridden.\n'


[LightGBM] [Warning] min_sum_hessian_in_leaf is set=8, min_child_weight=16.66875 will be ignored. Current value: min_sum_hessian_in_leaf=8
[LightGBM] [Info] Number of positive: 7767, number of negative: 7782
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.107167 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 78645
[LightGBM] [Info] Number of data points in the train set: 15549, number of used features: 890
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=8, min_child_weight=16.66875 will be ignored. Current value: min_sum_hessian_in_leaf=8
Training until validation scores don't improve for 200 rounds
[200]	training's auc: 0.919463	valid_1's auc: 0.722177
[400]	training's auc: 0.972662	valid_1's auc: 0.731202
[600]	training's auc: 0.990988	valid_1's auc: 0.735804


KeyboardInterrupt: 

In [ ]:
train_df['oof'] = oof_train

# player 1이 이긴 확률값이 들어가야 하기 때문에 swap을 수행한 data는 1-prob 수행
train_df.loc[train_df['swap']==1,'oof'] = 1-train_df.loc[train_df['swap']==1,'oof']
train_result = train_df[['swap','winner','oof','species_war_kind']]
# swap한 winner값도 다시 원복
train_result.loc[train_result['swap']==1,'winner'] = train_result.loc[train_result['swap']==1,'winner']^1

print(np.round(np.mean(auc_score_list),6))
print(roc_auc_score(train_result['winner'],train_result['oof']))

# 계속 종족별 auc 값을 check하면서 되도록이면 6개 종족전에 도움이 되는 feature 추가
# 각 feature를 추가할 때 마다 어떤 종족전에 도움이 되는지 알 수 있었는데
# 특히 first time같은 경우 Z vs Z 전에 큰 도움이 되었습니다.
species_war_kind = {0:'T_T', 1:'T_Z', 2:'T_P', 3:'Z_Z',4:'Z_P',5:'P_P'}
species_war_result ={}
for c in sorted(train_result['species_war_kind'].unique()):
    temp = train_result.query('species_war_kind==@c')
    species_war_result[species_war_kind[c]] = '{:.4f}'.format(roc_auc_score(temp['winner'],temp['oof']))
    print(species_war_kind[c], roc_auc_score(temp['winner'],temp['oof']))
batch_result_string = ', '.join([f'{k}:{v}' for k,v in species_war_result.items()])
print(batch_result_string)

In [ ]:
test_df['oof'] = oof_test
test_df.loc[test_df['swap']==1,'oof'] = 1-test_df.loc[test_df['swap']==1,'oof']
oof_test_swap = test_df['oof'].values

In [49]:
lgbm_sub = pd.read_csv('./data/sample_submission.csv')
print(np.sum(lgbm_sub['game_id']-test_df['game_id']))
lgbm_sub['winner'] = oof_test_swap

0


In [50]:
cat_params = {
                'n_estimators':10000,
                'learning_rate': 0.01,
                'eval_metric':'AUC',
                'loss_function':'Logloss',
                'random_seed':42,
                'metric_period':200,
                'od_wait':200,
                'task_type':'GPU',
                'depth': 11,
                'border_count':254,
                'l2_leaf_reg':20,
                'min_data_in_leaf':16,
                'bootstrap_type':'Bayesian',
                'bagging_temperature':0.9,
                'max_leaves':48,
                'grow_policy':'Lossguide'
                #'colsample_bylevel':0.7,
                } 

In [51]:
x_train = train_df.copy()
y_train = train_df['winner']
x_test = test_df.copy()

oof_train_cat = np.zeros((len(x_train),))
oof_test_cat = np.zeros((len(x_test),))
log_loss_score_list= []   
NFOLD = 5
SEED = 42

folds = StratifiedKFold(n_splits=NFOLD, shuffle=True, random_state=SEED)
for fold_, (trn_index, val_index) in enumerate(folds.split(x_train, y_train)):
    print(f"{fold_+1} FOLD Start!!")
    bad_feature_list = fold_perm_dict[fold_].copy()
    train_columns = get_train_columns(x_train, x_test, useless, bad_feature_list)
    trn_x, trn_y = x_train.iloc[trn_index][train_columns], y_train.iloc[trn_index]
    val_x, val_y = x_train.iloc[val_index][train_columns], y_train.iloc[val_index]
    
    # lgbm과 마찬가지이고 아래 부분만 변경됨
    estimator = CatBoostClassifier(**cat_params)        
    clf = estimator.fit(
        trn_x, trn_y,
        eval_set=(val_x, val_y),
        use_best_model=True)
    
    val_pred = estimator.predict_proba(val_x)
    oof_train_cat[val_index] = val_pred[:,1]
    
    log_loss_score = roc_auc_score(val_y, val_pred[:,1])
    log_loss_score_list.append(log_loss_score)
    print(f"{fold_+1} FOLD LogLoss: ", log_loss_score)
    
    oof_test_cat += estimator.predict_proba(x_test[train_columns])[:,1]/NFOLD

1 FOLD Start!!
1193
1152
0:	learn: 0.7036772	test: 0.6528246	best: 0.6528246 (0)	total: 89.4ms	remaining: 14m 54s
200:	learn: 0.8249515	test: 0.7061103	best: 0.7061553 (199)	total: 12.2s	remaining: 9m 55s
400:	learn: 0.8887042	test: 0.7222230	best: 0.7222545 (399)	total: 23.8s	remaining: 9m 29s
600:	learn: 0.9309994	test: 0.7307122	best: 0.7307183 (599)	total: 35.1s	remaining: 9m 8s
800:	learn: 0.9581277	test: 0.7351183	best: 0.7351804 (798)	total: 46.5s	remaining: 8m 54s
1000:	learn: 0.9750404	test: 0.7377919	best: 0.7378123 (987)	total: 57.8s	remaining: 8m 39s
1200:	learn: 0.9853359	test: 0.7394182	best: 0.7394870 (1176)	total: 1m 9s	remaining: 8m 27s
1400:	learn: 0.9917114	test: 0.7397019	best: 0.7398511 (1389)	total: 1m 20s	remaining: 8m 14s
1600:	learn: 0.9954421	test: 0.7402819	best: 0.7402819 (1600)	total: 1m 31s	remaining: 8m 1s
1800:	learn: 0.9975932	test: 0.7406434	best: 0.7406960 (1798)	total: 1m 42s	remaining: 7m 48s
2000:	learn: 0.9987575	test: 0.7407598	best: 0.7408823 (1

In [52]:
train_df['oof'] = oof_train_cat

train_df.loc[train_df['swap']==1,'oof'] = 1-train_df.loc[train_df['swap']==1,'oof']
train_result = train_df[['swap','winner','oof','species_war_kind']]
train_result.loc[train_result['swap']==1,'winner'] = train_result.loc[train_result['swap']==1,'winner']^1

print(np.round(np.mean(auc_score_list),6))
print(roc_auc_score(train_result['winner'],train_result['oof']))
# T, Z, P
species_war_kind = {0:'T_T', 1:'T_Z', 2:'T_P', 3:'Z_Z',4:'Z_P',5:'P_P'}
species_war_result ={}
for c in sorted(train_result['species_war_kind'].unique()):
    temp = train_result.query('species_war_kind==@c')
    species_war_result[species_war_kind[c]] = '{:.4f}'.format(roc_auc_score(temp['winner'],temp['oof']))
    print(species_war_kind[c], roc_auc_score(temp['winner'],temp['oof']))
batch_result_string = ', '.join([f'{k}:{v}' for k,v in species_war_result.items()])
print(batch_result_string)

0.742463
0.7427205662106088
T_T 0.7411091771916515
T_Z 0.7665402430895092
T_P 0.7539029410256617
Z_Z 0.7087528581170524
Z_P 0.7242366035010044
P_P 0.7079162554631327
T_T:0.7411, T_Z:0.7665, T_P:0.7539, Z_Z:0.7088, Z_P:0.7242, P_P:0.7079


C:\Users\ponno\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [53]:
test_df['oof'] = oof_test_cat
test_df.loc[test_df['swap']==1,'oof'] = 1-test_df.loc[test_df['swap']==1,'oof']
oof_test_swap = test_df['oof'].values

In [55]:
cat_sub = pd.read_csv('./data/sample_submission.csv')
print(np.sum(sample_submission['game_id']-test_df['game_id']))
cat_sub['winner'] = oof_test_swap

0


In [56]:
lgbm_sub['winner'].corr(cat_sub['winner'])

0.9847417982815919

In [58]:
lgbm_sub['winner'] = lgbm_sub['winner']*0.6 + cat_sub['winner']*0.4
lgbm_sub.to_csv('./data/submission.csv',index=False)